# Install Requirements

In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tavily-search (from versions: none)
ERROR: No matching distribution found for tavily-search


# Dependencies

In [69]:
import pandas as pd
import numpy as np
import os, requests, json, csv
import psycopg2

In [70]:
from pydantic import BaseModel, Field
from openai import OpenAI

# Initialize Ontology Translator

In [71]:
# Function to load ontology into memory
def load_ontology_from_db() -> dict:
    conn = psycopg2.connect(os.getenv("DATABASE_URL"))
    cur = conn.cursor()

    ontology_data = {
        "uberon-tissue": {},
        "cl-cell": {},
        "ensembl-gene": {}
    }

    table_mappings = [
        ("TISSUE", "UBERON", "tissueName", "uberon-tissue"),
        ("CELL", "CL", "cellName", "cl-cell"),
        ("GENE", "ENSEMBL", "geneName", "ensembl-gene")
    ]

    for table, key_col, value_col, json_key in table_mappings:
        cur.execute(f"SELECT {key_col}, {value_col} FROM {table}")
        rows = cur.fetchall()
        ontology_data[json_key] = {key: value for key, value in rows}

    cur.close()
    conn.close()

    return ontology_data

In [72]:
class OntologyTranslator:
    _ontology_dict = None  # Class-level cache

    @classmethod
    def _load_ontology(cls):
        if cls._ontology_dict is None:
            cls._ontology_dict = load_ontology_from_db()

    @classmethod
    def tissue(cls, value: str, rev: bool = False) -> str | None:
        cls._load_ontology()
        table = cls._ontology_dict.get("uberon-tissue", {})
        return (
            next((k for k, v in table.items() if v.lower() == value.lower()), None)
            if rev else table.get(value)
        )

    @classmethod
    def cell(cls, value: str, rev: bool = False) -> str | None:
        cls._load_ontology()
        table = cls._ontology_dict.get("cl-cell", {})
        return (
            next((k for k, v in table.items() if v.lower() == value.lower()), None)
            if rev else table.get(value)
        )

    @classmethod
    def gene(cls, value: str, rev: bool = False) -> str | None:
        cls._load_ontology()
        table = cls._ontology_dict.get("ensembl-gene", {})
        return (
            next((k for k, v in table.items() if v.lower() == value.lower()), None)
            if rev else table.get(value)
        )

In [73]:
ontl = OntologyTranslator

# Load CSV

In [ ]:
def csv_read(file_path):
    with open(file_path, 'r') as file:
        sample = file.read(1024) 
        file.seek(0)
        detected_delimiter = csv.Sniffer().sniff(sample).delimiter    
    # Read the file with the detected delimiter
    df = pd.read_csv(file_path, sep=detected_delimiter)
    return df

In [75]:
file_path = "cluster0.csv"
df = csv_read(file_path)

In [76]:
df

,myAUC,avg_diff,power,avg_log2FC,pct.1,pct.2,Cluster,Gene
0,0.939,0.918256,0.878,1.351082,1.000,0.996,0,KRT14
1,0.855,0.713125,0.710,1.219866,1.000,0.848,0,IFI27
2,0.755,0.507771,0.510,1.248768,0.875,0.509,0,LGALS3BP
3,0.741,0.476354,0.482,1.234963,0.848,0.460,0,RBM42
4,0.703,0.363869,0.406,1.289658,0.694,0.300,0,GAMT
...,...,...,...,...,...,...,...,...
128,0.839,0.837200,0.678,1.736036,0.942,0.438,25,COL6A3
129,0.738,0.427259,0.476,1.844166,0.626,0.142,25,COL11A1
130,0.724,0.189142,0.448,0.306940,1.000,0.828,26,SPARC
131,0.703,0.282657,0.406,0.431395,1.000,0.998,26,MT-CYB


# Preprocess Data

## Group Genes by Clusters

In [77]:
def extract_clusters(df):
    # Ensure the column names are correct
    if "Cluster" not in df.columns or "Gene" not in df.columns:
        raise ValueError("The file must contain 'Cluster' and 'Gene' columns")

    # Group the data by clusters and pad genes to ensure columns have equal lengths
    grouped = df.groupby("Cluster")["Gene"].apply(list)

    # Create a DataFrame where each column is a cluster and rows contain the genes
    max_length = max(grouped.apply(len))  # Determine the maximum number of genes in a cluster
    return pd.DataFrame({cluster: genes + [None] * (max_length - len(genes)) for cluster, genes in grouped.items()})

In [78]:
clustered_df = extract_clusters(df)

clustered_df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,KRT14,TXNIP,LY6D,S100A8,TIMP1,PERP,TINAGL1,LTF,KRT16,KRT14,...,IGHG4,THRSP,IGKC,SPRR2D,IGHG4,SAA1,LOR,S100A8,COL3A1,SPARC
1,IFI27,ATP1B3,KRT16,S100A7,APOD,GPNMB,IGFBP6,CCL27,SBSN,LY6D,...,IGHG1,PECR,IGHG3,SPRR1A,IGKC,AWAT2,LCE1B,S100A7,COL1A1,MT-CYB
2,LGALS3BP,MAF,HIST1H1C,SPRR1B,MMP3,GJB6,MMP28,FOSB,LGALS7,LGALS7,...,JCHAIN,HSD3B1,IGHA1,IL36RN,C1QA,ACSBG1,WFDC12,CSTA,DCN,MT-ND2
3,RBM42,MMP13,LSR,KRTDAP,COMP,CLDN1,CENPW,ATF3,KRTDAP,IFI27,...,IGFBP6,FABP7,IGLC2,LCE3E,MYL9,CYP4F8,LCE1A,S100A7A,COL6A3,None
4,GAMT,PLAU,TMEM79,PI3,CCL19,TNFSF10,CCNB2,AC103591.3,HIST1H1C,TAGLN2,...,SULF1,ACSM6,IGHG2,SLC6A14,IGHG1,GLDC,LCE1E,SPP1,COL11A1,None


## Translate Genes into Ensembl ID

In [79]:
def translate_genes(clustered_df):
    # Translate gene names to Ensembl IDs in the clustered DataFrame
    return clustered_df.apply(lambda col: col.map(lambda gene: ontl.gene(gene, rev=True) if pd.notna(gene) else None))

In [82]:
ensembl_df = translate_genes(clustered_df)
ensembl_df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,ENSG00000186847,ENSG00000265972,ENSG00000167656,ENSG00000143546,ENSG00000102265,ENSG00000112378,ENSG00000142910,ENSG00000012223,ENSG00000186832,ENSG00000186847,...,ENSG00000211892,ENSG00000151365,ENSG00000211592,ENSG00000163216,ENSG00000211892,ENSG00000173432,None,ENSG00000143546,ENSG00000168542,ENSG00000113140
1,ENSG00000165949,ENSG00000069849,ENSG00000186832,ENSG00000143556,ENSG00000189058,ENSG00000136235,ENSG00000167779,ENSG00000213927,ENSG00000189001,ENSG00000167656,...,ENSG00000211896,ENSG00000115425,ENSG00000211897,ENSG00000169474,ENSG00000211592,ENSG00000147160,ENSG00000196734,ENSG00000143556,ENSG00000108821,ENSG00000198727
2,ENSG00000108679,ENSG00000178573,None,ENSG00000169469,ENSG00000149968,ENSG00000121742,ENSG00000271447,ENSG00000125740,ENSG00000205076,ENSG00000205076,...,ENSG00000132465,ENSG00000203857,ENSG00000211895,ENSG00000136695,ENSG00000173372,ENSG00000103740,ENSG00000168703,ENSG00000121552,ENSG00000011465,ENSG00000198763
3,ENSG00000126254,ENSG00000137745,ENSG00000105699,ENSG00000188508,ENSG00000105664,ENSG00000163347,ENSG00000203760,ENSG00000162772,ENSG00000188508,ENSG00000165949,...,ENSG00000167779,ENSG00000164434,ENSG00000211677,ENSG00000185966,ENSG00000101335,ENSG00000186526,ENSG00000186844,ENSG00000184330,ENSG00000163359,None
4,ENSG00000130005,ENSG00000122861,ENSG00000163472,ENSG00000124102,ENSG00000172724,ENSG00000121858,ENSG00000157456,None,None,ENSG00000158710,...,ENSG00000137573,ENSG00000173124,ENSG00000211893,ENSG00000268104,ENSG00000211896,ENSG00000178445,ENSG00000186226,ENSG00000118785,ENSG00000060718,None


# Statistical Analysis with CellxGene's Gene Expression Data

## Function Initialization

Function to fetch gene expression data

In [83]:
def fetch_expression_data(cluster, ensembl_ids):
    """
    Fetches gene expression data from the Cellxgene API for a given cluster and list of Ensembl IDs.
    
    :param cluster: The cluster name.
    :param ensembl_ids: List of Ensembl IDs for the cluster.
    :return: JSON response from the API or None if an error occurred.
    """

    payload = {
        "filter": {
            "dataset_ids": [],
            "development_stage_ontology_term_ids": [],
            "disease_ontology_term_ids": [],
            "gene_ontology_term_ids": ensembl_ids,
            "organism_ontology_term_id": "NCBITaxon:9606",
            "self_reported_ethnicity_ontology_term_ids": [],
            "sex_ontology_term_ids": [],
            "publication_citations": [],
        },
        "is_rollup": True
    }

    # URL for the POST request
    API_URL = "https://api.cellxgene.cziscience.com/wmg/v2/query"

    try:
        response = requests.post(API_URL, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()['expression_summary']
    except requests.RequestException as e:
        print(f"Error fetching data for Cluster {cluster}: {e}")
        return None

Function to convert the data to a DataFrame

In [84]:
def expression_data_to_df(data):
    flattened_data = []
    for gene_id, anatomical_structures in data.items():
        for anatomical_id, cell_types in anatomical_structures.items():
            for cell_type_id, aggregated_data in cell_types.items():
                metrics = aggregated_data['aggregated']
                flattened_data.append({
                    'gene': gene_id,
                    'tissue': anatomical_id,
                    'cell': cell_type_id,
                    'expression': metrics.get('me', None),
                    'cell count': metrics.get('n', None),
                    'cell percentage': metrics.get('pc', None),
                    'tissue composition': metrics.get('tpc', None)
                })
    return pd.DataFrame(flattened_data)

Function to filter the data

In [85]:
def filter_expression_data(response_df, TARGET_UBERON_ID):
    tissue_df = response_df[response_df['tissue'] == TARGET_UBERON_ID].drop(columns=['tissue'])
    return tissue_df[~tissue_df['cell'].isin(['tissue_stats', 'CL:0000000'])]

Function to translate the ontology terms

In [86]:
def translate_ontology(df):
    df['cell'] = df['cell'].apply(lambda x: ontl.cell(x))
    df['gene'] = df['gene'].apply(lambda x: ontl.gene(x)).fillna(df['gene'])

    return df

In [87]:
def calculate_cell_score(results):
    cell_df = results.copy()

    # Group by 'cell' and calculate the total cell count for each group
    sum_cell_count = cell_df.groupby('cell')['cell count'].sum().reset_index()
    sum_cell_count.columns = ['cell', 'total cell count']
    cell_df = cell_df.merge(sum_cell_count, on='cell')

    cell_df = cell_df[cell_df['total cell count'] >= 100]

    # Calculate scores
    cell_df['score'] = (1) * ((cell_df['expression'] - cell_df['expression'].min()) / (cell_df['expression'].max() - cell_df['expression'].min())) + (1.5) * (cell_df['cell percentage']) + (2.5) * ((cell_df['cell count'] - cell_df['cell count'].min()) / (cell_df['cell count'].max() - cell_df['cell count'].min()))

    # Group by 'cell' and calculate the standard deviation of 'score' for each group
    std_scores = cell_df.groupby('cell')['score'].std().reset_index()
    std_scores.columns = ['cell', 'std']
    cell_df = cell_df.merge(std_scores, on='cell')

    # Group by 'cell' and calculate the mean of 'score' for each group
    mean_scores = cell_df.groupby('cell')['score'].mean().reset_index().fillna(0)
    mean_scores.columns = ['cell', 'mean']
    cell_df = cell_df.merge(mean_scores, on='cell')

    # Calculate the coefficient of variation (CV) for each cell
    cell_df['CV'] = cell_df['std'] / cell_df['mean']

    # Group by 'cell' and calculate the kurtosis of 'score' for each group
    kurtosis_scores = cell_df.groupby('cell')['score'].apply(pd.Series.kurt).reset_index()
    kurtosis_scores.columns = ['cell', 'kurtosis']
    cell_df = cell_df.merge(kurtosis_scores, on='cell')

    # Normalize the scores
    cell_df['std norm'] = (cell_df['std'] - cell_df['std'].min()) / (cell_df['std'].max() - cell_df['std'].min())
    cell_df['mean norm'] = (cell_df['mean'] - cell_df['mean'].min()) / (cell_df['mean'].max() - cell_df['mean'].min())
    cell_df['CV norm'] = (cell_df['CV'] - cell_df['CV'].min()) / (cell_df['CV'].max() - cell_df['CV'].min())
    cell_df['kurtosis norm'] = ((cell_df['kurtosis'] - cell_df['kurtosis'].min()) / (cell_df['kurtosis'].max() - cell_df['kurtosis'].min()))

    # Count the number of entries for each cell
    cell_counts = cell_df['cell'].value_counts().reset_index()
    cell_counts.columns = ['cell', 'count']
    cell_df = cell_df.merge(cell_counts, on='cell')

    cell_df['cell score'] = (0.1 * (1 - cell_df['std norm']).fillna(0) + 0.7 * cell_df['mean norm'] + 0.1 * (1 - cell_df['CV norm']) + 0.1 * (1 - cell_df['kurtosis norm']).fillna(0)) * cell_df['count']
    return cell_df.sort_values(by=['count', 'cell score', 'cell', 'gene'], ascending=[False, False, False, True]).reset_index(drop=True)

In [88]:
def calculate_cell_score(
    results,
    w_e=1.0, w_p=1.5, w_ct=2.5,             # base score weights
    w_m=7.0, w_f=3.0,                       # mean vs flatness weights
    alpha=1.0, beta=1.0, gamma=1.0,         # flatness breakdown
    w_c=1.0                                 # count weight
):
    import pandas as pd
    import numpy as np

    cell_df = results.copy()

    # Calculate total cell count per cell
    sum_cell_count = cell_df.groupby('cell')['cell count'].sum().reset_index()
    sum_cell_count.columns = ['cell', 'total cell count']
    cell_df = cell_df.merge(sum_cell_count, on='cell')

    # Adaptive cutoff: max(μ - 0.5σ, min(total count))
    mu = cell_df['total cell count'].mean()
    sigma = cell_df['total cell count'].std()
    minimum = cell_df['total cell count'].min()
    min_count = max(mu - 0.5 * sigma, minimum)
    cell_df = cell_df[cell_df['total cell count'] >= min_count]

    # Normalize expression and cell count
    norm_expr = (cell_df['expression'] - cell_df['expression'].min()) / (cell_df['expression'].max() - cell_df['expression'].min())
    norm_count = (cell_df['cell count'] - cell_df['cell count'].min()) / (cell_df['cell count'].max() - cell_df['cell count'].min())

    # Compute base score
    cell_df['score'] = (
        w_e * norm_expr +
        w_p * cell_df['cell percentage'] +
        w_ct * norm_count
    )

    # Compute flatness components
    std_scores = cell_df.groupby('cell')['score'].std().reset_index().rename(columns={'score': 'std'})
    mean_scores = cell_df.groupby('cell')['score'].mean().reset_index().rename(columns={'score': 'mean'})
    kurtosis_scores = cell_df.groupby('cell')['score'].apply(pd.Series.kurt).reset_index(name='kurtosis')

    cell_df = cell_df.merge(std_scores, on='cell')
    cell_df = cell_df.merge(mean_scores, on='cell')
    cell_df = cell_df.merge(kurtosis_scores, on='cell')

    cell_df['CV'] = cell_df['std'] / (cell_df['mean'].replace(0, np.nan))

    # Normalize flatness metrics
    for col in ['std', 'CV', 'kurtosis', 'mean']:
        col_min, col_max = cell_df[col].min(), cell_df[col].max()
        cell_df[f'{col} norm'] = (cell_df[col] - col_min) / (col_max - col_min + 1e-9)

    # Stability (flatness) score
    stability = (
        alpha * (1 - cell_df['std norm'].fillna(0)) +
        beta * (1 - cell_df['CV norm'].fillna(0)) +
        gamma * (1 - cell_df['kurtosis norm'].fillna(0))
    )
    flatness_weight = stability / (alpha + beta + gamma)

    # Count-based log-scaled soft weight
    cell_counts = cell_df['cell'].value_counts().reset_index()
    cell_counts.columns = ['cell', 'count']
    cell_df = cell_df.merge(cell_counts, on='cell')

    log_scaled_count = np.log1p(cell_df['count']) / np.log1p(cell_df['count'].max())

    # Final cell score
    cell_df['cell score'] = (
        (w_m * cell_df['mean norm'] + w_f * flatness_weight) *
        (1 + w_c * log_scaled_count)
    )

    return cell_df.sort_values(by=['count', 'cell score', 'cell', 'gene'], ascending=[False, False, False, True]).reset_index(drop=True)


In [89]:
def run_data_processing(cluster, ensembl_ids, target_uberon_id):
    try:
        data = fetch_expression_data(cluster, ensembl_ids)

        response_df = expression_data_to_df(data)

        filtered_df = filter_expression_data(response_df, target_uberon_id)

        translated_df = translate_ontology(filtered_df)

        ranked_df = calculate_cell_score(translated_df)

        return ranked_df

    except Exception as e:
        print(f"An error occurred during the pipeline execution: {e}")
        return pd.DataFrame()

In [90]:
def main_data_analysis(ensembl_df, target_uberon_id):
    results = {}

    # Iterate over each cluster (column) in the DataFrame
    for cluster in ensembl_df.columns:
        # Extract Ensembl IDs for the current cluster, dropping any NaN values
        ensembl_ids = ensembl_df[cluster].dropna().tolist()
        
        if not ensembl_ids:
            continue
        
        # Run the expression pipeline for the current cluster
        result_df = run_data_processing(cluster, ensembl_ids, target_uberon_id)
        
        # Store the result in the dictionary
        results[cluster] = result_df.reset_index(drop=True)

    return results

Function to Display the Results

In [91]:
from IPython.display import Markdown, display
def display_dataframe_as_markdown(df):
    markdown_table = df.to_markdown(index=False)
    display(Markdown(markdown_table))

## Run the Section

In [92]:
# Specify the UBERON ID to filter
TARGET_UBERON_ID = "UBERON:0002097"

In [93]:
# Run the main pipeline
results = main_data_analysis(ensembl_df, TARGET_UBERON_ID)

In [94]:
display_dataframe_as_markdown(results[1])

| gene   | cell                                                    |   expression |   cell count |   cell percentage |   tissue composition |   total cell count |       score |      std |     mean |   kurtosis |       CV |   std norm |   CV norm |   kurtosis norm |   mean norm |   count |   cell score |
|:-------|:--------------------------------------------------------|-------------:|-------------:|------------------:|---------------------:|-------------------:|------------:|---------:|---------:|-----------:|---------:|-----------:|----------:|----------------:|------------:|--------:|-------------:|
| ATP1B3 | hematopoietic cell                                      |      1.96266 |        69209 |       0.224443    |          0.0847753   |             188775 | 2.57641     | 1.2642   | 1.58521  |  -2.42958  | 0.797495 | 0.775047   | 0.491174  |       0.292183  |   1         |       5 |     16.8832  |
| MAF    | hematopoietic cell                                      |      2.02133 |        33625 |       0.11267     |          0.0411878   |             188775 | 1.49561     | 1.2642   | 1.58521  |  -2.42958  | 0.797495 | 0.775047   | 0.491174  |       0.292183  |   1         |       5 |     16.8832  |
| MMP13  | hematopoietic cell                                      |      1.68807 |         1792 |       0.00738753  |          0.00219505  |             188775 | 0.239547    | 1.2642   | 1.58521  |  -2.42958  | 0.797495 | 0.775047   | 0.491174  |       0.292183  |   1         |       5 |     16.8832  |
| PLAU   | hematopoietic cell                                      |      1.90397 |         4615 |       0.0159959   |          0.00565299  |             188775 | 0.485719    | 1.2642   | 1.58521  |  -2.42958  | 0.797495 | 0.775047   | 0.491174  |       0.292183  |   1         |       5 |     16.8832  |
| TXNIP  | hematopoietic cell                                      |      2.26996 |        79534 |       0.258402    |          0.0974225   |             188775 | 3.12878     | 1.2642   | 1.58521  |  -2.42958  | 0.797495 | 0.775047   | 0.491174  |       0.292183  |   1         |       5 |     16.8832  |
| ATP1B3 | leukocyte                                               |      1.96448 |        68687 |       0.225378    |          0.0841359   |             187351 | 2.56511     | 1.25528  | 1.57875  |  -2.43961  | 0.795111 | 0.768526   | 0.488119  |       0.29122   |   0.994955  |       5 |     16.8336  |
| MAF    | leukocyte                                               |      2.02167 |        33586 |       0.113263    |          0.0411401   |             187351 | 1.49571     | 1.25528  | 1.57875  |  -2.43961  | 0.795111 | 0.768526   | 0.488119  |       0.29122   |   0.994955  |       5 |     16.8336  |
| MMP13  | leukocyte                                               |      1.68807 |         1792 |       0.00738753  |          0.00219505  |             187351 | 0.239547    | 1.25528  | 1.57875  |  -2.43961  | 0.795111 | 0.768526   | 0.488119  |       0.29122   |   0.994955  |       5 |     16.8336  |
| PLAU   | leukocyte                                               |      1.90549 |         4588 |       0.0160711   |          0.00561992  |             187351 | 0.486216    | 1.25528  | 1.57875  |  -2.43961  | 0.795111 | 0.768526   | 0.488119  |       0.29122   |   0.994955  |       5 |     16.8336  |
| TXNIP  | leukocyte                                               |      2.2705  |        78698 |       0.258687    |          0.0963985   |             187351 | 3.10714     | 1.25528  | 1.57875  |  -2.43961  | 0.795111 | 0.768526   | 0.488119  |       0.29122   |   0.994955  |       5 |     16.8336  |
| ATP1B3 | mononuclear cell                                        |      1.96129 |        67295 |       0.224851    |          0.0824308   |             183535 | 2.5246      | 1.23811  | 1.55494  |  -2.4439   | 0.796244 | 0.755979   | 0.489571  |       0.290808  |   0.976389  |       5 |     16.5967  |
| MAF    | mononuclear cell                                        |      2.02028 |        33141 |       0.113865    |          0.040595    |             183535 | 1.48364     | 1.23811  | 1.55494  |  -2.4439   | 0.796244 | 0.755979   | 0.489571  |       0.290808  |   0.976389  |       5 |     16.5967  |
| MMP13  | mononuclear cell                                        |      1.68429 |         1757 |       0.00737532  |          0.00215218  |             183535 | 0.235834    | 1.23811  | 1.55494  |  -2.4439   | 0.796244 | 0.755979   | 0.489571  |       0.290808  |   0.976389  |       5 |     16.5967  |
| PLAU   | mononuclear cell                                        |      1.89698 |         4250 |       0.0151735   |          0.0052059   |             183535 | 0.469589    | 1.23811  | 1.55494  |  -2.4439   | 0.796244 | 0.755979   | 0.489571  |       0.290808  |   0.976389  |       5 |     16.5967  |
| TXNIP  | mononuclear cell                                        |      2.26864 |        77092 |       0.257616    |          0.0944313   |             183535 | 3.06104     | 1.23811  | 1.55494  |  -2.4439   | 0.796244 | 0.755979   | 0.489571  |       0.290808  |   0.976389  |       5 |     16.5967  |
| ATP1B3 | epithelial cell                                         |      2.03209 |        93064 |       0.516514    |          0.113996    |             172695 | 3.70597     | 1.57197  | 1.57007  |  -2.1698   | 1.00121  | 1          | 0.752262  |       0.31713   |   0.988191  |       5 |     15.6959  |
| MAF    | epithelial cell                                         |      1.75391 |         6982 |       0.0389975   |          0.00855237  |             172695 | 0.474393    | 1.57197  | 1.57007  |  -2.1698   | 1.00121  | 1          | 0.752262  |       0.31713   |   0.988191  |       5 |     15.6959  |
| MMP13  | epithelial cell                                         |      1.7066  |          448 |       0.00310482  |          0.000548763 |             172695 | 0.210537    | 1.57197  | 1.57007  |  -2.1698   | 1.00121  | 1          | 0.752262  |       0.31713   |   0.988191  |       5 |     15.6959  |
| PLAU   | epithelial cell                                         |      2.00531 |         7495 |       0.0417242   |          0.00918075  |             172695 | 0.675573    | 1.57197  | 1.57007  |  -2.1698   | 1.00121  | 1          | 0.752262  |       0.31713   |   0.988191  |       5 |     15.6959  |
| TXNIP  | epithelial cell                                         |      2.13605 |        64706 |       0.359125    |          0.0792595   |             172695 | 2.78389     | 1.57197  | 1.57007  |  -2.1698   | 1.00121  | 1          | 0.752262  |       0.31713   |   0.988191  |       5 |     15.6959  |
| ATP1B3 | squamous epithelial cell                                |      2.01621 |        70184 |       0.515551    |          0.0859696   |             134858 | 3.07831     | 1.31037  | 1.37216  |  -2.55305  | 0.954966 | 0.80879    | 0.692994  |       0.280326  |   0.833834  |       5 |     14.1095  |
| MAF    | squamous epithelial cell                                |      1.74131 |         6308 |       0.046728    |          0.00772677  |             134858 | 0.458694    | 1.31037  | 1.37216  |  -2.55305  | 0.954966 | 0.80879    | 0.692994  |       0.280326  |   0.833834  |       5 |     14.1095  |
| MMP13  | squamous epithelial cell                                |      1.69463 |          222 |       0.00209053  |          0.000271932 |             134858 | 0.194211    | 1.31037  | 1.37216  |  -2.55305  | 0.954966 | 0.80879    | 0.692994  |       0.280326  |   0.833834  |       5 |     14.1095  |
| PLAU   | squamous epithelial cell                                |      2.00848 |         6319 |       0.046494    |          0.00774025  |             134858 | 0.65345     | 1.31037  | 1.37216  |  -2.55305  | 0.954966 | 0.80879    | 0.692994  |       0.280326  |   0.833834  |       5 |     14.1095  |
| TXNIP  | squamous epithelial cell                                |      2.14417 |        51825 |       0.380694    |          0.0634813   |             134858 | 2.47614     | 1.31037  | 1.37216  |  -2.55305  | 0.954966 | 0.80879    | 0.692994  |       0.280326  |   0.833834  |       5 |     14.1095  |
| ATP1B3 | epidermal cell                                          |      2.01847 |        69933 |       0.514758    |          0.0856621   |             134246 | 3.07203     | 1.30765  | 1.37033  |  -2.55811  | 0.954257 | 0.806802   | 0.692085  |       0.27984   |   0.832407  |       5 |     14.0962  |
| MAF    | epidermal cell                                          |      1.74486 |         6227 |       0.0462232   |          0.00762756  |             134246 | 0.458353    | 1.30765  | 1.37033  |  -2.55811  | 0.954257 | 0.806802   | 0.692085  |       0.27984   |   0.832407  |       5 |     14.0962  |
| MMP13  | epidermal cell                                          |      1.69599 |          221 |       0.00209564  |          0.000270707 |             134246 | 0.195183    | 1.30765  | 1.37033  |  -2.55811  | 0.954257 | 0.806802   | 0.692085  |       0.27984   |   0.832407  |       5 |     14.0962  |
| PLAU   | epidermal cell                                          |      2.01094 |         6249 |       0.046128    |          0.0076545   |             134246 | 0.652813    | 1.30765  | 1.37033  |  -2.55811  | 0.954257 | 0.806802   | 0.692085  |       0.27984   |   0.832407  |       5 |     14.0962  |
| TXNIP  | epidermal cell                                          |      2.14951 |        51616 |       0.379932    |          0.0632253   |             134246 | 2.47327     | 1.30765  | 1.37033  |  -2.55811  | 0.954257 | 0.806802   | 0.692085  |       0.27984   |   0.832407  |       5 |     14.0962  |
| ATP1B3 | stratified squamous epithelial cell                     |      2.01708 |        69793 |       0.515511    |          0.0854906   |             133737 | 3.06838     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MAF    | stratified squamous epithelial cell                     |      1.73854 |         6151 |       0.0458189   |          0.00753446  |             133737 | 0.451097    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MMP13  | stratified squamous epithelial cell                     |      1.69599 |          221 |       0.00209564  |          0.000270707 |             133737 | 0.195183    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| PLAU   | stratified squamous epithelial cell                     |      2.01091 |         6246 |       0.0462071   |          0.00765083  |             133737 | 0.65283     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| TXNIP  | stratified squamous epithelial cell                     |      2.14454 |        51326 |       0.379109    |          0.0628701   |             133737 | 2.46063     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| ATP1B3 | stratified epithelial cell                              |      2.01708 |        69793 |       0.515511    |          0.0854906   |             133737 | 3.06838     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MAF    | stratified epithelial cell                              |      1.73854 |         6151 |       0.0458189   |          0.00753446  |             133737 | 0.451097    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MMP13  | stratified epithelial cell                              |      1.69599 |          221 |       0.00209564  |          0.000270707 |             133737 | 0.195183    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| PLAU   | stratified epithelial cell                              |      2.01091 |         6246 |       0.0462071   |          0.00765083  |             133737 | 0.65283     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| TXNIP  | stratified epithelial cell                              |      2.14454 |        51326 |       0.379109    |          0.0628701   |             133737 | 2.46063     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| ATP1B3 | keratinocyte                                            |      2.01708 |        69793 |       0.515511    |          0.0854906   |             133737 | 3.06838     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MAF    | keratinocyte                                            |      1.73854 |         6151 |       0.0458189   |          0.00753446  |             133737 | 0.451097    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MMP13  | keratinocyte                                            |      1.69599 |          221 |       0.00209564  |          0.000270707 |             133737 | 0.195183    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| PLAU   | keratinocyte                                            |      2.01091 |         6246 |       0.0462071   |          0.00765083  |             133737 | 0.65283     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| TXNIP  | keratinocyte                                            |      2.14454 |        51326 |       0.379109    |          0.0628701   |             133737 | 2.46063     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| ATP1B3 | myeloid leukocyte                                       |      1.95002 |        28196 |       0.316088    |          0.0345378   |              67161 | 1.60291     | 0.568262 | 0.965862 |  -1.89727  | 0.588347 | 0.266383   | 0.223124  |       0.343302  |   0.516954  |       5 |     11.5717  |
| MAF    | myeloid leukocyte                                       |      2.16538 |        17607 |       0.197621    |          0.0215671   |              67161 | 1.29778     | 0.568262 | 0.965862 |  -1.89727  | 0.588347 | 0.266383   | 0.223124  |       0.343302  |   0.516954  |       5 |     11.5717  |
| MMP13  | myeloid leukocyte                                       |      1.74324 |          206 |       0.00236817  |          0.000252333 |              67161 | 0.229644    | 0.568262 | 0.965862 |  -1.89727  | 0.588347 | 0.266383   | 0.223124  |       0.343302  |   0.516954  |       5 |     11.5717  |
| PLAU   | myeloid leukocyte                                       |      1.92875 |         3943 |       0.0442308   |          0.00482985  |              67161 | 0.528094    | 0.568262 | 0.965862 |  -1.89727  | 0.588347 | 0.266383   | 0.223124  |       0.343302  |   0.516954  |       5 |     11.5717  |
| TXNIP  | myeloid leukocyte                                       |      2.01569 |        17209 |       0.19292     |          0.0210796   |              67161 | 1.17089     | 0.568262 | 0.965862 |  -1.89727  | 0.588347 | 0.266383   | 0.223124  |       0.343302  |   0.516954  |       5 |     11.5717  |
| ATP1B3 | myeloid cell                                            |      1.94823 |        28389 |       0.310018    |          0.0347742   |              67547 | 1.59768     | 0.566616 | 0.963757 |  -1.9083   | 0.587924 | 0.26518    | 0.222582  |       0.342243  |   0.515313  |       5 |     11.5544  |
| MAF    | myeloid cell                                            |      2.16491 |        17624 |       0.195555    |          0.0215879   |              67547 | 1.29479     | 0.566616 | 0.963757 |  -1.9083   | 0.587924 | 0.26518    | 0.222582  |       0.342243  |   0.515313  |       5 |     11.5544  |
| MMP13  | myeloid cell                                            |      1.74324 |          206 |       0.00236817  |          0.000252333 |              67547 | 0.229644    | 0.566616 | 0.963757 |  -1.9083   | 0.587924 | 0.26518    | 0.222582  |       0.342243  |   0.515313  |       5 |     11.5544  |
| PLAU   | myeloid cell                                            |      1.92748 |         3963 |       0.0434078   |          0.00485435  |              67547 | 0.526465    | 0.566616 | 0.963757 |  -1.9083   | 0.587924 | 0.26518    | 0.222582  |       0.342243  |   0.515313  |       5 |     11.5544  |
| TXNIP  | myeloid cell                                            |      2.01554 |        17365 |       0.189738    |          0.0212707   |              67547 | 1.1702      | 0.566616 | 0.963757 |  -1.9083   | 0.587924 | 0.26518    | 0.222582  |       0.342243  |   0.515313  |       5 |     11.5544  |
| ATP1B3 | lymphocyte                                              |      1.97974 |        37587 |       0.190901    |          0.0460409   |             115545 | 1.68907     | 1.08772  | 1.13886  |  -0.530505 | 0.955095 | 0.646055   | 0.69316   |       0.474555  |   0.651877  |       5 |     11.4987  |
| MAF    | lymphocyte                                              |      1.86568 |        15547 |       0.081938    |          0.0190438   |             115545 | 0.850387    | 1.08772  | 1.13886  |  -0.530505 | 0.955095 | 0.646055   | 0.69316   |       0.474555  |   0.651877  |       5 |     11.4987  |
| MMP13  | lymphocyte                                              |      1.6793  |         1497 |       0.0108069   |          0.0018337   |             115545 | 0.230363    | 1.08772  | 1.13886  |  -0.530505 | 0.955095 | 0.646055   | 0.69316   |       0.474555  |   0.651877  |       5 |     11.4987  |
| PLAU   | lymphocyte                                              |      1.66354 |          366 |       0.00205908  |          0.00044832  |             115545 | 0.175365    | 1.08772  | 1.13886  |  -0.530505 | 0.955095 | 0.646055   | 0.69316   |       0.474555  |   0.651877  |       5 |     11.4987  |
| TXNIP  | lymphocyte                                              |      2.34772 |        60548 |       0.3075      |          0.0741663   |             115545 | 2.7491      | 1.08772  | 1.13886  |  -0.530505 | 0.955095 | 0.646055   | 0.69316   |       0.474555  |   0.651877  |       5 |     11.4987  |
| ATP1B3 | phagocyte                                               |      1.93793 |        29708 |       0.290134    |          0.0363898   |              67990 | 1.59578     | 0.562836 | 0.929985 |  -1.66048  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| MAF    | phagocyte                                               |      2.15689 |        17594 |       0.173658    |          0.0215512   |              67990 | 1.25529     | 0.562836 | 0.929985 |  -1.66048  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| MMP13  | phagocyte                                               |      1.71298 |          260 |       0.00260772  |          0.000318478 |              67990 | 0.209392    | 0.562836 | 0.929985 |  -1.66048  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| PLAU   | phagocyte                                               |      1.91898 |         3884 |       0.0379501   |          0.00475758  |              67990 | 0.509962    | 0.562836 | 0.929985 |  -1.66048  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| TXNIP  | phagocyte                                               |      1.97919 |        16544 |       0.161645    |          0.020265    |              67990 | 1.0795      | 0.562836 | 0.929985 |  -1.66048  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| ATP1B3 | mononuclear phagocyte                                   |      1.93793 |        29708 |       0.290134    |          0.0363898   |              67990 | 1.59578     | 0.562836 | 0.929985 |  -1.66048  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| MAF    | mononuclear phagocyte                                   |      2.15689 |        17594 |       0.173658    |          0.0215512   |              67990 | 1.25529     | 0.562836 | 0.929985 |  -1.66048  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| MMP13  | mononuclear phagocyte                                   |      1.71298 |          260 |       0.00260772  |          0.000318478 |              67990 | 0.209392    | 0.562836 | 0.929985 |  -1.66048  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| PLAU   | mononuclear phagocyte                                   |      1.91898 |         3884 |       0.0379501   |          0.00475758  |              67990 | 0.509962    | 0.562836 | 0.929985 |  -1.66048  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| TXNIP  | mononuclear phagocyte                                   |      1.97919 |        16544 |       0.161645    |          0.020265    |              67990 | 1.0795      | 0.562836 | 0.929985 |  -1.66048  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| ATP1B3 | T cell                                                  |      2.0015  |        33486 |       0.21714     |          0.0410176   |              97598 | 1.63413     | 0.983809 | 1.06541  |  -1.12645  | 0.923411 | 0.570109   | 0.652553  |       0.417325  |   0.594591  |       5 |     11.0443  |
| MAF    | T cell                                                  |      1.87653 |        14423 |       0.0935307   |          0.017667    |              97598 | 0.845492    | 0.983809 | 1.06541  |  -1.12645  | 0.923411 | 0.570109   | 0.652553  |       0.417325  |   0.594591  |       5 |     11.0443  |
| MMP13  | T cell                                                  |      1.67873 |          986 |       0.00923982  |          0.00120777  |              97598 | 0.213866    | 0.983809 | 1.06541  |  -1.12645  | 0.923411 | 0.570109   | 0.652553  |       0.417325  |   0.594591  |       5 |     11.0443  |
| PLAU   | T cell                                                  |      1.65596 |          243 |       0.00175342  |          0.000297655 |              97598 | 0.166076    | 0.983809 | 1.06541  |  -1.12645  | 0.923411 | 0.570109   | 0.652553  |       0.417325  |   0.594591  |       5 |     11.0443  |
| TXNIP  | T cell                                                  |      2.39302 |        48460 |       0.314216    |          0.0593595   |              97598 | 2.46748     | 0.983809 | 1.06541  |  -1.12645  | 0.923411 | 0.570109   | 0.652553  |       0.417325  |   0.594591  |       5 |     11.0443  |
| ATP1B3 | professional antigen presenting cell                    |      1.94512 |        28387 |       0.29964     |          0.0347717   |              64385 | 1.57979     | 0.555087 | 0.918554 |  -1.67696  | 0.604306 | 0.256754   | 0.243577  |       0.364459  |   0.480057  |       5 |     10.9912  |
| MAF    | professional antigen presenting cell                    |      2.16865 |        17053 |       0.182514    |          0.0208885   |              64385 | 1.26262     | 0.555087 | 0.918554 |  -1.67696  | 0.604306 | 0.256754   | 0.243577  |       0.364459  |   0.480057  |       5 |     10.9912  |
| MMP13  | professional antigen presenting cell                    |      1.71997 |          237 |       0.00260927  |          0.000290305 |              64385 | 0.213872    | 0.555087 | 0.918554 |  -1.67696  | 0.604306 | 0.256754   | 0.243577  |       0.364459  |   0.480057  |       5 |     10.9912  |
| PLAU   | professional antigen presenting cell                    |      1.92183 |         3685 |       0.0390426   |          0.00451382  |              64385 | 0.508333    | 0.555087 | 0.918554 |  -1.67696  | 0.604306 | 0.256754   | 0.243577  |       0.364459  |   0.480057  |       5 |     10.9912  |
| TXNIP  | professional antigen presenting cell                    |      1.97112 |        15023 |       0.158576    |          0.0184019   |              64385 | 1.02815     | 0.555087 | 0.918554 |  -1.67696  | 0.604306 | 0.256754   | 0.243577  |       0.364459  |   0.480057  |       5 |     10.9912  |
| ATP1B3 | stem cell of epidermis                                  |      2.20196 |        25431 |       0.784593    |          0.0311509   |              47452 | 2.41508     | 1.11238  | 1.05872  |  -3.10141  | 1.05068  | 0.66408    | 0.815663  |       0.227667  |   0.589378  |       5 |     10.8365  |
| MAF    | stem cell of epidermis                                  |      1.79559 |          907 |       0.0279826   |          0.001111    |              47452 | 0.32507     | 1.11238  | 1.05872  |  -3.10141  | 1.05068  | 0.66408    | 0.815663  |       0.227667  |   0.589378  |       5 |     10.8365  |
| MMP13  | stem cell of epidermis                                  |      1.70608 |          151 |       0.00465862  |          0.000184962 |              47452 | 0.204505    | 1.11238  | 1.05872  |  -3.10141  | 1.05068  | 0.66408    | 0.815663  |       0.227667  |   0.589378  |       5 |     10.8365  |
| PLAU   | stem cell of epidermis                                  |      1.70676 |          381 |       0.0117545   |          0.000466693 |              47452 | 0.221827    | 1.11238  | 1.05872  |  -3.10141  | 1.05068  | 0.66408    | 0.815663  |       0.227667  |   0.589378  |       5 |     10.8365  |
| TXNIP  | stem cell of epidermis                                  |      2.24417 |        20582 |       0.65894     |          0.0252112   |              47452 | 2.12713     | 1.11238  | 1.05872  |  -3.10141  | 1.05068  | 0.66408    | 0.815663  |       0.227667  |   0.589378  |       5 |     10.8365  |
| ATP1B3 | macrophage                                              |      1.95562 |        14190 |       0.280634    |          0.0173816   |              45686 | 1.17756     | 0.51098  | 0.892632 |  -1.42783  | 0.572442 | 0.224516   | 0.20274   |       0.388383  |   0.459841  |       5 |     10.8065  |
| MAF    | macrophage                                              |      2.17852 |        16637 |       0.329029    |          0.0203789   |              45686 | 1.47841     | 0.51098  | 0.892632 |  -1.42783  | 0.572442 | 0.224516   | 0.20274   |       0.388383  |   0.459841  |       5 |     10.8065  |
| MMP13  | macrophage                                              |      1.7172  |          142 |       0.00289571  |          0.000173938 |              45686 | 0.209728    | 0.51098  | 0.892632 |  -1.42783  | 0.572442 | 0.224516   | 0.20274   |       0.388383  |   0.459841  |       5 |     10.8065  |
| PLAU   | macrophage                                              |      1.92977 |         3207 |       0.0634384   |          0.00392831  |              45686 | 0.537876    | 0.51098  | 0.892632 |  -1.42783  | 0.572442 | 0.224516   | 0.20274   |       0.388383  |   0.459841  |       5 |     10.8065  |
| TXNIP  | macrophage                                              |      2.0016  |        11510 |       0.227632    |          0.0140988   |              45686 | 1.05959     | 0.51098  | 0.892632 |  -1.42783  | 0.572442 | 0.224516   | 0.20274   |       0.388383  |   0.459841  |       5 |     10.8065  |
| ATP1B3 | basal cell of epidermis                                 |      2.20972 |        24813 |       0.794397    |          0.0303939   |              46715 | 2.41885     | 1.1185   | 1.05434  |  -3.09513  | 1.06085  | 0.668554   | 0.828695  |       0.228269  |   0.585963  |       5 |     10.7525  |
| MAF    | basal cell of epidermis                                 |      1.80535 |          834 |       0.0267008   |          0.00102158  |              46715 | 0.328302    | 1.1185   | 1.05434  |  -3.09513  | 1.06085  | 0.668554   | 0.828695  |       0.228269  |   0.585963  |       5 |     10.7525  |
| MMP13  | basal cell of epidermis                                 |      1.68844 |          149 |       0.00477029  |          0.000182513 |              46715 | 0.191759    | 1.1185   | 1.05434  |  -3.09513  | 1.06085  | 0.668554   | 0.828695  |       0.228269  |   0.585963  |       5 |     10.7525  |
| PLAU   | basal cell of epidermis                                 |      1.68823 |          337 |       0.0107892   |          0.000412797 |              46715 | 0.205683    | 1.1185   | 1.05434  |  -3.09513  | 1.06085  | 0.668554   | 0.828695  |       0.228269  |   0.585963  |       5 |     10.7525  |
| TXNIP  | basal cell of epidermis                                 |      2.24417 |        20582 |       0.65894     |          0.0252112   |              46715 | 2.12713     | 1.1185   | 1.05434  |  -3.09513  | 1.06085  | 0.668554   | 0.828695  |       0.228269  |   0.585963  |       5 |     10.7525  |
| ATP1B3 | mature T cell                                           |      1.99876 |        31622 |       0.214816    |          0.0387343   |              91929 | 1.57857     | 0.939014 | 1.02664  |  -1.18557  | 0.91465  | 0.537367   | 0.641324  |       0.411648  |   0.564354  |       5 |     10.7203  |
| MAF    | mature T cell                                           |      1.86831 |        13511 |       0.0917767   |          0.0165499   |              91929 | 0.812366    | 0.939014 | 1.02664  |  -1.18557  | 0.91465  | 0.537367   | 0.641324  |       0.411648  |   0.564354  |       5 |     10.7203  |
| MMP13  | mature T cell                                           |      1.67873 |          986 |       0.00923982  |          0.00120777  |              91929 | 0.213866    | 0.939014 | 1.02664  |  -1.18557  | 0.91465  | 0.537367   | 0.641324  |       0.411648  |   0.564354  |       5 |     10.7203  |
| PLAU   | mature T cell                                           |      1.65934 |          227 |       0.00172392  |          0.000278056 |              91929 | 0.168065    | 0.939014 | 1.02664  |  -1.18557  | 0.91465  | 0.537367   | 0.641324  |       0.411648  |   0.564354  |       5 |     10.7203  |
| TXNIP  | mature T cell                                           |      2.36148 |        45583 |       0.309633    |          0.0558354   |              91929 | 2.36032     | 0.939014 | 1.02664  |  -1.18557  | 0.91465  | 0.537367   | 0.641324  |       0.411648  |   0.564354  |       5 |     10.7203  |
| ATP1B3 | connective tissue cell                                  |      1.82118 |        30844 |       0.118108    |          0.0377813   |             114128 | 1.28313     | 0.918472 | 1.0574   |   1.68368  | 0.86861  | 0.522354   | 0.582318  |       0.687187  |   0.58835   |       5 |     10.6532  |
| MAF    | connective tissue cell                                  |      1.79169 |        14238 |       0.0545202   |          0.0174404   |             114128 | 0.720145    | 0.918472 | 1.0574   |   1.68368  | 0.86861  | 0.522354   | 0.582318  |       0.687187  |   0.58835   |       5 |     10.6532  |
| MMP13  | connective tissue cell                                  |      1.72222 |         1135 |       0.00446443  |          0.00139028  |             114128 | 0.24242     | 0.918472 | 1.0574   |   1.68368  | 0.86861  | 0.522354   | 0.582318  |       0.687187  |   0.58835   |       5 |     10.6532  |
| PLAU   | connective tissue cell                                  |      1.93535 |         4029 |       0.0154295   |          0.00493519  |             114128 | 0.492014    | 0.918472 | 1.0574   |   1.68368  | 0.86861  | 0.522354   | 0.582318  |       0.687187  |   0.58835   |       5 |     10.6532  |
| TXNIP  | connective tissue cell                                  |      2.08026 |        63882 |       0.244617    |          0.0782501   |             114128 | 2.54932     | 0.918472 | 1.0574   |   1.68368  | 0.86861  | 0.522354   | 0.582318  |       0.687187  |   0.58835   |       5 |     10.6532  |
| ATP1B3 | progenitor cell                                         |      2.18338 |        26390 |       0.636164    |          0.0323256   |              51021 | 2.20466     | 1.00773  | 0.999214 |  -3.16391  | 1.00853  | 0.587596   | 0.761641  |       0.221664  |   0.542966  |       5 |     10.4597  |
| MAF    | progenitor cell                                         |      1.8011  |         1407 |       0.0344304   |          0.00172346  |              51021 | 0.352191    | 1.00773  | 0.999214 |  -3.16391  | 1.00853  | 0.587596   | 0.761641  |       0.221664  |   0.542966  |       5 |     10.4597  |
| MMP13  | progenitor cell                                         |      1.68209 |          172 |       0.00428831  |          0.000210686 |              51021 | 0.187025    | 1.00773  | 0.999214 |  -3.16391  | 1.00853  | 0.587596   | 0.761641  |       0.221664  |   0.542966  |       5 |     10.4597  |
| PLAU   | progenitor cell                                         |      1.75005 |          551 |       0.0134456   |          0.000674929 |              51021 | 0.260496    | 1.00773  | 0.999214 |  -3.16391  | 1.00853  | 0.587596   | 0.761641  |       0.221664  |   0.542966  |       5 |     10.4597  |
| TXNIP  | progenitor cell                                         |      2.22683 |        22501 |       0.542716    |          0.0275619   |              51021 | 1.9917      | 1.00773  | 0.999214 |  -3.16391  | 1.00853  | 0.587596   | 0.761641  |       0.221664  |   0.542966  |       5 |     10.4597  |
| ATP1B3 | stem cell                                               |      2.20121 |        25507 |       0.696095    |          0.031244    |              47874 | 2.28384     | 1.04886  | 1.01054  |  -3.1164   | 1.03792  | 0.617652   | 0.79931   |       0.226227  |   0.551797  |       5 |     10.4388  |
| MAF    | stem cell                                               |      1.80317 |          929 |       0.0253832   |          0.00113795  |              47874 | 0.327283    | 1.04886  | 1.01054  |  -3.1164   | 1.03792  | 0.617652   | 0.79931   |       0.226227  |   0.551797  |       5 |     10.4388  |
| MMP13  | stem cell                                               |      1.68779 |          190 |       0.00523243  |          0.000232734 |              47874 | 0.193075    | 1.04886  | 1.01054  |  -3.1164   | 1.03792  | 0.617652   | 0.79931   |       0.226227  |   0.551797  |       5 |     10.4388  |
| PLAU   | stem cell                                               |      1.71234 |          394 |       0.0107653   |          0.000482617 |              47874 | 0.224756    | 1.04886  | 1.01054  |  -3.1164   | 1.03792  | 0.617652   | 0.79931   |       0.226227  |   0.551797  |       5 |     10.4388  |
| TXNIP  | stem cell                                               |      2.23843 |        20854 |       0.587933    |          0.0255444   |              47874 | 2.02373     | 1.04886  | 1.01054  |  -3.1164   | 1.03792  | 0.617652   | 0.79931   |       0.226227  |   0.551797  |       5 |     10.4388  |
| ATP1B3 | epithelial fate stem cell                               |      2.20872 |        24869 |       0.707833    |          0.0304625   |              47068 | 2.28978     | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| MAF    | epithelial fate stem cell                               |      1.80314 |          845 |       0.0240508   |          0.00103505  |              47068 | 0.323012    | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| MMP13  | epithelial fate stem cell                               |      1.67361 |          188 |       0.00535094  |          0.000230284 |              47068 | 0.182865    | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| PLAU   | epithelial fate stem cell                               |      1.68714 |          347 |       0.00987647  |          0.000425046 |              47068 | 0.20379     | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| TXNIP  | epithelial fate stem cell                               |      2.23814 |        20819 |       0.59256     |          0.0255015   |              47068 | 2.02952     | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| ATP1B3 | basal cell                                              |      2.20872 |        24869 |       0.707833    |          0.0304625   |              47068 | 2.28978     | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| MAF    | basal cell                                              |      1.80314 |          845 |       0.0240508   |          0.00103505  |              47068 | 0.323012    | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| MMP13  | basal cell                                              |      1.67361 |          188 |       0.00535094  |          0.000230284 |              47068 | 0.182865    | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| PLAU   | basal cell                                              |      1.68714 |          347 |       0.00987647  |          0.000425046 |              47068 | 0.20379     | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| TXNIP  | basal cell                                              |      2.23814 |        20819 |       0.59256     |          0.0255015   |              47068 | 2.02952     | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| ATP1B3 | fibroblast                                              |      1.80838 |        28138 |       0.120089    |          0.0344667   |             105848 | 1.20407     | 0.869236 | 1.0215   |   1.9098   | 0.850941 | 0.486367   | 0.559672  |       0.708902  |   0.560347  |       5 |     10.335   |
| MAF    | fibroblast                                              |      1.79198 |        13997 |       0.0597374   |          0.0171452   |             105848 | 0.721708    | 0.869236 | 1.0215   |   1.9098   | 0.850941 | 0.486367   | 0.559672  |       0.708902  |   0.560347  |       5 |     10.335   |
| MMP13  | fibroblast                                              |      1.75692 |          686 |       0.00292987  |          0.000840293 |             105848 | 0.253353    | 0.869236 | 1.0215   |   1.9098   | 0.850941 | 0.486367   | 0.559672  |       0.708902  |   0.560347  |       5 |     10.335   |
| PLAU   | fibroblast                                              |      1.93656 |         3716 |       0.0158594   |          0.00455179  |             105848 | 0.485132    | 0.869236 | 1.0215   |   1.9098   | 0.850941 | 0.486367   | 0.559672  |       0.708902  |   0.560347  |       5 |     10.335   |
| TXNIP  | fibroblast                                              |      2.08567 |        59311 |       0.253132    |          0.072651    |             105848 | 2.44324     | 0.869236 | 1.0215   |   1.9098   | 0.850941 | 0.486367   | 0.559672  |       0.708902  |   0.560347  |       5 |     10.335   |
| ATP1B3 | skin fibroblast                                         |      1.83878 |        19174 |       0.169566    |          0.0234866   |              69807 | 1.05965     | 0.691157 | 0.977011 |   2.75644  | 0.70742  | 0.356208   | 0.375732  |       0.790206  |   0.525649  |       5 |     10.3148  |
| MAF    | skin fibroblast                                         |      1.84161 |         9418 |       0.0832884   |          0.0115363   |              69807 | 0.670217    | 0.691157 | 0.977011 |   2.75644  | 0.70742  | 0.356208   | 0.375732  |       0.790206  |   0.525649  |       5 |     10.3148  |
| MMP13  | skin fibroblast                                         |      2.22251 |            1 |       8.84353e-06 |          1.22492e-06 |              69807 | 0.570055    | 0.691157 | 0.977011 |   2.75644  | 0.70742  | 0.356208   | 0.375732  |       0.790206  |   0.525649  |       5 |     10.3148  |
| PLAU   | skin fibroblast                                         |      1.93228 |         2097 |       0.0185449   |          0.00256865  |              69807 | 0.442546    | 0.691157 | 0.977011 |   2.75644  | 0.70742  | 0.356208   | 0.375732  |       0.790206  |   0.525649  |       5 |     10.3148  |
| TXNIP  | skin fibroblast                                         |      2.22643 |        39117 |       0.345932    |          0.0479151   |              69807 | 2.14259     | 0.691157 | 0.977011 |   2.75644  | 0.70742  | 0.356208   | 0.375732  |       0.790206  |   0.525649  |       5 |     10.3148  |
| ATP1B3 | precursor cell                                          |      2.16921 |        27417 |       0.521087    |          0.0335835   |              53753 | 2.0493      | 0.934877 | 0.940823 |  -3.19395  | 0.99368  | 0.534344   | 0.742611  |       0.21878   |   0.497426  |       5 |      9.97249 |
| MAF    | precursor cell                                          |      1.77136 |         1783 |       0.0342904   |          0.00218403  |              53753 | 0.340397    | 0.934877 | 0.940823 |  -3.19395  | 0.99368  | 0.534344   | 0.742611  |       0.21878   |   0.497426  |       5 |      9.97249 |
| MMP13  | precursor cell                                          |      1.68273 |          213 |       0.00471385  |          0.000260907 |              53753 | 0.189229    | 0.934877 | 0.940823 |  -3.19395  | 0.99368  | 0.534344   | 0.742611  |       0.21878   |   0.497426  |       5 |      9.97249 |
| PLAU   | precursor cell                                          |      1.71888 |          895 |       0.0171745   |          0.0010963   |              53753 | 0.252596    | 0.934877 | 0.940823 |  -3.19395  | 0.99368  | 0.534344   | 0.742611  |       0.21878   |   0.497426  |       5 |      9.97249 |
| TXNIP  | precursor cell                                          |      2.20711 |        23445 |       0.455995    |          0.0287182   |              53753 | 1.87259     | 0.934877 | 0.940823 |  -3.19395  | 0.99368  | 0.534344   | 0.742611  |       0.21878   |   0.497426  |       5 |      9.97249 |
| ATP1B3 | mast cell                                               |      2.11348 |         1206 |       0.277624    |          0.00147725  |               3116 | 0.939353    | 0.348643 | 0.698903 |  -1.93209  | 0.498843 | 0.105863   | 0.108413  |       0.339958  |   0.308747  |       5 |      9.21399 |
| MAF    | mast cell                                               |      2.14928 |          389 |       0.0895488   |          0.000476493 |               3116 | 0.661395    | 0.348643 | 0.698903 |  -1.93209  | 0.498843 | 0.105863   | 0.108413  |       0.339958  |   0.308747  |       5 |      9.21399 |
| MMP13  | mast cell                                               |      1.87773 |           35 |       0.00805709  |          4.28721e-05 |               3116 | 0.331645    | 0.348643 | 0.698903 |  -1.93209  | 0.498843 | 0.105863   | 0.108413  |       0.339958  |   0.308747  |       5 |      9.21399 |
| PLAU   | mast cell                                               |      1.92606 |          145 |       0.0333794   |          0.000177613 |               3116 | 0.407827    | 0.348643 | 0.698903 |  -1.93209  | 0.498843 | 0.105863   | 0.108413  |       0.339958  |   0.308747  |       5 |      9.21399 |
| TXNIP  | mast cell                                               |      2.33936 |         1341 |       0.308702    |          0.00164261  |               3116 | 1.15429     | 0.348643 | 0.698903 |  -1.93209  | 0.498843 | 0.105863   | 0.108413  |       0.339958  |   0.308747  |       5 |      9.21399 |
| ATP1B3 | endothelial cell                                        |      2.07187 |        23014 |       0.525206    |          0.0281902   |              38284 | 1.86622     | 0.713042 | 0.826887 |  -1.10956  | 0.862321 | 0.372204   | 0.574258  |       0.418947  |   0.408564  |       5 |      8.98908 |
| MAF    | endothelial cell                                        |      1.8268  |          736 |       0.0167964   |          0.000901539 |              38284 | 0.326454    | 0.713042 | 0.826887 |  -1.10956  | 0.862321 | 0.372204   | 0.574258  |       0.418947  |   0.408564  |       5 |      8.98908 |
| MMP13  | endothelial cell                                        |      1.71662 |          226 |       0.00587272  |          0.000276831 |              38284 | 0.216024    | 0.713042 | 0.826887 |  -1.10956  | 0.862321 | 0.372204   | 0.574258  |       0.418947  |   0.408564  |       5 |      8.98908 |
| PLAU   | endothelial cell                                        |      1.96624 |         1230 |       0.0280771   |          0.00150665  |              38284 | 0.458315    | 0.713042 | 0.826887 |  -1.10956  | 0.862321 | 0.372204   | 0.574258  |       0.418947  |   0.408564  |       5 |      8.98908 |
| TXNIP  | endothelial cell                                        |      2.08317 |        13078 |       0.298455    |          0.0160195   |              38284 | 1.26742     | 0.713042 | 0.826887 |  -1.10956  | 0.862321 | 0.372204   | 0.574258  |       0.418947  |   0.408564  |       5 |      8.98908 |
| ATP1B3 | alpha-beta T cell                                       |      2.05807 |        20506 |       0.312144    |          0.0251181   |              44270 | 1.46919     | 0.599986 | 0.751196 |  -2.71185  | 0.798708 | 0.289571   | 0.492729  |       0.265077  |   0.349531  |       5 |      8.79869 |
| MAF    | alpha-beta T cell                                       |      1.87156 |         6952 |       0.105806    |          0.00851562  |              44270 | 0.659587    | 0.599986 | 0.751196 |  -2.71185  | 0.798708 | 0.289571   | 0.492729  |       0.265077  |   0.349531  |       5 |      8.79869 |
| MMP13  | alpha-beta T cell                                       |      1.67738 |          218 |       0.00789455  |          0.000267032 |              44270 | 0.190236    | 0.599986 | 0.751196 |  -2.71185  | 0.798708 | 0.289571   | 0.492729  |       0.265077  |   0.349531  |       5 |      8.79869 |
| PLAU   | alpha-beta T cell                                       |      1.6696  |           75 |       0.00149745  |          9.18688e-05 |              44270 | 0.171122    | 0.599986 | 0.751196 |  -2.71185  | 0.798708 | 0.289571   | 0.492729  |       0.265077  |   0.349531  |       5 |      8.79869 |
| TXNIP  | alpha-beta T cell                                       |      2.05101 |        16519 |       0.251412    |          0.0202344   |              44270 | 1.26584     | 0.599986 | 0.751196 |  -2.71185  | 0.798708 | 0.289571   | 0.492729  |       0.265077  |   0.349531  |       5 |      8.79869 |
| ATP1B3 | mature alpha-beta T cell                                |      2.05816 |        20502 |       0.312469    |          0.0251132   |              44227 | 1.46964     | 0.599826 | 0.751053 |  -2.70609  | 0.798646 | 0.289454   | 0.49265   |       0.265629  |   0.34942   |       5 |      8.79641 |
| MAF    | mature alpha-beta T cell                                |      1.87161 |         6951 |       0.105922    |          0.0085144   |              44227 | 0.659767    | 0.599826 | 0.751053 |  -2.70609  | 0.798646 | 0.289454   | 0.49265   |       0.265629  |   0.34942   |       5 |      8.79641 |
| MMP13  | mature alpha-beta T cell                                |      1.67738 |          218 |       0.00789455  |          0.000267032 |              44227 | 0.190236    | 0.599826 | 0.751053 |  -2.70609  | 0.798646 | 0.289454   | 0.49265   |       0.265629  |   0.34942   |       5 |      8.79641 |
| PLAU   | mature alpha-beta T cell                                |      1.6696  |           75 |       0.00149745  |          9.18688e-05 |              44227 | 0.171122    | 0.599826 | 0.751053 |  -2.70609  | 0.798646 | 0.289454   | 0.49265   |       0.265629  |   0.34942   |       5 |      8.79641 |
| TXNIP  | mature alpha-beta T cell                                |      2.05112 |        16481 |       0.251143    |          0.0201879   |              44227 | 1.2645      | 0.599826 | 0.751053 |  -2.70609  | 0.798646 | 0.289454   | 0.49265   |       0.265629  |   0.34942   |       5 |      8.79641 |
| ATP1B3 | endothelial cell of vascular tree                       |      2.0668  |        20659 |       0.536681    |          0.0253056   |              33675 | 1.81647     | 0.701841 | 0.790041 |  -1.12984  | 0.88836  | 0.364017   | 0.607631  |       0.417     |   0.379828  |       5 |      8.54029 |
| MAF    | endothelial cell of vascular tree                       |      1.86569 |          566 |       0.0147036   |          0.000693303 |              33675 | 0.347098    | 0.701841 | 0.790041 |  -1.12984  | 0.88836  | 0.364017   | 0.607631  |       0.417     |   0.379828  |       5 |      8.54029 |
| MMP13  | endothelial cell of vascular tree                       |      1.71662 |          226 |       0.00587272  |          0.000276831 |              33675 | 0.216024    | 0.701841 | 0.790041 |  -1.12984  | 0.88836  | 0.364017   | 0.607631  |       0.417     |   0.379828  |       5 |      8.54029 |
| PLAU   | endothelial cell of vascular tree                       |      1.84174 |          771 |       0.0200348   |          0.000944411 |              33675 | 0.343138    | 0.701841 | 0.790041 |  -1.12984  | 0.88836  | 0.364017   | 0.607631  |       0.417     |   0.379828  |       5 |      8.54029 |
| TXNIP  | endothelial cell of vascular tree                       |      2.09016 |        11453 |       0.297527    |          0.014029    |              33675 | 1.22747     | 0.701841 | 0.790041 |  -1.12984  | 0.88836  | 0.364017   | 0.607631  |       0.417     |   0.379828  |       5 |      8.54029 |
| ATP1B3 | secretory cell                                          |      2.11102 |         1266 |       0.189777    |          0.00155074  |               3267 | 0.807394    | 0.295366 | 0.61792  |  -1.40253  | 0.478    | 0.0669229  | 0.0817007 |       0.390813  |   0.245587  |       5 |      8.35934 |
| MAF    | secretory cell                                          |      2.14297 |          395 |       0.0604345   |          0.000483842 |               3267 | 0.613284    | 0.295366 | 0.61792  |  -1.40253  | 0.478    | 0.0669229  | 0.0817007 |       0.390813  |   0.245587  |       5 |      8.35934 |
| MMP13  | secretory cell                                          |      1.76192 |           88 |       0.0134639   |          0.000107793 |               3267 | 0.256739    | 0.295366 | 0.61792  |  -1.40253  | 0.478    | 0.0669229  | 0.0817007 |       0.390813  |   0.245587  |       5 |      8.35934 |
| PLAU   | secretory cell                                          |      1.95996 |          153 |       0.0229351   |          0.000187412 |               3267 | 0.417091    | 0.295366 | 0.61792  |  -1.40253  | 0.478    | 0.0669229  | 0.0817007 |       0.390813  |   0.245587  |       5 |      8.35934 |
| TXNIP  | secretory cell                                          |      2.33432 |         1365 |       0.204586    |          0.00167201  |               3267 | 0.995092    | 0.295366 | 0.61792  |  -1.40253  | 0.478    | 0.0669229  | 0.0817007 |       0.390813  |   0.245587  |       5 |      8.35934 |
| ATP1B3 | endothelial cell of lymphatic vessel                    |      1.98413 |         2926 |       0.465331    |          0.00358411  |               5501 | 1.1728      | 0.440731 | 0.641348 |  -2.72955  | 0.687195 | 0.17317    | 0.34981   |       0.263377  |   0.263858  |       5 |      8.1213  |
| MAF    | endothelial cell of lymphatic vessel                    |      1.89042 |          340 |       0.0540712   |          0.000416472 |               5501 | 0.418111    | 0.440731 | 0.641348 |  -2.72955  | 0.687195 | 0.17317    | 0.34981   |       0.263377  |   0.263858  |       5 |      8.1213  |
| MMP13  | endothelial cell of lymphatic vessel                    |      1.67265 |           50 |       0.00795165  |          6.12458e-05 |               5501 | 0.182354    | 0.440731 | 0.641348 |  -2.72955  | 0.687195 | 0.17317    | 0.34981   |       0.263377  |   0.263858  |       5 |      8.1213  |
| PLAU   | endothelial cell of lymphatic vessel                    |      1.96358 |            7 |       0.00111323  |          8.57442e-06 |               5501 | 0.383079    | 0.440731 | 0.641348 |  -2.72955  | 0.687195 | 0.17317    | 0.34981   |       0.263377  |   0.263858  |       5 |      8.1213  |
| TXNIP  | endothelial cell of lymphatic vessel                    |      2.08853 |         2178 |       0.346374    |          0.00266787  |               5501 | 1.0504      | 0.440731 | 0.641348 |  -2.72955  | 0.687195 | 0.17317    | 0.34981   |       0.263377  |   0.263858  |       5 |      8.1213  |
| ATP1B3 | regulatory T cell                                       |      2.07479 |         8255 |       0.245502    |          0.0101117   |              21592 | 1.05232     | 0.43267  | 0.620733 |  -2.97137  | 0.69703  | 0.167279   | 0.362415  |       0.240155  |   0.247781  |       5 |      7.92924 |
| MAF    | regulatory T cell                                       |      1.88688 |         4552 |       0.135375    |          0.00557582  |              21592 | 0.650635    | 0.43267  | 0.620733 |  -2.97137  | 0.69703  | 0.167279   | 0.362415  |       0.240155  |   0.247781  |       5 |      7.92924 |
| MMP13  | regulatory T cell                                       |      1.68809 |          288 |       0.011405    |          0.000352776 |              21592 | 0.205188    | 0.43267  | 0.620733 |  -2.97137  | 0.69703  | 0.167279   | 0.362415  |       0.240155  |   0.247781  |       5 |      7.92924 |
| PLAU   | regulatory T cell                                       |      1.65088 |           58 |       0.00172491  |          7.10452e-05 |              21592 | 0.157355    | 0.43267  | 0.620733 |  -2.97137  | 0.69703  | 0.167279   | 0.362415  |       0.240155  |   0.247781  |       5 |      7.92924 |
| TXNIP  | regulatory T cell                                       |      2.03735 |         8439 |       0.250974    |          0.0103371   |              21592 | 1.03817     | 0.43267  | 0.620733 |  -2.97137  | 0.69703  | 0.167279   | 0.362415  |       0.240155  |   0.247781  |       5 |      7.92924 |
| ATP1B3 | CD8-positive, alpha-beta T cell                         |      2.07732 |        16216 |       0.334523    |          0.0198633   |              31371 | 1.40155     | 0.551626 | 0.674045 |  -2.09369  | 0.818381 | 0.254224   | 0.517943  |       0.324439  |   0.28936   |       5 |      7.85783 |
| MAF    | CD8-positive, alpha-beta T cell                         |      1.87958 |         3985 |       0.0822073   |          0.00488129  |              31371 | 0.550332    | 0.551626 | 0.674045 |  -2.09369  | 0.818381 | 0.254224   | 0.517943  |       0.324439  |   0.28936   |       5 |      7.85783 |
| MMP13  | CD8-positive, alpha-beta T cell                         |      1.67738 |          218 |       0.00789455  |          0.000267032 |              31371 | 0.190236    | 0.551626 | 0.674045 |  -2.09369  | 0.818381 | 0.254224   | 0.517943  |       0.324439  |   0.28936   |       5 |      7.85783 |
| PLAU   | CD8-positive, alpha-beta T cell                         |      1.64116 |           57 |       0.00133543  |          6.98203e-05 |              31371 | 0.149659    | 0.551626 | 0.674045 |  -2.09369  | 0.818381 | 0.254224   | 0.517943  |       0.324439  |   0.28936   |       5 |      7.85783 |
| TXNIP  | CD8-positive, alpha-beta T cell                         |      2.05604 |        10895 |       0.224755    |          0.0133455   |              31371 | 1.07845     | 0.551626 | 0.674045 |  -2.09369  | 0.818381 | 0.254224   | 0.517943  |       0.324439  |   0.28936   |       5 |      7.85783 |
| ATP1B3 | neoplastic cell                                         |      1.87792 |         5747 |       0.220665    |          0.0070396   |              10599 | 0.804143    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| MAF    | neoplastic cell                                         |      1.77166 |          392 |       0.0150515   |          0.000480167 |              10599 | 0.27439     | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| MMP13  | neoplastic cell                                         |      1.74236 |           18 |       0.000691138 |          2.20485e-05 |              10599 | 0.221439    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| PLAU   | neoplastic cell                                         |      2.30923 |         1084 |       0.0416219   |          0.00132781  |              10599 | 0.724799    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| TXNIP  | neoplastic cell                                         |      1.89133 |         3358 |       0.128936    |          0.00411327  |              10599 | 0.61215     | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| ATP1B3 | malignant cell                                          |      1.87792 |         5747 |       0.220665    |          0.0070396   |              10599 | 0.804143    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| MAF    | malignant cell                                          |      1.77166 |          392 |       0.0150515   |          0.000480167 |              10599 | 0.27439     | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| MMP13  | malignant cell                                          |      1.74236 |           18 |       0.000691138 |          2.20485e-05 |              10599 | 0.221439    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| PLAU   | malignant cell                                          |      2.30923 |         1084 |       0.0416219   |          0.00132781  |              10599 | 0.724799    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| TXNIP  | malignant cell                                          |      1.89133 |         3358 |       0.128936    |          0.00411327  |              10599 | 0.61215     | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| ATP1B3 | abnormal cell                                           |      1.87792 |         5747 |       0.220665    |          0.0070396   |              10599 | 0.804143    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| MAF    | abnormal cell                                           |      1.77166 |          392 |       0.0150515   |          0.000480167 |              10599 | 0.27439     | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| MMP13  | abnormal cell                                           |      1.74236 |           18 |       0.000691138 |          2.20485e-05 |              10599 | 0.221439    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| PLAU   | abnormal cell                                           |      2.30923 |         1084 |       0.0416219   |          0.00132781  |              10599 | 0.724799    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| TXNIP  | abnormal cell                                           |      1.89133 |         3358 |       0.128936    |          0.00411327  |              10599 | 0.61215     | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| ATP1B3 | neural cell                                             |      2.6321  |         1763 |       0.215842    |          0.00215953  |               4497 | 1.23979     | 0.389289 | 0.629092 |   1.20602  | 0.61881  | 0.135571   | 0.262167  |       0.641316  |   0.2543    |       5 |      7.48209 |
| MAF    | neural cell                                             |      1.94996 |          683 |       0.0971827   |          0.000836618 |               4497 | 0.535406    | 0.389289 | 0.629092 |   1.20602  | 0.61881  | 0.135571   | 0.262167  |       0.641316  |   0.2543    |       5 |      7.48209 |
| MMP13  | neural cell                                             |      1.7049  |           35 |       0.0185087   |          4.28721e-05 |               4497 | 0.221304    | 0.389289 | 0.629092 |   1.20602  | 0.61881  | 0.135571   | 0.262167  |       0.641316  |   0.2543    |       5 |      7.48209 |
| PLAU   | neural cell                                             |      1.91888 |          284 |       0.0377911   |          0.000347876 |               4497 | 0.412937    | 0.389289 | 0.629092 |   1.20602  | 0.61881  | 0.135571   | 0.262167  |       0.641316  |   0.2543    |       5 |      7.48209 |
| TXNIP  | neural cell                                             |      1.95015 |         1732 |       0.212047    |          0.00212156  |               4497 | 0.73602     | 0.389289 | 0.629092 |   1.20602  | 0.61881  | 0.135571   | 0.262167  |       0.641316  |   0.2543    |       5 |      7.48209 |
| ATP1B3 | Langerhans cell                                         |      1.94776 |         4770 |       0.382487    |          0.00584285  |               5450 | 1.07155     | 0.280398 | 0.570717 |   4.94107  | 0.491308 | 0.0559828  | 0.0987566 |       1         |   0.208772  |       5 |      6.61333 |
| MAF    | Langerhans cell                                         |      2.0582  |           10 |       0.00080186  |          1.22492e-05 |               5450 | 0.451678    | 0.280398 | 0.570717 |   4.94107  | 0.491308 | 0.0559828  | 0.0987566 |       1         |   0.208772  |       5 |      6.61333 |
| MMP13  | Langerhans cell                                         |      2.07237 |            5 |       0.000402156 |          6.12458e-06 |               5450 | 0.461277    | 0.280398 | 0.570717 |   4.94107  | 0.491308 | 0.0559828  | 0.0987566 |       1         |   0.208772  |       5 |      6.61333 |
| PLAU   | Langerhans cell                                         |      1.99192 |          123 |       0.00989303  |          0.000150665 |               5450 | 0.420023    | 0.280398 | 0.570717 |   4.94107  | 0.491308 | 0.0559828  | 0.0987566 |       1         |   0.208772  |       5 |      6.61333 |
| TXNIP  | Langerhans cell                                         |      1.94724 |          542 |       0.0434608   |          0.000663905 |               5450 | 0.449058    | 0.280398 | 0.570717 |   4.94107  | 0.491308 | 0.0559828  | 0.0987566 |       1         |   0.208772  |       5 |      6.61333 |
| ATP1B3 | smooth muscle cell                                      |      1.89532 |         1478 |       0.146598    |          0.00181043  |               3365 | 0.591045    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| MAF    | smooth muscle cell                                      |      1.60786 |          142 |       0.0140845   |          0.000173938 |               3365 | 0.146785    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| MMP13  | smooth muscle cell                                      |      1.6824  |           18 |       0.0269461   |          2.20485e-05 |               3365 | 0.217101    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| PLAU   | smooth muscle cell                                      |      2.2642  |          123 |       0.0122      |          0.000150665 |               3365 | 0.622015    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| TXNIP  | smooth muscle cell                                      |      1.81684 |         1604 |       0.159095    |          0.00196477  |               3365 | 0.555953    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| ATP1B3 | muscle cell                                             |      1.89532 |         1478 |       0.146598    |          0.00181043  |               3365 | 0.591045    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| MAF    | muscle cell                                             |      1.60786 |          142 |       0.0140845   |          0.000173938 |               3365 | 0.146785    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| MMP13  | muscle cell                                             |      1.6824  |           18 |       0.0269461   |          2.20485e-05 |               3365 | 0.217101    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| PLAU   | muscle cell                                             |      2.2642  |          123 |       0.0122      |          0.000150665 |               3365 | 0.622015    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| TXNIP  | muscle cell                                             |      1.81684 |         1604 |       0.159095    |          0.00196477  |               3365 | 0.555953    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| ATP1B3 | contractile cell                                        |      1.91592 |         4068 |       0.120945    |          0.00498296  |              11338 | 0.637167    | 0.269866 | 0.453807 |  -1.99726  | 0.594671 | 0.0482849  | 0.23123   |       0.3337    |   0.117592  |       5 |      6.41986 |
| MAF    | contractile cell                                        |      1.6544  |          358 |       0.0106437   |          0.00043852  |              11338 | 0.181365    | 0.269866 | 0.453807 |  -1.99726  | 0.594671 | 0.0482849  | 0.23123   |       0.3337    |   0.117592  |       5 |      6.41986 |
| MMP13  | contractile cell                                        |      1.66772 |          414 |       0.0222964   |          0.000507116 |              11338 | 0.210055    | 0.269866 | 0.453807 |  -1.99726  | 0.594671 | 0.0482849  | 0.23123   |       0.3337    |   0.117592  |       5 |      6.41986 |
| PLAU   | contractile cell                                        |      1.9961  |          420 |       0.012487    |          0.000514465 |              11338 | 0.43494     | 0.269866 | 0.453807 |  -1.99726  | 0.594671 | 0.0482849  | 0.23123   |       0.3337    |   0.117592  |       5 |      6.41986 |
| TXNIP  | contractile cell                                        |      1.94981 |         6078 |       0.180705    |          0.00744504  |              11338 | 0.80551     | 0.269866 | 0.453807 |  -1.99726  | 0.594671 | 0.0482849  | 0.23123   |       0.3337    |   0.117592  |       5 |      6.41986 |
| ATP1B3 | melanocyte                                              |      1.87432 |         4143 |       0.342737    |          0.00507483  |               7124 | 0.941537    | 0.318934 | 0.480748 |  -0.93933  | 0.663411 | 0.0841484  | 0.319329  |       0.435295  |   0.138603  |       5 |      6.2629  |
| MAF    | melanocyte                                              |      1.79706 |          456 |       0.0377234   |          0.000558562 |               7124 | 0.328638    | 0.318934 | 0.480748 |  -0.93933  | 0.663411 | 0.0841484  | 0.319329  |       0.435295  |   0.138603  |       5 |      6.2629  |
| MMP13  | melanocyte                                              |      1.72212 |          154 |       0.0604396   |          0.000188637 |               7124 | 0.299953    | 0.318934 | 0.480748 |  -0.93933  | 0.663411 | 0.0841484  | 0.319329  |       0.435295  |   0.138603  |       5 |      6.2629  |
| PLAU   | melanocyte                                              |      1.65413 |           38 |       0.00314361  |          4.65468e-05 |               7124 | 0.16132     | 0.318934 | 0.480748 |  -0.93933  | 0.663411 | 0.0841484  | 0.319329  |       0.435295  |   0.138603  |       5 |      6.2629  |
| TXNIP  | melanocyte                                              |      1.87978 |         2333 |       0.193001    |          0.00285773  |               7124 | 0.672292    | 0.318934 | 0.480748 |  -0.93933  | 0.663411 | 0.0841484  | 0.319329  |       0.435295  |   0.138603  |       5 |      6.2629  |
| ATP1B3 | supporting cell                                         |      1.93929 |         2362 |       0.131955    |          0.00289325  |               4997 | 0.624893    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| MAF    | supporting cell                                         |      1.68149 |          153 |       0.00854749  |          0.000187412 |               4997 | 0.192463    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| MMP13  | supporting cell                                         |      1.66705 |          396 |       0.0221229   |          0.000485067 |               4997 | 0.208825    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| PLAU   | supporting cell                                         |      1.88096 |          264 |       0.0147486   |          0.000323378 |               4997 | 0.350194    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| TXNIP  | supporting cell                                         |      1.9649  |         1822 |       0.101788    |          0.0022318   |               4997 | 0.583806    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| ATP1B3 | pericyte                                                |      1.93929 |         2362 |       0.131955    |          0.00289325  |               4997 | 0.624893    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| MAF    | pericyte                                                |      1.68149 |          153 |       0.00854749  |          0.000187412 |               4997 | 0.192463    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| MMP13  | pericyte                                                |      1.66705 |          396 |       0.0221229   |          0.000485067 |               4997 | 0.208825    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| PLAU   | pericyte                                                |      1.88096 |          264 |       0.0147486   |          0.000323378 |               4997 | 0.350194    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| TXNIP  | pericyte                                                |      1.9649  |         1822 |       0.101788    |          0.0022318   |               4997 | 0.583806    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| ATP1B3 | mural cell                                              |      1.93929 |         2362 |       0.131955    |          0.00289325  |               4997 | 0.624893    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| MAF    | mural cell                                              |      1.68149 |          153 |       0.00854749  |          0.000187412 |               4997 | 0.192463    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| MMP13  | mural cell                                              |      1.66705 |          396 |       0.0221229   |          0.000485067 |               4997 | 0.208825    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| PLAU   | mural cell                                              |      1.88096 |          264 |       0.0147486   |          0.000323378 |               4997 | 0.350194    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| TXNIP  | mural cell                                              |      1.9649  |         1822 |       0.101788    |          0.0022318   |               4997 | 0.583806    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| ATP1B3 | neuron associated cell                                  |      2.81218 |         1473 |       0.234891    |          0.0018043   |               3269 | 1.39188     | 0.52548  | 0.57297  |   0.215921 | 0.917117 | 0.235114   | 0.644486  |       0.546235  |   0.210529  |       5 |      6.09574 |
| MAF    | neuron associated cell                                  |      1.67543 |           54 |       0.0105243   |          6.61455e-05 |               3269 | 0.188353    | 0.52548  | 0.57297  |   0.215921 | 0.917117 | 0.235114   | 0.644486  |       0.546235  |   0.210529  |       5 |      6.09574 |
| MMP13  | neuron associated cell                                  |      1.7049  |           35 |       0.0185087   |          4.28721e-05 |               3269 | 0.221304    | 0.52548  | 0.57297  |   0.215921 | 0.917117 | 0.235114   | 0.644486  |       0.546235  |   0.210529  |       5 |      6.09574 |
| PLAU   | neuron associated cell                                  |      1.75889 |           67 |       0.0106841   |          8.20694e-05 |               3269 | 0.249793    | 0.52548  | 0.57297  |   0.215921 | 0.917117 | 0.235114   | 0.644486  |       0.546235  |   0.210529  |       5 |      6.09574 |
| TXNIP  | neuron associated cell                                  |      1.95804 |         1640 |       0.261521    |          0.00200886  |               3269 | 0.813519    | 0.52548  | 0.57297  |   0.215921 | 0.917117 | 0.235114   | 0.644486  |       0.546235  |   0.210529  |       5 |      6.09574 |
| ATP1B3 | cytotoxic T cell                                        |      1.78991 |         2151 |       0.192157    |          0.0026348   |               7108 | 0.600602    | 0.45946  | 0.582307 |   3.17297  | 0.789034 | 0.18686    | 0.480331  |       0.830206  |   0.217811  |       5 |      6.05457 |
| MAF    | cytotoxic T cell                                        |      1.84153 |          260 |       0.0232267   |          0.000318478 |               7108 | 0.33405     | 0.45946  | 0.582307 |   3.17297  | 0.789034 | 0.18686    | 0.480331  |       0.830206  |   0.217811  |       5 |      6.05457 |
| MMP13  | cytotoxic T cell                                        |      1.97821 |           55 |       0.00491335  |          6.73704e-05 |               7108 | 0.400734    | 0.45946  | 0.582307 |   3.17297  | 0.789034 | 0.18686    | 0.480331  |       0.830206  |   0.217811  |       5 |      6.05457 |
| PLAU   | cytotoxic T cell                                        |      1.72557 |           24 |       0.00214401  |          2.9398e-05  |               7108 | 0.211534    | 0.45946  | 0.582307 |   3.17297  | 0.789034 | 0.18686    | 0.480331  |       0.830206  |   0.217811  |       5 |      6.05457 |
| TXNIP  | cytotoxic T cell                                        |      2.29346 |         4618 |       0.412542    |          0.00565667  |               7108 | 1.36461     | 0.45946  | 0.582307 |   3.17297  | 0.789034 | 0.18686    | 0.480331  |       0.830206  |   0.217811  |       5 |      6.05457 |
| ATP1B3 | central memory CD8-positive, alpha-beta T cell          |      2.07155 |         8361 |       0.484415    |          0.0102415   |              13275 | 1.41117     | 0.531176 | 0.579303 |   0.942431 | 0.916923 | 0.239277   | 0.644237  |       0.616003  |   0.215468  |       5 |      6.01752 |
| MAF    | central memory CD8-positive, alpha-beta T cell          |      1.8212  |         1359 |       0.078737    |          0.00166466  |              13275 | 0.432014    | 0.531176 | 0.579303 |   0.942431 | 0.916923 | 0.239277   | 0.644237  |       0.616003  |   0.215468  |       5 |      6.01752 |
| MMP13  | central memory CD8-positive, alpha-beta T cell          |      1.78719 |            1 |       5.79374e-05 |          1.22492e-06 |              13275 | 0.252717    | 0.531176 | 0.579303 |   0.942431 | 0.916923 | 0.239277   | 0.644237  |       0.616003  |   0.215468  |       5 |      6.01752 |
| PLAU   | central memory CD8-positive, alpha-beta T cell          |      1.50932 |            8 |       0.000463499 |          9.79933e-06 |              13275 | 0.0509069   | 0.531176 | 0.579303 |   0.942431 | 0.916923 | 0.239277   | 0.644237  |       0.616003  |   0.215468  |       5 |      6.01752 |
| TXNIP  | central memory CD8-positive, alpha-beta T cell          |      1.91567 |         3546 |       0.205446    |          0.00434355  |              13275 | 0.749708    | 0.531176 | 0.579303 |   0.942431 | 0.916923 | 0.239277   | 0.644237  |       0.616003  |   0.215468  |       5 |      6.01752 |
| ATP1B3 | perivascular cell                                       |      1.86761 |         3491 |       0.12781     |          0.00427618  |               7845 | 0.596734    | 0.208547 | 0.377286 |  -3.06226  | 0.552757 | 0.0034665  | 0.177511  |       0.231426  |   0.0579108 |       5 |      5.98594 |
| MAF    | perivascular cell                                       |      1.6434  |          286 |       0.0104708   |          0.000350326 |               7845 | 0.171149    | 0.208547 | 0.377286 |  -3.06226  | 0.552757 | 0.0034665  | 0.177511  |       0.231426  |   0.0579108 |       5 |      5.98594 |
| MMP13  | perivascular cell                                       |      1.66705 |          396 |       0.0221229   |          0.000485067 |               7845 | 0.208825    | 0.208547 | 0.377286 |  -3.06226  | 0.552757 | 0.0034665  | 0.177511  |       0.231426  |   0.0579108 |       5 |      5.98594 |
| PLAU   | perivascular cell                                       |      1.82771 |          321 |       0.0117522   |          0.000393198 |               7845 | 0.308396    | 0.208547 | 0.377286 |  -3.06226  | 0.552757 | 0.0034665  | 0.177511  |       0.231426  |   0.0579108 |       5 |      5.98594 |
| TXNIP  | perivascular cell                                       |      1.8896  |         3351 |       0.122684    |          0.0041047   |               7845 | 0.601323    | 0.208547 | 0.377286 |  -3.06226  | 0.552757 | 0.0034665  | 0.177511  |       0.231426  |   0.0579108 |       5 |      5.98594 |
| ATP1B3 | memory T cell                                           |      2.06932 |         8423 |       0.305026    |          0.0103175   |              14405 | 1.14213     | 0.406097 | 0.51241  |   0.42835  | 0.792523 | 0.147857   | 0.484803  |       0.566635  |   0.163298  |       5 |      5.88758 |
| MAF    | memory T cell                                           |      1.81529 |         1497 |       0.05419     |          0.0018337   |              14405 | 0.394591    | 0.406097 | 0.51241  |   0.42835  | 0.792523 | 0.147857   | 0.484803  |       0.566635  |   0.163298  |       5 |      5.88758 |
| MMP13  | memory T cell                                           |      1.67738 |          218 |       0.00789455  |          0.000267032 |              14405 | 0.190236    | 0.406097 | 0.51241  |   0.42835  | 0.792523 | 0.147857   | 0.484803  |       0.566635  |   0.163298  |       5 |      5.88758 |
| PLAU   | memory T cell                                           |      1.66187 |           48 |       0.00173825  |          5.8796e-05  |              14405 | 0.165124    | 0.406097 | 0.51241  |   0.42835  | 0.792523 | 0.147857   | 0.484803  |       0.566635  |   0.163298  |       5 |      5.88758 |
| TXNIP  | memory T cell                                           |      1.88997 |         4219 |       0.152724    |          0.00516792  |              14405 | 0.669969    | 0.406097 | 0.51241  |   0.42835  | 0.792523 | 0.147857   | 0.484803  |       0.566635  |   0.163298  |       5 |      5.88758 |
| ATP1B3 | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      2.06932 |         8423 |       0.305026    |          0.0103175   |              14397 | 1.14213     | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| MAF    | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.8146  |         1496 |       0.0541754   |          0.00183248  |              14397 | 0.394043    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| MMP13  | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.67738 |          218 |       0.00789455  |          0.000267032 |              14397 | 0.190236    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| PLAU   | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.66187 |           48 |       0.00173825  |          5.8796e-05  |              14397 | 0.165124    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| TXNIP  | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.88834 |         4212 |       0.152531    |          0.00515935  |              14397 | 0.668302    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| ATP1B3 | CD8-positive, alpha-beta memory T cell                  |      2.06932 |         8423 |       0.305026    |          0.0103175   |              14397 | 1.14213     | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| MAF    | CD8-positive, alpha-beta memory T cell                  |      1.8146  |         1496 |       0.0541754   |          0.00183248  |              14397 | 0.394043    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| MMP13  | CD8-positive, alpha-beta memory T cell                  |      1.67738 |          218 |       0.00789455  |          0.000267032 |              14397 | 0.190236    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| PLAU   | CD8-positive, alpha-beta memory T cell                  |      1.66187 |           48 |       0.00173825  |          5.8796e-05  |              14397 | 0.165124    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| TXNIP  | CD8-positive, alpha-beta memory T cell                  |      1.88834 |         4212 |       0.152531    |          0.00515935  |              14397 | 0.668302    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| ATP1B3 | monocyte                                                |      1.77597 |         1361 |       0.153369    |          0.00166711  |               3916 | 0.511041    | 0.239436 | 0.435089 |   1.02949  | 0.550314 | 0.026043   | 0.17438   |       0.624364  |   0.102993  |       5 |      5.79232 |
| MAF    | monocyte                                                |      1.79885 |          551 |       0.0620915   |          0.000674929 |               3916 | 0.369046    | 0.239436 | 0.435089 |   1.02949  | 0.550314 | 0.026043   | 0.17438   |       0.624364  |   0.102993  |       5 |      5.79232 |
| MMP13  | monocyte                                                |      1.64097 |           23 |       0.00259184  |          2.81731e-05 |               3916 | 0.150491    | 0.239436 | 0.435089 |   1.02949  | 0.550314 | 0.026043   | 0.17438   |       0.624364  |   0.102993  |       5 |      5.79232 |
| PLAU   | monocyte                                                |      1.86552 |          197 |       0.0221997   |          0.000241309 |               3916 | 0.348308    | 0.239436 | 0.435089 |   1.02949  | 0.550314 | 0.026043   | 0.17438   |       0.624364  |   0.102993  |       5 |      5.79232 |
| TXNIP  | monocyte                                                |      2.05391 |         1784 |       0.201037    |          0.00218525  |               3916 | 0.796559    | 0.239436 | 0.435089 |   1.02949  | 0.550314 | 0.026043   | 0.17438   |       0.624364  |   0.102993  |       5 |      5.79232 |
| ATP1B3 | helper T cell                                           |      1.78925 |         3660 |       0.0858107   |          0.0044832   |              24501 | 0.481139    | 0.797813 | 0.671921 |   4.11822  | 1.18736  | 0.434163   | 0.990838  |       0.92098   |   0.287703  |       5 |      5.33589 |
| MAF    | helper T cell                                           |      1.88059 |         2960 |       0.0693989   |          0.00362575  |              24501 | 0.504322    | 0.797813 | 0.671921 |   4.11822  | 1.18736  | 0.434163   | 0.990838  |       0.92098   |   0.287703  |       5 |      5.33589 |
| MMP13  | helper T cell                                           |      1.63432 |          425 |       0.00996436  |          0.00052059  |              24501 | 0.1675      | 0.797813 | 0.671921 |   4.11822  | 1.18736  | 0.434163   | 0.990838  |       0.92098   |   0.287703  |       5 |      5.33589 |
| PLAU   | helper T cell                                           |      1.6272  |           70 |       0.00164119  |          8.57442e-05 |              24501 | 0.140287    | 0.797813 | 0.671921 |   4.11822  | 1.18736  | 0.434163   | 0.990838  |       0.92098   |   0.287703  |       5 |      5.33589 |
| TXNIP  | helper T cell                                           |      2.79559 |        17386 |       0.407624    |          0.0212964   |              24501 | 2.06636     | 0.797813 | 0.671921 |   4.11822  | 1.18736  | 0.434163   | 0.990838  |       0.92098   |   0.287703  |       5 |      5.33589 |
| ATP1B3 | dendritic cell                                          |      1.93541 |        14140 |       0.329527    |          0.0173203   |              18367 | 1.23481     | 0.420232 | 0.510313 |   3.55749  | 0.823479 | 0.158188   | 0.524477  |       0.867132  |   0.161662  |       5 |      5.16367 |
| MAF    | dendritic cell                                          |      1.75742 |          405 |       0.00968205  |          0.000496091 |              18367 | 0.256297    | 0.420232 | 0.510313 |   3.55749  | 0.823479 | 0.158188   | 0.524477  |       0.867132  |   0.161662  |       5 |      5.16367 |
| MMP13  | dendritic cell                                          |      1.72412 |           95 |       0.00227316  |          0.000116367 |              18367 | 0.212575    | 0.420232 | 0.510313 |   3.55749  | 0.823479 | 0.158188   | 0.524477  |       0.867132  |   0.161662  |       5 |      5.16367 |
| PLAU   | dendritic cell                                          |      1.86951 |          477 |       0.0111261   |          0.000584285 |              18367 | 0.342131    | 0.420232 | 0.510313 |   3.55749  | 0.823479 | 0.158188   | 0.524477  |       0.867132  |   0.161662  |       5 |      5.16367 |
| TXNIP  | dendritic cell                                          |      1.85882 |         3250 |       0.0757399   |          0.00398098  |              18367 | 0.505749    | 0.420232 | 0.510313 |   3.55749  | 0.823479 | 0.158188   | 0.524477  |       0.867132  |   0.161662  |       5 |      5.16367 |
| ATP1B3 | conventional dendritic cell                             |      1.94525 |        11522 |       0.35806     |          0.0141135   |              14328 | 1.21446     | 0.420846 | 0.484911 |   3.74973  | 0.867882 | 0.158636   | 0.581384  |       0.885593  |   0.14185   |       5 |      4.73468 |
| MAF    | conventional dendritic cell                             |      1.7256  |           67 |       0.00208911  |          8.20694e-05 |              14328 | 0.212628    | 0.420846 | 0.484911 |   3.74973  | 0.867882 | 0.158636   | 0.581384  |       0.885593  |   0.14185   |       5 |      4.73468 |
| MMP13  | conventional dendritic cell                             |      1.73032 |           73 |       0.0022789   |          8.94189e-05 |              14328 | 0.216513    | 0.420846 | 0.484911 |   3.74973  | 0.867882 | 0.158636   | 0.581384  |       0.885593  |   0.14185   |       5 |      4.73468 |
| PLAU   | conventional dendritic cell                             |      1.84683 |          222 |       0.00690707  |          0.000271932 |              14328 | 0.31241     | 0.420846 | 0.484911 |   3.74973  | 0.867882 | 0.158636   | 0.581384  |       0.885593  |   0.14185   |       5 |      4.73468 |
| TXNIP  | conventional dendritic cell                             |      1.83706 |         2444 |       0.0759502   |          0.0029937   |              14328 | 0.468542    | 0.420846 | 0.484911 |   3.74973  | 0.867882 | 0.158636   | 0.581384  |       0.885593  |   0.14185   |       5 |      4.73468 |
| ATP1B3 | innate lymphoid cell                                    |      1.80577 |         2815 |       0.0917655   |          0.00344814  |              12556 | 0.479418    | 0.415358 | 0.469863 |   3.39929  | 0.883997 | 0.154625   | 0.602038  |       0.85194   |   0.130114  |       5 |      4.60439 |
| MAF    | innate lymphoid cell                                    |      1.7271  |         1117 |       0.0364128   |          0.00136823  |              12556 | 0.293411    | 0.415358 | 0.469863 |   3.39929  | 0.883997 | 0.154625   | 0.602038  |       0.85194   |   0.130114  |       5 |      4.60439 |
| MMP13  | innate lymphoid cell                                    |      1.67761 |          503 |       0.0175598   |          0.000616133 |              12556 | 0.212551    | 0.415358 | 0.469863 |   3.39929  | 0.883997 | 0.154625   | 0.602038  |       0.85194   |   0.130114  |       5 |      4.60439 |
| PLAU   | innate lymphoid cell                                    |      1.67664 |          114 |       0.00371626  |          0.000139641 |              12556 | 0.180632    | 0.415358 | 0.469863 |   3.39929  | 0.883997 | 0.154625   | 0.602038  |       0.85194   |   0.130114  |       5 |      4.60439 |
| TXNIP  | innate lymphoid cell                                    |      2.23165 |         8007 |       0.261018    |          0.00980791  |              12556 | 1.1833      | 0.415358 | 0.469863 |   3.39929  | 0.883997 | 0.154625   | 0.602038  |       0.85194   |   0.130114  |       5 |      4.60439 |
| ATP1B3 | lymphocyte of B lineage                                 |      1.79415 |         1286 |       0.10714     |          0.00157524  |               5391 | 0.452933    | 0.370148 | 0.42898  |   2.9191   | 0.862857 | 0.121581   | 0.574945  |       0.805827  |   0.0982283 |       5 |      4.37049 |
| MAF    | lymphocyte of B lineage                                 |      1.62569 |            7 |       0.00144063  |          8.57442e-06 |               5391 | 0.137192    | 0.370148 | 0.42898  |   2.9191   | 0.862857 | 0.121581   | 0.574945  |       0.805827  |   0.0982283 |       5 |      4.37049 |
| MMP13  | lymphocyte of B lineage                                 |      1.85698 |            8 |       0.00252685  |          9.79933e-06 |               5391 | 0.307494    | 0.370148 | 0.42898  |   2.9191   | 0.862857 | 0.121581   | 0.574945  |       0.805827  |   0.0982283 |       5 |      4.37049 |
| PLAU   | lymphocyte of B lineage                                 |      1.70223 |            9 |       0.00106045  |          1.10243e-05 |               5391 | 0.19249     | 0.370148 | 0.42898  |   2.9191   | 0.862857 | 0.121581   | 0.574945  |       0.805827  |   0.0982283 |       5 |      4.37049 |
| TXNIP  | lymphocyte of B lineage                                 |      2.03756 |         4081 |       0.339998    |          0.00499889  |               5391 | 1.05479     | 0.370148 | 0.42898  |   2.9191   | 0.862857 | 0.121581   | 0.574945  |       0.805827  |   0.0982283 |       5 |      4.37049 |
| ATP1B3 | natural killer cell                                     |      1.7757  |         1008 |       0.0543631   |          0.00123472  |               5508 | 0.352849    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| MAF    | natural killer cell                                     |      1.71231 |          653 |       0.0352173   |          0.000799871 |               5508 | 0.268369    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| MMP13  | natural killer cell                                     |      1.67841 |          396 |       0.0213569   |          0.000485067 |               5508 | 0.215958    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| PLAU   | natural killer cell                                     |      1.69045 |           94 |       0.00506957  |          0.000115142 |               5508 | 0.192194    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| TXNIP  | natural killer cell                                     |      2.07958 |         3357 |       0.181048    |          0.00411205  |               5508 | 0.827552    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| ATP1B3 | group 1 innate lymphoid cell                            |      1.7757  |         1008 |       0.0543631   |          0.00123472  |               5508 | 0.352849    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| MAF    | group 1 innate lymphoid cell                            |      1.71231 |          653 |       0.0352173   |          0.000799871 |               5508 | 0.268369    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| MMP13  | group 1 innate lymphoid cell                            |      1.67841 |          396 |       0.0213569   |          0.000485067 |               5508 | 0.215958    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| PLAU   | group 1 innate lymphoid cell                            |      1.69045 |           94 |       0.00506957  |          0.000115142 |               5508 | 0.192194    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| TXNIP  | group 1 innate lymphoid cell                            |      2.07958 |         3357 |       0.181048    |          0.00411205  |               5508 | 0.827552    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| ATP1B3 | myeloid dendritic cell                                  |      1.94602 |         6592 |       0.531356    |          0.00807465  |               8508 | 1.34253     | 0.5357   | 0.448469 |   2.46243  | 1.19451  | 0.242584   | 1         |       0.761972  |   0.113428  |       5 |      3.57888 |
| MAF    | myeloid dendritic cell                                  |      1.70022 |           11 |       0.000894454 |          1.34741e-05 |               8508 | 0.19083     | 0.5357   | 0.448469 |   2.46243  | 1.19451  | 0.242584   | 1         |       0.761972  |   0.113428  |       5 |      3.57888 |
| MMP13  | myeloid dendritic cell                                  |      1.44072 |            1 |       8.1314e-05  |          1.22492e-06 |               8508 | 0.000121971 | 0.5357   | 0.448469 |   2.46243  | 1.19451  | 0.242584   | 1         |       0.761972  |   0.113428  |       5 |      3.57888 |
| PLAU   | myeloid dendritic cell                                  |      1.66306 |           96 |       0.00773819  |          0.000117592 |               8508 | 0.17628     | 0.5357   | 0.448469 |   2.46243  | 1.19451  | 0.242584   | 1         |       0.761972  |   0.113428  |       5 |      3.57888 |
| TXNIP  | myeloid dendritic cell                                  |      1.80476 |         1808 |       0.145736    |          0.00221465  |               8508 | 0.532584    | 0.5357   | 0.448469 |   2.46243  | 1.19451  | 0.242584   | 1         |       0.761972  |   0.113428  |       5 |      3.57888 |
| ATP1B3 | monocyte-derived dendritic cell                         |      1.93235 |         6506 |       0.529029    |          0.00796931  |               8398 | 1.32676     | 0.529119 | 0.444091 |   2.45008  | 1.19146  | 0.237774   | 0.996098  |       0.760786  |   0.110014  |       5 |      3.55088 |
| MAF    | monocyte-derived dendritic cell                         |      1.70022 |           11 |       0.000894454 |          1.34741e-05 |               8398 | 0.19083     | 0.529119 | 0.444091 |   2.45008  | 1.19146  | 0.237774   | 0.996098  |       0.760786  |   0.110014  |       5 |      3.55088 |
| MMP13  | monocyte-derived dendritic cell                         |      1.44072 |            1 |       8.1314e-05  |          1.22492e-06 |               8398 | 0.000121971 | 0.529119 | 0.444091 |   2.45008  | 1.19146  | 0.237774   | 0.996098  |       0.760786  |   0.110014  |       5 |      3.55088 |
| PLAU   | monocyte-derived dendritic cell                         |      1.66157 |           93 |       0.00756221  |          0.000113917 |               8398 | 0.174851    | 0.529119 | 0.444091 |   2.45008  | 1.19146  | 0.237774   | 0.996098  |       0.760786  |   0.110014  |       5 |      3.55088 |
| TXNIP  | monocyte-derived dendritic cell                         |      1.79998 |         1787 |       0.145308    |          0.00218893  |               8398 | 0.527893    | 0.529119 | 0.444091 |   2.45008  | 1.19146  | 0.237774   | 0.996098  |       0.760786  |   0.110014  |       5 |      3.55088 |
| ATP1B3 | hair follicular keratinocyte                            |      2.2172  |         2852 |       0.839812    |          0.00349346  |               5207 | 1.90247     | 0.927636 | 0.956212 |  -5.47215  | 0.970115 | 0.529051   | 0.71241   |       0         |   0.509428  |       4 |     10.1073  |
| MAF    | hair follicular keratinocyte                            |      1.61235 |           27 |       0.00795053  |          3.30728e-05 |               5207 | 0.137768    | 0.927636 | 0.956212 |  -5.47215  | 0.970115 | 0.529051   | 0.71241   |       0         |   0.509428  |       4 |     10.1073  |
| PLAU   | hair follicular keratinocyte                            |      1.68247 |           13 |       0.00382803  |          1.59239e-05 |               5207 | 0.18234     | 0.927636 | 0.956212 |  -5.47215  | 0.970115 | 0.529051   | 0.71241   |       0         |   0.509428  |       4 |     10.1073  |
| TXNIP  | hair follicular keratinocyte                            |      2.15056 |         2315 |       0.681684    |          0.00283568  |               5207 | 1.60227     | 0.927636 | 0.956212 |  -5.47215  | 0.970115 | 0.529051   | 0.71241   |       0         |   0.509428  |       4 |     10.1073  |
| ATP1B3 | prickle cell                                            |      1.79982 |        15325 |       0.582235    |          0.0187718   |              27607 | 1.54685     | 0.734668 | 0.773438 |  -5.17137  | 0.949874 | 0.388011   | 0.686468  |       0.0288849 |   0.366878  |       4 |      8.47525 |
| MAF    | prickle cell                                            |      1.58369 |          964 |       0.0366247   |          0.00118082  |              27607 | 0.185052    | 0.734668 | 0.773438 |  -5.17137  | 0.949874 | 0.388011   | 0.686468  |       0.0288849 |   0.366878  |       4 |      8.47525 |
| PLAU   | prickle cell                                            |      1.57571 |          108 |       0.00410319  |          0.000132291 |              27607 | 0.107456    | 0.734668 | 0.773438 |  -5.17137  | 0.949874 | 0.388011   | 0.686468  |       0.0288849 |   0.366878  |       4 |      8.47525 |
| TXNIP  | prickle cell                                            |      1.87196 |        11210 |       0.425896    |          0.0137313   |              27607 | 1.2544      | 0.734668 | 0.773438 |  -5.17137  | 0.949874 | 0.388011   | 0.686468  |       0.0288849 |   0.366878  |       4 |      8.47525 |
| ATP1B3 | activated CD8-positive, alpha-beta T cell, human        |      1.92137 |         1433 |       0.274784    |          0.00175531  |               3919 | 0.801112    | 0.435294 | 0.631395 |  -1.89072  | 0.689416 | 0.169197   | 0.352657  |       0.343931  |   0.256096  |       4 |      7.45419 |
| MAF    | activated CD8-positive, alpha-beta T cell, human        |      1.80023 |          474 |       0.0908917   |          0.000580611 |               3919 | 0.411183    | 0.435294 | 0.631395 |  -1.89072  | 0.689416 | 0.169197   | 0.352657  |       0.343931  |   0.256096  |       4 |      7.45419 |
| PLAU   | activated CD8-positive, alpha-beta T cell, human        |      1.6617  |            2 |       0.000383509 |          2.44983e-06 |               3919 | 0.161733    | 0.435294 | 0.631395 |  -1.89072  | 0.689416 | 0.169197   | 0.352657  |       0.343931  |   0.256096  |       4 |      7.45419 |
| TXNIP  | activated CD8-positive, alpha-beta T cell, human        |      2.15311 |         2010 |       0.385427    |          0.00246208  |               3919 | 1.15155     | 0.435294 | 0.631395 |  -1.89072  | 0.689416 | 0.169197   | 0.352657  |       0.343931  |   0.256096  |       4 |      7.45419 |
| ATP1B3 | naive T cell                                            |      2.04124 |         2656 |       0.259705    |          0.00325338  |               5783 | 0.898748    | 0.338543 | 0.557873 |  -4.05766  | 0.606846 | 0.0984812  | 0.246834  |       0.135836  |   0.198755  |       4 |      7.42239 |
| MAF    | naive T cell                                            |      1.82434 |          506 |       0.0494769   |          0.000619808 |               5783 | 0.367497    | 0.338543 | 0.557873 |  -4.05766  | 0.606846 | 0.0984812  | 0.246834  |       0.135836  |   0.198755  |       4 |      7.42239 |
| PLAU   | naive T cell                                            |      1.69035 |            2 |       0.000344412 |          2.44983e-06 |               5783 | 0.182565    | 0.338543 | 0.557873 |  -4.05766  | 0.606846 | 0.0984812  | 0.246834  |       0.135836  |   0.198755  |       4 |      7.42239 |
| TXNIP  | naive T cell                                            |      1.89086 |         2619 |       0.256087    |          0.00320806  |               5783 | 0.782682    | 0.338543 | 0.557873 |  -4.05766  | 0.606846 | 0.0984812  | 0.246834  |       0.135836  |   0.198755  |       4 |      7.42239 |
| ATP1B3 | activated CD8-positive, alpha-beta T cell               |      1.92898 |         1468 |       0.261303    |          0.00179818  |               4041 | 0.787378    | 0.426328 | 0.624341 |  -1.7715   | 0.682845 | 0.162643   | 0.344236  |       0.35538   |   0.250594  |       4 |      7.38778 |
| MAF    | activated CD8-positive, alpha-beta T cell               |      1.81226 |          486 |       0.0865077   |          0.00059531  |               4041 | 0.413703    | 0.426328 | 0.624341 |  -1.7715   | 0.682845 | 0.162643   | 0.344236  |       0.35538   |   0.250594  |       4 |      7.38778 |
| PLAU   | activated CD8-positive, alpha-beta T cell               |      1.6617  |            2 |       0.000383509 |          2.44983e-06 |               4041 | 0.161733    | 0.426328 | 0.624341 |  -1.7715   | 0.682845 | 0.162643   | 0.344236  |       0.35538   |   0.250594  |       4 |      7.38778 |
| TXNIP  | activated CD8-positive, alpha-beta T cell               |      2.15644 |         2085 |       0.371129    |          0.00255395  |               4041 | 1.13455     | 0.426328 | 0.624341 |  -1.7715   | 0.682845 | 0.162643   | 0.344236  |       0.35538   |   0.250594  |       4 |      7.38778 |
| ATP1B3 | CD4-positive, alpha-beta T cell                         |      1.987   |         3991 |       0.24815     |          0.00488864  |              12233 | 0.877731    | 0.445439 | 0.665728 |   1.41563  | 0.669101 | 0.176612   | 0.32662   |       0.661445  |   0.282873  |       4 |      7.24263 |
| MAF    | CD4-positive, alpha-beta T cell                         |      1.86248 |         2945 |       0.182987    |          0.00360738  |              12233 | 0.661095    | 0.445439 | 0.665728 |   1.41563  | 0.669101 | 0.176612   | 0.32662   |       0.661445  |   0.282873  |       4 |      7.24263 |
| PLAU   | CD4-positive, alpha-beta T cell                         |      1.50567 |            2 |       0.000313381 |          2.44983e-06 |              12233 | 0.0478545   | 0.445439 | 0.665728 |   1.41563  | 0.669101 | 0.176612   | 0.32662   |       0.661445  |   0.282873  |       4 |      7.24263 |
| TXNIP  | CD4-positive, alpha-beta T cell                         |      2.04486 |         5295 |       0.329005    |          0.00648593  |              12233 | 1.07623     | 0.445439 | 0.665728 |   1.41563  | 0.669101 | 0.176612   | 0.32662   |       0.661445  |   0.282873  |       4 |      7.24263 |
| ATP1B3 | T follicular helper cell                                |      2.11603 |         1446 |       0.471163    |          0.00177123  |               3029 | 1.23796     | 0.509294 | 0.681895 |   0.871406 | 0.746879 | 0.223283   | 0.426304  |       0.609183  |   0.295482  |       4 |      7.23156 |
| MAF    | T follicular helper cell                                |      1.89576 |          991 |       0.322906    |          0.00121389  |               3029 | 0.842747    | 0.509294 | 0.681895 |   0.871406 | 0.746879 | 0.223283   | 0.426304  |       0.609183  |   0.295482  |       4 |      7.23156 |
| PLAU   | T follicular helper cell                                |      1.46531 |            1 |       0.000325839 |          1.22492e-06 |               3029 | 0.0184218   | 0.509294 | 0.681895 |   0.871406 | 0.746879 | 0.223283   | 0.426304  |       0.609183  |   0.295482  |       4 |      7.23156 |
| TXNIP  | T follicular helper cell                                |      1.88472 |          591 |       0.192571    |          0.000723926 |               3029 | 0.628448    | 0.509294 | 0.681895 |   0.871406 | 0.746879 | 0.223283   | 0.426304  |       0.609183  |   0.295482  |       4 |      7.23156 |
| ATP1B3 | CD8-positive, alpha-beta regulatory T cell              |      2.17565 |         4191 |       0.500537    |          0.00513363  |               7632 | 1.39924     | 0.568043 | 0.706254 |   1.45581  | 0.804304 | 0.266223   | 0.499901  |       0.665304  |   0.31448   |       4 |      7.15625 |
| MAF    | CD8-positive, alpha-beta regulatory T cell              |      1.95094 |         1503 |       0.179506    |          0.00184105  |               7632 | 0.681639    | 0.568043 | 0.706254 |   1.45581  | 0.804304 | 0.266223   | 0.499901  |       0.665304  |   0.31448   |       4 |      7.15625 |
| PLAU   | CD8-positive, alpha-beta regulatory T cell              |      1.45264 |            1 |       0.000119432 |          1.22492e-06 |               7632 | 0.00887406  | 0.568043 | 0.706254 |   1.45581  | 0.804304 | 0.266223   | 0.499901  |       0.665304  |   0.31448   |       4 |      7.15625 |
| TXNIP  | CD8-positive, alpha-beta regulatory T cell              |      1.90187 |         1937 |       0.231339    |          0.00237266  |               7632 | 0.735261    | 0.568043 | 0.706254 |   1.45581  | 0.804304 | 0.266223   | 0.499901  |       0.665304  |   0.31448   |       4 |      7.15625 |
| ATP1B3 | CD4-positive helper T cell                              |      2.06818 |         2053 |       0.426908    |          0.00251475  |               4492 | 1.153       | 0.473528 | 0.654313 |   1.46573  | 0.723702 | 0.197142   | 0.3966    |       0.666256  |   0.27397   |       4 |      6.94339 |
| MAF    | CD4-positive helper T cell                              |      1.87842 |         1403 |       0.291745    |          0.00171856  |               4492 | 0.794427    | 0.473528 | 0.654313 |   1.46573  | 0.723702 | 0.197142   | 0.3966    |       0.666256  |   0.27397   |       4 |      6.94339 |
| PLAU   | CD4-positive helper T cell                              |      1.46531 |            1 |       0.000325839 |          1.22492e-06 |               4492 | 0.0184218   | 0.473528 | 0.654313 |   1.46573  | 0.723702 | 0.197142   | 0.3966    |       0.666256  |   0.27397   |       4 |      6.94339 |
| TXNIP  | CD4-positive helper T cell                              |      1.85325 |         1035 |       0.215221    |          0.00126779  |               4492 | 0.651404    | 0.473528 | 0.654313 |   1.46573  | 0.723702 | 0.197142   | 0.3966    |       0.666256  |   0.27397   |       4 |      6.94339 |
| ATP1B3 | B cell                                                  |      1.79863 |         1215 |       0.13749     |          0.00148827  |               4946 | 0.49982     | 0.470932 | 0.499573 |   1.49511  | 0.942669 | 0.195245   | 0.677234  |       0.669078  |   0.153286  |       4 |      4.80529 |
| MAF    | B cell                                                  |      1.62808 |            5 |       0.00295334  |          6.12458e-06 |               4946 | 0.141153    | 0.470932 | 0.499573 |   1.49511  | 0.942669 | 0.195245   | 0.677234  |       0.669078  |   0.153286  |       4 |      4.80529 |
| PLAU   | B cell                                                  |      1.70223 |            9 |       0.00106045  |          1.10243e-05 |               4946 | 0.19249     | 0.470932 | 0.499573 |   1.49511  | 0.942669 | 0.195245   | 0.677234  |       0.669078  |   0.153286  |       4 |      4.80529 |
| TXNIP  | B cell                                                  |      2.03604 |         3717 |       0.420618    |          0.00455302  |               4946 | 1.16483     | 0.470932 | 0.499573 |   1.49511  | 0.942669 | 0.195245   | 0.677234  |       0.669078  |   0.153286  |       4 |      4.80529 |
| ATP1B3 | vascular associated smooth muscle cell                  |      1.71763 |         1129 |       0.119928    |          0.00138293  |               2848 | 0.412104    | 0.211142 | 0.303033 |  -4.0041   | 0.696763 | 0.00536349 | 0.362073  |       0.14098   |   0         |       4 |      4.72963 |
| MAF    | vascular associated smooth muscle cell                  |      1.59959 |          133 |       0.0141279   |          0.000162914 |               2848 | 0.140577    | 0.211142 | 0.303033 |  -4.0041   | 0.696763 | 0.00536349 | 0.362073  |       0.14098   |   0         |       4 |      4.72963 |
| PLAU   | vascular associated smooth muscle cell                  |      1.58103 |           57 |       0.00605481  |          6.98203e-05 |               2848 | 0.112897    | 0.211142 | 0.303033 |  -4.0041   | 0.696763 | 0.00536349 | 0.362073  |       0.14098   |   0         |       4 |      4.72963 |
| TXNIP  | vascular associated smooth muscle cell                  |      1.79988 |         1529 |       0.162418    |          0.0018729   |               2848 | 0.546555    | 0.211142 | 0.303033 |  -4.0041   | 0.696763 | 0.00536349 | 0.362073  |       0.14098   |   0         |       4 |      4.72963 |
| ATP1B3 | myofibroblast cell                                      |      1.73398 |          203 |       0.0366691   |          0.000248658 |               2931 | 0.274265    | 0.47762  | 0.498901 |   3.52146  | 0.957343 | 0.200133   | 0.696041  |       0.863672  |   0.152761  |       4 |      4.38396 |
| MAF    | myofibroblast cell                                      |      1.64368 |           57 |       0.0102962   |          6.98203e-05 |               2931 | 0.164935    | 0.47762  | 0.498901 |   3.52146  | 0.957343 | 0.200133   | 0.696041  |       0.863672  |   0.152761  |       4 |      4.38396 |
| PLAU   | myofibroblast cell                                      |      1.90799 |           32 |       0.00578035  |          3.91973e-05 |               2931 | 0.350213    | 0.47762  | 0.498901 |   3.52146  | 0.957343 | 0.200133   | 0.696041  |       0.863672  |   0.152761  |       4 |      4.38396 |
| TXNIP  | myofibroblast cell                                      |      2.01711 |         2639 |       0.476698    |          0.00323256  |               2931 | 1.20619     | 0.47762  | 0.498901 |   3.52146  | 0.957343 | 0.200133   | 0.696041  |       0.863672  |   0.152761  |       4 |      4.38396 |
| ATP1B3 | naive thymus-derived CD8-positive, alpha-beta T cell    |      2.03345 |         1302 |       0.29457     |          0.00159484  |               2651 | 0.908996    | 0.279754 | 0.675321 | nan        | 0.414253 | 0.0555116  | 0         |     nan         |   0.290355  |       3 |      8.82768 |
| MAF    | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.84033 |          202 |       0.0457014   |          0.000247433 |               2651 | 0.365328    | 0.279754 | 0.675321 | nan        | 0.414253 | 0.0555116  | 0         |     nan         |   0.290355  |       3 |      8.82768 |
| TXNIP  | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.89549 |         1147 |       0.259502    |          0.00140498  |               2651 | 0.751637    | 0.279754 | 0.675321 | nan        | 0.414253 | 0.0555116  | 0         |     nan         |   0.290355  |       3 |      8.82768 |

In [95]:
results[0]['cell'].unique()[:5]

array(['epithelial cell', 'squamous epithelial cell', 'epidermal cell',
       'stratified squamous epithelial cell',
       'stratified epithelial cell'], dtype=object)

In [96]:
ex_df = results[0].copy()
unique_cells = ex_df['cell'].unique()[:10]

ex_df = ex_df[ex_df['cell'].isin(unique_cells)][['cell', 'cell score']].drop_duplicates().reset_index(drop=True)
ex_df

,cell,cell score
0,epithelial cell,11.664398
1,squamous epithelial cell,9.319514
2,epidermal cell,9.307081
3,stratified squamous epithelial cell,9.299244
4,stratified epithelial cell,9.299244
5,keratinocyte,9.299244
6,connective tissue cell,8.449418
7,skin fibroblast,8.381004
8,neoplastic cell,8.316629
9,malignant cell,8.316629


In [97]:
results[0][results[0]['cell']=='epithelial cell'][['gene', 'expression', 'cell count', 'cell percentage', 'tissue composition', 'score']]

,gene,expression,cell count,cell percentage,tissue composition,score
0,GAMT,1.625025,4500,0.025368,0.005512,0.203267
1,IFI27,2.646860,35913,0.199487,0.043990,1.461181
2,KRT14,4.508505,105286,0.587271,0.128967,4.147107
3,LGALS3BP,1.696281,1768,0.009825,0.002166,0.132572
4,RBM42,1.627480,4892,0.027429,0.005992,0.216269


In [98]:
def export_dataframe(df, n=10):
    unique_cells = df['cell'].unique()[:n]
    filtered_df = df[df['cell'].isin(unique_cells)]

    final_df = (
        filtered_df.groupby('cell', as_index=False)
        .agg({'gene': lambda x: ", ".join(map(str, set(x))), 
              'cell score': lambda x: ", ".join(map(str, set(x)))})
    ).sort_values(by=['cell score'], ascending=[False])

    return final_df

In [99]:
final_df = export_dataframe(results[0])
display_dataframe_as_markdown(final_df)

| cell                                | gene                                |   cell score |
|:------------------------------------|:------------------------------------|-------------:|
| squamous epithelial cell            | IFI27, RBM42, KRT14, GAMT, LGALS3BP |      9.31951 |
| epidermal cell                      | IFI27, RBM42, KRT14, GAMT, LGALS3BP |      9.30708 |
| keratinocyte                        | IFI27, RBM42, KRT14, GAMT, LGALS3BP |      9.29924 |
| stratified epithelial cell          | IFI27, RBM42, KRT14, GAMT, LGALS3BP |      9.29924 |
| stratified squamous epithelial cell | IFI27, RBM42, KRT14, GAMT, LGALS3BP |      9.29924 |
| connective tissue cell              | IFI27, RBM42, KRT14, GAMT, LGALS3BP |      8.44942 |
| skin fibroblast                     | IFI27, RBM42, KRT14, GAMT, LGALS3BP |      8.381   |
| malignant cell                      | IFI27, RBM42, KRT14, GAMT, LGALS3BP |      8.31663 |
| neoplastic cell                     | IFI27, RBM42, KRT14, GAMT, LGALS3BP |      8.31663 |
| epithelial cell                     | IFI27, RBM42, KRT14, GAMT, LGALS3BP |     11.6644  |

In [100]:
def get_top3_cells(df):
    return df['cell'].unique()[:3]

In [101]:
def export_top3_cells(results):

    top3cells = pd.DataFrame()
    for cluster in results:
        top3cells[f'{cluster}'] = get_top3_cells(results[cluster])

    top3cellsT = top3cells.T.reset_index()
    top3cellsT.columns = ['Cluster'] + [f'Cell {i+1}' for i in range(top3cellsT.shape[1]-1)]

    return top3cellsT

In [102]:
display_dataframe_as_markdown(export_top3_cells(results))

|   Cluster | Cell 1                              | Cell 2                              | Cell 3                              |
|----------:|:------------------------------------|:------------------------------------|:------------------------------------|
|         0 | epithelial cell                     | squamous epithelial cell            | epidermal cell                      |
|         1 | hematopoietic cell                  | leukocyte                           | mononuclear cell                    |
|         2 | epidermal cell                      | squamous epithelial cell            | stratified squamous epithelial cell |
|         3 | granular cell of epidermis          | squamous epithelial cell            | stratified squamous epithelial cell |
|         4 | skin fibroblast                     | fibroblast                          | connective tissue cell              |
|         5 | epidermal cell                      | stratified squamous epithelial cell | stratified epithelial cell          |
|         6 | connective tissue cell              | fibroblast                          | stromal cell                        |
|         7 | stratified squamous epithelial cell | stratified epithelial cell          | keratinocyte                        |
|         8 | epidermal cell                      | stratified squamous epithelial cell | stratified epithelial cell          |
|         9 | epithelial cell                     | squamous epithelial cell            | stratified squamous epithelial cell |
|        10 | connective tissue cell              | fibroblast                          | myofibroblast cell                  |
|        11 | connective tissue cell              | fibroblast                          | skin fibroblast                     |
|        12 | connective tissue cell              | fibroblast                          | secretory cell                      |
|        13 | connective tissue cell              | fibroblast                          | skin fibroblast                     |
|        14 | sebaceous gland cell                | connective tissue cell              | squamous epithelial cell            |
|        15 | stem cell of epidermis              | basal cell of epidermis             | connective tissue cell              |
|        16 | epithelial cell                     | stratified squamous epithelial cell | stratified epithelial cell          |
|        17 | lymphocyte of B lineage             | macrophage                          | connective tissue cell              |
|        18 | skin fibroblast                     | progenitor cell                     | stem cell of epidermis              |
|        19 | hematopoietic cell                  | leukocyte                           | mononuclear cell                    |
|        20 | epithelial cell                     | epidermal cell                      | stratified squamous epithelial cell |
|        21 | hematopoietic cell                  | leukocyte                           | mononuclear cell                    |
|        22 | connective tissue cell              | skin fibroblast                     | fibroblast                          |
|        23 | hematopoietic cell                  | leukocyte                           | mononuclear cell                    |
|        24 | hematopoietic cell                  | leukocyte                           | mononuclear cell                    |
|        25 | connective tissue cell              | fibroblast                          | skin fibroblast                     |
|        26 | connective tissue cell              | fibroblast                          | skin fibroblast                     |

# Validation with Tavily and Gemini Connected to a Database

In [103]:
import psycopg2
import os
from openai import OpenAI
from tavily import TavilyClient
from dotenv import load_dotenv

In [104]:
# Load environment variables
load_dotenv(override=True)

# Initialize Tavily and Gemini clients
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [105]:
# Connect to PostgreSQL
def get_db_connection():
    return psycopg2.connect(os.getenv("DATABASE_URL"))

In [106]:
def check_database(tissue_name, cell_name, gene_name):
    query = """
        SELECT relationcertainty FROM gene_expression
        WHERE uberon = (
            SELECT uberon 
            FROM tissue 
            WHERE tissuename = %s
        )
        AND cl = (
            SELECT cl 
            FROM cell 
            WHERE cellname = %s
        )
        AND ensembl = (
            SELECT ensembl 
            FROM gene 
            WHERE genename = %s
        )
    """
    try:
        with get_db_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(query, (tissue_name, cell_name, gene_name))
                result = cur.fetchone()
                return result[0] if result else None
    except Exception as e:
        print(f"Database error: {e}")
        return None

In [107]:
# Function to query Tavily using SDK
def query_tavily(tissue_name, cell_name, gene_name):
    query = f"Does {cell_name} express {gene_name} in {tissue_name}?"
    try:
        response = tavily_client.search(query= query, search_depth= "basic")
        sources = [result["url"] for result in response.get("results", [])]
        return sources if sources else None
    except Exception as e:
        print(f"Tavily Error: {e}")
        return None

In [108]:
# Function to summarize using OpenAI
def summarize_with_openai(tissue_name, cell_name, gene_name, sources):
    prompt = f"""
    Given the following sources, determine the confidence level that {cell_name} expresses {gene_name} in {tissue_name}.
    Return one of these values in valid JSON format:
    {{"status": 1.0}} → Yes
    {{"status": 0.67}} → Mostly Yes
    {{"status": 0.33}} → Mostly No
    {{"status": 0.0}} → No
    never return with markdown formatting.
    Sources: {sources}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": "You are an expert in cellular biology."},
                      {"role": "user", "content": prompt}]
        )
        output = response.choices[0].message.content
        return json.loads(output)["status"]
    except Exception as e:
        print(f"OpenAI Error: {e}")
        return None

In [109]:
def insert_into_database(tissue_name, cell_name, gene_name, source_urls, certainty):
    conn = psycopg2.connect(os.getenv("DATABASE_URL"))
    cur = conn.cursor()

    try:
        # Lookup IDs
        cur.execute("SELECT uberon FROM tissue WHERE tissuename = %s", (tissue_name,))
        uberon = cur.fetchone()
        if not uberon:
            raise ValueError(f"Tissue '{tissue_name}' not found.")
        uberon = uberon[0]

        cur.execute("SELECT cl FROM cell WHERE cellname = %s", (cell_name,))
        cl = cur.fetchone()
        if not cl:
            raise ValueError(f"Cell '{cell_name}' not found.")
        cl = cl[0]

        cur.execute("SELECT ensembl FROM gene WHERE genename = %s", (gene_name,))
        ensembl = cur.fetchone()
        if not ensembl:
            raise ValueError(f"Gene '{gene_name}' not found.")
        ensembl = ensembl[0]

        # Insert or update gene_expression
        cur.execute("""
            INSERT INTO gene_expression (uberon, cl, ensembl, relationcertainty)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (uberon, cl, ensembl)
            DO UPDATE SET relationcertainty = EXCLUDED.relationcertainty
            RETURNING expressionid
        """, (uberon, cl, ensembl, certainty))
        expressionid = cur.fetchone()[0]

        # Insert sources and validate links
        for url in source_urls:
            # Check or insert source
            cur.execute("SELECT sourceid FROM source WHERE url = %s", (url,))
            row = cur.fetchone()
            if row:
                sourceid = row[0]
            else:
                cur.execute("INSERT INTO source (url) VALUES (%s) RETURNING sourceid", (url,))
                sourceid = cur.fetchone()[0]

            # Insert validate link if not exists
            cur.execute("""
                INSERT INTO validate (expressionid, sourceid)
                VALUES (%s, %s)
                ON CONFLICT DO NOTHING
            """, (expressionid, sourceid))

        conn.commit()
        print(f"✅ Inserted/Updated expressionid {expressionid} with sources.")

    except Exception as e:
        conn.rollback()
        print(f"❌ Error: {e}")

    finally:
        cur.close()
        conn.close()

In [110]:
def validate_cell_gene_relation(tissue_name, cell_name, gene_name):
    print(f"\n[Validation Start] Processing: {tissue_name} - {cell_name} - {gene_name}")

    # Step 2: Check database first
    stored_status = check_database(tissue_name, cell_name, gene_name)
    if stored_status is not None:
        print(f"[Validation] Relation already exists in DB with status {stored_status}. Skipping process.")
        return float(stored_status)  # Ensure status is returned as a float

    # Step 3: Query Tavily
    try:
        sources = query_tavily(tissue_name, cell_name, gene_name)
    except Exception as e:
        print(f"Tavily Error: {e}")
        return None

    # Step 3a: If no sources found, set status to 0.0 and insert into database
    if not sources:
        status = 0.0
        print(f"[Validation] No references found in Tavily. Setting status to {status}.")
        insert_into_database(tissue_name, cell_name, gene_name, [], status)
        return float(status)  # Ensure status is a float

    # Step 4: Summarize with OpenAI
    status = summarize_with_openai(tissue_name, cell_name, gene_name, sources)
    status = float(status)  # Ensure status is a float
    print(f"[Validation] OpenAI summary status: {status}")

    # Step 5: Insert into Database
    insert_into_database(tissue_name, cell_name, gene_name, sources, status)
    print(f"[Validation] Inserted/Updated in DB with status: {status}")

    return status  # Return the validation score as a float

In [55]:
validate_cell_gene_relation("skin of body", "neural cell", "MMP3")


[Validation Start] Processing: skin of body - neural cell - MMP3
[Validation] Relation already exists in DB with status 0.33. Skipping process.


0.33

In [142]:
x_df = results.copy()
for cluster in x_df:
    unique_cells = x_df[cluster]['cell'].unique()[:10]
    x_df[cluster] = x_df[cluster][['gene', 'cell', 'cell score']][x_df[cluster]['cell'].isin(unique_cells)]

In [143]:
display_dataframe_as_markdown(x_df[0])

| gene     | cell                                |   cell score |
|:---------|:------------------------------------|-------------:|
| GAMT     | epithelial cell                     |     11.6644  |
| IFI27    | epithelial cell                     |     11.6644  |
| KRT14    | epithelial cell                     |     11.6644  |
| LGALS3BP | epithelial cell                     |     11.6644  |
| RBM42    | epithelial cell                     |     11.6644  |
| GAMT     | squamous epithelial cell            |      9.31951 |
| IFI27    | squamous epithelial cell            |      9.31951 |
| KRT14    | squamous epithelial cell            |      9.31951 |
| LGALS3BP | squamous epithelial cell            |      9.31951 |
| RBM42    | squamous epithelial cell            |      9.31951 |
| GAMT     | epidermal cell                      |      9.30708 |
| IFI27    | epidermal cell                      |      9.30708 |
| KRT14    | epidermal cell                      |      9.30708 |
| LGALS3BP | epidermal cell                      |      9.30708 |
| RBM42    | epidermal cell                      |      9.30708 |
| GAMT     | stratified squamous epithelial cell |      9.29924 |
| IFI27    | stratified squamous epithelial cell |      9.29924 |
| KRT14    | stratified squamous epithelial cell |      9.29924 |
| LGALS3BP | stratified squamous epithelial cell |      9.29924 |
| RBM42    | stratified squamous epithelial cell |      9.29924 |
| GAMT     | stratified epithelial cell          |      9.29924 |
| IFI27    | stratified epithelial cell          |      9.29924 |
| KRT14    | stratified epithelial cell          |      9.29924 |
| LGALS3BP | stratified epithelial cell          |      9.29924 |
| RBM42    | stratified epithelial cell          |      9.29924 |
| GAMT     | keratinocyte                        |      9.29924 |
| IFI27    | keratinocyte                        |      9.29924 |
| KRT14    | keratinocyte                        |      9.29924 |
| LGALS3BP | keratinocyte                        |      9.29924 |
| RBM42    | keratinocyte                        |      9.29924 |
| GAMT     | connective tissue cell              |      8.44942 |
| IFI27    | connective tissue cell              |      8.44942 |
| KRT14    | connective tissue cell              |      8.44942 |
| LGALS3BP | connective tissue cell              |      8.44942 |
| RBM42    | connective tissue cell              |      8.44942 |
| GAMT     | skin fibroblast                     |      8.381   |
| IFI27    | skin fibroblast                     |      8.381   |
| KRT14    | skin fibroblast                     |      8.381   |
| LGALS3BP | skin fibroblast                     |      8.381   |
| RBM42    | skin fibroblast                     |      8.381   |
| GAMT     | neoplastic cell                     |      8.31663 |
| IFI27    | neoplastic cell                     |      8.31663 |
| KRT14    | neoplastic cell                     |      8.31663 |
| LGALS3BP | neoplastic cell                     |      8.31663 |
| RBM42    | neoplastic cell                     |      8.31663 |
| GAMT     | malignant cell                      |      8.31663 |
| IFI27    | malignant cell                      |      8.31663 |
| KRT14    | malignant cell                      |      8.31663 |
| LGALS3BP | malignant cell                      |      8.31663 |
| RBM42    | malignant cell                      |      8.31663 |

In [150]:
def main_data_validation(results, target_uberon_id):
    ex_df = results.copy()
    for cluster in ex_df:
        unique_cells = ex_df[cluster]['cell'].unique()[:10]
        ex_df[cluster] = ex_df[cluster][['gene', 'cell', 'cell score']][ex_df[cluster]['cell'].isin(unique_cells)]
    tissue_name = ontl.tissue(target_uberon_id)

    for cluster in ex_df:
        ex_df[cluster]["certainty score"] = ex_df[cluster].apply(lambda row: validate_cell_gene_relation(tissue_name, row["cell"], row["gene"]), axis=1)

    return ex_df

In [151]:
res = main_data_validation(results, TARGET_UBERON_ID)


[Validation Start] Processing: skin of body - epithelial cell - GAMT
[Validation] Relation already exists in DB with status 0.33. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - IFI27
[Validation] Relation already exists in DB with status 1.0. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - KRT14
[Validation] Relation already exists in DB with status 1.0. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - LGALS3BP
[Validation] Relation already exists in DB with status 0.67. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - RBM42
[Validation] Relation already exists in DB with status 0.33. Skipping process.

[Validation Start] Processing: skin of body - squamous epithelial cell - GAMT
[Validation] Relation already exists in DB with status 0.0. Skipping process.

[Validation Start] Processing: skin of body - squamous epithelial cell - IFI27
[Validation] 

In [161]:
valid_df = res[0].copy()
mean_certainty = valid_df.groupby('cell')['certainty score'].mean().reset_index().rename(columns={'certainty score': 'cell certainty'})
final_df = valid_df.merge(mean_certainty, on='cell')


unique_cells = final_df['cell'].unique()
final_df = final_df[['cell', 'cell score', 'cell certainty']][final_df['cell'].isin(unique_cells)].drop_duplicates().reset_index(drop=True)



In [169]:
def export_validation_dataframe(results):
    ex_df = results.copy()
    for cluster in ex_df:
        mean_certainty = ex_df[cluster].groupby('cell')['certainty score'].mean().reset_index().rename(columns={'certainty score': 'cell certainty'})
        ex_df[cluster] = ex_df[cluster].merge(mean_certainty, on='cell')

        unique_cells = ex_df[cluster]['cell'].unique()[:10]
        ex_df[cluster] = ex_df[cluster][['cell', 'cell score', 'cell certainty']][ex_df[cluster]['cell'].isin(unique_cells)].drop_duplicates().reset_index(drop=True)
    return ex_df

In [170]:
tes = export_validation_dataframe(res)[0]

In [171]:
display_dataframe_as_markdown(tes)

| cell                                |   cell score |   cell certainty |
|:------------------------------------|-------------:|-----------------:|
| epithelial cell                     |     11.6644  |            0.666 |
| squamous epithelial cell            |      9.31951 |            0.534 |
| epidermal cell                      |      9.30708 |            0.802 |
| stratified squamous epithelial cell |      9.29924 |            0.4   |
| stratified epithelial cell          |      9.29924 |            0.668 |
| keratinocyte                        |      9.29924 |            0.6   |
| connective tissue cell              |      8.44942 |            0.468 |
| skin fibroblast                     |      8.381   |            0.868 |
| neoplastic cell                     |      8.31663 |            0.736 |
| malignant cell                      |      8.31663 |            0.8   |

In [162]:
display_dataframe_as_markdown(final_df)

| cell                                |   cell score |   cell certainty |
|:------------------------------------|-------------:|-----------------:|
| epithelial cell                     |     11.6644  |            0.666 |
| squamous epithelial cell            |      9.31951 |            0.534 |
| epidermal cell                      |      9.30708 |            0.802 |
| stratified squamous epithelial cell |      9.29924 |            0.4   |
| stratified epithelial cell          |      9.29924 |            0.668 |
| keratinocyte                        |      9.29924 |            0.6   |
| connective tissue cell              |      8.44942 |            0.468 |
| skin fibroblast                     |      8.381   |            0.868 |
| neoplastic cell                     |      8.31663 |            0.736 |
| malignant cell                      |      8.31663 |            0.8   |

In [156]:
display_dataframe_as_markdown(mean_certainty)

| cell                                |   mean |
|:------------------------------------|-------:|
| connective tissue cell              |  0.468 |
| epidermal cell                      |  0.802 |
| epithelial cell                     |  0.666 |
| keratinocyte                        |  0.6   |
| malignant cell                      |  0.8   |
| neoplastic cell                     |  0.736 |
| skin fibroblast                     |  0.868 |
| squamous epithelial cell            |  0.534 |
| stratified epithelial cell          |  0.668 |
| stratified squamous epithelial cell |  0.4   |

In [45]:
for cluster in results:
    valids = {}
    print(f"\nProcessing Cluster: {cluster}")
    valids[cluster] = data_validation(results[cluster], tl.tl_uberon(TARGET_UBERON_ID)).reset_index(drop=True)


Processing Cluster: 0

[Validation Start] Processing: skin of body - epithelial cell - COL17A1
[Validation] Relation already exists in DB with status 1.0. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - EFNA1
[Validation] Relation already exists in DB with status 0.67. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - GAMT
[Validation] Relation already exists in DB with status 0.33. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - HSPA1A
[Validation] Relation already exists in DB with status 0.67. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - IFI27
[Validation] Relation already exists in DB with status 1.0. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - KRT14
[Validation] Relation already exists in DB with status 1.0. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - LGALS3BP
[Val

KeyboardInterrupt: 

In [53]:
display_dataframe_as_markdown(results[0])

| gene     | cell                                                    |   expression |   cell count |   cell percentage |   tissue composition |   total cell count |       score |       std |      mean |       CV |   kurtosis |   std norm |   mean norm |   CV norm |   kurtosis norm |   count |   cell score |
|:---------|:--------------------------------------------------------|-------------:|-------------:|------------------:|---------------------:|-------------------:|------------:|----------:|----------:|---------:|-----------:|-----------:|------------:|----------:|----------------:|--------:|-------------:|
| COL17A1  | epithelial cell                                         |      2.24252 |        53112 |       0.295526    |          0.0653639   |             348909 | 1.91273     | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| EFNA1    | epithelial cell                                         |      1.83681 |        49963 |       0.277986    |          0.0614885   |             348909 | 1.71185     | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| GAMT     | epithelial cell                                         |      1.62434 |         4490 |       0.0253536   |          0.00552575  |             348909 | 0.200841    | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| HSPA1A   | epithelial cell                                         |      2.55937 |        91631 |       0.509398    |          0.112768    |             348909 | 3.22615     | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| IFI27    | epithelial cell                                         |      2.64729 |        35789 |       0.199125    |          0.0440448   |             348909 | 1.45635     | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| KRT14    | epithelial cell                                         |      4.50854 |       105283 |       0.588226    |          0.12957     |             348909 | 4.14805     | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| LGALS3BP | epithelial cell                                         |      1.62171 |         1670 |       0.00929549  |          0.00205524  |             348909 | 0.109144    | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| RBM42    | epithelial cell                                         |      1.62746 |         4885 |       0.0274353   |          0.00601187  |             348909 | 0.214111    | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| SCPEP1   | epithelial cell                                         |      1.75099 |         2086 |       0.0115973   |          0.0025672   |             348909 | 0.154275    | 1.46973   | 1.45928   | 1.00716  | -0.369714  |  0.941422  |   1         | 0.374701  |       0.155387  |       9 |      7.67564 |
| COL17A1  | epidermal cell                                          |      2.24453 |        52858 |       0.389074    |          0.0650513   |             294917 | 2.04752     | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| EFNA1    | epidermal cell                                          |      1.83415 |        37420 |       0.275439    |          0.046052    |             294917 | 1.40953     | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| GAMT     | epidermal cell                                          |      1.62195 |         4435 |       0.033142    |          0.00545807  |             294917 | 0.210628    | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| HSPA1A   | epidermal cell                                          |      2.57748 |        81735 |       0.60163     |          0.10059     |             294917 | 3.13396     | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| IFI27    | epidermal cell                                          |      2.14654 |         9126 |       0.0671741   |          0.0112312   |             294917 | 0.502115    | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| KRT14    | epidermal cell                                          |      4.53267 |       103908 |       0.764839    |          0.127877    |             294917 | 4.38626     | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| LGALS3BP | epidermal cell                                          |      1.60195 |         1509 |       0.0111247   |          0.0018571   |             294917 | 0.103203    | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| RBM42    | epidermal cell                                          |      1.62528 |         2100 |       0.0156681   |          0.00258443  |             294917 | 0.12979     | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| SCPEP1   | epidermal cell                                          |      1.76011 |         1826 |       0.0134407   |          0.00224722  |             294917 | 0.153111    | 1.55659   | 1.34179   | 1.16009  |  0.224499  |  0.999877  |   0.914317  | 0.474383  |       0.20937   |       9 |      6.94493 |
| COL17A1  | squamous epithelial cell                                |      2.24266 |        52717 |       0.387278    |          0.0648778   |             294999 | 2.04102     | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| EFNA1    | squamous epithelial cell                                |      1.83331 |        37570 |       0.27598     |          0.0462366   |             294999 | 1.41369     | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| GAMT     | squamous epithelial cell                                |      1.62028 |         4422 |       0.0331614   |          0.00544207  |             294999 | 0.209937    | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| HSPA1A   | squamous epithelial cell                                |      2.577   |        81720 |       0.600295    |          0.100571    |             294999 | 3.13148     | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| IFI27    | squamous epithelial cell                                |      2.1366  |         9401 |       0.0690575   |          0.0115696   |             294999 | 0.509025    | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| KRT14    | squamous epithelial cell                                |      4.53433 |       103735 |       0.766217    |          0.127665    |             294999 | 4.38462     | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| LGALS3BP | squamous epithelial cell                                |      1.601   |         1513 |       0.0111324   |          0.00186202  |             294999 | 0.103075    | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| RBM42    | squamous epithelial cell                                |      1.62399 |         2104 |       0.0156655   |          0.00258935  |             294999 | 0.129565    | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| SCPEP1   | squamous epithelial cell                                |      1.75054 |         1817 |       0.0133483   |          0.00223615  |             294999 | 0.150404    | 1.55539   | 1.34143   | 1.1595   |  0.23015   |  0.999065  |   0.914051  | 0.474002  |       0.209883  |       9 |      6.94387 |
| COL17A1  | stratified squamous epithelial cell                     |      2.2427  |        52713 |       0.389353    |          0.0648728   |             294413 | 2.04404     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| EFNA1    | stratified squamous epithelial cell                     |      1.83356 |        37353 |       0.2759      |          0.0459696   |             294413 | 1.40848     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| GAMT     | stratified squamous epithelial cell                     |      1.62028 |         4422 |       0.0331614   |          0.00544207  |             294413 | 0.209937    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| HSPA1A   | stratified squamous epithelial cell                     |      2.57731 |        81690 |       0.603386    |          0.100534    |             294413 | 3.13549     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| IFI27    | stratified squamous epithelial cell                     |      2.14633 |         9121 |       0.0673703   |          0.011225    |             294413 | 0.502237    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| KRT14    | stratified squamous epithelial cell                     |      4.53433 |       103735 |       0.766217    |          0.127665    |             294413 | 4.38462     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| LGALS3BP | stratified squamous epithelial cell                     |      1.60042 |         1506 |       0.0111412   |          0.0018534   |             294413 | 0.102781    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| RBM42    | stratified squamous epithelial cell                     |      1.62156 |         2079 |       0.015566    |          0.00255858  |             294413 | 0.128225    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| SCPEP1   | stratified squamous epithelial cell                     |      1.75095 |         1794 |       0.013251    |          0.00220784  |             294413 | 0.149813    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| COL17A1  | stratified epithelial cell                              |      2.2427  |        52713 |       0.389353    |          0.0648728   |             294413 | 2.04404     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| EFNA1    | stratified epithelial cell                              |      1.83356 |        37353 |       0.2759      |          0.0459696   |             294413 | 1.40848     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| GAMT     | stratified epithelial cell                              |      1.62028 |         4422 |       0.0331614   |          0.00544207  |             294413 | 0.209937    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| HSPA1A   | stratified epithelial cell                              |      2.57731 |        81690 |       0.603386    |          0.100534    |             294413 | 3.13549     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| IFI27    | stratified epithelial cell                              |      2.14633 |         9121 |       0.0673703   |          0.011225    |             294413 | 0.502237    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| KRT14    | stratified epithelial cell                              |      4.53433 |       103735 |       0.766217    |          0.127665    |             294413 | 4.38462     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| LGALS3BP | stratified epithelial cell                              |      1.60042 |         1506 |       0.0111412   |          0.0018534   |             294413 | 0.102781    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| RBM42    | stratified epithelial cell                              |      1.62156 |         2079 |       0.015566    |          0.00255858  |             294413 | 0.128225    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| SCPEP1   | stratified epithelial cell                              |      1.75095 |         1794 |       0.013251    |          0.00220784  |             294413 | 0.149813    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| COL17A1  | keratinocyte                                            |      2.2427  |        52713 |       0.389353    |          0.0648728   |             294413 | 2.04404     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| EFNA1    | keratinocyte                                            |      1.83356 |        37353 |       0.2759      |          0.0459696   |             294413 | 1.40848     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| GAMT     | keratinocyte                                            |      1.62028 |         4422 |       0.0331614   |          0.00544207  |             294413 | 0.209937    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| HSPA1A   | keratinocyte                                            |      2.57731 |        81690 |       0.603386    |          0.100534    |             294413 | 3.13549     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| IFI27    | keratinocyte                                            |      2.14633 |         9121 |       0.0673703   |          0.011225    |             294413 | 0.502237    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| KRT14    | keratinocyte                                            |      4.53433 |       103735 |       0.766217    |          0.127665    |             294413 | 4.38462     | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| LGALS3BP | keratinocyte                                            |      1.60042 |         1506 |       0.0111412   |          0.0018534   |             294413 | 0.102781    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| RBM42    | keratinocyte                                            |      1.62156 |         2079 |       0.015566    |          0.00255858  |             294413 | 0.128225    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| SCPEP1   | keratinocyte                                            |      1.75095 |         1794 |       0.013251    |          0.00220784  |             294413 | 0.149813    | 1.55678   | 1.34063   | 1.16123  |  0.222793  |  1         |   0.913468  | 0.475129  |       0.209215  |       9 |      6.93894 |
| COL17A1  | basal cell of epidermis                                 |      2.2197  |        24534 |       0.785465    |          0.0301935   |              85041 | 1.96342     | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| EFNA1    | basal cell of epidermis                                 |      1.71246 |         7133 |       0.228366    |          0.00877844  |              85041 | 0.589794    | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| GAMT     | basal cell of epidermis                                 |      1.60329 |         2228 |       0.0713302   |          0.00274195  |              85041 | 0.210913    | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| HSPA1A   | basal cell of epidermis                                 |      2.46056 |        20246 |       0.648183    |          0.0249163   |              85041 | 1.71493     | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| IFI27    | basal cell of epidermis                                 |      2.19921 |         2083 |       0.066688    |          0.00256351  |              85041 | 0.347101    | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| KRT14    | basal cell of epidermis                                 |      5.4584  |        27283 |       0.873475    |          0.0335766   |              85041 | 2.95742     | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| LGALS3BP | basal cell of epidermis                                 |      1.59463 |          913 |       0.02923     |          0.00112361  |              85041 | 0.114408    | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| RBM42    | basal cell of epidermis                                 |      1.59668 |          317 |       0.0101489   |          0.000390126 |              85041 | 0.0721381   | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| SCPEP1   | basal cell of epidermis                                 |      1.71152 |          304 |       0.00973267  |          0.000374127 |              85041 | 0.0994544   | 1.05166   | 0.89662   | 1.17292  |  0.101729  |  0.660103  |   0.589659  | 0.482749  |       0.198216  |       9 |      5.20789 |
| COL17A1  | stem cell of epidermis                                  |      2.21458 |        25193 |       0.77725     |          0.0310045   |              87642 | 1.96549     | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| EFNA1    | stem cell of epidermis                                  |      1.71224 |         7251 |       0.223707    |          0.00892366  |              87642 | 0.585554    | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| GAMT     | stem cell of epidermis                                  |      1.60667 |         2296 |       0.0708358   |          0.00282564  |              87642 | 0.212618    | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| HSPA1A   | stem cell of epidermis                                  |      2.451   |        20598 |       0.635486    |          0.0253495   |              87642 | 1.70189     | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| IFI27    | stem cell of epidermis                                  |      2.17103 |         2230 |       0.0687996   |          0.00274442  |              87642 | 0.346827    | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| KRT14    | stem cell of epidermis                                  |      5.46095 |        28455 |       0.877889    |          0.035019    |              87642 | 2.99249     | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| LGALS3BP | stem cell of epidermis                                  |      1.59595 |          937 |       0.0289082   |          0.00115315  |              87642 | 0.11482     | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| RBM42    | stem cell of epidermis                                  |      1.59951 |          358 |       0.011045    |          0.000440583 |              87642 | 0.0751511   | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| SCPEP1   | stem cell of epidermis                                  |      1.70432 |          324 |       0.00999599  |          0.00039874  |              87642 | 0.0985531   | 1.05909   | 0.899266  | 1.17773  |  0.214918  |  0.6651    |   0.591589  | 0.485882  |       0.208499  |       9 |      5.20348 |
| COL17A1  | stem cell                                               |      2.21421 |        25216 |       0.694426    |          0.0310328   |              88427 | 1.84171     | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| EFNA1    | stem cell                                               |      1.7116  |         7758 |       0.211973    |          0.00954761  |              88427 | 0.579836    | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| GAMT     | stem cell                                               |      1.6117  |         2309 |       0.0705167   |          0.00284164  |              88427 | 0.213685    | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| HSPA1A   | stem cell                                               |      2.45247 |        20738 |       0.565885    |          0.0255218   |              88427 | 1.60117     | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| IFI27    | stem cell                                               |      2.16988 |         2235 |       0.0615499   |          0.00275057  |              88427 | 0.335788    | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| KRT14    | stem cell                                               |      5.45935 |        28467 |       0.783956    |          0.0350338   |              88427 | 2.85149     | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| LGALS3BP | stem cell                                               |      1.59592 |          961 |       0.0264651   |          0.00118268  |              88427 | 0.111717    | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| RBM42    | stem cell                                               |      1.65534 |          395 |       0.0107926   |          0.000486119 |              88427 | 0.0893862   | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| SCPEP1   | stem cell                                               |      1.76078 |          348 |       0.00949704  |          0.000428277 |              88427 | 0.112263    | 0.997685  | 0.859672  | 1.16054  |  0.350251  |  0.623779  |   0.562713  | 0.47468   |       0.220794  |       9 |      5.05776 |
| COL17A1  | epithelial fate stem cell                               |      2.21932 |        24557 |       0.698953    |          0.0302218   |              85652 | 1.83411     | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| EFNA1    | epithelial fate stem cell                               |      1.71104 |         7635 |       0.217311    |          0.00939624  |              85652 | 0.584783    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| GAMT     | epithelial fate stem cell                               |      1.60329 |         2228 |       0.0713302   |          0.00274195  |              85652 | 0.210913    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| HSPA1A   | epithelial fate stem cell                               |      2.45952 |        20276 |       0.577105    |          0.0249533   |              85652 | 1.60877     | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| IFI27    | epithelial fate stem cell                               |      2.19791 |         2088 |       0.0594296   |          0.00256966  |              85652 | 0.336011    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| KRT14    | epithelial fate stem cell                               |      5.45673 |        27295 |       0.776883    |          0.0335914   |              85652 | 2.8124      | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| LGALS3BP | epithelial fate stem cell                               |      1.59463 |          937 |       0.0266693   |          0.00115315  |              85652 | 0.111137    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| RBM42    | epithelial fate stem cell                               |      1.59714 |          331 |       0.00942107  |          0.000407355 |              85652 | 0.0714925   | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| SCPEP1   | epithelial fate stem cell                               |      1.7119  |          305 |       0.00868105  |          0.000375357 |              85652 | 0.0979937   | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| COL17A1  | basal cell                                              |      2.21932 |        24557 |       0.698953    |          0.0302218   |              85652 | 1.83411     | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| EFNA1    | basal cell                                              |      1.71104 |         7635 |       0.217311    |          0.00939624  |              85652 | 0.584783    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| GAMT     | basal cell                                              |      1.60329 |         2228 |       0.0713302   |          0.00274195  |              85652 | 0.210913    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| HSPA1A   | basal cell                                              |      2.45952 |        20276 |       0.577105    |          0.0249533   |              85652 | 1.60877     | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| IFI27    | basal cell                                              |      2.19791 |         2088 |       0.0594296   |          0.00256966  |              85652 | 0.336011    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| KRT14    | basal cell                                              |      5.45673 |        27295 |       0.776883    |          0.0335914   |              85652 | 2.8124      | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| LGALS3BP | basal cell                                              |      1.59463 |          937 |       0.0266693   |          0.00115315  |              85652 | 0.111137    | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| RBM42    | basal cell                                              |      1.59714 |          331 |       0.00942107  |          0.000407355 |              85652 | 0.0714925   | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| SCPEP1   | basal cell                                              |      1.7119  |          305 |       0.00868105  |          0.000375357 |              85652 | 0.0979937   | 0.990937  | 0.851956  | 1.16313  |  0.226061  |  0.619238  |   0.557086  | 0.476367  |       0.209512  |       9 |      5.03504 |
| COL17A1  | progenitor cell                                         |      2.21943 |        24548 |       0.611362    |          0.0302107   |              89930 | 1.70253     | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| EFNA1    | progenitor cell                                         |      1.71297 |         7366 |       0.18365     |          0.00906519  |              89930 | 0.528379    | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| GAMT     | progenitor cell                                         |      1.60395 |         2337 |       0.0563961   |          0.0028761   |              89930 | 0.191264    | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| HSPA1A   | progenitor cell                                         |      2.61188 |        23720 |       0.571745    |          0.0291917   |              89930 | 1.71999     | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| IFI27    | progenitor cell                                         |      2.19649 |         2091 |       0.0519774   |          0.00257335  |              89930 | 0.324555    | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| KRT14    | progenitor cell                                         |      5.45703 |        27293 |       0.680471    |          0.0335889   |              89930 | 2.66781     | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| LGALS3BP | progenitor cell                                         |      1.59831 |          923 |       0.0228703   |          0.00113592  |              89930 | 0.106011    | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| RBM42    | progenitor cell                                         |      1.59964 |          590 |       0.0142809   |          0.000726101 |              89930 | 0.0855454   | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| SCPEP1   | progenitor cell                                         |      1.68384 |         1062 |       0.0257417   |          0.00130698  |              89930 | 0.134659    | 0.95179   | 0.828971  | 1.14816  | -0.10139   |  0.592896  |   0.540324  | 0.466607  |       0.179764  |       9 |      4.9887  |
| COL17A1  | precursor cell                                          |      2.21395 |        25230 |       0.557816    |          0.0310501   |              96282 | 1.63706     | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| EFNA1    | precursor cell                                          |      1.71185 |         8006 |       0.156242    |          0.00985282  |              96282 | 0.50219     | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| GAMT     | precursor cell                                          |      1.61113 |         2659 |       0.054631    |          0.00327238  |              96282 | 0.198029    | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| HSPA1A   | precursor cell                                          |      2.63742 |        26038 |       0.49484     |          0.0320444   |              96282 | 1.66596     | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| IFI27    | precursor cell                                          |      2.16309 |         2260 |       0.0442495   |          0.00278134  |              96282 | 0.308759    | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| KRT14    | precursor cell                                          |      5.45803 |        28477 |       0.630217    |          0.0350461   |              96282 | 2.62079     | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| LGALS3BP | precursor cell                                          |      1.59784 |         1076 |       0.0210144   |          0.00132421  |              96282 | 0.106745    | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| RBM42    | precursor cell                                          |      1.62371 |         1323 |       0.0252259   |          0.00162819  |              96282 | 0.125289    | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| SCPEP1   | precursor cell                                          |      1.68718 |         1213 |       0.0231542   |          0.00149281  |              96282 | 0.135184    | 0.924318  | 0.811112  | 1.13957  |  0.0576932 |  0.57441   |   0.527299  | 0.461009  |       0.194216  |       9 |      4.91531 |
| COL17A1  | prickle cell                                            |      1.80445 |         2147 |       0.0815698   |          0.00264227  |              61334 | 0.273835    | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| EFNA1    | prickle cell                                            |      1.75523 |        10520 |       0.399681    |          0.0129468   |              61334 | 0.937716    | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| GAMT     | prickle cell                                            |      1.51306 |          131 |       0.00497701  |          0.000161219 |              61334 | 0.0393929   | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| HSPA1A   | prickle cell                                            |      2.42382 |        19968 |       0.758634    |          0.0245742   |              61334 | 1.86496     | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| IFI27    | prickle cell                                            |      2.21559 |         5086 |       0.19323     |          0.00625924  |              61334 | 0.612252    | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| KRT14    | prickle cell                                            |      3.27004 |        23107 |       0.877892    |          0.0284373   |              61334 | 2.32656     | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| LGALS3BP | prickle cell                                            |      1.53987 |          163 |       0.00619277  |          0.000200601 |              61334 | 0.0485727   | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| RBM42    | prickle cell                                            |      1.50598 |          153 |       0.00581285  |          0.000188294 |              61334 | 0.0394271   | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| SCPEP1   | prickle cell                                            |      1.52625 |           59 |       0.00224156  |          7.26101e-05 |              61334 | 0.036824    | 0.864867  | 0.686615  | 1.25961  |  0.16281   |  0.534404  |   0.436505  | 0.539254  |       0.203766  |       9 |      4.3003  |
| COL17A1  | skin fibroblast                                         |      1.8836  |          575 |       0.00508503  |          0.000707641 |              77696 | 0.14125     | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| EFNA1    | skin fibroblast                                         |      1.79486 |         1375 |       0.0121599   |          0.00169218  |              77696 | 0.149027    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| GAMT     | skin fibroblast                                         |      1.73991 |         2775 |       0.0245408   |          0.00341514  |              77696 | 0.187327    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| HSPA1A   | skin fibroblast                                         |      2.78309 |         9483 |       0.0838632   |          0.0116705   |              77696 | 0.692213    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| IFI27    | skin fibroblast                                         |      2.37941 |         7002 |       0.0619224   |          0.00861722  |              77696 | 0.501086    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| KRT14    | skin fibroblast                                         |      2.22303 |        11550 |       0.102143    |          0.0142144   |              77696 | 0.630944    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| LGALS3BP | skin fibroblast                                         |      1.89644 |        20796 |       0.18391     |          0.0255932   |              77696 | 0.892807    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| RBM42    | skin fibroblast                                         |      1.75585 |         6708 |       0.0593224   |          0.0082554   |              77696 | 0.336812    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| SCPEP1   | skin fibroblast                                         |      1.81023 |        17432 |       0.15416     |          0.0214532   |              77696 | 0.747096    | 0.282904  | 0.475396  | 0.595092 | -1.65809   |  0.142792  |   0.282464  | 0.106106  |       0.0383413 |       9 |      4.22101 |
| COL17A1  | columnar/cuboidal epithelial cell                       |      1.86484 |          267 |       0.234211    |          0.000328592 |               3149 | 0.473009    | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| EFNA1    | columnar/cuboidal epithelial cell                       |      1.80474 |          366 |       0.320771    |          0.000450429 |               3149 | 0.590417    | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| GAMT     | columnar/cuboidal epithelial cell                       |      1.72735 |            2 |       0.00175439  |          2.46136e-06 |               3149 | 0.0842105   | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| HSPA1A   | columnar/cuboidal epithelial cell                       |      2.40445 |          891 |       0.780894    |          0.00109654  |               3149 | 1.44059     | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| IFI27    | columnar/cuboidal epithelial cell                       |      1.78471 |            1 |       0.000877193 |          1.23068e-06 |               3149 | 0.096981    | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| KRT14    | columnar/cuboidal epithelial cell                       |      4.63062 |         1139 |       0.999123    |          0.00140174  |               3149 | 2.32145     | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| LGALS3BP | columnar/cuboidal epithelial cell                       |      1.66691 |           10 |       0.00877193  |          1.23068e-05 |               3149 | 0.0800587   | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| RBM42    | columnar/cuboidal epithelial cell                       |      1.57944 |           16 |       0.0140351   |          1.96909e-05 |               3149 | 0.0665777   | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| SCPEP1   | columnar/cuboidal epithelial cell                       |      1.83831 |          457 |       0.400877    |          0.000562421 |               3149 | 0.720993    | 0.770535  | 0.652699  | 1.18054  |  1.93794   |  0.470927  |   0.41177   | 0.487713  |       0.365031  |       9 |      4.10285 |
| COL17A1  | neuron associated cell (sensu Vertebrata)               |      1.86484 |          267 |       0.234211    |          0.000328592 |               3147 | 0.473009    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| EFNA1    | neuron associated cell (sensu Vertebrata)               |      1.80201 |          365 |       0.320175    |          0.000449198 |               3147 | 0.588827    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| GAMT     | neuron associated cell (sensu Vertebrata)               |      1.72735 |            2 |       0.00175439  |          2.46136e-06 |               3147 | 0.0842105   | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| HSPA1A   | neuron associated cell (sensu Vertebrata)               |      2.40281 |          890 |       0.780702    |          0.00109531  |               3147 | 1.43988     | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| IFI27    | neuron associated cell (sensu Vertebrata)               |      1.78471 |            1 |       0.000877193 |          1.23068e-06 |               3147 | 0.096981    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| KRT14    | neuron associated cell (sensu Vertebrata)               |      4.63062 |         1139 |       0.999123    |          0.00140174  |               3147 | 2.32145     | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| LGALS3BP | neuron associated cell (sensu Vertebrata)               |      1.66691 |           10 |       0.00877193  |          1.23068e-05 |               3147 | 0.0800587   | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| RBM42    | neuron associated cell (sensu Vertebrata)               |      1.57944 |           16 |       0.0140351   |          1.96909e-05 |               3147 | 0.0665777   | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| SCPEP1   | neuron associated cell (sensu Vertebrata)               |      1.83831 |          457 |       0.400877    |          0.000562421 |               3147 | 0.720993    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| COL17A1  | Merkel cell                                             |      1.86484 |          267 |       0.234211    |          0.000328592 |               3147 | 0.473009    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| EFNA1    | Merkel cell                                             |      1.80201 |          365 |       0.320175    |          0.000449198 |               3147 | 0.588827    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| GAMT     | Merkel cell                                             |      1.72735 |            2 |       0.00175439  |          2.46136e-06 |               3147 | 0.0842105   | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| HSPA1A   | Merkel cell                                             |      2.40281 |          890 |       0.780702    |          0.00109531  |               3147 | 1.43988     | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| IFI27    | Merkel cell                                             |      1.78471 |            1 |       0.000877193 |          1.23068e-06 |               3147 | 0.096981    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| KRT14    | Merkel cell                                             |      4.63062 |         1139 |       0.999123    |          0.00140174  |               3147 | 2.32145     | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| LGALS3BP | Merkel cell                                             |      1.66691 |           10 |       0.00877193  |          1.23068e-05 |               3147 | 0.0800587   | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| RBM42    | Merkel cell                                             |      1.57944 |           16 |       0.0140351   |          1.96909e-05 |               3147 | 0.0665777   | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| SCPEP1   | Merkel cell                                             |      1.83831 |          457 |       0.400877    |          0.000562421 |               3147 | 0.720993    | 0.77046   | 0.652443  | 1.18089  |  1.94283   |  0.470876  |   0.411583  | 0.48794   |       0.365476  |       9 |      4.10111 |
| COL17A1  | connective tissue cell                                  |      1.80107 |         1627 |       0.00638012  |          0.00200232  |             149479 | 0.147871    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| EFNA1    | connective tissue cell                                  |      1.76844 |         6200 |       0.0237699   |          0.00763022  |             149479 | 0.274517    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| GAMT     | connective tissue cell                                  |      1.65474 |         7145 |       0.0273752   |          0.00879321  |             149479 | 0.274395    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| HSPA1A   | connective tissue cell                                  |      2.63005 |        55814 |       0.213821    |          0.0686892   |             149479 | 1.94967     | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| IFI27    | connective tissue cell                                  |      2.29561 |         8640 |       0.0331395   |          0.0106331   |             149479 | 0.476193    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| KRT14    | connective tissue cell                                  |      2.22595 |        12669 |       0.0486246   |          0.0155915   |             149479 | 0.577956    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| LGALS3BP | connective tissue cell                                  |      1.87572 |        23415 |       0.0898007   |          0.0288164   |             149479 | 0.808737    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| RBM42    | connective tissue cell                                  |      1.69745 |        13559 |       0.051944    |          0.0166868   |             149479 | 0.474061    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| SCPEP1   | connective tissue cell                                  |      1.78669 |        20410 |       0.0781983   |          0.0251182   |             149479 | 0.698078    | 0.537824  | 0.631275  | 0.851966 |  5.38632   |  0.314332  |   0.396146  | 0.273542  |       0.678307  |       9 |      4.05615 |
| COL17A1  | fibroblast                                              |      1.81969 |         1195 |       0.0051064   |          0.00147066  |             135486 | 0.140282    | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| EFNA1    | fibroblast                                              |      1.77218 |         4137 |       0.017678    |          0.00509132  |             135486 | 0.217313    | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| GAMT     | fibroblast                                              |      1.64918 |         6818 |       0.0291132   |          0.00839078  |             135486 | 0.26787     | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| HSPA1A   | fibroblast                                              |      2.63467 |        49302 |       0.210522    |          0.060675    |             135486 | 1.79122     | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| IFI27    | fibroblast                                              |      2.32391 |         7836 |       0.0334602   |          0.00964361  |             135486 | 0.464544    | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| KRT14    | fibroblast                                              |      2.21882 |        11765 |       0.0502735   |          0.0144789   |             135486 | 0.557208    | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| LGALS3BP | fibroblast                                              |      1.88332 |        22223 |       0.0948934   |          0.0273494   |             135486 | 0.78994     | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| RBM42    | fibroblast                                              |      1.69493 |        12395 |       0.0529273   |          0.0152543   |             135486 | 0.447276    | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| SCPEP1   | fibroblast                                              |      1.78663 |        19815 |       0.0846111   |          0.0243859   |             135486 | 0.693552    | 0.496908  | 0.596579  | 0.832929 |  4.71984   |  0.286799  |   0.370842  | 0.261134  |       0.61776   |       9 |      3.98718 |
| COL17A1  | neoplastic cell                                         |      1.99107 |          282 |       0.0116274   |          0.000347052 |              21536 | 0.170543    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| EFNA1    | neoplastic cell                                         |      2.08464 |         2218 |       0.0914526   |          0.00272965  |              21536 | 0.35927     | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| GAMT     | neoplastic cell                                         |      1.83142 |          512 |       0.0211108   |          0.000630108 |              21536 | 0.150956    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| HSPA1A   | neoplastic cell                                         |      2.60145 |         6151 |       0.253618    |          0.00756991  |              21536 | 0.823042    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| IFI27    | neoplastic cell                                         |      1.68681 |          698 |       0.0287799   |          0.000859015 |              21536 | 0.131303    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| KRT14    | neoplastic cell                                         |      3.75004 |         3256 |       0.134251    |          0.00400709  |              21536 | 0.857795    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| LGALS3BP | neoplastic cell                                         |      1.99507 |         5211 |       0.21486     |          0.00641307  |              21536 | 0.593418    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| RBM42    | neoplastic cell                                         |      2.14087 |         2220 |       0.0915351   |          0.00273211  |              21536 | 0.373272    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| SCPEP1   | neoplastic cell                                         |      1.70619 |          988 |       0.0407372   |          0.00121591  |              21536 | 0.160891    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| COL17A1  | malignant cell                                          |      1.99107 |          282 |       0.0116274   |          0.000347052 |              21536 | 0.170543    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| EFNA1    | malignant cell                                          |      2.08464 |         2218 |       0.0914526   |          0.00272965  |              21536 | 0.35927     | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| GAMT     | malignant cell                                          |      1.83142 |          512 |       0.0211108   |          0.000630108 |              21536 | 0.150956    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| HSPA1A   | malignant cell                                          |      2.60145 |         6151 |       0.253618    |          0.00756991  |              21536 | 0.823042    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| IFI27    | malignant cell                                          |      1.68681 |          698 |       0.0287799   |          0.000859015 |              21536 | 0.131303    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| KRT14    | malignant cell                                          |      3.75004 |         3256 |       0.134251    |          0.00400709  |              21536 | 0.857795    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| LGALS3BP | malignant cell                                          |      1.99507 |         5211 |       0.21486     |          0.00641307  |              21536 | 0.593418    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| RBM42    | malignant cell                                          |      2.14087 |         2220 |       0.0915351   |          0.00273211  |              21536 | 0.373272    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| SCPEP1   | malignant cell                                          |      1.70619 |          988 |       0.0407372   |          0.00121591  |              21536 | 0.160891    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| COL17A1  | abnormal cell                                           |      1.99107 |          282 |       0.0116274   |          0.000347052 |              21536 | 0.170543    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| EFNA1    | abnormal cell                                           |      2.08464 |         2218 |       0.0914526   |          0.00272965  |              21536 | 0.35927     | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| GAMT     | abnormal cell                                           |      1.83142 |          512 |       0.0211108   |          0.000630108 |              21536 | 0.150956    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| HSPA1A   | abnormal cell                                           |      2.60145 |         6151 |       0.253618    |          0.00756991  |              21536 | 0.823042    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| IFI27    | abnormal cell                                           |      1.68681 |          698 |       0.0287799   |          0.000859015 |              21536 | 0.131303    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| KRT14    | abnormal cell                                           |      3.75004 |         3256 |       0.134251    |          0.00400709  |              21536 | 0.857795    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| LGALS3BP | abnormal cell                                           |      1.99507 |         5211 |       0.21486     |          0.00641307  |              21536 | 0.593418    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| RBM42    | abnormal cell                                           |      2.14087 |         2220 |       0.0915351   |          0.00273211  |              21536 | 0.373272    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| SCPEP1   | abnormal cell                                           |      1.70619 |          988 |       0.0407372   |          0.00121591  |              21536 | 0.160891    | 0.289897  | 0.402277  | 0.720641 | -1.10514   |  0.147498  |   0.229139  | 0.187942  |       0.0885756 |       9 |      3.76196 |
| COL17A1  | hair follicular keratinocyte                            |      1.76391 |          789 |       0.232332    |          0.000971006 |               8122 | 0.457757    | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| EFNA1    | hair follicular keratinocyte                            |      1.67417 |          637 |       0.187574    |          0.000783943 |               8122 | 0.364936    | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| GAMT     | hair follicular keratinocyte                            |      1.55148 |           38 |       0.0111896   |          4.67658e-05 |               8122 | 0.055955    | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| HSPA1A   | hair follicular keratinocyte                            |      3.00312 |         2906 |       0.855713    |          0.00357636  |               8122 | 1.74794     | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| IFI27    | hair follicular keratinocyte                            |      1.94323 |          111 |       0.0326855   |          0.000136605 |               8122 | 0.1863      | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| KRT14    | hair follicular keratinocyte                            |      4.37114 |         3372 |       0.992933    |          0.00414985  |               8122 | 2.30136     | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| LGALS3BP | hair follicular keratinocyte                            |      1.57412 |           51 |       0.0150177   |          6.27647e-05 |               8122 | 0.0675741   | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| RBM42    | hair follicular keratinocyte                            |      1.49171 |           22 |       0.00647821  |          2.7075e-05  |               8122 | 0.033804    | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| SCPEP1   | hair follicular keratinocyte                            |      1.64349 |          196 |       0.057715    |          0.000241213 |               8122 | 0.152129    | 0.833631  | 0.596417  | 1.39773  |  1.27561   |  0.513385  |   0.370724  | 0.629286  |       0.304861  |       9 |      3.73278 |
| COL17A1  | melanocyte                                              |      1.80981 |          335 |       0.0277134   |          0.000412278 |              16294 | 0.15134     | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| EFNA1    | melanocyte                                              |      1.86621 |         2059 |       0.170334    |          0.00253397  |              16294 | 0.420083    | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| GAMT     | melanocyte                                              |      1.73909 |          119 |       0.00984447  |          0.000146451 |              16294 | 0.102011    | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| HSPA1A   | melanocyte                                              |      2.75078 |         5364 |       0.443746    |          0.00660137  |              16294 | 1.12628     | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| IFI27    | melanocyte                                              |      2.50522 |         1372 |       0.113501    |          0.00168849  |              16294 | 0.475714    | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| KRT14    | melanocyte                                              |      2.37538 |         3799 |       0.314279    |          0.00467535  |              16294 | 0.802571    | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| LGALS3BP | melanocyte                                              |      1.80847 |         2240 |       0.185308    |          0.00275672  |              16294 | 0.432638    | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| RBM42    | melanocyte                                              |      1.74111 |          862 |       0.0713104   |          0.00106085  |              16294 | 0.21235     | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| SCPEP1   | melanocyte                                              |      1.81156 |          144 |       0.0119126   |          0.000177218 |              16294 | 0.123535    | 0.345013  | 0.427391  | 0.807252 |  0.836688  |  0.184586  |   0.247455  | 0.244397  |       0.264986  |       9 |      3.6344  |
| COL17A1  | sebaceous gland cell                                    |      2.37004 |           11 |       0.0366667   |          1.35375e-05 |                213 | 0.294891    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| EFNA1    | sebaceous gland cell                                    |      2.17396 |           62 |       0.206667    |          7.63022e-05 |                213 | 0.502867    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| GAMT     | sebaceous gland cell                                    |      1.8935  |            1 |       0.00740741  |          1.23068e-06 |                213 | 0.133538    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| HSPA1A   | sebaceous gland cell                                    |      2.6795  |           43 |       0.318519    |          5.29192e-05 |                213 | 0.794554    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| IFI27    | sebaceous gland cell                                    |      2.64172 |            3 |       0.0222222   |          3.69204e-06 |                213 | 0.339865    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| KRT14    | sebaceous gland cell                                    |      2.50666 |           65 |       0.216667    |          7.99942e-05 |                213 | 0.599781    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| LGALS3BP | sebaceous gland cell                                    |      1.91798 |            7 |       0.0233333   |          8.61476e-06 |                213 | 0.16359     | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| RBM42    | sebaceous gland cell                                    |      1.96204 |            3 |       0.01        |          3.69204e-06 |                213 | 0.154336    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| SCPEP1   | sebaceous gland cell                                    |      2.14275 |           18 |       0.06        |          2.21522e-05 |                213 | 0.274146    | 0.226629  | 0.361952  | 0.626129 | -0.0198275 |  0.104924  |   0.199731  | 0.126337  |       0.187173  |       9 |      3.58172 |
| COL17A1  | neuron associated cell                                  |      1.875   |          308 |       0.162877    |          0.000379049 |               7763 | 0.369482    | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| EFNA1    | neuron associated cell                                  |      1.83332 |          538 |       0.0857917   |          0.000662106 |               7763 | 0.249062    | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| GAMT     | neuron associated cell                                  |      1.59755 |          230 |       0.0366768   |          0.000283056 |               7763 | 0.110077    | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| HSPA1A   | neuron associated cell                                  |      3.06618 |         3894 |       0.620954    |          0.00479227  |               7763 | 1.43477     | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| IFI27    | neuron associated cell                                  |      2.30548 |          116 |       0.0184978   |          0.000142759 |               7763 | 0.254249    | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| KRT14    | neuron associated cell                                  |      4.53381 |         1178 |       0.187849    |          0.00144974  |               7763 | 1.08165     | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| LGALS3BP | neuron associated cell                                  |      1.67851 |          227 |       0.0361984   |          0.000279364 |               7763 | 0.129205    | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| RBM42    | neuron associated cell                                  |      1.6439  |          606 |       0.0966353   |          0.000745792 |               7763 | 0.220346    | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| SCPEP1   | neuron associated cell                                  |      1.84467 |          666 |       0.106203    |          0.000819633 |               7763 | 0.28551     | 0.467895  | 0.459373  | 1.01855  |  1.51946   |  0.267275  |   0.270779  | 0.382127  |       0.327014  |       9 |      3.52713 |
| COL17A1  | neural cell                                             |      1.875   |          308 |       0.162877    |          0.000379049 |               8590 | 0.369482    | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| EFNA1    | neural cell                                             |      1.83405 |          542 |       0.0782785   |          0.000667028 |               8590 | 0.238066    | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| GAMT     | neural cell                                             |      1.59977 |          313 |       0.0383203   |          0.000385203 |               8590 | 0.115059    | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| HSPA1A   | neural cell                                             |      3.02641 |         4395 |       0.538075    |          0.00540884  |               8590 | 1.31257     | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| IFI27    | neural cell                                             |      2.30548 |          116 |       0.0184978   |          0.000142759 |               8590 | 0.254249    | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| KRT14    | neural cell                                             |      4.53381 |         1178 |       0.187849    |          0.00144974  |               8590 | 1.08165     | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| LGALS3BP | neural cell                                             |      1.67689 |          240 |       0.034662    |          0.000295363 |               8590 | 0.12681     | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| RBM42    | neural cell                                             |      1.64096 |          729 |       0.0892507   |          0.000897166 |               8590 | 0.211467    | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| SCPEP1   | neural cell                                             |      1.81465 |          769 |       0.0941479   |          0.000946393 |               8590 | 0.262489    | 0.438854  | 0.441316  | 0.994422 |  1.00507   |  0.247733  |   0.25761   | 0.366399  |       0.280282  |       9 |      3.51797 |
| COL17A1  | Schwann cell                                            |      1.94119 |           41 |       0.0545939   |          5.04579e-05 |                623 | 0.216999    | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| EFNA1    | Schwann cell                                            |      1.91843 |          165 |       0.219707    |          0.000203062 |                623 | 0.462013    | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| GAMT     | Schwann cell                                            |      1.74218 |            2 |       0.00266312  |          2.46136e-06 |                623 | 0.0892212   | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| HSPA1A   | Schwann cell                                            |      2.2605  |          103 |       0.13715     |          0.00012676  |                623 | 0.420853    | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| IFI27    | Schwann cell                                            |      2.38399 |          102 |       0.135819    |          0.000125529 |                623 | 0.44921     | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| KRT14    | Schwann cell                                            |      2.09418 |            6 |       0.00798935  |          7.38408e-06 |                623 | 0.183897    | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| LGALS3BP | Schwann cell                                            |      1.78797 |           53 |       0.0705726   |          6.5226e-05  |                623 | 0.20356     | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| RBM42    | Schwann cell                                            |      1.71655 |           25 |       0.0332889   |          3.0767e-05  |                623 | 0.129401    | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| SCPEP1   | Schwann cell                                            |      2.02504 |          126 |       0.167776    |          0.000155066 |                623 | 0.409417    | 0.148431  | 0.284952  | 0.520897 | -2.08013   |  0.0523031 |   0.143576  | 0.0577439 |       0         |       9 |      3.50549 |
| COL17A1  | endothelial cell                                        |      1.82391 |          250 |       0.00571873  |          0.00030767  |              53757 | 0.1198      | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| EFNA1    | endothelial cell                                        |      1.84292 |        12508 |       0.286048    |          0.0153933   |              53757 | 0.836044    | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| GAMT     | endothelial cell                                        |      1.69743 |           38 |       0.000884132 |          4.67658e-05 |              53757 | 0.0763995   | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| HSPA1A   | endothelial cell                                        |      2.40738 |         9797 |       0.224049    |          0.012057    |              53757 | 0.817526    | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| IFI27    | endothelial cell                                        |      2.81868 |        26662 |       0.609738    |          0.0328124   |              53757 | 1.89771     | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| KRT14    | endothelial cell                                        |      2.68658 |         1369 |       0.031852    |          0.0016848   |              53757 | 0.397781    | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| LGALS3BP | endothelial cell                                        |      1.7489  |          125 |       0.00285936  |          0.000153835 |              53757 | 0.094089    | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| RBM42    | endothelial cell                                        |      1.62469 |         2757 |       0.0630503   |          0.00339298  |              53757 | 0.21632     | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| SCPEP1   | endothelial cell                                        |      1.68267 |          251 |       0.0057416   |          0.000308901 |              53757 | 0.0851121   | 0.604044  | 0.504531  | 1.19724  |  3.29707   |  0.358893  |   0.303712  | 0.4986    |       0.488505  |       9 |      3.40199 |
| COL17A1  | sebum secreting cell                                    |      1.82275 |            1 |       0.00740741  |          1.23068e-06 |                117 | 0.116135    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| EFNA1    | sebum secreting cell                                    |      2.19523 |           55 |       0.407407    |          6.76874e-05 |                117 | 0.809045    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| GAMT     | sebum secreting cell                                    |      1.8935  |            1 |       0.00740741  |          1.23068e-06 |                117 | 0.133538    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| HSPA1A   | sebum secreting cell                                    |      2.6795  |           43 |       0.318519    |          5.29192e-05 |                117 | 0.794554    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| IFI27    | sebum secreting cell                                    |      2.64172 |            3 |       0.0222222   |          3.69204e-06 |                117 | 0.339865    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| KRT14    | sebum secreting cell                                    |      3.17732 |            2 |       0.0148148   |          2.46136e-06 |                117 | 0.460484    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| LGALS3BP | sebum secreting cell                                    |      1.85507 |            6 |       0.0444444   |          7.38408e-06 |                117 | 0.179759    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| RBM42    | sebum secreting cell                                    |      1.67727 |            1 |       0.00740741  |          1.23068e-06 |                117 | 0.0803468   | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| SCPEP1   | sebum secreting cell                                    |      1.92974 |            5 |       0.037037    |          6.1534e-06  |                117 | 0.186991    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| COL17A1  | exocrine cell                                           |      1.82275 |            1 |       0.00740741  |          1.23068e-06 |                117 | 0.116135    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| EFNA1    | exocrine cell                                           |      2.19523 |           55 |       0.407407    |          6.76874e-05 |                117 | 0.809045    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| GAMT     | exocrine cell                                           |      1.8935  |            1 |       0.00740741  |          1.23068e-06 |                117 | 0.133538    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| HSPA1A   | exocrine cell                                           |      2.6795  |           43 |       0.318519    |          5.29192e-05 |                117 | 0.794554    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| IFI27    | exocrine cell                                           |      2.64172 |            3 |       0.0222222   |          3.69204e-06 |                117 | 0.339865    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| KRT14    | exocrine cell                                           |      3.17732 |            2 |       0.0148148   |          2.46136e-06 |                117 | 0.460484    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| LGALS3BP | exocrine cell                                           |      1.85507 |            6 |       0.0444444   |          7.38408e-06 |                117 | 0.179759    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| RBM42    | exocrine cell                                           |      1.67727 |            1 |       0.00740741  |          1.23068e-06 |                117 | 0.0803468   | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| SCPEP1   | exocrine cell                                           |      1.92974 |            5 |       0.037037    |          6.1534e-06  |                117 | 0.186991    | 0.285089  | 0.344524  | 0.827486 | -0.569851  |  0.144263  |   0.187021  | 0.257586  |       0.137205  |       9 |      3.39308 |
| COL17A1  | endothelial cell of vascular tree                       |      1.83058 |          235 |       0.00610659  |          0.00028921  |              47624 | 0.121666    | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| EFNA1    | endothelial cell of vascular tree                       |      1.83957 |        11238 |       0.291942    |          0.0138304   |              47624 | 0.813904    | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| GAMT     | endothelial cell of vascular tree                       |      1.66883 |           26 |       0.000688796 |          3.19977e-05 |              47624 | 0.0687861   | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| HSPA1A   | endothelial cell of vascular tree                       |      2.21813 |         7679 |       0.199486    |          0.00945039  |              47624 | 0.683833    | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| IFI27    | endothelial cell of vascular tree                       |      2.84035 |        24309 |       0.631501    |          0.0299166   |              47624 | 1.87981     | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| KRT14    | endothelial cell of vascular tree                       |      2.69891 |         1316 |       0.0348637   |          0.00161957  |              47624 | 0.404074    | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| LGALS3BP | endothelial cell of vascular tree                       |      1.72881 |          107 |       0.00278045  |          0.000131683 |              47624 | 0.0886016   | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| RBM42    | endothelial cell of vascular tree                       |      1.62413 |         2503 |       0.0650231   |          0.00308039  |              47624 | 0.213109    | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| SCPEP1   | endothelial cell of vascular tree                       |      1.68699 |          211 |       0.00548294  |          0.000259673 |              47624 | 0.0848379   | 0.591205  | 0.484291  | 1.22076  |  3.99833   |  0.350253  |   0.288952  | 0.513933  |       0.552212  |       9 |      3.24564 |
| COL17A1  | stromal cell                                            |      1.72962 |           46 |       0.0148819   |          5.66113e-05 |               1014 | 0.105505    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| EFNA1    | stromal cell                                            |      1.89409 |          322 |       0.104173    |          0.000396279 |               1014 | 0.286455    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| GAMT     | stromal cell                                            |      2.47744 |           40 |       0.0129408   |          4.92272e-05 |               1014 | 0.286409    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| HSPA1A   | stromal cell                                            |      2.88942 |          417 |       0.134908    |          0.000513194 |               1014 | 0.579656    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| IFI27    | stromal cell                                            |      2.2656  |            9 |       0.00291168  |          1.10761e-05 |               1014 | 0.218519    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| KRT14    | stromal cell                                            |      1.76955 |           10 |       0.0032352   |          1.23068e-05 |               1014 | 0.097001    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| LGALS3BP | stromal cell                                            |      2.12996 |           21 |       0.00679392  |          2.58443e-05 |               1014 | 0.19126     | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| RBM42    | stromal cell                                            |      2.54413 |           76 |       0.0245875   |          9.35317e-05 |               1014 | 0.321141    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| SCPEP1   | stromal cell                                            |      2.57798 |           73 |       0.023617    |          8.98396e-05 |               1014 | 0.327939    | 0.145004  | 0.268209  | 0.540636 |  2.11195   |  0.0499969 |   0.131365  | 0.0706102 |       0.38084   |       9 |      3.0763  |
| COL17A1  | endothelial cell of lymphatic vessel                    |      1.69263 |           23 |       0.00365776  |          2.83056e-05 |               5230 | 0.0790219   | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| EFNA1    | endothelial cell of lymphatic vessel                    |      1.92953 |         2155 |       0.342716    |          0.00265212  |               5230 | 0.696513    | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| GAMT     | endothelial cell of lymphatic vessel                    |      1.69216 |           14 |       0.00222646  |          1.72295e-05 |               5230 | 0.0765454   | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| HSPA1A   | endothelial cell of lymphatic vessel                    |      2.61616 |         2071 |       0.329358    |          0.00254874  |               5230 | 0.843387    | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| IFI27    | endothelial cell of lymphatic vessel                    |      1.83856 |          215 |       0.0341921   |          0.000264596 |               5230 | 0.165281    | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| KRT14    | endothelial cell of lymphatic vessel                    |      2.61177 |          255 |       0.0405534   |          0.000313823 |               5230 | 0.365978    | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| LGALS3BP | endothelial cell of lymphatic vessel                    |      1.78494 |           27 |       0.00429389  |          3.32284e-05 |               5230 | 0.102779    | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| RBM42    | endothelial cell of lymphatic vessel                    |      1.66889 |          386 |       0.0613868   |          0.000475042 |               5230 | 0.168397    | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| SCPEP1   | endothelial cell of lymphatic vessel                    |      1.67807 |           84 |       0.0133588   |          0.000103377 |               5230 | 0.0914403   | 0.289851  | 0.287705  | 1.00746  |  0.372594  |  0.147467  |   0.145583  | 0.374897  |       0.222824  |       9 |      2.9465  |
| COL17A1  | melanocyte of skin                                      |      1.75148 |          183 |       0.071821    |          0.000225214 |                977 | 0.199545    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| EFNA1    | melanocyte of skin                                      |      1.78617 |          628 |       0.246468    |          0.000772867 |                977 | 0.480615    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| GAMT     | melanocyte of skin                                      |      2.60789 |            2 |       0.000784929 |          2.46136e-06 |                977 | 0.299364    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| HSPA1A   | melanocyte of skin                                      |      2.05244 |           40 |       0.0156986   |          4.92272e-05 |                977 | 0.185999    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| IFI27    | melanocyte of skin                                      |      1.77584 |           15 |       0.00588697  |          1.84602e-05 |                977 | 0.102645    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| KRT14    | melanocyte of skin                                      |      2.51294 |           44 |       0.0172684   |          5.41499e-05 |                977 | 0.301729    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| LGALS3BP | melanocyte of skin                                      |      1.8637  |           51 |       0.0200157   |          6.27647e-05 |                977 | 0.146308    | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| RBM42    | melanocyte of skin                                      |      1.75981 |           10 |       0.00392465  |          1.23068e-05 |                977 | 0.0956396   | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| SCPEP1   | melanocyte of skin                                      |      1.78054 |            4 |       0.00156986  |          4.92272e-06 |                977 | 0.0970663   | 0.128303  | 0.212101  | 0.604912 |  1.20392   |  0.0387586 |   0.0904462 | 0.112507  |       0.298347  |       9 |      2.86516 |
| COL17A1  | supporting cell                                         |      1.75195 |          386 |       0.0215642   |          0.000475042 |               8252 | 0.129094    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| EFNA1    | supporting cell                                         |      1.73515 |         1701 |       0.0950279   |          0.00209339  |               8252 | 0.266382    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| GAMT     | supporting cell                                         |      1.68498 |          131 |       0.00731844  |          0.000161219 |               8252 | 0.0851974   | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| HSPA1A   | supporting cell                                         |      2.289   |         2573 |       0.143743    |          0.00316654  |               8252 | 0.496405    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| IFI27    | supporting cell                                         |      2.02098 |          786 |       0.0439106   |          0.000967314 |               8252 | 0.238293    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| KRT14    | supporting cell                                         |      2.32868 |          889 |       0.0496648   |          0.00109407  |               8252 | 0.325062    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| LGALS3BP | supporting cell                                         |      1.75297 |          688 |       0.0384358   |          0.000846708 |               8252 | 0.161825    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| RBM42    | supporting cell                                         |      1.66225 |          849 |       0.0474302   |          0.00104485  |               8252 | 0.156823    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| SCPEP1   | supporting cell                                         |      1.66625 |          249 |       0.0139106   |          0.000306439 |               8252 | 0.093279    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| COL17A1  | pericyte                                                |      1.75195 |          386 |       0.0215642   |          0.000475042 |               8252 | 0.129094    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| EFNA1    | pericyte                                                |      1.73515 |         1701 |       0.0950279   |          0.00209339  |               8252 | 0.266382    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| GAMT     | pericyte                                                |      1.68498 |          131 |       0.00731844  |          0.000161219 |               8252 | 0.0851974   | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| HSPA1A   | pericyte                                                |      2.289   |         2573 |       0.143743    |          0.00316654  |               8252 | 0.496405    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| IFI27    | pericyte                                                |      2.02098 |          786 |       0.0439106   |          0.000967314 |               8252 | 0.238293    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| KRT14    | pericyte                                                |      2.32868 |          889 |       0.0496648   |          0.00109407  |               8252 | 0.325062    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| LGALS3BP | pericyte                                                |      1.75297 |          688 |       0.0384358   |          0.000846708 |               8252 | 0.161825    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| RBM42    | pericyte                                                |      1.66225 |          849 |       0.0474302   |          0.00104485  |               8252 | 0.156823    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| SCPEP1   | pericyte                                                |      1.66625 |          249 |       0.0139106   |          0.000306439 |               8252 | 0.093279    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| COL17A1  | mural cell                                              |      1.75195 |          386 |       0.0215642   |          0.000475042 |               8252 | 0.129094    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| EFNA1    | mural cell                                              |      1.73515 |         1701 |       0.0950279   |          0.00209339  |               8252 | 0.266382    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| GAMT     | mural cell                                              |      1.68498 |          131 |       0.00731844  |          0.000161219 |               8252 | 0.0851974   | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| HSPA1A   | mural cell                                              |      2.289   |         2573 |       0.143743    |          0.00316654  |               8252 | 0.496405    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| IFI27    | mural cell                                              |      2.02098 |          786 |       0.0439106   |          0.000967314 |               8252 | 0.238293    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| KRT14    | mural cell                                              |      2.32868 |          889 |       0.0496648   |          0.00109407  |               8252 | 0.325062    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| LGALS3BP | mural cell                                              |      1.75297 |          688 |       0.0384358   |          0.000846708 |               8252 | 0.161825    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| RBM42    | mural cell                                              |      1.66225 |          849 |       0.0474302   |          0.00104485  |               8252 | 0.156823    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| SCPEP1   | mural cell                                              |      1.66625 |          249 |       0.0139106   |          0.000306439 |               8252 | 0.093279    | 0.13215   | 0.216929  | 0.609185 |  1.45388   |  0.0413474 |   0.0939672 | 0.115292  |       0.321056  |       9 |      2.86207 |
| COL17A1  | hematopoietic cell                                      |      1.95432 |         3367 |       0.0142786   |          0.0041437   |             135695 | 0.238734    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| EFNA1    | hematopoietic cell                                      |      1.73061 |         8466 |       0.0296338   |          0.0104189   |             135695 | 0.327816    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| GAMT     | hematopoietic cell                                      |      1.64297 |         3563 |       0.0121022   |          0.00438491  |             135695 | 0.163534    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| HSPA1A   | hematopoietic cell                                      |      3.26665 |        92787 |       0.302484    |          0.114191    |             135695 | 3.11721     | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| IFI27    | hematopoietic cell                                      |      2.05496 |         1758 |       0.00701402  |          0.00216354  |             135695 | 0.214388    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| KRT14    | hematopoietic cell                                      |      2.21316 |         4626 |       0.0162531   |          0.00569313  |             135695 | 0.335266    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| LGALS3BP | hematopoietic cell                                      |      1.68439 |         1722 |       0.00575106  |          0.00211923  |             135695 | 0.12048     | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| RBM42    | hematopoietic cell                                      |      1.67137 |        14976 |       0.0488668   |          0.0184307   |             135695 | 0.496677    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| SCPEP1   | hematopoietic cell                                      |      1.6628  |         4430 |       0.0144593   |          0.00545191  |             135695 | 0.192534    | 0.958692  | 0.578516  | 1.65716  |  8.64247   |  0.59754   |   0.357669  | 0.798386  |       0.97412   |       9 |      2.82027 |
| COL17A1  | leukocyte                                               |      1.95615 |         3337 |       0.0142609   |          0.00410678  |             133052 | 0.238446    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| EFNA1    | leukocyte                                               |      1.73047 |         8461 |       0.0298044   |          0.0104128   |             133052 | 0.327918    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| GAMT     | leukocyte                                               |      1.64267 |         3462 |       0.0119037   |          0.00426061  |             133052 | 0.160763    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| HSPA1A   | leukocyte                                               |      3.26645 |        91218 |       0.300922    |          0.11226     |             133052 | 3.07756     | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| IFI27    | leukocyte                                               |      2.05907 |         1736 |       0.00697878  |          0.00213646  |             133052 | 0.214822    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| KRT14    | leukocyte                                               |      2.17483 |         3953 |       0.0139753   |          0.00486488  |             133052 | 0.306438    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| LGALS3BP | leukocyte                                               |      1.68396 |         1713 |       0.00575978  |          0.00210815  |             133052 | 0.120175    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| RBM42    | leukocyte                                               |      1.67132 |        14848 |       0.048997    |          0.0182731   |             133052 | 0.49382     | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| SCPEP1   | leukocyte                                               |      1.66265 |         4324 |       0.0142678   |          0.00532146  |             133052 | 0.189694    | 0.946855  | 0.56996   | 1.66127  |  8.64725   |  0.589575  |   0.351429  | 0.801063  |       0.974554  |       9 |      2.78533 |
| COL17A1  | mononuclear cell                                        |      1.95482 |         3301 |       0.0144308   |          0.00406247  |             131093 | 0.237518    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| EFNA1    | mononuclear cell                                        |      1.7273  |         8293 |       0.0297544   |          0.010206    |             131093 | 0.323073    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| GAMT     | mononuclear cell                                        |      1.6418  |         3395 |       0.0118935   |          0.00417816  |             131093 | 0.158942    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| HSPA1A   | mononuclear cell                                        |      3.27094 |        90001 |       0.302367    |          0.110762    |             131093 | 3.05194     | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| IFI27    | mononuclear cell                                        |      2.06079 |         1714 |       0.00703656  |          0.00210939  |             131093 | 0.214811    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| KRT14    | mononuclear cell                                        |      2.17744 |         3895 |       0.0140108   |          0.0047935   |             131093 | 0.305756    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| LGALS3BP | mononuclear cell                                        |      1.68219 |         1695 |       0.0057937   |          0.002086    |             131093 | 0.119361    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| RBM42    | mononuclear cell                                        |      1.67086 |        14549 |       0.0488934   |          0.0179052   |             131093 | 0.486452    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| SCPEP1   | mononuclear cell                                        |      1.65891 |         4250 |       0.0142812   |          0.00523039  |             131093 | 0.187036    | 0.938947  | 0.564988  | 1.66189  |  8.65323   |  0.584254  |   0.347803  | 0.801469  |       0.975098  |       9 |      2.76642 |
| COL17A1  | monocyte-derived dendritic cell                         |      1.89074 |            1 |       8.1314e-05  |          1.23068e-06 |               1769 | 0.121871    | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| EFNA1    | monocyte-derived dendritic cell                         |      1.59082 |            1 |       8.1314e-05  |          1.23068e-06 |               1769 | 0.0480913   | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| GAMT     | monocyte-derived dendritic cell                         |      1.4534  |            1 |       8.1314e-05  |          1.23068e-06 |               1769 | 0.0142853   | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| HSPA1A   | monocyte-derived dendritic cell                         |      1.80784 |         1560 |       0.12685     |          0.00191986  |               1769 | 0.328649    | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| IFI27    | monocyte-derived dendritic cell                         |      2.39792 |           54 |       0.00439096  |          6.64567e-05 |               1769 | 0.254356    | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| KRT14    | monocyte-derived dendritic cell                         |      2.34077 |           26 |       0.00211416  |          3.19977e-05 |               1769 | 0.236218    | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| LGALS3BP | monocyte-derived dendritic cell                         |      1.7421  |            2 |       0.000162628 |          2.46136e-06 |               1769 | 0.0854512   | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| RBM42    | monocyte-derived dendritic cell                         |      1.60052 |          112 |       0.00910717  |          0.000137836 |               1769 | 0.0666515   | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| SCPEP1   | monocyte-derived dendritic cell                         |      1.61067 |           12 |       0.000975768 |          1.47682e-05 |               1769 | 0.0545757   | 0.110612  | 0.134461  | 0.822634 | -0.871617  |  0.0268543 |   0.0338239 | 0.254423  |       0.109791  |       9 |      2.56113 |
| COL17A1  | myeloid dendritic cell                                  |      1.89074 |            1 |       8.1314e-05  |          1.23068e-06 |               1778 | 0.121871    | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| EFNA1    | myeloid dendritic cell                                  |      1.59082 |            1 |       8.1314e-05  |          1.23068e-06 |               1778 | 0.0480913   | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| GAMT     | myeloid dendritic cell                                  |      1.4534  |            1 |       8.1314e-05  |          1.23068e-06 |               1778 | 0.0142853   | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| HSPA1A   | myeloid dendritic cell                                  |      1.81085 |         1568 |       0.12639     |          0.00192971  |               1778 | 0.328891    | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| IFI27    | myeloid dendritic cell                                  |      2.39792 |           54 |       0.00439096  |          6.64567e-05 |               1778 | 0.254356    | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| KRT14    | myeloid dendritic cell                                  |      2.34077 |           26 |       0.00211416  |          3.19977e-05 |               1778 | 0.236218    | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| LGALS3BP | myeloid dendritic cell                                  |      1.7421  |            2 |       0.000162628 |          2.46136e-06 |               1778 | 0.0854512   | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| RBM42    | myeloid dendritic cell                                  |      1.60052 |          112 |       0.00910717  |          0.000137836 |               1778 | 0.0666515   | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| SCPEP1   | myeloid dendritic cell                                  |      1.59358 |           13 |       0.00104788  |          1.59988e-05 |               1778 | 0.0505052   | 0.11104   | 0.134036  | 0.82844  | -0.879714  |  0.0271425 |   0.0335137 | 0.258208  |       0.109055  |       9 |      2.55617 |
| COL17A1  | erythroid lineage cell                                  |      1.56496 |            4 |       0.00396825  |          4.92272e-06 |                435 | 0.0476319   | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| EFNA1    | erythroid lineage cell                                  |      2.23455 |            3 |       0.00287356  |          3.69204e-06 |                435 | 0.210681    | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| GAMT     | erythroid lineage cell                                  |      2.04087 |           10 |       0.00933707  |          1.23068e-05 |                435 | 0.172899    | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| HSPA1A   | erythroid lineage cell                                  |      2.8578  |          320 |       0.298786    |          0.000393818 |                435 | 0.815394    | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| IFI27    | erythroid lineage cell                                  |      1.70169 |           12 |       0.0119048   |          1.47682e-05 |                435 | 0.0933603   | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| KRT14    | erythroid lineage cell                                  |      3.04299 |           34 |       0.0337302   |          4.18431e-05 |                435 | 0.456574    | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| LGALS3BP | erythroid lineage cell                                  |      2.00253 |            2 |       0.00193237  |          2.46136e-06 |                435 | 0.15217     | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| RBM42    | erythroid lineage cell                                  |      1.81088 |           38 |       0.0354809   |          4.67658e-05 |                435 | 0.156201    | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| SCPEP1   | erythroid lineage cell                                  |      1.77934 |           12 |       0.0112045   |          1.47682e-05 |                435 | 0.111413    | 0.243098  | 0.246258  | 0.987166 |  3.82586   |  0.116006  |   0.115357  | 0.361669  |       0.536544  |       9 |      2.51395 |
| COL17A1  | perivascular cell                                       |      1.75119 |          387 |       0.0141686   |          0.000476273 |              15137 | 0.117838    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| EFNA1    | perivascular cell                                       |      1.72517 |         1846 |       0.0675794   |          0.00227184  |              15137 | 0.226199    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| GAMT     | perivascular cell                                       |      1.61781 |          665 |       0.0243465   |          0.000818402 |              15137 | 0.106894    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| HSPA1A   | perivascular cell                                       |      2.35917 |         6126 |       0.224281    |          0.00753914  |              15137 | 0.718842    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| IFI27    | perivascular cell                                       |      2.01735 |          792 |       0.0289961   |          0.000974698 |              15137 | 0.21517     | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| KRT14    | perivascular cell                                       |      2.32948 |          890 |       0.032584    |          0.00109531  |              15137 | 0.29966     | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| LGALS3BP | perivascular cell                                       |      1.69274 |         1848 |       0.0676576   |          0.0022743   |              15137 | 0.218386    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| RBM42    | perivascular cell                                       |      1.64076 |         1926 |       0.0705133   |          0.00237029  |              15137 | 0.211734    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| SCPEP1   | perivascular cell                                       |      1.62932 |          657 |       0.0240518   |          0.000808557 |              15137 | 0.109094    | 0.188496  | 0.247091  | 0.762862 |  6.15764   |  0.0792639 |   0.115964  | 0.215463  |       0.748379  |       9 |      2.49178 |
| COL17A1  | vascular associated smooth muscle cell                  |      1.45844 |            1 |       0.000106225 |          1.23068e-06 |               6885 | 0.0155633   | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| EFNA1    | vascular associated smooth muscle cell                  |      1.60818 |          145 |       0.0153993   |          0.000178449 |               6885 | 0.078757    | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| GAMT     | vascular associated smooth muscle cell                  |      1.60133 |          534 |       0.056724    |          0.000657183 |               6885 | 0.148296    | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| HSPA1A   | vascular associated smooth muscle cell                  |      2.40999 |         3553 |       0.377417    |          0.00437261  |               6885 | 0.899949    | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| IFI27    | vascular associated smooth muscle cell                  |      1.54129 |            6 |       0.000637349 |          7.38408e-06 |               6885 | 0.0368588   | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| KRT14    | vascular associated smooth muscle cell                  |      3.03553 |            1 |       0.000106225 |          1.23068e-06 |               6885 | 0.40352     | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| LGALS3BP | vascular associated smooth muscle cell                  |      1.65702 |         1160 |       0.123221    |          0.00142759  |               6885 | 0.276606    | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| RBM42    | vascular associated smooth muscle cell                  |      1.62381 |         1077 |       0.114404    |          0.00132544  |               6885 | 0.253242    | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| SCPEP1   | vascular associated smooth muscle cell                  |      1.60678 |          408 |       0.0433305   |          0.000502117 |               6885 | 0.126554    | 0.274318  | 0.248816  | 1.10249  |  4.24896   |  0.137015  |   0.117222  | 0.436842  |       0.574981  |       9 |      2.40455 |
| COL17A1  | contractile cell                                        |      1.75993 |          397 |       0.0141286   |          0.00048858  |              19951 | 0.120166    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| EFNA1    | contractile cell                                        |      1.72989 |         1927 |       0.0574881   |          0.00237152  |              19951 | 0.214145    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| GAMT     | contractile cell                                        |      1.61579 |          813 |       0.0242556   |          0.00100054  |              19951 | 0.109777    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| HSPA1A   | contractile cell                                        |      2.51    |         9671 |       0.287528    |          0.0119019   |              19951 | 0.934997    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| IFI27    | contractile cell                                        |      2.01714 |          830 |       0.0246767   |          0.00102146  |              19951 | 0.209542    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| KRT14    | contractile cell                                        |      2.32184 |          905 |       0.0269065   |          0.00111377  |              19951 | 0.289623    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| LGALS3BP | contractile cell                                        |      1.69473 |         2345 |       0.069719    |          0.00288594  |              19951 | 0.23377     | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| RBM42    | contractile cell                                        |      1.63529 |         2147 |       0.0640551   |          0.00264227  |              19951 | 0.205949    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| SCPEP1   | contractile cell                                        |      1.62969 |          916 |       0.0272319   |          0.0011273   |              19951 | 0.120105    | 0.256266  | 0.270897  | 0.945991 |  7.61748   |  0.124867  |   0.133325  | 0.33483   |       0.881002  |       9 |      2.33332 |
| COL17A1  | lymphocyte                                              |      1.96358 |         3146 |       0.0246708   |          0.00387172  |              83193 | 0.251352    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| EFNA1    | lymphocyte                                              |      1.73057 |         7024 |       0.0395747   |          0.0086443   |              83193 | 0.308475    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| GAMT     | lymphocyte                                              |      1.5942  |          992 |       0.00541254  |          0.00122083  |              83193 | 0.0804518   | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| HSPA1A   | lymphocyte                                              |      3.22827 |        54165 |       0.277291    |          0.0666598   |              83193 | 2.15287     | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| IFI27    | lymphocyte                                              |      1.69849 |          638 |       0.00448084  |          0.000785174 |              83193 | 0.096302    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| KRT14    | lymphocyte                                              |      1.90213 |         2073 |       0.0115932   |          0.0025512   |              83193 | 0.191141    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| LGALS3BP | lymphocyte                                              |      1.66466 |         1190 |       0.00621835  |          0.00146451  |              83193 | 0.103695    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| RBM42    | lymphocyte                                              |      1.66632 |        11566 |       0.0592017   |          0.014234    |              83193 | 0.429964    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| SCPEP1   | lymphocyte                                              |      1.63094 |         2399 |       0.0122845   |          0.0029524   |              83193 | 0.133207    | 0.661293  | 0.416385  | 1.58818  |  8.22388   |  0.397416  |   0.239428  | 0.753422  |       0.936092  |       9 |      2.33016 |
| COL17A1  | blood cell                                              |      2.06895 |           11 |       0.0082769   |          1.35375e-05 |                564 | 0.178239    | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| EFNA1    | blood cell                                              |      1.63199 |            2 |       0.00150489  |          2.46136e-06 |                564 | 0.0603788   | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| GAMT     | blood cell                                              |      1.67718 |           17 |       0.0110104   |          2.09216e-05 |                564 | 0.086108    | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| HSPA1A   | blood cell                                              |      2.68595 |          426 |       0.274485    |          0.00052427  |                564 | 0.739183    | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| IFI27    | blood cell                                              |      1.68766 |           13 |       0.00978179  |          1.59988e-05 |                564 | 0.0867477   | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| KRT14    | blood cell                                              |      3.00716 |           35 |       0.0344488   |          4.30738e-05 |                564 | 0.448862    | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| LGALS3BP | blood cell                                              |      1.39773 |            1 |       0.000992063 |          1.23068e-06 |                564 | 0.00195747  | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| RBM42    | blood cell                                              |      1.65671 |           45 |       0.0289948   |          5.53806e-05 |                564 | 0.108715    | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| SCPEP1   | blood cell                                              |      1.73129 |           14 |       0.00906736  |          1.72295e-05 |                564 | 0.0964341   | 0.238927  | 0.200736  | 1.19026  |  2.85299   |  0.1132    |   0.0821578 | 0.494047  |       0.448161  |       9 |      2.26773 |
| COL17A1  | T cell                                                  |      1.99123 |         2814 |       0.0294019   |          0.00346313  |              66842 | 0.257367    | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| EFNA1    | T cell                                                  |      1.71713 |         4708 |       0.0341325   |          0.00579404  |              66842 | 0.24201     | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| GAMT     | T cell                                                  |      1.59456 |          687 |       0.00484803  |          0.000845477 |              66842 | 0.0724519   | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| HSPA1A   | T cell                                                  |      3.19788 |        43444 |       0.282945    |          0.0534657   |              66842 | 1.8993      | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| IFI27    | T cell                                                  |      1.71346 |          522 |       0.00479489  |          0.000642415 |              66842 | 0.0977011   | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| KRT14    | T cell                                                  |      1.91405 |         1955 |       0.0132992   |          0.00240598  |              66842 | 0.193831    | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| LGALS3BP | T cell                                                  |      1.65952 |         1015 |       0.00667957  |          0.00124914  |              66842 | 0.0989657   | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| RBM42    | T cell                                                  |      1.67018 |         9967 |       0.0649012   |          0.0122662   |              66842 | 0.401493    | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| SCPEP1   | T cell                                                  |      1.62645 |         1730 |       0.0112673   |          0.00212908  |              66842 | 0.114692    | 0.581047  | 0.375313  | 1.54817  |  8.16666   |  0.343417  |   0.209475  | 0.727344  |       0.930894  |       9 |      2.2182  |
| COL17A1  | myeloid cell                                            |      1.81394 |          161 |       0.00179068  |          0.000198139 |              45633 | 0.109342    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| EFNA1    | myeloid cell                                            |      1.75033 |         1069 |       0.0118907   |          0.0013156   |              45633 | 0.130404    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| GAMT     | myeloid cell                                            |      1.66409 |         2424 |       0.0265292   |          0.00298317  |              45633 | 0.163322    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| HSPA1A   | myeloid cell                                            |      3.29825 |        33711 |       0.368149    |          0.0414874   |              45633 | 1.82068     | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| IFI27    | myeloid cell                                            |      2.27487 |         1065 |       0.0118545   |          0.00131067  |              45633 | 0.259288    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| KRT14    | myeloid cell                                            |      2.50277 |         1813 |       0.020245    |          0.00223122  |              45633 | 0.345699    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| LGALS3BP | myeloid cell                                            |      1.72185 |          494 |       0.00551746  |          0.000607956 |              45633 | 0.100184    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| RBM42    | myeloid cell                                            |      1.68895 |         3022 |       0.0330906   |          0.00371911  |              45633 | 0.193482    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| SCPEP1   | myeloid cell                                            |      1.69823 |         1874 |       0.0205031   |          0.00230629  |              45633 | 0.149622    | 0.551989  | 0.363558  | 1.5183   |  8.48308   |  0.323864  |   0.200902  | 0.707873  |       0.95964   |       9 |      2.17344 |
| COL17A1  | myeloid leukocyte                                       |      1.82266 |          154 |       0.0017331   |          0.000189525 |              44570 | 0.111233    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| EFNA1    | myeloid leukocyte                                       |      1.74897 |         1066 |       0.0119967   |          0.0013119   |              44570 | 0.130157    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| GAMT     | myeloid leukocyte                                       |      1.66388 |         2358 |       0.0264807   |          0.00290194  |              44570 | 0.161631    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| HSPA1A   | myeloid leukocyte                                       |      3.30568 |        32937 |       0.369249    |          0.0405349   |              44570 | 1.80578     | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| IFI27    | myeloid leukocyte                                       |      2.2814  |         1053 |       0.011854    |          0.00129591  |              44570 | 0.260609    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| KRT14    | myeloid leukocyte                                       |      2.49245 |         1779 |       0.0200915   |          0.00218938  |              44570 | 0.342122    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| LGALS3BP | myeloid leukocyte                                       |      1.72071 |          492 |       0.00555938  |          0.000605494 |              44570 | 0.0999186   | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| RBM42    | myeloid leukocyte                                       |      1.68817 |         2931 |       0.0329026   |          0.00360712  |              44570 | 0.190846    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| SCPEP1   | myeloid leukocyte                                       |      1.69969 |         1800 |       0.0201837   |          0.00221522  |              44570 | 0.147744    | 0.547247  | 0.361115  | 1.51543  |  8.48512   |  0.320672  |   0.199121  | 0.706007  |       0.959825  |       9 |      2.16661 |
| COL17A1  | glial cell                                              |      1.94119 |           41 |       0.0545939   |          5.04579e-05 |               4616 | 0.216999    | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| EFNA1    | glial cell                                              |      1.89938 |          173 |       0.0337166   |          0.000212908 |               4616 | 0.178532    | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| GAMT     | glial cell                                              |      1.59641 |          228 |       0.0444358   |          0.000280595 |               4616 | 0.121387    | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| HSPA1A   | glial cell                                              |      3.26272 |         3004 |       0.585461    |          0.00369696  |               4616 | 1.40875     | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| IFI27    | glial cell                                              |      2.31001 |          115 |       0.0224128   |          0.000141528 |               4616 | 0.261211    | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| KRT14    | glial cell                                              |      1.70654 |           39 |       0.00760086  |          4.79965e-05 |               4616 | 0.0887382   | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| LGALS3BP | glial cell                                              |      1.67905 |          217 |       0.042292    |          0.000267058 |               4616 | 0.13824     | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| RBM42    | glial cell                                              |      1.64565 |          590 |       0.114987    |          0.000726101 |               4616 | 0.247924    | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| SCPEP1   | glial cell                                              |      1.85858 |          209 |       0.0407328   |          0.000257212 |               4616 | 0.179874    | 0.413836  | 0.315739  | 1.31069  |  8.51659   |  0.230898  |   0.166028  | 0.572549  |       0.962684  |       9 |      2.15646 |
| COL17A1  | smooth muscle cell                                      |      1.56376 |            7 |       0.000694307 |          8.61476e-06 |               7125 | 0.042496    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| EFNA1    | smooth muscle cell                                      |      1.70133 |          191 |       0.0189409   |          0.00023506  |               7125 | 0.108077    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| GAMT     | smooth muscle cell                                      |      1.60753 |          538 |       0.0533624   |          0.000662106 |               7125 | 0.144874    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| HSPA1A   | smooth muscle cell                                      |      2.4113  |         3677 |       0.364709    |          0.00452521  |               7125 | 0.884156    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| IFI27    | smooth muscle cell                                      |      2.00899 |           34 |       0.00337235  |          4.18431e-05 |               7125 | 0.156679    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| KRT14    | smooth muscle cell                                      |      2.07262 |           10 |       0.000991867 |          1.23068e-05 |               7125 | 0.168191    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| LGALS3BP | smooth muscle cell                                      |      1.6601  |         1171 |       0.116148    |          0.00144113  |               7125 | 0.267015    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| RBM42    | smooth muscle cell                                      |      1.62437 |         1085 |       0.107618    |          0.00133529  |               7125 | 0.243389    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| SCPEP1   | smooth muscle cell                                      |      1.61323 |          412 |       0.0408568   |          0.00050704  |               7125 | 0.124527    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| COL17A1  | muscle cell                                             |      1.56376 |            7 |       0.000694307 |          8.61476e-06 |               7125 | 0.042496    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| EFNA1    | muscle cell                                             |      1.70133 |          191 |       0.0189409   |          0.00023506  |               7125 | 0.108077    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| GAMT     | muscle cell                                             |      1.60753 |          538 |       0.0533624   |          0.000662106 |               7125 | 0.144874    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| HSPA1A   | muscle cell                                             |      2.4113  |         3677 |       0.364709    |          0.00452521  |               7125 | 0.884156    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| IFI27    | muscle cell                                             |      2.00899 |           34 |       0.00337235  |          4.18431e-05 |               7125 | 0.156679    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| KRT14    | muscle cell                                             |      2.07262 |           10 |       0.000991867 |          1.23068e-05 |               7125 | 0.168191    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| LGALS3BP | muscle cell                                             |      1.6601  |         1171 |       0.116148    |          0.00144113  |               7125 | 0.267015    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| RBM42    | muscle cell                                             |      1.62437 |         1085 |       0.107618    |          0.00133529  |               7125 | 0.243389    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| SCPEP1   | muscle cell                                             |      1.61323 |          412 |       0.0408568   |          0.00050704  |               7125 | 0.124527    | 0.251626  | 0.237712  | 1.05853  |  7.23319   |  0.121745  |   0.109124  | 0.408189  |       0.84609   |       9 |      2.14906 |
| COL17A1  | macrophage                                              |      1.64287 |           76 |       0.00150337  |          9.35317e-05 |              30751 | 0.0648086   | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| EFNA1    | macrophage                                              |      1.71583 |          646 |       0.0127787   |          0.000795019 |              30751 | 0.113205    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| GAMT     | macrophage                                              |      1.66559 |         2241 |       0.0443201   |          0.00275795  |              30751 | 0.186032    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| HSPA1A   | macrophage                                              |      3.34191 |        21909 |       0.433292    |          0.026963    |              30751 | 1.64889     | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| IFI27    | macrophage                                              |      2.29345 |          969 |       0.0191638   |          0.00119253  |              30751 | 0.272543    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| KRT14    | macrophage                                              |      2.49903 |         1436 |       0.0284058   |          0.00176726  |              30751 | 0.348068    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| LGALS3BP | macrophage                                              |      1.71491 |          470 |       0.00929717  |          0.00057842  |              30751 | 0.103577    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| RBM42    | macrophage                                              |      1.70106 |         2033 |       0.0402152   |          0.00250197  |              30751 | 0.183663    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| SCPEP1   | macrophage                                              |      1.71638 |          971 |       0.0192076   |          0.00119499  |              30751 | 0.130701    | 0.4991    | 0.339054  | 1.47204  |  8.19681   |  0.288274  |   0.183032  | 0.67772   |       0.933633  |       9 |      2.14344 |
| COL17A1  | inflammatory macrophage                                 |      1.39582 |            1 |       0.000660066 |          1.23068e-06 |                281 | 0.000990099 | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| EFNA1    | inflammatory macrophage                                 |      1.70227 |            2 |       0.00132013  |          2.46136e-06 |                281 | 0.077389    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| GAMT     | inflammatory macrophage                                 |      2.00453 |            1 |       0.000660066 |          1.23068e-06 |                281 | 0.15073     | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| HSPA1A   | inflammatory macrophage                                 |      2.64375 |          225 |       0.148515    |          0.000276903 |                281 | 0.535075    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| IFI27    | inflammatory macrophage                                 |      2.1218  |            3 |       0.0019802   |          3.69204e-06 |                281 | 0.181604    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| KRT14    | inflammatory macrophage                                 |      1.92635 |           26 |       0.0171617   |          3.19977e-05 |                281 | 0.156844    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| LGALS3BP | inflammatory macrophage                                 |      2.24559 |            1 |       0.000660066 |          1.23068e-06 |                281 | 0.210028    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| RBM42    | inflammatory macrophage                                 |      1.73662 |           20 |       0.0132013   |          2.46136e-05 |                281 | 0.104089    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| SCPEP1   | inflammatory macrophage                                 |      1.73518 |            2 |       0.00132013  |          2.46136e-06 |                281 | 0.0854838   | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| COL17A1  | elicited macrophage                                     |      1.39582 |            1 |       0.000660066 |          1.23068e-06 |                281 | 0.000990099 | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| EFNA1    | elicited macrophage                                     |      1.70227 |            2 |       0.00132013  |          2.46136e-06 |                281 | 0.077389    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| GAMT     | elicited macrophage                                     |      2.00453 |            1 |       0.000660066 |          1.23068e-06 |                281 | 0.15073     | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| HSPA1A   | elicited macrophage                                     |      2.64375 |          225 |       0.148515    |          0.000276903 |                281 | 0.535075    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| IFI27    | elicited macrophage                                     |      2.1218  |            3 |       0.0019802   |          3.69204e-06 |                281 | 0.181604    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| KRT14    | elicited macrophage                                     |      1.92635 |           26 |       0.0171617   |          3.19977e-05 |                281 | 0.156844    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| LGALS3BP | elicited macrophage                                     |      2.24559 |            1 |       0.000660066 |          1.23068e-06 |                281 | 0.210028    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| RBM42    | elicited macrophage                                     |      1.73662 |           20 |       0.0132013   |          2.46136e-05 |                281 | 0.104089    | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| SCPEP1   | elicited macrophage                                     |      1.73518 |            2 |       0.00132013  |          2.46136e-06 |                281 | 0.0854838   | 0.1517    | 0.166915  | 0.908849 |  5.16827   |  0.0545033 |   0.0574922 | 0.31062   |       0.658498  |       9 |      2.14094 |
| COL17A1  | mature T cell                                           |      1.99298 |         2792 |       0.0312123   |          0.00343606  |              63688 | 0.259991    | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| EFNA1    | mature T cell                                           |      1.71619 |         4674 |       0.0355178   |          0.0057522   |              63688 | 0.24305     | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| GAMT     | mature T cell                                           |      1.59136 |          646 |       0.00477211  |          0.000795019 |              63688 | 0.0705755   | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| HSPA1A   | mature T cell                                           |      3.22094 |        41573 |       0.282416    |          0.0511631   |              63688 | 1.85975     | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| IFI27    | mature T cell                                           |      1.67383 |          393 |       0.00383004  |          0.000483657 |              63688 | 0.0834413   | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| KRT14    | mature T cell                                           |      1.72142 |         1268 |       0.0090092   |          0.0015605   |              63688 | 0.123696    | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| LGALS3BP | mature T cell                                           |      1.66005 |          992 |       0.00680851  |          0.00122083  |              63688 | 0.0987449   | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| RBM42    | mature T cell                                           |      1.67054 |         9730 |       0.0660934   |          0.0119745   |              63688 | 0.397741    | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| SCPEP1   | mature T cell                                           |      1.61888 |         1620 |       0.0110051   |          0.0019937   |              63688 | 0.109824    | 0.572417  | 0.360757  | 1.58671  |  8.08791   |  0.33761   |   0.19886   | 0.752465  |       0.92374   |       9 |      2.14038 |
| COL17A1  | phagocyte                                               |      1.77704 |          155 |       0.0015312   |          0.000190755 |              47900 | 0.0997307   | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| EFNA1    | phagocyte                                               |      1.70921 |         1269 |       0.0125361   |          0.00156173  |              47900 | 0.126006    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| GAMT     | phagocyte                                               |      1.66145 |         2403 |       0.0235189   |          0.00295732  |              47900 | 0.157658    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| HSPA1A   | phagocyte                                               |      3.33544 |        35836 |       0.350238    |          0.0441026   |              47900 | 1.85342     | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| IFI27    | phagocyte                                               |      2.27562 |         1076 |       0.0106323   |          0.00132421  |              47900 | 0.257901    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| KRT14    | phagocyte                                               |      2.49068 |         1822 |       0.0183692   |          0.0022423   |              47900 | 0.340123    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| LGALS3BP | phagocyte                                               |      1.7235  |          505 |       0.00499061  |          0.000621493 |              47900 | 0.100061    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| RBM42    | phagocyte                                               |      1.68847 |         2983 |       0.0291879   |          0.00367112  |              47900 | 0.186581    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| SCPEP1   | phagocyte                                               |      1.69516 |         1851 |       0.0180924   |          0.00227799  |              47900 | 0.144704    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| COL17A1  | mononuclear phagocyte                                   |      1.77704 |          155 |       0.0015312   |          0.000190755 |              47900 | 0.0997307   | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| EFNA1    | mononuclear phagocyte                                   |      1.70921 |         1269 |       0.0125361   |          0.00156173  |              47900 | 0.126006    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| GAMT     | mononuclear phagocyte                                   |      1.66145 |         2403 |       0.0235189   |          0.00295732  |              47900 | 0.157658    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| HSPA1A   | mononuclear phagocyte                                   |      3.33544 |        35836 |       0.350238    |          0.0441026   |              47900 | 1.85342     | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| IFI27    | mononuclear phagocyte                                   |      2.27562 |         1076 |       0.0106323   |          0.00132421  |              47900 | 0.257901    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| KRT14    | mononuclear phagocyte                                   |      2.49068 |         1822 |       0.0183692   |          0.0022423   |              47900 | 0.340123    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| LGALS3BP | mononuclear phagocyte                                   |      1.7235  |          505 |       0.00499061  |          0.000621493 |              47900 | 0.100061    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| RBM42    | mononuclear phagocyte                                   |      1.68847 |         2983 |       0.0291879   |          0.00367112  |              47900 | 0.186581    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| SCPEP1   | mononuclear phagocyte                                   |      1.69516 |         1851 |       0.0180924   |          0.00227799  |              47900 | 0.144704    | 0.564411  | 0.36291   | 1.55524  |  8.50322   |  0.332223  |   0.200429  | 0.731953  |       0.961469  |       9 |      2.13962 |
| COL17A1  | professional antigen presenting cell                    |      1.78531 |          145 |       0.00156867  |          0.000178449 |              43981 | 0.101586    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| EFNA1    | professional antigen presenting cell                    |      1.70483 |         1036 |       0.0112177   |          0.00127498  |              43981 | 0.117418    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| GAMT     | professional antigen presenting cell                    |      1.66309 |         2354 |       0.024974    |          0.00289702  |              43981 | 0.159082    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| HSPA1A   | professional antigen presenting cell                    |      3.31426 |        33064 |       0.349115    |          0.0406912   |              43981 | 1.7807      | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| IFI27    | professional antigen presenting cell                    |      2.28074 |         1069 |       0.0115784   |          0.0013156   |              43981 | 0.260413    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| KRT14    | professional antigen presenting cell                    |      2.49498 |         1812 |       0.0200633   |          0.00222999  |              43981 | 0.343486    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| LGALS3BP | professional antigen presenting cell                    |      1.72078 |          501 |       0.00537121  |          0.000616571 |              43981 | 0.099867    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| RBM42    | professional antigen presenting cell                    |      1.69602 |         2799 |       0.0295912   |          0.00344467  |              43981 | 0.184675    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| SCPEP1   | professional antigen presenting cell                    |      1.70874 |         1201 |       0.0126891   |          0.00147805  |              43981 | 0.124506    | 0.541748  | 0.352415  | 1.53724  |  8.41971   |  0.316973  |   0.192776  | 0.720223  |       0.953883  |       9 |      2.12252 |
| COL17A1  | secretory cell                                          |      1.75586 |           69 |       0.0103479   |          8.49169e-05 |               1752 | 0.105705    | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| EFNA1    | secretory cell                                          |      1.82132 |          471 |       0.0706359   |          0.00057965  |               1752 | 0.221785    | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| GAMT     | secretory cell                                          |      1.66955 |           52 |       0.00779844  |          6.39954e-05 |               1752 | 0.0802438   | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| HSPA1A   | secretory cell                                          |      2.68533 |          796 |       0.119376    |          0.000979621 |               1752 | 0.515155    | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| IFI27    | secretory cell                                          |      1.82996 |           21 |       0.00314937  |          2.58443e-05 |               1752 | 0.111995    | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| KRT14    | secretory cell                                          |      1.90006 |           21 |       0.00314937  |          2.58443e-05 |               1752 | 0.129239    | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| LGALS3BP | secretory cell                                          |      1.73906 |           27 |       0.00404919  |          3.32284e-05 |               1752 | 0.0911256   | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| RBM42    | secretory cell                                          |      1.67618 |          248 |       0.0371926   |          0.000305209 |               1752 | 0.130622    | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| SCPEP1   | secretory cell                                          |      1.83239 |           47 |       0.00704859  |          5.7842e-05  |               1752 | 0.11906     | 0.136623  | 0.167214  | 0.817052 |  6.79796   |  0.0443574 |   0.0577107 | 0.250785  |       0.806551  |       9 |      2.07205 |
| COL17A1  | regulatory T cell                                       |      2.05545 |         2153 |       0.0852606   |          0.00264965  |              17196 | 0.341257    | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| EFNA1    | regulatory T cell                                       |      1.72124 |         1322 |       0.0523523   |          0.00162696  |              17196 | 0.189948    | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| GAMT     | regulatory T cell                                       |      1.58923 |          110 |       0.00327138  |          0.000135375 |              17196 | 0.0550732   | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| HSPA1A   | regulatory T cell                                       |      3.02232 |        10030 |       0.29829     |          0.0123437   |              17196 | 1.08569     | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| IFI27    | regulatory T cell                                       |      1.68632 |           85 |       0.00336607  |          0.000104608 |              17196 | 0.078505    | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| KRT14    | regulatory T cell                                       |      1.76705 |          281 |       0.00835688  |          0.000345821 |              17196 | 0.110505    | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| LGALS3BP | regulatory T cell                                       |      1.6566  |          312 |       0.00927881  |          0.000383972 |              17196 | 0.0854548   | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| RBM42    | regulatory T cell                                       |      1.67595 |         2487 |       0.0739628   |          0.0030607   |              17196 | 0.238887    | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| SCPEP1   | regulatory T cell                                       |      1.6268  |          416 |       0.0123717   |          0.000511963 |              17196 | 0.085232    | 0.326243  | 0.252284  | 1.29316  |  6.94083   |  0.171956  |   0.119751  | 0.561124  |       0.81953   |       9 |      2.05708 |
| COL17A1  | erythrocyte                                             |      1.56496 |            4 |       0.00396825  |          4.92272e-06 |                381 | 0.0476319   | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| EFNA1    | erythrocyte                                             |      1.745   |            1 |       0.000992063 |          1.23068e-06 |                381 | 0.0873844   | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| GAMT     | erythrocyte                                             |      1.63528 |            5 |       0.00496032  |          6.1534e-06  |                381 | 0.0664418   | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| HSPA1A   | erythrocyte                                             |      2.86837 |          289 |       0.286706    |          0.000355666 |                381 | 0.799137    | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| IFI27    | erythrocyte                                             |      1.70169 |           12 |       0.0119048   |          1.47682e-05 |                381 | 0.0933603   | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| KRT14    | erythrocyte                                             |      3.04299 |           34 |       0.0337302   |          4.18431e-05 |                381 | 0.456574    | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| LGALS3BP | erythrocyte                                             |      1.39773 |            1 |       0.000992063 |          1.23068e-06 |                381 | 0.00195747  | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| RBM42    | erythrocyte                                             |      1.62902 |           27 |       0.0267857   |          3.32284e-05 |                381 | 0.0981624   | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| SCPEP1   | erythrocyte                                             |      1.61771 |            8 |       0.00793651  |          9.84544e-06 |                381 | 0.0666545   | 0.263751  | 0.190811  | 1.38226  |  3.4202    |  0.129904  |   0.0749199 | 0.6192    |       0.499691  |       9 |      2.04808 |
| COL17A1  | collagen secreting cell                                 |      1.65399 |           42 |       0.0191606   |          5.16886e-05 |                336 | 0.093223    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| EFNA1    | collagen secreting cell                                 |      1.69314 |          252 |       0.114964    |          0.000310131 |                336 | 0.251544    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| GAMT     | collagen secreting cell                                 |      1.48286 |            2 |       0.000912409 |          2.46136e-06 |                336 | 0.0228047   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| HSPA1A   | collagen secreting cell                                 |      1.76498 |           13 |       0.00593066  |          1.59988e-05 |                336 | 0.0999925   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| IFI27    | collagen secreting cell                                 |      1.51336 |            3 |       0.00136861  |          3.69204e-06 |                336 | 0.031014    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| KRT14    | collagen secreting cell                                 |      1.66766 |            9 |       0.00410584  |          1.10761e-05 |                336 | 0.073221    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| LGALS3BP | collagen secreting cell                                 |      1.51402 |            8 |       0.00364964  |          9.84544e-06 |                336 | 0.0347185   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| RBM42    | collagen secreting cell                                 |      1.6201  |            4 |       0.00182482  |          4.92272e-06 |                336 | 0.0579801   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| SCPEP1   | collagen secreting cell                                 |      1.90858 |            3 |       0.00136861  |          3.69204e-06 |                336 | 0.128237    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| COL17A1  | chondrocyte                                             |      1.65399 |           42 |       0.0191606   |          5.16886e-05 |                336 | 0.093223    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| EFNA1    | chondrocyte                                             |      1.69314 |          252 |       0.114964    |          0.000310131 |                336 | 0.251544    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| GAMT     | chondrocyte                                             |      1.48286 |            2 |       0.000912409 |          2.46136e-06 |                336 | 0.0228047   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| HSPA1A   | chondrocyte                                             |      1.76498 |           13 |       0.00593066  |          1.59988e-05 |                336 | 0.0999925   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| IFI27    | chondrocyte                                             |      1.51336 |            3 |       0.00136861  |          3.69204e-06 |                336 | 0.031014    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| KRT14    | chondrocyte                                             |      1.66766 |            9 |       0.00410584  |          1.10761e-05 |                336 | 0.073221    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| LGALS3BP | chondrocyte                                             |      1.51402 |            8 |       0.00364964  |          9.84544e-06 |                336 | 0.0347185   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| RBM42    | chondrocyte                                             |      1.6201  |            4 |       0.00182482  |          4.92272e-06 |                336 | 0.0579801   | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| SCPEP1   | chondrocyte                                             |      1.90858 |            3 |       0.00136861  |          3.69204e-06 |                336 | 0.128237    | 0.0707046 | 0.0880816 | 0.802717 |  3.51338   |  0         |   0         | 0.241441  |       0.508156  |       9 |      2.02536 |
| COL17A1  | cytotoxic T cell                                        |      2.0734  |          108 |       0.00964803  |          0.000132913 |               4237 | 0.183695    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| EFNA1    | cytotoxic T cell                                        |      1.9245  |          218 |       0.0194747   |          0.000268288 |               4237 | 0.164416    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| GAMT     | cytotoxic T cell                                        |      1.5869  |           60 |       0.00536001  |          7.38408e-05 |               4237 | 0.0564467   | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| HSPA1A   | cytotoxic T cell                                        |      2.73629 |         2203 |       0.196802    |          0.00271119  |               4237 | 0.677239    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| IFI27    | cytotoxic T cell                                        |      1.84428 |           44 |       0.00393068  |          5.41499e-05 |               4237 | 0.117236    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| KRT14    | cytotoxic T cell                                        |      1.72273 |          427 |       0.0381454   |          0.0005255   |               4237 | 0.147752    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| LGALS3BP | cytotoxic T cell                                        |      1.83385 |           77 |       0.00687869  |          9.47623e-05 |               4237 | 0.119876    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| RBM42    | cytotoxic T cell                                        |      1.65433 |          808 |       0.0721815   |          0.000994389 |               4237 | 0.191028    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| SCPEP1   | cytotoxic T cell                                        |      1.61554 |          292 |       0.0260854   |          0.000359359 |               4237 | 0.100089    | 0.185661  | 0.195309  | 0.950605 |  7.70856   |  0.0773561 |   0.0781995 | 0.337838  |       0.889276  |       9 |      2.01863 |
| COL17A1  | Langerhans cell                                         |      2.07157 |           33 |       0.00264614  |          4.06124e-05 |               6294 | 0.17096     | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| EFNA1    | Langerhans cell                                         |      1.88806 |           21 |       0.00168391  |          2.58443e-05 |               6294 | 0.12409     | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| GAMT     | Langerhans cell                                         |      1.98046 |            1 |       8.04311e-05 |          1.23068e-06 |               6294 | 0.14394     | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| HSPA1A   | Langerhans cell                                         |      3.55051 |         5538 |       0.44407     |          0.00681551  |               6294 | 1.32763     | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| IFI27    | Langerhans cell                                         |      1.79029 |            7 |       0.000563018 |          8.61476e-06 |               6294 | 0.0980248   | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| KRT14    | Langerhans cell                                         |      2.52419 |          296 |       0.0237351   |          0.000364281 |               6294 | 0.320181    | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| LGALS3BP | Langerhans cell                                         |      2.00397 |            1 |       8.04311e-05 |          1.23068e-06 |               6294 | 0.149724    | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| RBM42    | Langerhans cell                                         |      1.71835 |          311 |       0.0249379   |          0.000382741 |               6294 | 0.12411     | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| SCPEP1   | Langerhans cell                                         |      1.68867 |           86 |       0.006896    |          0.000105838 |               6294 | 0.0844026   | 0.397875  | 0.282562  | 1.4081   |  8.2293    |  0.220158  |   0.141833  | 0.636042  |       0.936585  |       9 |      1.98004 |
| COL17A1  | mast cell                                               |      1.91785 |           26 |       0.0059894   |          3.19977e-05 |               1299 | 0.137995    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| EFNA1    | mast cell                                               |      1.89288 |          164 |       0.0377793   |          0.000201831 |               1299 | 0.182813    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| GAMT     | mast cell                                               |      1.67259 |           49 |       0.0112877   |          6.03033e-05 |               1299 | 0.0861565   | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| HSPA1A   | mast cell                                               |      2.70184 |          740 |       0.170468    |          0.000910703 |               1299 | 0.594523    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| IFI27    | mast cell                                               |      1.73093 |           15 |       0.00345543  |          1.84602e-05 |               1299 | 0.087951    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| KRT14    | mast cell                                               |      1.85376 |           10 |       0.00230362  |          1.23068e-05 |               1299 | 0.11632     | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| LGALS3BP | mast cell                                               |      1.82399 |           13 |       0.0029947   |          1.59988e-05 |               1299 | 0.110105    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| RBM42    | mast cell                                               |      1.6771  |          243 |       0.0559779   |          0.000299055 |               1299 | 0.158907    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| SCPEP1   | mast cell                                               |      1.81405 |           39 |       0.00898411  |          4.79965e-05 |               1299 | 0.117262    | 0.159733  | 0.176892  | 0.902993 |  8.01908   |  0.0599083 |   0.0647688 | 0.306803  |       0.917487  |       9 |      1.95227 |
| COL17A1  | conventional dendritic cell                             |      1.96267 |           56 |       0.00174613  |          6.89181e-05 |               8544 | 0.143368    | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| EFNA1    | conventional dendritic cell                             |      1.69358 |          332 |       0.010352    |          0.000408586 |               8544 | 0.0966352   | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| GAMT     | conventional dendritic cell                             |      1.662   |            3 |       9.36534e-05 |          3.69204e-06 |               8544 | 0.0656678   | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| HSPA1A   | conventional dendritic cell                             |      3.15739 |         7174 |       0.22294     |          0.0088289   |               8544 | 0.938075    | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| IFI27    | conventional dendritic cell                             |      2.20118 |           80 |       0.00249742  |          9.84544e-05 |               8544 | 0.203735    | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| KRT14    | conventional dendritic cell                             |      2.4741  |          343 |       0.010695    |          0.000422123 |               8544 | 0.289414    | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| LGALS3BP | conventional dendritic cell                             |      1.69148 |           17 |       0.000530703 |          2.09216e-05 |               8544 | 0.0739076   | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| RBM42    | conventional dendritic cell                             |      1.69122 |          437 |       0.013626    |          0.000537807 |               8544 | 0.10346     | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| SCPEP1   | conventional dendritic cell                             |      1.68653 |          102 |       0.00316977  |          0.000125529 |               8544 | 0.078667    | 0.278502  | 0.221437  | 1.2577   |  7.25977   |  0.13983   |   0.0972546 | 0.538012  |       0.848505  |       9 |      1.93899 |
| COL17A1  | dendritic cell                                          |      1.93185 |           68 |       0.00162676  |          8.36862e-05 |              12884 | 0.135893    | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| EFNA1    | dendritic cell                                          |      1.68661 |          390 |       0.00932992  |          0.000479965 |              12884 | 0.0947651   | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| GAMT     | dendritic cell                                          |      1.61806 |          108 |       0.0025272   |          0.000132913 |              12884 | 0.061002    | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| HSPA1A   | dendritic cell                                          |      3.25919 |        10882 |       0.253772    |          0.0133923   |              12884 | 1.09741     | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| IFI27    | dendritic cell                                          |      2.15758 |          100 |       0.00239446  |          0.000123068 |              12884 | 0.193331    | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| KRT14    | dendritic cell                                          |      2.4795  |          376 |       0.0094565   |          0.000462736 |              12884 | 0.289669    | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| LGALS3BP | dendritic cell                                          |      1.82685 |           29 |       0.000694395 |          3.56897e-05 |              12884 | 0.107737    | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| RBM42    | dendritic cell                                          |      1.68537 |          736 |       0.0172071   |          0.00090578  |              12884 | 0.114492    | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| SCPEP1   | dendritic cell                                          |      1.67329 |          195 |       0.00454747  |          0.000239983 |              12884 | 0.0796834   | 0.328387  | 0.241554  | 1.35947  |  7.85478   |  0.173398  |   0.111926  | 0.604348  |       0.902561  |       9 |      1.89286 |
| COL17A1  | innate lymphoid cell                                    |      1.72381 |          314 |       0.0109618   |          0.000386433 |               7739 | 0.104559    | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| EFNA1    | innate lymphoid cell                                    |      1.75861 |         2281 |       0.0751516   |          0.00280718  |               7739 | 0.256112    | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| GAMT     | innate lymphoid cell                                    |      1.58901 |          234 |       0.00762811  |          0.000287979 |               7739 | 0.0644984   | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| HSPA1A   | innate lymphoid cell                                    |      3.39112 |         3058 |       0.0996871   |          0.00376342  |               7739 | 0.712953    | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| IFI27    | innate lymphoid cell                                    |      1.62581 |          114 |       0.00375593  |          0.000140298 |               7739 | 0.064893    | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| KRT14    | innate lymphoid cell                                    |      1.70616 |          113 |       0.00394484  |          0.000139067 |               7739 | 0.0849189   | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| LGALS3BP | innate lymphoid cell                                    |      1.70464 |          155 |       0.00541107  |          0.000190755 |               7739 | 0.0877412   | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| RBM42    | innate lymphoid cell                                    |      1.65819 |         1107 |       0.0360868   |          0.00136236  |               7739 | 0.144935    | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| SCPEP1   | innate lymphoid cell                                    |      1.64173 |          363 |       0.0118334   |          0.000446737 |               7739 | 0.0868382   | 0.209033  | 0.178605  | 1.17036  |  6.95342   |  0.093083  |   0.0660181 | 0.481079  |       0.820674  |       9 |      1.86056 |
| COL17A1  | natural killer cell                                     |      1.73014 |          260 |       0.0140222   |          0.000319977 |               5255 | 0.109425    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| EFNA1    | natural killer cell                                     |      1.76345 |         1810 |       0.0976162   |          0.00222753  |               5255 | 0.279816    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| GAMT     | natural killer cell                                     |      1.58108 |          157 |       0.00846726  |          0.000193217 |               5255 | 0.0619788   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| HSPA1A   | natural killer cell                                     |      3.82528 |         2098 |       0.113149    |          0.00258197  |               5255 | 0.817151    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| IFI27    | natural killer cell                                     |      1.63937 |           92 |       0.00496171  |          0.000113223 |               5255 | 0.0695156   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| KRT14    | natural killer cell                                     |      1.6922  |           78 |       0.00420667  |          9.5993e-05  |               5255 | 0.0810473   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| LGALS3BP | natural killer cell                                     |      1.7394  |          111 |       0.00598641  |          0.000136605 |               5255 | 0.0961099   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| RBM42    | natural killer cell                                     |      1.66056 |          555 |       0.029932    |          0.000683027 |               5255 | 0.123178    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| SCPEP1   | natural killer cell                                     |      1.62895 |           94 |       0.00506957  |          0.000115684 |               5255 | 0.0671619   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| COL17A1  | group 1 innate lymphoid cell                            |      1.73014 |          260 |       0.0140222   |          0.000319977 |               5255 | 0.109425    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| EFNA1    | group 1 innate lymphoid cell                            |      1.76345 |         1810 |       0.0976162   |          0.00222753  |               5255 | 0.279816    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| GAMT     | group 1 innate lymphoid cell                            |      1.58108 |          157 |       0.00846726  |          0.000193217 |               5255 | 0.0619788   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| HSPA1A   | group 1 innate lymphoid cell                            |      3.82528 |         2098 |       0.113149    |          0.00258197  |               5255 | 0.817151    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| IFI27    | group 1 innate lymphoid cell                            |      1.63937 |           92 |       0.00496171  |          0.000113223 |               5255 | 0.0695156   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| KRT14    | group 1 innate lymphoid cell                            |      1.6922  |           78 |       0.00420667  |          9.5993e-05  |               5255 | 0.0810473   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| LGALS3BP | group 1 innate lymphoid cell                            |      1.7394  |          111 |       0.00598641  |          0.000136605 |               5255 | 0.0961099   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| RBM42    | group 1 innate lymphoid cell                            |      1.66056 |          555 |       0.029932    |          0.000683027 |               5255 | 0.123178    | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| SCPEP1   | group 1 innate lymphoid cell                            |      1.62895 |           94 |       0.00506957  |          0.000115684 |               5255 | 0.0671619   | 0.244699  | 0.189487  | 1.29138  |  7.10131   |  0.117084  |   0.0739541 | 0.55996   |       0.83411   |       9 |      1.80587 |
| COL17A1  | alpha-beta T cell                                       |      1.78396 |          259 |       0.0250145   |          0.000318746 |              35237 | 0.139128    | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| EFNA1    | alpha-beta T cell                                       |      1.72252 |          848 |       0.0169312   |          0.00104362  |              35237 | 0.125877    | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| GAMT     | alpha-beta T cell                                       |      1.59657 |          348 |       0.00646132  |          0.000428277 |              35237 | 0.0673152   | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| HSPA1A   | alpha-beta T cell                                       |      3.40341 |        26371 |       0.401422    |          0.0324543   |              35237 | 1.72216     | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| IFI27    | alpha-beta T cell                                       |      1.68894 |           90 |       0.00382783  |          0.000110761 |              35237 | 0.0799609   | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| KRT14    | alpha-beta T cell                                       |      1.75644 |          170 |       0.0028739   |          0.000209216 |              35237 | 0.0970345   | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| LGALS3BP | alpha-beta T cell                                       |      1.63986 |          474 |       0.00739377  |          0.000583342 |              35237 | 0.082356    | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| RBM42    | alpha-beta T cell                                       |      1.6877  |         6110 |       0.0929914   |          0.00751945  |              35237 | 0.356351    | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| SCPEP1   | alpha-beta T cell                                       |      1.62284 |          567 |       0.00863093  |          0.000697795 |              35237 | 0.0822333   | 0.538536  | 0.305825  | 1.76093  |  8.28983   |  0.314811  |   0.158798  | 0.866027  |       0.942084  |       9 |      1.7898  |
| COL17A1  | mature alpha-beta T cell                                |      1.78396 |          259 |       0.0250145   |          0.000318746 |              35188 | 0.139128    | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| EFNA1    | mature alpha-beta T cell                                |      1.72232 |          845 |       0.0168986   |          0.00103992  |              35188 | 0.125706    | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| GAMT     | mature alpha-beta T cell                                |      1.59701 |          346 |       0.00643386  |          0.000425815 |              35188 | 0.0673348   | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| HSPA1A   | mature alpha-beta T cell                                |      3.40246 |        26333 |       0.401338    |          0.0324075   |              35188 | 1.7209      | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| IFI27    | mature alpha-beta T cell                                |      1.68894 |           90 |       0.00382783  |          0.000110761 |              35188 | 0.0799609   | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| KRT14    | mature alpha-beta T cell                                |      1.75644 |          170 |       0.0028739   |          0.000209216 |              35188 | 0.0970345   | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| LGALS3BP | mature alpha-beta T cell                                |      1.63986 |          474 |       0.00739377  |          0.000583342 |              35188 | 0.082356    | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| RBM42    | mature alpha-beta T cell                                |      1.68769 |         6105 |       0.09303     |          0.0075133   |              35188 | 0.356288    | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| SCPEP1   | mature alpha-beta T cell                                |      1.62304 |          566 |       0.00862634  |          0.000696565 |              35188 | 0.0822502   | 0.538125  | 0.305662  | 1.76052  |  8.2891    |  0.314535  |   0.158679  | 0.86576   |       0.942017  |       9 |      1.7896  |
| COL17A1  | CD8-positive, alpha-beta T cell                         |      1.78396 |          259 |       0.0250145   |          0.000318746 |              23663 | 0.139128    | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| EFNA1    | CD8-positive, alpha-beta T cell                         |      1.72573 |          813 |       0.0209915   |          0.00100054  |              23663 | 0.131925    | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| GAMT     | CD8-positive, alpha-beta T cell                         |      1.61122 |          244 |       0.00665939  |          0.000300286 |              23663 | 0.0687472   | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| HSPA1A   | CD8-positive, alpha-beta T cell                         |      3.39246 |        16625 |       0.34296     |          0.0204601   |              23663 | 1.40035     | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| IFI27    | CD8-positive, alpha-beta T cell                         |      1.67716 |           87 |       0.007109    |          0.000107069 |              23663 | 0.0819151   | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| KRT14    | CD8-positive, alpha-beta T cell                         |      1.75969 |          142 |       0.00301467  |          0.000174757 |              23663 | 0.0973793   | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| LGALS3BP | CD8-positive, alpha-beta T cell                         |      1.65067 |          351 |       0.00746904  |          0.000431969 |              23663 | 0.0822067   | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| RBM42    | CD8-positive, alpha-beta T cell                         |      1.70278 |         4705 |       0.0970603   |          0.00579035  |              23663 | 0.332802    | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| SCPEP1   | CD8-positive, alpha-beta T cell                         |      1.625   |          437 |       0.00901496  |          0.000537807 |              23663 | 0.0802534   | 0.432255  | 0.268301  | 1.61108  |  8.08137   |  0.243293  |   0.131432  | 0.768351  |       0.923145  |       9 |      1.78671 |
| COL17A1  | memory T cell                                           |      1.78396 |          259 |       0.0250145   |          0.000318746 |              10052 | 0.139128    | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| EFNA1    | memory T cell                                           |      1.73028 |          772 |       0.0279568   |          0.000950085 |              10052 | 0.142518    | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| GAMT     | memory T cell                                           |      1.59918 |           90 |       0.00521437  |          0.000110761 |              10052 | 0.0599617   | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| HSPA1A   | memory T cell                                           |      3.36238 |         6338 |       0.229521    |          0.00780005  |              10052 | 0.978522    | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| IFI27    | memory T cell                                           |      1.6689  |           83 |       0.00801623  |          0.000102146 |              10052 | 0.0811473   | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| KRT14    | memory T cell                                           |      1.73576 |           85 |       0.00307815  |          0.000104608 |              10052 | 0.0902343   | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| LGALS3BP | memory T cell                                           |      1.6645  |          197 |       0.00713122  |          0.000242444 |              10052 | 0.0814455   | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| RBM42    | memory T cell                                           |      1.68107 |         2051 |       0.0742443   |          0.00252412  |              10052 | 0.230216    | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| SCPEP1   | memory T cell                                           |      1.59302 |          177 |       0.00640979  |          0.00021783  |              10052 | 0.0623033   | 0.294182  | 0.207275  | 1.41928  |  8.14093   |  0.150381  |   0.0869268 | 0.643332  |       0.928556  |       9 |      1.6976  |
| COL17A1  | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.78396 |          259 |       0.0250145   |          0.000318746 |              10050 | 0.139128    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| EFNA1    | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.73028 |          772 |       0.0279568   |          0.000950085 |              10050 | 0.142518    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| GAMT     | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.59918 |           90 |       0.00521437  |          0.000110761 |              10050 | 0.0599617   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| HSPA1A   | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      3.36238 |         6338 |       0.229521    |          0.00780005  |              10050 | 0.978522    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| IFI27    | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.6689  |           83 |       0.00801623  |          0.000102146 |              10050 | 0.0811473   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| KRT14    | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.73576 |           85 |       0.00307815  |          0.000104608 |              10050 | 0.0902343   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| LGALS3BP | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.66102 |          196 |       0.00709785  |          0.000241213 |              10050 | 0.0805143   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| RBM42    | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.68038 |         2050 |       0.0742377   |          0.00252289  |              10050 | 0.230011    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| SCPEP1   | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.59302 |          177 |       0.00640979  |          0.00021783  |              10050 | 0.0623033   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| COL17A1  | CD8-positive, alpha-beta memory T cell                  |      1.78396 |          259 |       0.0250145   |          0.000318746 |              10050 | 0.139128    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| EFNA1    | CD8-positive, alpha-beta memory T cell                  |      1.73028 |          772 |       0.0279568   |          0.000950085 |              10050 | 0.142518    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| GAMT     | CD8-positive, alpha-beta memory T cell                  |      1.59918 |           90 |       0.00521437  |          0.000110761 |              10050 | 0.0599617   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| HSPA1A   | CD8-positive, alpha-beta memory T cell                  |      3.36238 |         6338 |       0.229521    |          0.00780005  |              10050 | 0.978522    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| IFI27    | CD8-positive, alpha-beta memory T cell                  |      1.6689  |           83 |       0.00801623  |          0.000102146 |              10050 | 0.0811473   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| KRT14    | CD8-positive, alpha-beta memory T cell                  |      1.73576 |           85 |       0.00307815  |          0.000104608 |              10050 | 0.0902343   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| LGALS3BP | CD8-positive, alpha-beta memory T cell                  |      1.66102 |          196 |       0.00709785  |          0.000241213 |              10050 | 0.0805143   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| RBM42    | CD8-positive, alpha-beta memory T cell                  |      1.68038 |         2050 |       0.0742377   |          0.00252289  |              10050 | 0.230011    | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| SCPEP1   | CD8-positive, alpha-beta memory T cell                  |      1.59302 |          177 |       0.00640979  |          0.00021783  |              10050 | 0.0623033   | 0.29423   | 0.207149  | 1.42038  |  8.14097   |  0.150414  |   0.0868347 | 0.644046  |       0.92856   |       9 |      1.69634 |
| COL17A1  | helper T cell                                           |      1.6656  |          272 |       0.00637719  |          0.000334745 |              10399 | 0.0823658   | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| EFNA1    | helper T cell                                           |      1.69137 |         2286 |       0.0535965   |          0.00281333  |              10399 | 0.207356    | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| GAMT     | helper T cell                                           |      1.57104 |          156 |       0.00365751  |          0.000191986 |              10399 | 0.0522696   | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| HSPA1A   | helper T cell                                           |      2.96916 |         5231 |       0.122644    |          0.00643769  |              10399 | 0.695188    | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| IFI27    | helper T cell                                           |      1.6168  |          174 |       0.00407953  |          0.000214138 |              10399 | 0.0645876   | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| KRT14    | helper T cell                                           |      1.67825 |          396 |       0.00928444  |          0.000487349 |              10399 | 0.0927815   | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| LGALS3BP | helper T cell                                           |      1.63574 |          140 |       0.00328238  |          0.000172295 |              10399 | 0.0672426   | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| RBM42    | helper T cell                                           |      1.61835 |         1322 |       0.030995    |          0.00162696  |              10399 | 0.132602    | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| SCPEP1   | helper T cell                                           |      1.61014 |          422 |       0.00989403  |          0.000519347 |              10399 | 0.0775598   | 0.204927  | 0.16355   | 1.25299  |  7.6319    |  0.0903204 |   0.0550386 | 0.53494   |       0.882313  |       9 |      1.68993 |
| COL17A1  | monocyte                                                |      1.74695 |           11 |       0.00123958  |          1.35375e-05 |               4265 | 0.0884729   | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| EFNA1    | monocyte                                                |      1.72868 |          233 |       0.0262565   |          0.000286748 |               4265 | 0.126775    | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| GAMT     | monocyte                                                |      1.57631 |           54 |       0.00608519  |          6.64567e-05 |               4265 | 0.0547857   | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| HSPA1A   | monocyte                                                |      3.56141 |         3045 |       0.343137    |          0.00374742  |               4265 | 1.11971     | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| IFI27    | monocyte                                                |      1.49393 |            7 |       0.000788821 |          8.61476e-06 |               4265 | 0.0254613   | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| KRT14    | monocyte                                                |      1.71089 |           10 |       0.00112689  |          1.23068e-05 |               4265 | 0.0794101   | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| LGALS3BP | monocyte                                                |      1.89668 |            6 |       0.000676133 |          7.38408e-06 |               4265 | 0.124342    | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| RBM42    | monocyte                                                |      1.57943 |          214 |       0.0241154   |          0.000263365 |               4265 | 0.0863973   | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| SCPEP1   | monocyte                                                |      1.6713  |          685 |       0.0771918   |          0.000843016 |               4265 | 0.199797    | 0.344086  | 0.211684  | 1.62547  |  8.47082   |  0.183963  |   0.0901417 | 0.777734  |       0.958526  |       9 |      1.53969 |
| COL17A1  | lymphocyte of B lineage                                 |      1.82389 |           18 |       0.00568541  |          2.21522e-05 |               8612 | 0.114235    | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| EFNA1    | lymphocyte of B lineage                                 |      1.71068 |           35 |       0.00380352  |          4.30738e-05 |               8612 | 0.0839665   | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| GAMT     | lymphocyte of B lineage                                 |      1.6078  |           71 |       0.00651675  |          8.73783e-05 |               8612 | 0.0635832   | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| HSPA1A   | lymphocyte of B lineage                                 |      3.33558 |         7663 |       0.689243    |          0.0094307   |               8612 | 1.69297     | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| IFI27    | lymphocyte of B lineage                                 |      1.93412 |            2 |       0.000631712 |          2.46136e-06 |               8612 | 0.133391    | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| KRT14    | lymphocyte of B lineage                                 |      1.66902 |            5 |       0.00157928  |          6.1534e-06  |               8612 | 0.0696697   | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| LGALS3BP | lymphocyte of B lineage                                 |      1.61585 |           20 |       0.00185736  |          2.46136e-05 |               8612 | 0.0573639   | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| RBM42    | lymphocyte of B lineage                                 |      1.60648 |          492 |       0.0442526   |          0.000605494 |               8612 | 0.12986     | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| SCPEP1   | lymphocyte of B lineage                                 |      1.64351 |          306 |       0.0276448   |          0.000376588 |               8612 | 0.109641    | 0.533342  | 0.272743  | 1.95548  |  8.92734   |  0.311316  |   0.134672  | 0.992836  |       1         |       9 |      1.47469 |
| COL17A1  | plasma cell                                             |      1.82389 |           18 |       0.00568541  |          2.21522e-05 |               1692 | 0.114235    | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| EFNA1    | plasma cell                                             |      1.73082 |           29 |       0.00915982  |          3.56897e-05 |               1692 | 0.0968128   | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| GAMT     | plasma cell                                             |      1.57039 |           16 |       0.0050537   |          1.96909e-05 |               1692 | 0.0508791   | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| HSPA1A   | plasma cell                                             |      3.91254 |         1562 |       0.493367    |          0.00192232  |               1692 | 1.39622     | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| IFI27    | plasma cell                                             |      1.93412 |            2 |       0.000631712 |          2.46136e-06 |               1692 | 0.133391    | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| KRT14    | plasma cell                                             |      1.66902 |            5 |       0.00157928  |          6.1534e-06  |               1692 | 0.0696697   | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| LGALS3BP | plasma cell                                             |      1.69397 |            8 |       0.00252685  |          9.84544e-06 |               1692 | 0.0772989   | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| RBM42    | plasma cell                                             |      1.59037 |           25 |       0.0078964   |          3.0767e-05  |               1692 | 0.0602733   | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| SCPEP1   | plasma cell                                             |      1.65661 |           27 |       0.00852811  |          3.32284e-05 |               1692 | 0.077563    | 0.437848  | 0.230704  | 1.89787  |  8.90831   |  0.247056  |   0.104013  | 0.955289  |       0.998271  |       9 |      1.37473 |
| COL17A1  | granulocyte                                             |      2.35694 |            7 |       0.0218069   |          8.61476e-06 |                183 | 0.269284    | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| EFNA1    | granulocyte                                             |      1.51899 |            1 |       0.00311526  |          1.23068e-06 |                183 | 0.034972    | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| GAMT     | granulocyte                                             |      1.69463 |           12 |       0.0223881   |          1.47682e-05 |                183 | 0.10735     | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| HSPA1A   | granulocyte                                             |      2.30113 |          137 |       0.251838    |          0.000168603 |                183 | 0.603689    | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| IFI27    | granulocyte                                             |      1.51927 |            1 |       0.00311526  |          1.23068e-06 |                183 | 0.0350405   | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| KRT14    | granulocyte                                             |      1.78896 |            1 |       0.125       |          1.23068e-06 |                183 | 0.284212    | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| RBM42    | granulocyte                                             |      1.69825 |           18 |       0.0330882   |          2.21522e-05 |                183 | 0.124432    | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| SCPEP1   | granulocyte                                             |      1.88274 |            6 |       0.011194    |          7.38408e-06 |                183 | 0.136689    | 0.188074  | 0.199458  | 0.942923 |  2.87084   |  0.0789796 |   0.081226  | 0.332831  |       0.449783  |       8 |      2.16559 |
| COL17A1  | plasmacytoid dendritic cell                             |      1.71081 |            7 |       0.00343137  |          8.61476e-06 |                941 | 0.082776    | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| EFNA1    | plasmacytoid dendritic cell                             |      1.62931 |           34 |       0.0166667   |          4.18431e-05 |                941 | 0.0832205   | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| GAMT     | plasmacytoid dendritic cell                             |      1.71207 |           29 |       0.00962815  |          3.56897e-05 |                941 | 0.0929024   | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| HSPA1A   | plasmacytoid dendritic cell                             |      3.48787 |          691 |       0.229416    |          0.0008504   |                941 | 0.875142    | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| IFI27    | plasmacytoid dendritic cell                             |      2.26346 |            2 |       0.000980392 |          2.46136e-06 |                941 | 0.21493     | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| LGALS3BP | plasmacytoid dendritic cell                             |      2.19024 |            5 |       0.00245098  |          6.1534e-06  |                941 | 0.199194    | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| RBM42    | plasmacytoid dendritic cell                             |      1.69907 |          150 |       0.0498008   |          0.000184602 |                941 | 0.152838    | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| SCPEP1   | plasmacytoid dendritic cell                             |      1.69106 |           23 |       0.00763612  |          2.83056e-05 |                941 | 0.084605    | 0.268893  | 0.223201  | 1.20471  |  7.04301   |  0.133364  |   0.0985412 | 0.50347   |       0.828813  |       8 |      1.77931 |
| EFNA1    | activated CD8-positive, alpha-beta T cell               |      1.64463 |           21 |       0.00402685  |          2.58443e-05 |               5157 | 0.0677206   | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| GAMT     | activated CD8-positive, alpha-beta T cell               |      1.68873 |           59 |       0.010502    |          7.26101e-05 |               5157 | 0.0891836   | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| HSPA1A   | activated CD8-positive, alpha-beta T cell               |      4.06944 |         4378 |       0.779281    |          0.00538792  |               5157 | 1.93055     | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| IFI27    | activated CD8-positive, alpha-beta T cell               |      2.0178  |            2 |       0.00496278  |          2.46136e-06 |               5157 | 0.160471    | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| KRT14    | activated CD8-positive, alpha-beta T cell               |      1.78231 |            7 |       0.00134228  |          8.61476e-06 |               5157 | 0.0972302   | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| LGALS3BP | activated CD8-positive, alpha-beta T cell               |      1.6727  |           61 |       0.010858    |          7.50715e-05 |               5157 | 0.0858224   | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| RBM42    | activated CD8-positive, alpha-beta T cell               |      1.80002 |          557 |       0.0991456   |          0.000685489 |               5157 | 0.261351    | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| SCPEP1   | activated CD8-positive, alpha-beta T cell               |      1.7063  |           72 |       0.0128159   |          8.8609e-05  |               5157 | 0.0972858   | 0.642224  | 0.348702  | 1.84176  |  7.76916   |  0.384584  |   0.190068  | 0.918711  |       0.894782  |       8 |      1.70592 |
| EFNA1    | naive T cell                                            |      1.62391 |           44 |       0.00430234  |          5.41499e-05 |               5149 | 0.0635847   | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| GAMT     | naive T cell                                            |      1.57514 |          110 |       0.0107558   |          0.000135375 |               5149 | 0.0628349   | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| HSPA1A   | naive T cell                                            |      2.93081 |         3724 |       0.364134    |          0.00458305  |               5149 | 1.01221     | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| IFI27    | naive T cell                                            |      2.19368 |            1 |       0.000301841 |          1.23068e-06 |               5149 | 0.196721    | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| KRT14    | naive T cell                                            |      1.73381 |           43 |       0.00621927  |          5.29192e-05 |               5149 | 0.093471    | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| LGALS3BP | naive T cell                                            |      1.62849 |          102 |       0.0099736   |          0.000125529 |               5149 | 0.0745954   | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| RBM42    | naive T cell                                            |      1.69566 |         1033 |       0.101007    |          0.00127129  |               5149 | 0.249775    | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| SCPEP1   | naive T cell                                            |      1.58941 |           92 |       0.0089958   |          0.000113223 |               5149 | 0.0632771   | 0.325028  | 0.227058  | 1.43147  |  6.86729   |  0.171138  |   0.101354  | 0.651279  |       0.812849  |       8 |      1.65937 |
| EFNA1    | myofibroblast cell                                      |      1.63002 |           35 |       0.00632225  |          4.30738e-05 |               4558 | 0.0679039   | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| GAMT     | myofibroblast cell                                      |      1.58373 |          144 |       0.0260116   |          0.000177218 |               4558 | 0.0886375   | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| HSPA1A   | myofibroblast cell                                      |      2.78215 |         3419 |       0.617594    |          0.00420769  |               4558 | 1.34858     | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| IFI27    | myofibroblast cell                                      |      1.66971 |            9 |       0.00162572  |          1.10761e-05 |               4558 | 0.0700042   | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| KRT14    | myofibroblast cell                                      |      1.65642 |            5 |       0.000903179 |          6.1534e-06  |               4558 | 0.0655568   | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| LGALS3BP | myofibroblast cell                                      |      1.69038 |          482 |       0.0870665   |          0.000593188 |               4558 | 0.214482    | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| RBM42    | myofibroblast cell                                      |      1.58343 |          213 |       0.0384754   |          0.000262135 |               4558 | 0.108899    | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| SCPEP1   | myofibroblast cell                                      |      1.60999 |          251 |       0.0453396   |          0.000308901 |               4558 | 0.12663     | 0.442051  | 0.261337  | 1.6915   |  7.70051   |  0.249885  |   0.126354  | 0.820769  |       0.888546  |       8 |      1.54022 |
| EFNA1    | CD4-positive, alpha-beta T cell                         |      1.63555 |           32 |       0.00283839  |          3.93818e-05 |              10660 | 0.0639652   | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| GAMT     | CD4-positive, alpha-beta T cell                         |      1.56443 |          100 |       0.00621775  |          0.000123068 |              10660 | 0.0531553   | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| HSPA1A   | CD4-positive, alpha-beta T cell                         |      3.43401 |         8958 |       0.556986    |          0.0110244   |              10660 | 1.54955     | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| IFI27    | CD4-positive, alpha-beta T cell                         |      2.03039 |            3 |       0.000266099 |          3.69204e-06 |              10660 | 0.156548    | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| KRT14    | CD4-positive, alpha-beta T cell                         |      1.73375 |           23 |       0.00208522  |          2.83056e-05 |              10660 | 0.0867798   | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| LGALS3BP | CD4-positive, alpha-beta T cell                         |      1.61059 |          121 |       0.00751833  |          0.000148912 |              10660 | 0.0669588   | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| RBM42    | CD4-positive, alpha-beta T cell                         |      1.6418  |         1312 |       0.0815211   |          0.00161465  |              10660 | 0.213922    | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| SCPEP1   | CD4-positive, alpha-beta T cell                         |      1.61102 |          111 |       0.0069017   |          0.000136605 |              10660 | 0.065903    | 0.515201  | 0.282098  | 1.82632  |  7.71231   |  0.299108  |   0.141494  | 0.908648  |       0.889617  |       8 |      1.51447 |
| COL17A1  | granular cell of epidermis                              |      1.60525 |            3 |       0.0141509   |          3.69204e-06 |                496 | 0.0727938   | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| EFNA1    | granular cell of epidermis                              |      1.65991 |           45 |       0.212264    |          5.53806e-05 |                496 | 0.384406    | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| HSPA1A   | granular cell of epidermis                              |      2.86162 |          186 |       0.877358    |          0.000228906 |                496 | 1.68101     | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| IFI27    | granular cell of epidermis                              |      2.22769 |          104 |       0.490566    |          0.000127991 |                496 | 0.94293     | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| KRT14    | granular cell of epidermis                              |      2.20923 |          155 |       0.731132    |          0.000190755 |                496 | 1.30045     | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| RBM42    | granular cell of epidermis                              |      1.48347 |            2 |       0.00943396  |          2.46136e-06 |                496 | 0.0357361   | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| SCPEP1   | granular cell of epidermis                              |      1.57681 |            1 |       0.00471698  |          1.23068e-06 |                496 | 0.0515989   | 0.672067  | 0.638418  | 1.05271  | -1.42142   |  0.404666  |   0.401355  | 0.40439   |       0.0598422 |       7 |      3.45841 |
| COL17A1  | blood vessel endothelial cell                           |      1.77903 |            4 |       0.00543478  |          4.92272e-06 |                585 | 0.102492    | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| EFNA1    | blood vessel endothelial cell                           |      1.79002 |          217 |       0.290495    |          0.000267058 |                585 | 0.537843    | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| HSPA1A   | blood vessel endothelial cell                           |      1.72235 |           30 |       0.0401606   |          3.69204e-05 |                585 | 0.141254    | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| IFI27    | blood vessel endothelial cell                           |      1.81992 |          280 |       0.374833    |          0.00034459  |                585 | 0.673199    | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| LGALS3BP | blood vessel endothelial cell                           |      1.72467 |            7 |       0.00951087  |          8.61476e-06 |                585 | 0.0953035   | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| RBM42    | blood vessel endothelial cell                           |      1.82397 |           24 |       0.0321285   |          2.95363e-05 |                585 | 0.154062    | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| SCPEP1   | blood vessel endothelial cell                           |      1.71868 |           23 |       0.03125     |          2.83056e-05 |                585 | 0.12682     | 0.239063  | 0.261568  | 0.913962 | -0.168862  |  0.113291  |   0.126522  | 0.313953  |       0.173634  |       7 |      2.29934 |
| COL17A1  | suprabasal keratinocyte                                 |      1.88917 |           12 |       0.00657174  |          1.47682e-05 |                417 | 0.131481    | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| EFNA1    | suprabasal keratinocyte                                 |      1.81992 |          370 |       0.202629    |          0.000455352 |                417 | 0.417031    | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| HSPA1A   | suprabasal keratinocyte                                 |      2.42656 |            1 |       0.000547645 |          1.23068e-06 |                417 | 0.254379    | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| IFI27    | suprabasal keratinocyte                                 |      1.68748 |            3 |       0.00164294  |          3.69204e-06 |                417 | 0.0742586   | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| KRT14    | suprabasal keratinocyte                                 |      1.75237 |           10 |       0.00547645  |          1.23068e-05 |                417 | 0.0961366   | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| LGALS3BP | suprabasal keratinocyte                                 |      1.74968 |           19 |       0.0104053   |          2.33829e-05 |                417 | 0.103083    | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| SCPEP1   | suprabasal keratinocyte                                 |      1.8543  |            2 |       0.00109529  |          2.46136e-06 |                417 | 0.114449    | 0.123703  | 0.170117  | 0.727162 |  2.39987   |  0.0356631 |   0.0598275 | 0.192192  |       0.406996  |       7 |      1.94876 |
| COL17A1  | neutrophil                                              |      2.35694 |            7 |       0.0218069   |          8.61476e-06 |                105 | 0.269284    | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| EFNA1    | neutrophil                                              |      1.51899 |            1 |       0.00311526  |          1.23068e-06 |                105 | 0.034972    | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| GAMT     | neutrophil                                              |      1.89927 |            2 |       0.00623053  |          2.46136e-06 |                105 | 0.133217    | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| HSPA1A   | neutrophil                                              |      2.15891 |           86 |       0.267913    |          0.000105838 |                105 | 0.591603    | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| IFI27    | neutrophil                                              |      1.51927 |            1 |       0.00311526  |          1.23068e-06 |                105 | 0.0350405   | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| RBM42    | neutrophil                                              |      2.08825 |            5 |       0.0155763   |          6.1534e-06  |                105 | 0.193793    | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| SCPEP1   | neutrophil                                              |      2.03322 |            3 |       0.00934579  |          3.69204e-06 |                105 | 0.170864    | 0.190538  | 0.204111  | 0.933504 |  3.17725   |  0.0806377 |   0.0846187 | 0.326691  |       0.477619  |       7 |      1.89517 |
| COL17A1  | hematopoietic precursor cell                            |      1.69847 |            3 |       0.0681818   |          3.69204e-06 |                651 | 0.176771    | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| GAMT     | hematopoietic precursor cell                            |      1.66424 |           59 |       0.0420527   |          7.26101e-05 |                651 | 0.130487    | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| HSPA1A   | hematopoietic precursor cell                            |      3.00193 |          449 |       0.309442    |          0.000552575 |                651 | 0.869895    | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| IFI27    | hematopoietic precursor cell                            |      1.43986 |            1 |       0.00833333  |          1.23068e-06 |                651 | 0.0233331   | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| LGALS3BP | hematopoietic precursor cell                            |      1.98995 |            4 |       0.0160643   |          4.92272e-06 |                651 | 0.17032     | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| RBM42    | hematopoietic precursor cell                            |      1.68981 |           62 |       0.0493631   |          7.63022e-05 |                651 | 0.147812    | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| SCPEP1   | hematopoietic precursor cell                            |      1.68626 |           73 |       0.0636443   |          8.98396e-05 |                651 | 0.168623    | 0.282295  | 0.241034  | 1.17118  |  6.27904   |  0.142383  |   0.111547  | 0.481616  |       0.759408  |       7 |      1.67819 |
| EFNA1    | cell of skeletal muscle                                 |      1.75498 |           26 |       0.00440902  |          3.19977e-05 |               3085 | 0.0955577   | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| GAMT     | cell of skeletal muscle                                 |      1.74092 |          317 |       0.0537561   |          0.000390126 |               3085 | 0.173031    | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| HSPA1A   | cell of skeletal muscle                                 |      3.11061 |         1853 |       0.314228    |          0.00228045  |               3085 | 0.937148    | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| IFI27    | cell of skeletal muscle                                 |      1.58793 |           17 |       0.0029473   |          2.09216e-05 |               3085 | 0.0520584   | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| LGALS3BP | cell of skeletal muscle                                 |      1.58343 |          105 |       0.0182039   |          0.000129221 |               3085 | 0.0759272   | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| RBM42    | cell of skeletal muscle                                 |      1.61349 |          659 |       0.111752    |          0.000811018 |               3085 | 0.236797    | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| SCPEP1   | cell of skeletal muscle                                 |      1.5499  |          108 |       0.018724    |          0.000132913 |               3085 | 0.0685289   | 0.316959  | 0.23415   | 1.35366  |  6.02534   |  0.165709  |   0.106526  | 0.600559  |       0.736361  |       7 |      1.57014 |
| EFNA1    | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.57504 |           13 |       0.00294118  |          1.59988e-05 |               2873 | 0.048783    | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| GAMT     | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.597   |           48 |       0.0108597   |          5.90726e-05 |               2873 | 0.0668945   | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| HSPA1A   | naive thymus-derived CD8-positive, alpha-beta T cell    |      2.68139 |         2029 |       0.45905     |          0.00249705  |               2873 | 1.05297     | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| KRT14    | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.71923 |            7 |       0.00158371  |          8.61476e-06 |               2873 | 0.0820749   | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| LGALS3BP | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.54715 |           30 |       0.00678733  |          3.69204e-05 |               2873 | 0.048096    | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| RBM42    | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.71362 |          690 |       0.156109    |          0.000849169 |               2873 | 0.3287      | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| SCPEP1   | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.60199 |           56 |       0.0126697   |          6.89181e-05 |               2873 | 0.0710285   | 0.370943  | 0.24265   | 1.52872  |  5.35978   |  0.202035  |   0.112725  | 0.714664  |       0.675896  |       7 |      1.53754 |
| EFNA1    | activated CD8-positive, alpha-beta T cell, human        |      1.64463 |           21 |       0.00402685  |          2.58443e-05 |               5010 | 0.0677206   | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| GAMT     | activated CD8-positive, alpha-beta T cell, human        |      1.57597 |           49 |       0.00939597  |          6.03033e-05 |               5010 | 0.0595505   | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| HSPA1A   | activated CD8-positive, alpha-beta T cell, human        |      4.07181 |         4373 |       0.838543    |          0.00538176  |               5010 | 2.01991     | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| KRT14    | activated CD8-positive, alpha-beta T cell, human        |      1.78231 |            7 |       0.00134228  |          8.61476e-06 |               5010 | 0.0972302   | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| LGALS3BP | activated CD8-positive, alpha-beta T cell, human        |      1.64492 |           57 |       0.01093     |          7.01488e-05 |               5010 | 0.0790028   | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| RBM42    | activated CD8-positive, alpha-beta T cell, human        |      1.67584 |          446 |       0.0855225   |          0.000548883 |               5010 | 0.207735    | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| SCPEP1   | activated CD8-positive, alpha-beta T cell, human        |      1.58395 |           57 |       0.01093     |          7.01488e-05 |               5010 | 0.0640028   | 0.72904   | 0.370736  | 1.96647  |  6.88629   |  0.443004  |   0.206137  | 1         |       0.814576  |       7 |      1.52976 |
| EFNA1    | skeletal muscle satellite cell                          |      1.58024 |           15 |       0.00260055  |          1.84602e-05 |               2981 | 0.0495996   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| GAMT     | skeletal muscle satellite cell                          |      1.60194 |          242 |       0.0419556   |          0.000297825 |               2981 | 0.119361    | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| HSPA1A   | skeletal muscle satellite cell                          |      3.11306 |         1839 |       0.318828    |          0.00226322  |               2981 | 0.944318    | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| IFI27    | skeletal muscle satellite cell                          |      1.58793 |           17 |       0.0029473   |          2.09216e-05 |               2981 | 0.0520584   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| LGALS3BP | skeletal muscle satellite cell                          |      1.58343 |          105 |       0.0182039   |          0.000129221 |               2981 | 0.0759272   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| RBM42    | skeletal muscle satellite cell                          |      1.61323 |          655 |       0.113558    |          0.000806095 |               2981 | 0.239347    | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| SCPEP1   | skeletal muscle satellite cell                          |      1.5499  |          108 |       0.018724    |          0.000132913 |               2981 | 0.0685289   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| EFNA1    | muscle precursor cell                                   |      1.58024 |           15 |       0.00260055  |          1.84602e-05 |               2981 | 0.0495996   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| GAMT     | muscle precursor cell                                   |      1.60194 |          242 |       0.0419556   |          0.000297825 |               2981 | 0.119361    | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| HSPA1A   | muscle precursor cell                                   |      3.11306 |         1839 |       0.318828    |          0.00226322  |               2981 | 0.944318    | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| IFI27    | muscle precursor cell                                   |      1.58793 |           17 |       0.0029473   |          2.09216e-05 |               2981 | 0.0520584   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| LGALS3BP | muscle precursor cell                                   |      1.58343 |          105 |       0.0182039   |          0.000129221 |               2981 | 0.0759272   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| RBM42    | muscle precursor cell                                   |      1.61323 |          655 |       0.113558    |          0.000806095 |               2981 | 0.239347    | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| SCPEP1   | muscle precursor cell                                   |      1.5499  |          108 |       0.018724    |          0.000132913 |               2981 | 0.0685289   | 0.325594  | 0.221306  | 1.47124  |  6.06291   |  0.171519  |   0.097159  | 0.6772    |       0.739774  |       7 |      1.46413 |
| EFNA1    | central memory CD8-positive, alpha-beta T cell          |      1.64074 |            5 |       0.000289687 |          6.1534e-06  |               8656 | 0.0607775   | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| GAMT     | central memory CD8-positive, alpha-beta T cell          |      1.59918 |           90 |       0.00521437  |          0.000110761 |               8656 | 0.0599617   | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| HSPA1A   | central memory CD8-positive, alpha-beta T cell          |      3.37442 |         6293 |       0.3646      |          0.00774467  |               8656 | 1.18303     | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| KRT14    | central memory CD8-positive, alpha-beta T cell          |      1.61031 |           30 |       0.00173812  |          3.69204e-05 |               8656 | 0.05606     | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| LGALS3BP | central memory CD8-positive, alpha-beta T cell          |      1.61294 |           93 |       0.00538818  |          0.000114453 |               8656 | 0.0636775   | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| RBM42    | central memory CD8-positive, alpha-beta T cell          |      1.67662 |         1978 |       0.1146      |          0.00243428  |               8656 | 0.28792     | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| SCPEP1   | central memory CD8-positive, alpha-beta T cell          |      1.58884 |          167 |       0.00967555  |          0.000205524 |               8656 | 0.0659361   | 0.418332  | 0.25391   | 1.64756  |  6.06088   |  0.233924  |   0.120937  | 0.792132  |       0.73959   |       7 |      1.45664 |
| EFNA1    | naive thymus-derived CD4-positive, alpha-beta T cell    |      1.65535 |           28 |       0.00845155  |          3.4459e-05  |               1162 | 0.0771621   | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| GAMT     | naive thymus-derived CD4-positive, alpha-beta T cell    |      1.55486 |           44 |       0.013281    |          5.41499e-05 |               1162 | 0.0600654   | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| HSPA1A   | naive thymus-derived CD4-positive, alpha-beta T cell    |      3.64869 |          965 |       0.291277    |          0.00118761  |               1162 | 1.014       | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| IFI27    | naive thymus-derived CD4-positive, alpha-beta T cell    |      2.19368 |            1 |       0.000301841 |          1.23068e-06 |               1162 | 0.196721    | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| LGALS3BP | naive thymus-derived CD4-positive, alpha-beta T cell    |      1.6614  |           35 |       0.0105644   |          4.30738e-05 |               1162 | 0.0819843   | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| RBM42    | naive thymus-derived CD4-positive, alpha-beta T cell    |      1.61045 |           82 |       0.024751    |          0.000100916 |               1162 | 0.0918489   | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| SCPEP1   | naive thymus-derived CD4-positive, alpha-beta T cell    |      1.55478 |            7 |       0.00211289  |          8.61476e-06 |               1162 | 0.0424164   | 0.352109  | 0.223457  | 1.57573  |  6.54719   |  0.189361  |   0.0987278 | 0.745311  |       0.78377   |       7 |      1.38086 |
| GAMT     | muscle fibroblast                                       |      1.9998  |            3 |       0.0177515   |          3.69204e-06 |                105 | 0.175252    | 0.225436  | 0.332608  | 0.677784 |  2.05863   |  0.104121  |   0.178331  | 0.160007  |       0.375996  |       6 |      2.16492 |
| HSPA1A   | muscle fibroblast                                       |      2.05923 |            1 |       0.00591716  |          1.23068e-06 |                105 | 0.172071    | 0.225436  | 0.332608  | 0.677784 |  2.05863   |  0.104121  |   0.178331  | 0.160007  |       0.375996  |       6 |      2.16492 |
| IFI27    | muscle fibroblast                                       |      2.09803 |            6 |       0.035503    |          7.38408e-06 |                105 | 0.226113    | 0.225436  | 0.332608  | 0.677784 |  2.05863   |  0.104121  |   0.178331  | 0.160007  |       0.375996  |       6 |      2.16492 |
| LGALS3BP | muscle fibroblast                                       |      2.29223 |           25 |       0.147929    |          3.0767e-05  |                105 | 0.442974    | 0.225436  | 0.332608  | 0.677784 |  2.05863   |  0.104121  |   0.178331  | 0.160007  |       0.375996  |       6 |      2.16492 |
| RBM42    | muscle fibroblast                                       |      2.07413 |           65 |       0.384615    |          7.99942e-05 |                105 | 0.745303    | 0.225436  | 0.332608  | 0.677784 |  2.05863   |  0.104121  |   0.178331  | 0.160007  |       0.375996  |       6 |      2.16492 |
| SCPEP1   | muscle fibroblast                                       |      2.16601 |            5 |       0.0295858   |          6.1534e-06  |                105 | 0.233936    | 0.225436  | 0.332608  | 0.677784 |  2.05863   |  0.104121  |   0.178331  | 0.160007  |       0.375996  |       6 |      2.16492 |
| COL17A1  | myeloid lineage restricted progenitor cell              |      1.69847 |            3 |       0.0681818   |          3.69204e-06 |                358 | 0.176771    | 0.302208  | 0.321603  | 0.939692 |  4.57677   |  0.155782  |   0.170305  | 0.330724  |       0.604762  |       6 |      1.86052 |
| GAMT     | myeloid lineage restricted progenitor cell              |      1.77653 |           19 |       0.0286576   |          2.33829e-05 |                358 | 0.137066    | 0.302208  | 0.321603  | 0.939692 |  4.57677   |  0.155782  |   0.170305  | 0.330724  |       0.604762  |       6 |      1.86052 |
| HSPA1A   | myeloid lineage restricted progenitor cell              |      2.9989  |          244 |       0.34512     |          0.000300286 |                358 | 0.9178      | 0.302208  | 0.321603  | 0.939692 |  4.57677   |  0.155782  |   0.170305  | 0.330724  |       0.604762  |       6 |      1.86052 |
| LGALS3BP | myeloid lineage restricted progenitor cell              |      2.60733 |            1 |       0.037037    |          1.23068e-06 |                358 | 0.353581    | 0.302208  | 0.321603  | 0.939692 |  4.57677   |  0.155782  |   0.170305  | 0.330724  |       0.604762  |       6 |      1.86052 |
| RBM42    | myeloid lineage restricted progenitor cell              |      1.73506 |           34 |       0.0480905   |          4.18431e-05 |                358 | 0.15637     | 0.302208  | 0.321603  | 0.939692 |  4.57677   |  0.155782  |   0.170305  | 0.330724  |       0.604762  |       6 |      1.86052 |
| SCPEP1   | myeloid lineage restricted progenitor cell              |      1.66317 |           57 |       0.0806223   |          7.01488e-05 |                358 | 0.18803     | 0.302208  | 0.321603  | 0.939692 |  4.57677   |  0.155782  |   0.170305  | 0.330724  |       0.604762  |       6 |      1.86052 |
| EFNA1    | precursor B cell                                        |      1.77939 |            3 |       0.029703    |          3.69204e-06 |                215 | 0.138958    | 0.274598  | 0.266515  | 1.03033  |  5.15209   |  0.137203  |   0.13013   | 0.389802  |       0.657029  |       6 |      1.63612 |
| GAMT     | precursor B cell                                        |      1.65079 |           24 |       0.0533333   |          2.95363e-05 |                215 | 0.143267    | 0.274598  | 0.266515  | 1.03033  |  5.15209   |  0.137203  |   0.13013   | 0.389802  |       0.657029  |       6 |      1.63612 |
| HSPA1A   | precursor B cell                                        |      3.34609 |          100 |       0.222222    |          0.000123068 |                215 | 0.815439    | 0.274598  | 0.266515  | 1.03033  |  5.15209   |  0.137203  |   0.13013   | 0.389802  |       0.657029  |       6 |      1.63612 |
| LGALS3BP | precursor B cell                                        |      1.58173 |            7 |       0.0216718   |          8.61476e-06 |                215 | 0.0783844   | 0.274598  | 0.266515  | 1.03033  |  5.15209   |  0.137203  |   0.13013   | 0.389802  |       0.657029  |       6 |      1.63612 |
| RBM42    | precursor B cell                                        |      1.69821 |           52 |       0.115556    |          6.39954e-05 |                215 | 0.248931    | 0.274598  | 0.266515  | 1.03033  |  5.15209   |  0.137203  |   0.13013   | 0.389802  |       0.657029  |       6 |      1.63612 |
| SCPEP1   | precursor B cell                                        |      1.70794 |           29 |       0.0644444   |          3.56897e-05 |                215 | 0.174111    | 0.274598  | 0.266515  | 1.03033  |  5.15209   |  0.137203  |   0.13013   | 0.389802  |       0.657029  |       6 |      1.63612 |
| GAMT     | T follicular helper cell                                |      1.54709 |           10 |       0.00325839  |          1.23068e-05 |               2906 | 0.0423135   | 0.719337  | 0.403852  | 1.78119  |  5.55983   |  0.436475  |   0.230289  | 0.879231  |       0.694071  |       6 |      1.56135 |
| HSPA1A   | T follicular helper cell                                |      4.06852 |         2341 |       0.762789    |          0.00288102  |               2906 | 1.85722     | 0.719337  | 0.403852  | 1.78119  |  5.55983   |  0.436475  |   0.230289  | 0.879231  |       0.694071  |       6 |      1.56135 |
| KRT14    | T follicular helper cell                                |      1.78463 |           17 |       0.00553926  |          2.09216e-05 |               2906 | 0.104335    | 0.719337  | 0.403852  | 1.78119  |  5.55983   |  0.436475  |   0.230289  | 0.879231  |       0.694071  |       6 |      1.56135 |
| LGALS3BP | T follicular helper cell                                |      1.57704 |           28 |       0.00912349  |          3.4459e-05  |               2906 | 0.0589054   | 0.719337  | 0.403852  | 1.78119  |  5.55983   |  0.436475  |   0.230289  | 0.879231  |       0.694071  |       6 |      1.56135 |
| RBM42    | T follicular helper cell                                |      1.66144 |          484 |       0.157706    |          0.000595649 |               2906 | 0.31337     | 0.719337  | 0.403852  | 1.78119  |  5.55983   |  0.436475  |   0.230289  | 0.879231  |       0.694071  |       6 |      1.56135 |
| SCPEP1   | T follicular helper cell                                |      1.5327  |           26 |       0.00847181  |          3.19977e-05 |               2906 | 0.0469733   | 0.719337  | 0.403852  | 1.78119  |  5.55983   |  0.436475  |   0.230289  | 0.879231  |       0.694071  |       6 |      1.56135 |
| GAMT     | CD8-positive, alpha-beta regulatory T cell              |      1.54218 |           44 |       0.00525499  |          5.41499e-05 |               4446 | 0.0449076   | 0.390988  | 0.279145  | 1.40066  |  4.47838   |  0.215523  |   0.13934   | 0.631195  |       0.595823  |       6 |      1.5197  |
| HSPA1A   | CD8-positive, alpha-beta regulatory T cell              |      3.22247 |         2954 |       0.352801    |          0.00363543  |               4446 | 1.04867     | 0.390988  | 0.279145  | 1.40066  |  4.47838   |  0.215523  |   0.13934   | 0.631195  |       0.595823  |       6 |      1.5197  |
| KRT14    | CD8-positive, alpha-beta regulatory T cell              |      1.78774 |           42 |       0.00501612  |          5.16886e-05 |               4446 | 0.104907    | 0.390988  | 0.279145  | 1.40066  |  4.47838   |  0.215523  |   0.13934   | 0.631195  |       0.595823  |       6 |      1.5197  |
| LGALS3BP | CD8-positive, alpha-beta regulatory T cell              |      1.64866 |           48 |       0.00573271  |          5.90726e-05 |               4446 | 0.0719132   | 0.390988  | 0.279145  | 1.40066  |  4.47838   |  0.215523  |   0.13934   | 0.631195  |       0.595823  |       6 |      1.5197  |
| RBM42    | CD8-positive, alpha-beta regulatory T cell              |      1.70029 |         1253 |       0.149648    |          0.00154204  |               4446 | 0.329099    | 0.390988  | 0.279145  | 1.40066  |  4.47838   |  0.215523  |   0.13934   | 0.631195  |       0.595823  |       6 |      1.5197  |
| SCPEP1   | CD8-positive, alpha-beta regulatory T cell              |      1.61572 |          105 |       0.0125403   |          0.000129221 |               4446 | 0.0753732   | 0.390988  | 0.279145  | 1.40066  |  4.47838   |  0.215523  |   0.13934   | 0.631195  |       0.595823  |       6 |      1.5197  |
| EFNA1    | neuron                                                  |      1.9318  |            4 |       0.00612557  |          4.92272e-06 |                827 | 0.141107    | 0.247321  | 0.230306  | 1.07388  |  5.83406   |  0.118848  |   0.103723  | 0.418189  |       0.718984  |       6 |      1.48202 |
| GAMT     | neuron                                                  |      1.60593 |           83 |       0.0437533   |          0.000102146 |                827 | 0.119262    | 0.247321  | 0.230306  | 1.07388  |  5.83406   |  0.118848  |   0.103723  | 0.418189  |       0.718984  |       6 |      1.48202 |
| HSPA1A   | neuron                                                  |      2.71733 |          501 |       0.264101    |          0.000616571 |                827 | 0.733108    | 0.247321  | 0.230306  | 1.07388  |  5.83406   |  0.118848  |   0.103723  | 0.418189  |       0.718984  |       6 |      1.48202 |
| LGALS3BP | neuron                                                  |      1.64852 |           13 |       0.0199081   |          1.59988e-05 |                827 | 0.0923101   | 0.247321  | 0.230306  | 1.07388  |  5.83406   |  0.118848  |   0.103723  | 0.418189  |       0.718984  |       6 |      1.48202 |
| RBM42    | neuron                                                  |      1.62649 |          123 |       0.0648392   |          0.000151374 |                827 | 0.156899    | 0.247321  | 0.230306  | 1.07388  |  5.83406   |  0.118848  |   0.103723  | 0.418189  |       0.718984  |       6 |      1.48202 |
| SCPEP1   | neuron                                                  |      1.62056 |          103 |       0.0542963   |          0.00012676  |                827 | 0.139152    | 0.247321  | 0.230306  | 1.07388  |  5.83406   |  0.118848  |   0.103723  | 0.418189  |       0.718984  |       6 |      1.48202 |
| GAMT     | CD4-positive helper T cell                              |      1.51964 |           14 |       0.00291121  |          1.72295e-05 |               3987 | 0.0351355   | 0.625804  | 0.359663  | 1.73997  |  5.52127   |  0.373535  |   0.198062  | 0.852365  |       0.690568  |       6 |      1.48198 |
| HSPA1A   | CD4-positive helper T cell                              |      3.60764 |         3214 |       0.66833     |          0.00395541  |               3987 | 1.62289     | 0.625804  | 0.359663  | 1.73997  |  5.52127   |  0.373535  |   0.198062  | 0.852365  |       0.690568  |       6 |      1.48198 |
| KRT14    | CD4-positive helper T cell                              |      1.78463 |           17 |       0.00553926  |          2.09216e-05 |               3987 | 0.104335    | 0.625804  | 0.359663  | 1.73997  |  5.52127   |  0.373535  |   0.198062  | 0.852365  |       0.690568  |       6 |      1.48198 |
| LGALS3BP | CD4-positive helper T cell                              |      1.58012 |           38 |       0.00790185  |          4.67658e-05 |               3987 | 0.0580694   | 0.625804  | 0.359663  | 1.73997  |  5.52127   |  0.373535  |   0.198062  | 0.852365  |       0.690568  |       6 |      1.48198 |
| RBM42    | CD4-positive helper T cell                              |      1.65354 |          667 |       0.138698    |          0.000820863 |               3987 | 0.287261    | 0.625804  | 0.359663  | 1.73997  |  5.52127   |  0.373535  |   0.198062  | 0.852365  |       0.690568  |       6 |      1.48198 |
| SCPEP1   | CD4-positive helper T cell                              |      1.54988 |           37 |       0.00769391  |          4.55352e-05 |               3987 | 0.050294    | 0.625804  | 0.359663  | 1.73997  |  5.52127   |  0.373535  |   0.198062  | 0.852365  |       0.690568  |       6 |      1.48198 |
| EFNA1    | B cell                                                  |      1.61334 |            6 |       0.000994036 |          7.38408e-06 |               6920 | 0.055118    | 0.674496  | 0.362289  | 1.86176  |  5.92111   |  0.406301  |   0.199977  | 0.931752  |       0.726891  |       6 |      1.40094 |
| GAMT     | B cell                                                  |      1.61868 |           55 |       0.00711606  |          6.76874e-05 |               6920 | 0.0667797   | 0.674496  | 0.362289  | 1.86176  |  5.92111   |  0.406301  |   0.199977  | 0.931752  |       0.726891  |       6 |      1.40094 |
| HSPA1A   | B cell                                                  |      3.18786 |         6101 |       0.767228    |          0.00750838  |               6920 | 1.73652     | 0.674496  | 0.362289  | 1.86176  |  5.92111   |  0.406301  |   0.199977  | 0.931752  |       0.726891  |       6 |      1.40094 |
| LGALS3BP | B cell                                                  |      1.56378 |           12 |       0.00157853  |          1.47682e-05 |               6920 | 0.0439453   | 0.674496  | 0.362289  | 1.86176  |  5.92111   |  0.406301  |   0.199977  | 0.931752  |       0.726891  |       6 |      1.40094 |
| RBM42    | B cell                                                  |      1.60735 |          467 |       0.0587274   |          0.000574727 |               6920 | 0.151191    | 0.674496  | 0.362289  | 1.86176  |  5.92111   |  0.406301  |   0.199977  | 0.931752  |       0.726891  |       6 |      1.40094 |
| SCPEP1   | B cell                                                  |      1.64225 |          279 |       0.035303    |          0.00034336  |               6920 | 0.120175    | 0.674496  | 0.362289  | 1.86176  |  5.92111   |  0.406301  |   0.199977  | 0.931752  |       0.726891  |       6 |      1.40094 |
| GAMT     | mature NK T cell                                        |      1.59269 |            1 |       0.000980392 |          1.23068e-06 |                849 | 0.0498998   | 0.60169   | 0.333448  | 1.80445  |  5.86318   |  0.357308  |   0.178944  | 0.894391  |       0.721628  |       6 |      1.36757 |
| HSPA1A   | mature NK T cell                                        |      3.25101 |          737 |       0.722549    |          0.000907011 |                849 | 1.55767     | 0.60169   | 0.333448  | 1.80445  |  5.86318   |  0.357308  |   0.178944  | 0.894391  |       0.721628  |       6 |      1.36757 |
| KRT14    | mature NK T cell                                        |      1.7686  |            5 |       0.00490196  |          6.1534e-06  |                849 | 0.0991508   | 0.60169   | 0.333448  | 1.80445  |  5.86318   |  0.357308  |   0.178944  | 0.894391  |       0.721628  |       6 |      1.36757 |
| LGALS3BP | mature NK T cell                                        |      1.5144  |            2 |       0.00196078  |          2.46136e-06 |                849 | 0.0321351   | 0.60169   | 0.333448  | 1.80445  |  5.86318   |  0.357308  |   0.178944  | 0.894391  |       0.721628  |       6 |      1.36757 |
| RBM42    | mature NK T cell                                        |      1.56531 |           87 |       0.0852941   |          0.000107069 |                849 | 0.171677    | 0.60169   | 0.333448  | 1.80445  |  5.86318   |  0.357308  |   0.178944  | 0.894391  |       0.721628  |       6 |      1.36757 |
| SCPEP1   | mature NK T cell                                        |      1.65917 |           17 |       0.0166667   |          2.09216e-05 |                849 | 0.0901616   | 0.60169   | 0.333448  | 1.80445  |  5.86318   |  0.357308  |   0.178944  | 0.894391  |       0.721628  |       6 |      1.36757 |
| EFNA1    | group 3 innate lymphoid cell                            |      1.4744  |            2 |       0.00117165  |          2.46136e-06 |                348 | 0.0211112   | 0.247195  | 0.159634  | 1.54852  |  5.6878    |  0.118763  |   0.0521823 | 0.727569  |       0.705696  |       6 |      1.08795 |
| GAMT     | group 3 innate lymphoid cell                            |      1.59299 |           39 |       0.0228471   |          4.79965e-05 |                348 | 0.0836757   | 0.247195  | 0.159634  | 1.54852  |  5.6878    |  0.118763  |   0.0521823 | 0.727569  |       0.705696  |       6 |      1.08795 |
| HSPA1A   | group 3 innate lymphoid cell                            |      3.24405 |          228 |       0.133568    |          0.000280595 |                348 | 0.660397    | 0.247195  | 0.159634  | 1.54852  |  5.6878    |  0.118763  |   0.0521823 | 0.727569  |       0.705696  |       6 |      1.08795 |
| IFI27    | group 3 innate lymphoid cell                            |      1.50221 |            1 |       0.000585823 |          1.23068e-06 |                348 | 0.0270511   | 0.247195  | 0.159634  | 1.54852  |  5.6878    |  0.118763  |   0.0521823 | 0.727569  |       0.705696  |       6 |      1.08795 |
| RBM42    | group 3 innate lymphoid cell                            |      1.57458 |           59 |       0.0345636   |          7.26101e-05 |                348 | 0.0971958   | 0.247195  | 0.159634  | 1.54852  |  5.6878    |  0.118763  |   0.0521823 | 0.727569  |       0.705696  |       6 |      1.08795 |
| SCPEP1   | group 3 innate lymphoid cell                            |      1.60415 |           19 |       0.0111306   |          2.33829e-05 |                348 | 0.0683717   | 0.247195  | 0.159634  | 1.54852  |  5.6878    |  0.118763  |   0.0521823 | 0.727569  |       0.705696  |       6 |      1.08795 |
| EFNA1    | mesenchymal stem cell                                   |      2.8717  |            5 |       0.0174216   |          6.1534e-06  |                159 | 0.389286    | 0.209277  | 0.484093  | 0.432308 |  4.80638   |  0.0932477 |   0.288807  | 0         |       0.625622  |       5 |      2.15139 |
| GAMT     | mesenchymal stem cell                                   |      2.59288 |           12 |       0.0418118   |          1.47682e-05 |                159 | 0.35745     | 0.209277  | 0.484093  | 0.432308 |  4.80638   |  0.0932477 |   0.288807  | 0         |       0.625622  |       5 |      2.15139 |
| HSPA1A   | mesenchymal stem cell                                   |      2.80772 |           97 |       0.337979    |          0.000119376 |                159 | 0.856568    | 0.209277  | 0.484093  | 0.432308 |  4.80638   |  0.0932477 |   0.288807  | 0         |       0.625622  |       5 |      2.15139 |
| RBM42    | mesenchymal stem cell                                   |      2.55348 |           23 |       0.0801394   |          2.83056e-05 |                159 | 0.405509    | 0.209277  | 0.484093  | 0.432308 |  4.80638   |  0.0932477 |   0.288807  | 0         |       0.625622  |       5 |      2.15139 |
| SCPEP1   | mesenchymal stem cell                                   |      2.59979 |           22 |       0.0766551   |          2.7075e-05  |                159 | 0.411651    | 0.209277  | 0.484093  | 0.432308 |  4.80638   |  0.0932477 |   0.288807  | 0         |       0.625622  |       5 |      2.15139 |
| GAMT     | immature B cell                                         |      1.65071 |           14 |       0.0363636   |          1.72295e-05 |                310 | 0.117556    | 0.601996  | 0.408742  | 1.4728   |  4.71236   |  0.357514  |   0.233854  | 0.678219  |       0.61708   |       5 |      1.49208 |
| HSPA1A   | immature B cell                                         |      3.80141 |          226 |       0.587013    |          0.000278134 |                310 | 1.47762     | 0.601996  | 0.408742  | 1.4728   |  4.71236   |  0.357514  |   0.233854  | 0.678219  |       0.61708   |       5 |      1.49208 |
| LGALS3BP | immature B cell                                         |      1.52038 |            1 |       0.0025974   |          1.23068e-06 |                310 | 0.0345383   | 0.601996  | 0.408742  | 1.4728   |  4.71236   |  0.357514  |   0.233854  | 0.678219  |       0.61708   |       5 |      1.49208 |
| RBM42    | immature B cell                                         |      1.62108 |           44 |       0.114286    |          5.41499e-05 |                310 | 0.227861    | 0.601996  | 0.408742  | 1.4728   |  4.71236   |  0.357514  |   0.233854  | 0.678219  |       0.61708   |       5 |      1.49208 |
| SCPEP1   | immature B cell                                         |      1.75419 |           25 |       0.0649351   |          3.0767e-05  |                310 | 0.186131    | 0.601996  | 0.408742  | 1.4728   |  4.71236   |  0.357514  |   0.233854  | 0.678219  |       0.61708   |       5 |      1.49208 |
| GAMT     | small pre-B-II cell                                     |      1.63987 |            7 |       0.0315315   |          8.61476e-06 |                105 | 0.107474    | 0.306932  | 0.295212  | 1.0397   |  3.92762   |  0.158961  |   0.151058  | 0.395912  |       0.545789  |       5 |      1.47837 |
| HSPA1A   | small pre-B-II cell                                     |      3.41401 |           49 |       0.220721    |          6.03033e-05 |                105 | 0.828683    | 0.306932  | 0.295212  | 1.0397   |  3.92762   |  0.158961  |   0.151058  | 0.395912  |       0.545789  |       5 |      1.47837 |
| LGALS3BP | small pre-B-II cell                                     |      1.66686 |            1 |       0.0045045   |          1.23068e-06 |                105 | 0.0734321   | 0.306932  | 0.295212  | 1.0397   |  3.92762   |  0.158961  |   0.151058  | 0.395912  |       0.545789  |       5 |      1.47837 |
| RBM42    | small pre-B-II cell                                     |      1.65058 |           24 |       0.108108    |          2.95363e-05 |                105 | 0.225378    | 0.306932  | 0.295212  | 1.0397   |  3.92762   |  0.158961  |   0.151058  | 0.395912  |       0.545789  |       5 |      1.47837 |
| SCPEP1   | small pre-B-II cell                                     |      1.71446 |           24 |       0.108108    |          2.95363e-05 |                105 | 0.241093    | 0.306932  | 0.295212  | 1.0397   |  3.92762   |  0.158961  |   0.151058  | 0.395912  |       0.545789  |       5 |      1.47837 |
| GAMT     | pro-B cell                                              |      1.64269 |           24 |       0.0866426   |          2.95363e-05 |                101 | 0.191238    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| HSPA1A   | pro-B cell                                              |      2.77697 |           51 |       0.184116    |          6.27647e-05 |                101 | 0.617115    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| LGALS3BP | pro-B cell                                              |      1.51671 |            2 |       0.0196078   |          2.46136e-06 |                101 | 0.0591739   | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| RBM42    | pro-B cell                                              |      1.61114 |           22 |       0.0794224   |          2.7075e-05  |                101 | 0.172601    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| SCPEP1   | pro-B cell                                              |      2.17846 |            2 |       0.0114286   |          2.46136e-06 |                101 | 0.209691    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| GAMT     | lymphoid lineage restricted progenitor cell             |      1.64269 |           24 |       0.0866426   |          2.95363e-05 |                101 | 0.191238    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| HSPA1A   | lymphoid lineage restricted progenitor cell             |      2.77697 |           51 |       0.184116    |          6.27647e-05 |                101 | 0.617115    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| LGALS3BP | lymphoid lineage restricted progenitor cell             |      1.51671 |            2 |       0.0196078   |          2.46136e-06 |                101 | 0.0591739   | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| RBM42    | lymphoid lineage restricted progenitor cell             |      1.61114 |           22 |       0.0794224   |          2.7075e-05  |                101 | 0.172601    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| SCPEP1   | lymphoid lineage restricted progenitor cell             |      2.17846 |            2 |       0.0114286   |          2.46136e-06 |                101 | 0.209691    | 0.213457  | 0.249964  | 0.853954 |  3.76877   |  0.0960606 |   0.118059  | 0.274838  |       0.531358  |       5 |      1.46208 |
| GAMT     | effector CD8-positive, alpha-beta T cell                |      1.68828 |            3 |       0.00309598  |          3.69204e-06 |                826 | 0.0766347   | 0.565649  | 0.357209  | 1.58352  |  4.62182   |  0.333056  |   0.196272  | 0.750388  |       0.608855  |       5 |      1.3408  |
| HSPA1A   | effector CD8-positive, alpha-beta T cell                |      2.53425 |          687 |       0.708978    |          0.000845477 |                826 | 1.35981     | 0.565649  | 0.357209  | 1.58352  |  4.62182   |  0.333056  |   0.196272  | 0.750388  |       0.608855  |       5 |      1.3408  |
| LGALS3BP | effector CD8-positive, alpha-beta T cell                |      1.64008 |           16 |       0.0165119   |          1.96909e-05 |                826 | 0.0852116   | 0.565649  | 0.357209  | 1.58352  |  4.62182   |  0.333056  |   0.196272  | 0.750388  |       0.608855  |       5 |      1.3408  |
| RBM42    | effector CD8-positive, alpha-beta T cell                |      1.631   |          112 |       0.115583    |          0.000137836 |                826 | 0.233865    | 0.565649  | 0.357209  | 1.58352  |  4.62182   |  0.333056  |   0.196272  | 0.750388  |       0.608855  |       5 |      1.3408  |
| SCPEP1   | effector CD8-positive, alpha-beta T cell                |      1.46891 |            8 |       0.00825593  |          9.84544e-06 |                826 | 0.0305309   | 0.565649  | 0.357209  | 1.58352  |  4.62182   |  0.333056  |   0.196272  | 0.750388  |       0.608855  |       5 |      1.3408  |
| GAMT     | T-helper 17 cell                                        |      1.45102 |            4 |       0.00229885  |          4.92272e-06 |               1081 | 0.0170991   | 0.421236  | 0.272943  | 1.54331  |  4.27586   |  0.235878  |   0.134817  | 0.724177  |       0.577425  |       5 |      1.20312 |
| HSPA1A   | T-helper 17 cell                                        |      2.37176 |          873 |       0.501724    |          0.00107438  |               1081 | 1.01337     | 0.421236  | 0.272943  | 1.54331  |  4.27586   |  0.235878  |   0.134817  | 0.724177  |       0.577425  |       5 |      1.20312 |
| LGALS3BP | T-helper 17 cell                                        |      1.58876 |           10 |       0.00574713  |          1.23068e-05 |               1081 | 0.0562977   | 0.421236  | 0.272943  | 1.54331  |  4.27586   |  0.235878  |   0.134817  | 0.724177  |       0.577425  |       5 |      1.20312 |
| RBM42    | T-helper 17 cell                                        |      1.63265 |          183 |       0.105172    |          0.000225214 |               1081 | 0.22034     | 0.421236  | 0.272943  | 1.54331  |  4.27586   |  0.235878  |   0.134817  | 0.724177  |       0.577425  |       5 |      1.20312 |
| SCPEP1   | T-helper 17 cell                                        |      1.59049 |           11 |       0.00632184  |          1.35375e-05 |               1081 | 0.0576079   | 0.421236  | 0.272943  | 1.54331  |  4.27586   |  0.235878  |   0.134817  | 0.724177  |       0.577425  |       5 |      1.20312 |
| GAMT     | B-2 B cell                                              |      1.51697 |            5 |       0.00521376  |          6.1534e-06  |                293 | 0.0377189   | 0.348671  | 0.2183    | 1.59721  |  4.9199    |  0.187048  |   0.0949673 | 0.759307  |       0.635935  |       5 |      1.04124 |
| HSPA1A   | B-2 B cell                                              |      3.3033  |          234 |       0.244004    |          0.000287979 |                293 | 0.840769    | 0.348671  | 0.2183    | 1.59721  |  4.9199    |  0.187048  |   0.0949673 | 0.759307  |       0.635935  |       5 |      1.04124 |
| LGALS3BP | B-2 B cell                                              |      1.56125 |            2 |       0.00208551  |          2.46136e-06 |                293 | 0.0438482   | 0.348671  | 0.2183    | 1.59721  |  4.9199    |  0.187048  |   0.0949673 | 0.759307  |       0.635935  |       5 |      1.04124 |
| RBM42    | B-2 B cell                                              |      1.59611 |           24 |       0.0250261   |          2.95363e-05 |                293 | 0.0873568   | 0.348671  | 0.2183    | 1.59721  |  4.9199    |  0.187048  |   0.0949673 | 0.759307  |       0.635935  |       5 |      1.04124 |
| SCPEP1   | B-2 B cell                                              |      1.54775 |           28 |       0.0291971   |          3.4459e-05  |                293 | 0.08181     | 0.348671  | 0.2183    | 1.59721  |  4.9199    |  0.187048  |   0.0949673 | 0.759307  |       0.635935  |       5 |      1.04124 |
| GAMT     | mature B cell                                           |      1.51697 |            5 |       0.00521376  |          6.1534e-06  |                334 | 0.0377189   | 0.33603   | 0.211555  | 1.58838  |  4.9227    |  0.178541  |   0.0900479 | 0.753555  |       0.636189  |       5 |      1.03103 |
| HSPA1A   | mature B cell                                           |      3.28633 |          268 |       0.226734    |          0.000329822 |                334 | 0.811497    | 0.33603   | 0.211555  | 1.58838  |  4.9227    |  0.178541  |   0.0900479 | 0.753555  |       0.636189  |       5 |      1.03103 |
| LGALS3BP | mature B cell                                           |      1.56125 |            2 |       0.00208551  |          2.46136e-06 |                334 | 0.0438482   | 0.33603   | 0.211555  | 1.58838  |  4.9227    |  0.178541  |   0.0900479 | 0.753555  |       0.636189  |       5 |      1.03103 |
| RBM42    | mature B cell                                           |      1.5841  |           29 |       0.0245347   |          3.56897e-05 |                334 | 0.0837835   | 0.33603   | 0.211555  | 1.58838  |  4.9227    |  0.178541  |   0.0900479 | 0.753555  |       0.636189  |       5 |      1.03103 |
| SCPEP1   | mature B cell                                           |      1.56054 |           30 |       0.0264784   |          3.69204e-05 |                334 | 0.0809272   | 0.33603   | 0.211555  | 1.58838  |  4.9227    |  0.178541  |   0.0900479 | 0.753555  |       0.636189  |       5 |      1.03103 |
| GAMT     | plasmacytoid dendritic cell, human                      |      1.91788 |            4 |       0.00411523  |          4.92272e-06 |                679 | 0.134668    | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| HSPA1A   | plasmacytoid dendritic cell, human                      |      3.61397 |          566 |       0.582305    |          0.000696565 |                679 | 1.43253     | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| RBM42    | plasmacytoid dendritic cell, human                      |      1.67242 |          101 |       0.103909    |          0.000124299 |                679 | 0.22628     | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| SCPEP1   | plasmacytoid dendritic cell, human                      |      1.69969 |            8 |       0.00823045  |          9.84544e-06 |                679 | 0.0872629   | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| GAMT     | dendritic cell, human                                   |      1.91788 |            4 |       0.00411523  |          4.92272e-06 |                679 | 0.134668    | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| HSPA1A   | dendritic cell, human                                   |      3.61397 |          566 |       0.582305    |          0.000696565 |                679 | 1.43253     | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| RBM42    | dendritic cell, human                                   |      1.67242 |          101 |       0.103909    |          0.000124299 |                679 | 0.22628     | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| SCPEP1   | dendritic cell, human                                   |      1.69969 |            8 |       0.00823045  |          9.84544e-06 |                679 | 0.0872629   | 0.644151  | 0.470184  | 1.37     |  3.83539   |  0.385881  |   0.278664  | 0.611207  |       0.53741   |       4 |      1.36646 |
| GAMT     | megakaryocyte                                           |      1.61952 |           34 |       0.0923913   |          4.18431e-05 |                182 | 0.194399    | 0.393567  | 0.340853  | 1.15465  |  3.76622   |  0.217259  |   0.184344  | 0.47084   |       0.531126  |       4 |      1.22847 |
| HSPA1A   | megakaryocyte                                           |      3.10014 |          124 |       0.336957    |          0.000152604 |                182 | 0.927608    | 0.393567  | 0.340853  | 1.15465  |  3.76622   |  0.217259  |   0.184344  | 0.47084   |       0.531126  |       4 |      1.22847 |
| RBM42    | megakaryocyte                                           |      1.64921 |           22 |       0.0597826   |          2.7075e-05  |                182 | 0.152506    | 0.393567  | 0.340853  | 1.15465  |  3.76622   |  0.217259  |   0.184344  | 0.47084   |       0.531126  |       4 |      1.22847 |
| SCPEP1   | megakaryocyte                                           |      1.72398 |            2 |       0.00543478  |          2.46136e-06 |                182 | 0.0889006   | 0.393567  | 0.340853  | 1.15465  |  3.76622   |  0.217259  |   0.184344  | 0.47084   |       0.531126  |       4 |      1.22847 |
| GAMT     | eurydendroid cell                                       |      1.5887  |           72 |       0.0578778   |          8.8609e-05  |                614 | 0.135951    | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| HSPA1A   | eurydendroid cell                                       |      2.91159 |          379 |       0.304662    |          0.000466428 |                614 | 0.83884     | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| RBM42    | eurydendroid cell                                       |      1.58079 |           62 |       0.0498392   |          7.63022e-05 |                614 | 0.121709    | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| SCPEP1   | eurydendroid cell                                       |      1.61242 |          101 |       0.0811897   |          0.000124299 |                614 | 0.177442    | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| GAMT     | efferent neuron                                         |      1.5887  |           72 |       0.0578778   |          8.8609e-05  |                614 | 0.135951    | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| HSPA1A   | efferent neuron                                         |      2.91159 |          379 |       0.304662    |          0.000466428 |                614 | 0.83884     | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| RBM42    | efferent neuron                                         |      1.58079 |           62 |       0.0498392   |          7.63022e-05 |                614 | 0.121709    | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| SCPEP1   | efferent neuron                                         |      1.61242 |          101 |       0.0811897   |          0.000124299 |                614 | 0.177442    | 0.347708  | 0.318485  | 1.09175  |  3.90465   |  0.1864    |   0.168031  | 0.429842  |       0.543702  |       4 |      1.20651 |
| GAMT     | promonocyte                                             |      1.58959 |           11 |       0.0189003   |          1.35375e-05 |                281 | 0.0762541   | 0.398074  | 0.329009  | 1.20992  |  3.69284   |  0.220292  |   0.175706  | 0.506863  |       0.52446   |       4 |      1.19133 |
| HSPA1A   | promonocyte                                             |      3.09104 |          194 |       0.333333    |          0.000238752 |                281 | 0.921598    | 0.398074  | 0.329009  | 1.20992  |  3.69284   |  0.220292  |   0.175706  | 0.506863  |       0.52446   |       4 |      1.19133 |
| RBM42    | promonocyte                                             |      1.64234 |           24 |       0.0412371   |          2.95363e-05 |                281 | 0.123044    | 0.398074  | 0.329009  | 1.20992  |  3.69284   |  0.220292  |   0.175706  | 0.506863  |       0.52446   |       4 |      1.19133 |
| SCPEP1   | promonocyte                                             |      1.63936 |           52 |       0.0893471   |          6.39954e-05 |                281 | 0.195142    | 0.398074  | 0.329009  | 1.20992  |  3.69284   |  0.220292  |   0.175706  | 0.506863  |       0.52446   |       4 |      1.19133 |
| GAMT     | hematopoietic oligopotent progenitor cell               |      1.60085 |            6 |       0.0237154   |          7.38408e-06 |                113 | 0.0861279   | 0.448245  | 0.338345  | 1.32482  |  3.95364   |  0.254053  |   0.182515  | 0.581758  |       0.548152  |       4 |      1.15746 |
| HSPA1A   | hematopoietic oligopotent progenitor cell               |      3.12997 |           98 |       0.387352    |          0.000120607 |                113 | 1.00992     | 0.448245  | 0.338345  | 1.32482  |  3.95364   |  0.254053  |   0.182515  | 0.581758  |       0.548152  |       4 |      1.15746 |
| RBM42    | hematopoietic oligopotent progenitor cell               |      1.7339  |            3 |       0.0234375   |          3.69204e-06 |                113 | 0.11837     | 0.448245  | 0.338345  | 1.32482  |  3.95364   |  0.254053  |   0.182515  | 0.581758  |       0.548152  |       4 |      1.15746 |
| SCPEP1   | hematopoietic oligopotent progenitor cell               |      1.59799 |            6 |       0.0594059   |          7.38408e-06 |                113 | 0.138961    | 0.448245  | 0.338345  | 1.32482  |  3.95364   |  0.254053  |   0.182515  | 0.581758  |       0.548152  |       4 |      1.15746 |
| GAMT     | group 2 innate lymphoid cell                            |      1.59235 |            4 |       0.0123457   |          4.92272e-06 |                175 | 0.0669355   | 0.622311  | 0.378747  | 1.64308  |  3.94017   |  0.371185  |   0.211979  | 0.789209  |       0.546929  |       4 |      1.11061 |
| HSPA1A   | group 2 innate lymphoid cell                            |      3.77357 |          156 |       0.481481    |          0.000191986 |                175 | 1.31082     | 0.622311  | 0.378747  | 1.64308  |  3.94017   |  0.371185  |   0.211979  | 0.789209  |       0.546929  |       4 |      1.11061 |
| RBM42    | group 2 innate lymphoid cell                            |      1.59835 |           13 |       0.0401235   |          1.59988e-05 |                175 | 0.110291    | 0.622311  | 0.378747  | 1.64308  |  3.94017   |  0.371185  |   0.211979  | 0.789209  |       0.546929  |       4 |      1.11061 |
| SCPEP1   | group 2 innate lymphoid cell                            |      1.46761 |            2 |       0.00617284  |          2.46136e-06 |                175 | 0.0269434   | 0.622311  | 0.378747  | 1.64308  |  3.94017   |  0.371185  |   0.211979  | 0.789209  |       0.546929  |       4 |      1.11061 |

# Analysis using OpenAI and Tavily

In [91]:
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain_core.runnables import RunnableConfig, chain

In [92]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import chain
from tavily import TavilyClient
import ast
import asyncio

In [93]:
from langchain_openai import ChatOpenAI

In [94]:
from dotenv import load_dotenv
load_dotenv()

True

In [95]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini", 
    temperature=0
)

In [96]:
tavily = TavilyClient()

In [97]:
from langchain_community.tools import TavilySearchResults
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

In [98]:
tool = TavilySearchResults(
    max_results=10,
    search_depth="basic",
    include_answer=True,
    include_raw_content=False,
    include_images=False,
)

In [99]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0.0
)

In [100]:
res = tool.invoke('is epithelial cell of skin of body tissue related to these genes: [COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1]?')

In [101]:
llm.invoke(f"answer with yes or no. based on this search results,\n{res}\n, is epithelial cell of skin of body tissue related to these genes: [COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1]?")

AIMessage(content='Yes', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-941f9e1a-7857-4e9d-9838-32112f48a248-0', usage_metadata={'input_tokens': 1272, 'output_tokens': 2, 'total_tokens': 1274, 'input_token_details': {'cache_read': 0}})

In [102]:
tissue = tl.tl_uberon(TARGET_UBERON_ID)
tissue

NameError: name 'tl' is not defined

In [ ]:
cell = final_df['cell'][0]
cell

'epithelial cell'

In [ ]:
genes = [final_df['gene'][0]]
genes

['COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1']

In [ ]:
query = f"is {cell} of {tissue} tissue related to these genes: {genes}?"

In [ ]:
prompt = ChatPromptTemplate(query)

In [ ]:
prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert in cellular biology. answer with a boolean array"),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

In [ ]:
final_df = export_dataframe(results[0])
display_dataframe_as_markdown(final_df)

| cells                                     | genes                                                               |   cell score |
|:------------------------------------------|:--------------------------------------------------------------------|-------------:|
| epithelial cell                           | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      7.67564 |
| epidermal cell                            | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      6.94493 |
| squamous epithelial cell                  | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      6.94387 |
| stratified squamous epithelial cell       | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      6.93894 |
| stratified epithelial cell                | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      6.93894 |
| keratinocyte                              | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      6.93894 |
| basal cell of epidermis                   | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      5.20789 |
| stem cell of epidermis                    | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      5.20348 |
| stem cell                                 | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      5.05776 |
| epithelial fate stem cell                 | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      5.03504 |
| basal cell                                | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      5.03504 |
| progenitor cell                           | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.9887  |
| precursor cell                            | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.91531 |
| prickle cell                              | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.3003  |
| skin fibroblast                           | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.22101 |
| columnar/cuboidal epithelial cell         | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.10285 |
| neuron associated cell (sensu Vertebrata) | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.10111 |
| Merkel cell                               | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.10111 |
| connective tissue cell                    | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      4.05615 |
| fibroblast                                | COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1 |      3.98718 |

In [ ]:
final_df.head(1)

,cells,genes,cell score
0,epithelial cell,"COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LG...",7.675640368487279


In [ ]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

# initialize the agent
agent = create_conversational_retrieval_agent(
    llm,
    tools=[tavily_tool],
)

In [ ]:
def generate_queries(cell_name, gene_names: list):
    return f"answer with yes or no. Is {cell_name} related to this genes: {gene_names}?"

In [ ]:
cell_names = results[0]['cell'].unique()[:20].tolist()

In [ ]:
gene_names = results[0][results[0]['cell'] == cell_names[1]]['gene'].tolist()

In [ ]:
gene_names

['COL17A1',
 'EFNA1',
 'GAMT',
 'HSPA1A',
 'IFI27',
 'KRT14',
 'LGALS3BP',
 'RBM42',
 'SCPEP1']

In [ ]:
cell_names

['epithelial cell',
 'epidermal cell',
 'squamous epithelial cell',
 'stratified squamous epithelial cell',
 'stratified epithelial cell',
 'keratinocyte',
 'basal cell of epidermis',
 'stem cell of epidermis',
 'stem cell',
 'epithelial fate stem cell',
 'basal cell',
 'progenitor cell',
 'precursor cell',
 'prickle cell',
 'skin fibroblast',
 'columnar/cuboidal epithelial cell',
 'neuron associated cell (sensu Vertebrata)',
 'Merkel cell',
 'connective tissue cell',
 'fibroblast']

In [ ]:
test = generate_queries(cell_names[1], gene_names)
test

"answer with yes or no. Is epidermal cell related to this genes: ['COL17A1', 'EFNA1', 'GAMT', 'HSPA1A', 'IFI27', 'KRT14', 'LGALS3BP', 'RBM42', 'SCPEP1']?"

In [ ]:
# run the agent
resp = agent.invoke(test)

In [ ]:
resp

{'input': "answer with yes or no. Is epidermal cell related to this genes: ['COL17A1', 'EFNA1', 'GAMT', 'HSPA1A', 'IFI27', 'KRT14', 'LGALS3BP', 'RBM42', 'SCPEP1']?",
 'chat_history': [HumanMessage(content="answer with yes or no. Is epidermal cell related to this genes: ['COL17A1', 'EFNA1', 'GAMT', 'HSPA1A', 'IFI27', 'KRT14', 'LGALS3BP', 'RBM42', 'SCPEP1']?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="answer with yes or no. Is epidermal cell related to this genes: ['COL17A1', 'EFNA1', 'GAMT', 'HSPA1A', 'IFI27', 'KRT14', 'LGALS3BP', 'RBM42', 'SCPEP1']?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes.', additional_kwargs={}, response_metadata={})],
 'output': 'Yes.',
 'intermediate_steps': []}

In [ ]:
resp

{'input': "answer with yes or no. Is squamous epithelial cell related to this genes: ['ARG1', 'FLG2', 'KPRP', 'LCE1A', 'LCE1B', 'LCE1E', 'LCE2B', 'LCE2C', 'WFDC12']?",
 'chat_history': [HumanMessage(content="answer with yes or no. Is hematopoietic cell related to this genes: ['ARG1', 'FLG2', 'KPRP', 'LCE1A', 'LCE1B', 'LCE1E', 'LCE2B', 'LCE2C', 'WFDC12']?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='No.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="answer with yes or no. Is squamous epithelial cell related to this genes: ['ARG1', 'FLG2', 'KPRP', 'LCE1A', 'LCE1B', 'LCE1E', 'LCE2B', 'LCE2C', 'WFDC12']?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="answer with yes or no. Is squamous epithelial cell related to this genes: ['ARG1', 'FLG2', 'KPRP', 'LCE1A', 'LCE1B', 'LCE1E', 'LCE2B', 'LCE2C', 'WFDC12']?", additional_kwargs={}, response_metadata={}),

# Analysis using LangChain

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import JsonOutputParser

In [31]:
from dotenv import load_dotenv
load_dotenv()

True

In [45]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

load_dotenv(override=True)

client = MongoClient(os.getenv("MONGO_URI"), server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [30]:
llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
#JSON Output formatter  
class CellAnnotation(BaseModel):
    class cell(BaseModel):
        cell_type : str = Field(description="name of the chosen cell.")
        reason : str = Field(description="explain the reasoning for choosing the cell. give comments about parameters from the data that support the choice.")
        
    explanation : list[str] = Field(description="explanation of your reasoning step by step. focus on the parameters and number of genes expressed that support the choice of the cell.")
    cells : list[cell]
  

In [33]:
initial_df = df

In [ ]:
prompt = ChatPromptTemplate.from_template("""
                                          
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 5 most similar cell types to a given, unannotated cell cluster, based on a limited dataset.

The dataset is structured as follows:

Each row represents a known cell type expressing top genes from a cluster. The columns are: "Gene" (top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).

Given this dataset, outline the steps to create and apply a scoring system, then finally return a table of top 5 possible cell type identities for the cluster, and its calculated value.

Ensure that:
1. Describe each row of the dataset, and what it represents.
2. What metric is being used, and what value do they indicate.
3. The steps for scoring each possible cell type.
4. How to obtain the top five possible candidates.                                      
""")

chain = prompt | llm | StrOutputParser()

In [ ]:
parser = JsonOutputParser(pydantic_object=CellAnnotation)

prompt = PromptTemplate(
    template="""You are an expert in single-cell RNA sequencing.

        I have a cluster of cells with its genes as follow

        {initial_df}

        with the gene expression of each cell

        {expression_data}

        Based on the given data, which cell is the most valid to be annotated for this cluster of cells according to you?
        
        \n{format_instructions}""",
    input_variables=["initial_df", "expression_data"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

In [103]:
response = chain.invoke({'initial_df': initial_df, 'expression_data': results[0].to_csv(sep='\t', index=False)})

In [104]:
response

{'cell': 'stem cell of epidermis, stem cell, basal cell of epidermis, precursor cell, progenitor cell, epithelial fate stem cell, basal cell, columnar/cuboidal epithelial cell, neuron associated cell (sensu Vertebrata), Merkel cell',
 'explanation': 'The provided data strongly suggests that the cluster of cells is predominantly composed of epidermal cells.  The genes KRT14, which is highly expressed in keratinocytes (epidermal cells), is the most highly expressed gene in the cluster.  The cells listed above are all epidermal cell types or closely related stem/progenitor cells.  The high expression of KRT14, coupled with the presence of other epidermal-related cell types in the list, makes this the most likely annotation for the cluster.  The high cell counts and percentages for these cells further support this conclusion.  While other cell types might express some of these genes, the overall pattern points to a significant epidermal cell population.'}

# Analysis using OpenAI

In [ ]:
# MODEL = "o3-mini"

In [28]:
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [37]:
initial_df = df

In [279]:
instructions = """
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 5 most similar cell types to a given, unannotated cell cluster, based on a limited dataset. You will do this by creating and applying a scoring system.

The dataset is structured as follows:

Each row represents the cells that strongly express a top genes from the cluster. The columns are: "Gene" (a top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).

Follow these steps precisely to prepare the information, which will be automatically formatted into JSON:

**Step 1: Data Understanding**

*   Explain what each row in the dataset represents. Specifically, explain that each row shows a potential connection between the top gene from the cluster and one known cell type, and the metrics for THAT cell type, not for the cluster itself.
*   Explain the meaning of each column: "Gene", "Cell Name", "Expression Value", "Cell Percentage", "Cell Count", and "Tissue Composition." For each, explain what a higher or lower value might indicate *in the context of identifying similarity to the unannotated cluster*.

**Step 2: Develop a Scoring Function (CellTypeScore)**

*   Explain that the goal is to assign each known "Cell Name" (cell type) a score that reflects its similarity to the unannotated cluster.
*   Propose a formula for calculating a `GeneScore` for each *Gene* within each *Cell Name*. This formula should incorporate both "Cell Percentage" and "Expression Value". Justify why you chose to include these two metrics and how they are combined.
*   Propose a formula for combining the `GeneScore` values across all the `Gene` entries for a given "Cell Name" to calculate a final overall `CellTypeScore` for that cell type.

**Step 3: Calculate CellTypeScore for All Cell Names**

*   Explain that the goal is to now apply your scoring system to the known "Cell Name" values and find out which ones are the highest, with relation to what each Gene expresses.
*   Describe in precise steps what it should do: go through each unique "Cell Name" in the dataset, calculate a `GeneScore` and `CellTypeScore` based on Gene, Expression Value, and the percentage that are expressed.

**Step 4: Identify Top 5 Candidates**

*   Explain how to identify the top 5 candidates from the `CellTypeScore` values, now that they have been calculated in Step 3.
"""

In [73]:
instructions ="""
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 3 most similar cell types to a given, unannotated cell cluster, based on a limited dataset.

The dataset is structured as follows:

Each row represents a known cell type expressing top genes from a cluster. The columns are: "Gene" (top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).

Given this dataset, analyze the dataset to determine similarity, then finally return a list of top possible cell type identities for the cluster.

Ensure that:
1. Never make any assumption. If a gene is not expressed, it should be treated as such and not used for consideration.
2. Understand each row of the data and what it represents.
3. Prioritize cell types that's expressing more genes from the cluster, then consider the cell count, cell percentage, and expression value.
4. The steps for determining each possible cell type.
5. How to obtain the top three possible candidates.
6. give the explanation as if writing a report
"""

In [269]:
instructions ="""
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 3 most similar cell types to a given, unannotated cell cluster, based on a limited dataset.

The dataset is structured as follows:
Each row represents a known cell type expressing top genes from a cluster. The columns are: "Gene" (top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).
Given this dataset, analyze the dataset to determine similarity, then finally return a list of top possible cell type identities for the cluster. The dataset does not contain any representation of the unannotated cell cluster.

Cell similiarity is determined by the number of genes expressed in the cell then the cell count, cell percentage, and expression value. The more genes expressed in a cell, the higher the cell count, cell percentage, and expression value, the more similar the cell is to the unannotated cell cluster. the number of genes expressed in the cell is the most important factor in determining the similarity of the cell to the unannotated cell cluster.

Follow these steps precisely to prepare the information
Step 1 : Read the whole dataset
Step 2 : Explain to me what do you think about each cell's similiarity to the unannotated cell cluster based on the given data.
Step 3 : Based on you analysis, what are the top 3 possible cell type identities for the cluster?

Ensure that:
1. Never make any assumption. If a gene is not expressed, it should be treated as such and not used for consideration.
2. Do not consider cells that expressing lesser amount of genes from the cluster.
3. Do not consider cells that express a gene with parameters that deviate too much from other genes expressed by that cell.
4. Explanations must be written like a report.
"""

In [103]:
#JSON Output formatter  
class CellAnnotation(BaseModel):
    class top_cell(BaseModel):
        class gene(BaseModel):
            gene_name : str = Field(description="Name of the gene expressed by the cell type")
            metrics : str = Field(description="cell count, cell percentage, and expression value of the gene")
        
        cell_type : str = Field(description="name of the chosen cell.")
        gene_info : list[gene]
        reason : str = Field(description="explain the reasoning for choosing the cell. give comments about parameters from the data that support the choice.")
    
    explanation : list[str] = Field(description="explanation of your reasoning step by step. focus on the parameters and number of genes expressed that support the choice of the cell.")
    cells : list[top_cell]

In [270]:
print(results[0].to_csv(sep='\t', index=False))

cell	gene	expression	cell count	cell percentage	tissue composition	score	std	kurtosis	mean	CV
CD8-positive, alpha-beta T cell	RBM42	1.950844832292692	254	0.0186545240893067	0.0003493843819679144	9.243588807299892	3.6248595516837985	4.631287732326113	2.821587956603296	1.2846877742019789
CD8-positive, alpha-beta T cell	KRT14	1.8208990097045898	56	0.0047317279256442754	7.702962752048506e-05	0.48249592686384396	3.6248595516837985	4.631287732326113	2.821587956603296	1.2846877742019789
CD8-positive, alpha-beta T cell	GAMT	1.7516368865966796	40	0.022459292532285232	5.502116251463219e-05	1.5736210098566463	3.6248595516837985	4.631287732326113	2.821587956603296	1.2846877742019789
CD8-positive, alpha-beta T cell	LGALS3BP	1.713193620954241	112	0.0092295014421096	0.00015405925504097012	1.770935375463535	3.6248595516837985	4.631287732326113	2.821587956603296	1.2846877742019789
CD8-positive, alpha-beta T cell	IFI27	1.6771648889300468	87	0.0071090047393364926	0.00011967102846932501	1.0372986635325645

In [280]:
content = f"expression data : {results[23].drop(columns=['tissue composition']).to_csv(sep='\t', index=False)}"

In [281]:
#normal
completion = client.beta.chat.completions.parse(
    model = 'gpt-4o-mini-2024-07-18',
    messages=[
        {"role": "developer", 
         "content": instructions},
        {"role": "user", 
         "content": content}
    ],
    temperature=0.0,
    response_format=CellAnnotation,
)

In [ ]:
#Reasoning Model
response = client.chat.completions.create(
    model = "o3-mini",
    reasoning_effort="medium",
    messages=[
        {"role": "developer", 
         "content": """
            - You are an expert in single-cell RNA sequencing. 
            - You will be provided with cell clustering data and gene expression data for each cell. 
            - Based on the given data, you need to identify the 4 most valid cell to be annotated for the cluster. 
            - Provide a list of cells in the order of validity and explain your reasoning step by step. only give out the name of the cells from the given data.
            - Analyze only the data provided. Do not reference or infer from any external sources
            - the answer is in the format of 
                {
                cells : [cell1, cell2, cell3, cell4],
                explanation : <explanation>
                }
            """},
        {"role": "user", 
         "content": content}
    ],
)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `o3-mini` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [282]:
print(completion.choices[0].message.parsed.model_dump_json(indent=4))

{
    "explanation": [
        "Each row in the dataset represents a known cell type and its expression of a specific gene, along with various metrics that describe the expression level and prevalence of that gene in the cell type.",
        "The 'Gene' column indicates the specific gene being analyzed, which is crucial for understanding the biological functions associated with the cell type.",
        "The 'Cell Name' column identifies the known cell type that expresses the gene, allowing for comparison with the unannotated cluster.",
        "The 'Expression Value' column shows the level of gene expression; higher values suggest stronger expression, which may indicate a more significant role of that gene in the cell type's function.",
        "The 'Cell Percentage' column indicates the proportion of cells expressing the gene; a higher percentage suggests that the gene is more commonly expressed within that cell type, which may correlate with its relevance to the unannotated cluster."

In [39]:
import json

output_dict = json.loads(completion.choices[0].message.parsed.model_dump_json(indent=4))

In [40]:
output_dict['cells']

[{'rank': 1,
  'cell_type': 'T cell',
  'reason': 'The T cell type expresses 5 genes from the cluster: KRT14 (1.929), IFI27 (1.713), LGALS3BP (1.698), RBM42 (1.662), and GAMT (1.604). It has a high cell count of 3174, a cell percentage of 0.0336, and a tissue composition of 0.0044, indicating a strong presence in the tissue.'},
 {'rank': 2,
  'cell_type': 'CD8-positive, alpha-beta T cell',
  'reason': 'This cell type expresses 5 genes: RBM42 (1.950), KRT14 (1.820), GAMT (1.751), LGALS3BP (1.713), and IFI27 (1.677). It has a cell count of 254, a cell percentage of 0.0187, and a tissue composition of 0.0003, showing a significant expression of relevant genes.'},
 {'rank': 3,
  'cell_type': 'CD4-positive, alpha-beta T cell',
  'reason': 'This cell type expresses 4 genes: IFI27 (2.030), LGALS3BP (1.680), RBM42 (1.637), and GAMT (1.554). It has a cell count of 91, a cell percentage of 0.0247, and a tissue composition of 0.0001, indicating a relevant but slightly lower expression compared to

# Color Test

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Sample DataFrame
data = {'Values': [5, 3, 8, 2, 9, 1, 7, 4, 6, 5, 3, 2]}
df = pd.DataFrame(data)

# Define normalization (scale from min=1 to max=9)
norm = mcolors.Normalize(vmin=1, vmax=9)
# Choose a colormap (reversed for light→dark)
cmap = plt.get_cmap('YlGnBu_r')  # Or 'Blues_r', 'Greys_r', etc.

# Map values to hex color codes
df['Color'] = df['Values'].apply(
    lambda x: mcolors.to_hex(cmap(norm(x)))  # Convert to hex
)

# Display DataFrame with hex color codes
print(df)

    Values    Color
0        5  #42b6c4
1        3  #225ea8
2        8  #edf8b2
3        2  #253494
4        9  #ffffd9
5        1  #081d58
6        7  #c8e9b4
7        4  #1d91c0
8        6  #80cebb
9        5  #42b6c4
10       3  #225ea8
11       2  #253494


# Test Marker

In [1]:
import pandas as pd
import requests
def get_marker(tissue, cell):
   # URL to send the POST request to
    url = "https://api.cellxgene.cziscience.com/wmg/v2/markers"

    # Payload to send in the POST request
    payload = {
        "celltype": cell,
        "n_markers":50,
        "organism":"NCBITaxon:9606",
        "test":"ttest",
        "tissue":tissue
    }

    # Send POST request
    return requests.post(url, json=payload).json()['marker_genes']

In [4]:
get_marker("UBERON:0001043", "cell CL:1000615")

[]

# Test Function 2

In [36]:
resultsv2 = main_data_analysisv2(ensembl_df, TARGET_UBERON_ID)

In [47]:
display_dataframe_as_markdown(resultsv2[23])

| gene   | cell                                                    |   expression |   cell count |   cell percentage |   tissue composition |   total cell count |       score |       std |     mean |       CV |   kurtosis |   std norm |   mean norm |   CV norm |   kurtosis norm |   count |   cell score |
|:-------|:--------------------------------------------------------|-------------:|-------------:|------------------:|---------------------:|-------------------:|------------:|----------:|---------:|---------:|-----------:|-----------:|------------:|----------:|----------------:|--------:|-------------:|
| ARG1   | hematopoietic cell                                      |      1.81413 |         2407 |       0.0113148   |          0.00294837  |               5793 | 2.66937     | 0.901246  | 0.908911 | 0.991567 |  0.894088  | 0.930234   |   1         | 0.76278   |       0.334256  |       9 |      7.17546 |
| FLG2   | hematopoietic cell                                      |      1.73625 |         2048 |       0.00960213  |          0.00250863  |               5793 | 2.26232     | 0.901246  | 0.908911 | 0.991567 |  0.894088  | 0.930234   |   1         | 0.76278   |       0.334256  |       9 |      7.17546 |
| KPRP   | hematopoietic cell                                      |      2.34377 |            9 |       0.000117989 |          1.10243e-05 |               5793 | 0.374786    | 0.901246  | 0.908911 | 0.991567 |  0.894088  | 0.930234   |   1         | 0.76278   |       0.334256  |       9 |      7.17546 |
| LCE1A  | hematopoietic cell                                      |      1.78233 |           83 |       0.00037717  |          0.000101668 |               5793 | 0.225319    | 0.901246  | 0.908911 | 0.991567 |  0.894088  | 0.930234   |   1         | 0.76278   |       0.334256  |       9 |      7.17546 |
| LCE1B  | hematopoietic cell                                      |      1.67291 |          494 |       0.00231664  |          0.000605109 |               5793 | 0.611096    | 0.901246  | 0.908911 | 0.991567 |  0.894088  | 0.930234   |   1         | 0.76278   |       0.334256  |       9 |      7.17546 |
| LCE1E  | hematopoietic cell                                      |      1.80143 |          303 |       0.00146245  |          0.00037115  |               5793 | 0.463256    | 0.901246  | 0.908911 | 0.991567 |  0.894088  | 0.930234   |   1         | 0.76278   |       0.334256  |       9 |      7.17546 |
| LCE2B  | hematopoietic cell                                      |      1.7641  |          435 |       0.00200616  |          0.000532839 |               5793 | 0.586155    | 0.901246  | 0.908911 | 0.991567 |  0.894088  | 0.930234   |   1         | 0.76278   |       0.334256  |       9 |      7.17546 |
| LCE2C  | hematopoietic cell                                      |      3.14413 |            3 |       0.000378931 |          3.67475e-06 |               5793 | 0.692185    | 0.901246  | 0.908911 | 0.991567 |  0.894088  | 0.930234   |   1         | 0.76278   |       0.334256  |       9 |      7.17546 |
| WFDC12 | hematopoietic cell                                      |      2.14252 |           11 |       0.000205488 |          1.34741e-05 |               5793 | 0.295717    | 0.901246  | 0.908911 | 0.991567 |  0.894088  | 0.930234   |   1         | 0.76278   |       0.334256  |       9 |      7.17546 |
| ARG1   | leukocyte                                               |      1.81535 |         2379 |       0.0112406   |          0.00291408  |               5749 | 2.64065     | 0.965509  | 0.790315 | 1.22168  |  0.706954  | 1          |   0.849338  | 0.984704  |       0.309569  |       9 |      5.98598 |
| FLG2   | leukocyte                                               |      1.73337 |         2046 |       0.0096383   |          0.00250618  |               5749 | 2.25913     | 0.965509  | 0.790315 | 1.22168  |  0.706954  | 1          |   0.849338  | 0.984704  |       0.309569  |       9 |      5.98598 |
| KPRP   | leukocyte                                               |      1.68142 |            7 |       9.29985e-05 |          8.57442e-06 |               5749 | 0.10517     | 0.965509  | 0.790315 | 1.22168  |  0.706954  | 1          |   0.849338  | 0.984704  |       0.309569  |       9 |      5.98598 |
| LCE1A  | leukocyte                                               |      1.69135 |           81 |       0.000369775 |          9.92183e-05 |               5749 | 0.186486    | 0.965509  | 0.790315 | 1.22168  |  0.706954  | 1          |   0.849338  | 0.984704  |       0.309569  |       9 |      5.98598 |
| LCE1B  | leukocyte                                               |      1.66202 |          492 |       0.00231822  |          0.000602659 |               5749 | 0.604623    | 0.965509  | 0.790315 | 1.22168  |  0.706954  | 1          |   0.849338  | 0.984704  |       0.309569  |       9 |      5.98598 |
| LCE1E  | leukocyte                                               |      1.78153 |          300 |       0.00146042  |          0.000367475 |               5749 | 0.452102    | 0.965509  | 0.790315 | 1.22168  |  0.706954  | 1          |   0.849338  | 0.984704  |       0.309569  |       9 |      5.98598 |
| LCE2B  | leukocyte                                               |      1.74993 |          433 |       0.00200626  |          0.000530389 |               5749 | 0.578351    | 0.965509  | 0.790315 | 1.22168  |  0.706954  | 1          |   0.849338  | 0.984704  |       0.309569  |       9 |      5.98598 |
| LCE2C  | leukocyte                                               |      1.60669 |            1 |       0.000144739 |          1.22492e-06 |               5749 | 0.0688321   | 0.965509  | 0.790315 | 1.22168  |  0.706954  | 1          |   0.849338  | 0.984704  |       0.309569  |       9 |      5.98598 |
| WFDC12 | leukocyte                                               |      1.95144 |           10 |       0.000190393 |          1.22492e-05 |               5749 | 0.217487    | 0.965509  | 0.790315 | 1.22168  |  0.706954  | 1          |   0.849338  | 0.984704  |       0.309569  |       9 |      5.98598 |
| ARG1   | mononuclear cell                                        |      1.81038 |         2355 |       0.011378    |          0.00288468  |               5678 | 2.61391     | 0.956792  | 0.780954 | 1.22516  |  0.708762  | 0.990536   |   0.837445  | 0.988062  |       0.309808  |       9 |      5.91634 |
| FLG2   | mononuclear cell                                        |      1.73174 |         2027 |       0.00974829  |          0.00248291  |               5678 | 2.2389      | 0.956792  | 0.780954 | 1.22516  |  0.708762  | 0.990536   |   0.837445  | 0.988062  |       0.309808  |       9 |      5.91634 |
| KPRP   | mononuclear cell                                        |      1.68142 |            7 |       9.29985e-05 |          8.57442e-06 |               5678 | 0.10517     | 0.956792  | 0.780954 | 1.22516  |  0.708762  | 0.990536   |   0.837445  | 0.988062  |       0.309808  |       9 |      5.91634 |
| LCE1A  | mononuclear cell                                        |      1.68094 |           78 |       0.000363284 |          9.55435e-05 |               5678 | 0.179157    | 0.956792  | 0.780954 | 1.22516  |  0.708762  | 0.990536   |   0.837445  | 0.988062  |       0.309808  |       9 |      5.91634 |
| LCE1B  | mononuclear cell                                        |      1.65915 |          483 |       0.00232337  |          0.000591635 |               5678 | 0.59412     | 0.956792  | 0.780954 | 1.22516  |  0.708762  | 0.990536   |   0.837445  | 0.988062  |       0.309808  |       9 |      5.91634 |
| LCE1E  | mononuclear cell                                        |      1.7763  |          292 |       0.00145219  |          0.000357676 |               5678 | 0.441661    | 0.956792  | 0.780954 | 1.22516  |  0.708762  | 0.990536   |   0.837445  | 0.988062  |       0.309808  |       9 |      5.91634 |
| LCE2B  | mononuclear cell                                        |      1.74819 |          425 |       0.00200965  |          0.00052059  |               5678 | 0.569342    | 0.956792  | 0.780954 | 1.22516  |  0.708762  | 0.990536   |   0.837445  | 0.988062  |       0.309808  |       9 |      5.91634 |
| LCE2C  | mononuclear cell                                        |      1.60669 |            1 |       0.000144739 |          1.22492e-06 |               5678 | 0.0688321   | 0.956792  | 0.780954 | 1.22516  |  0.708762  | 0.990536   |   0.837445  | 0.988062  |       0.309808  |       9 |      5.91634 |
| WFDC12 | mononuclear cell                                        |      1.95144 |           10 |       0.000190393 |          1.22492e-05 |               5678 | 0.217487    | 0.956792  | 0.780954 | 1.22516  |  0.708762  | 0.990536   |   0.837445  | 0.988062  |       0.309808  |       9 |      5.91634 |
| ARG1   | lymphocyte                                              |      1.76361 |         2127 |       0.015393    |          0.0026054   |               5198 | 2.36414     | 0.88714   | 0.716859 | 1.23754  |  0.548267  | 0.914921   |   0.756021  | 1         |       0.288635  |       9 |      5.47973 |
| FLG2   | lymphocyte                                              |      1.7305  |         1911 |       0.0141564   |          0.00234082  |               5198 | 2.12447     | 0.88714   | 0.716859 | 1.23754  |  0.548267  | 0.914921   |   0.756021  | 1         |       0.288635  |       9 |      5.47973 |
| KPRP   | lymphocyte                                              |      1.56872 |            2 |       0.000107863 |          2.44983e-06 |               5198 | 0.0544826   | 0.88714   | 0.716859 | 1.23754  |  0.548267  | 0.914921   |   0.756021  | 1         |       0.288635  |       9 |      5.47973 |
| LCE1A  | lymphocyte                                              |      1.66244 |           67 |       0.000496326 |          8.20694e-05 |               5198 | 0.160456    | 0.88714   | 0.716859 | 1.23754  |  0.548267  | 0.914921   |   0.756021  | 1         |       0.288635  |       9 |      5.47973 |
| LCE1B  | lymphocyte                                              |      1.65239 |          446 |       0.0034797   |          0.000546313 |               5198 | 0.554678    | 0.88714   | 0.716859 | 1.23754  |  0.548267  | 0.914921   |   0.756021  | 1         |       0.288635  |       9 |      5.47973 |
| LCE1E  | lymphocyte                                              |      1.7715  |          251 |       0.00195831  |          0.000307454 |               5198 | 0.39788     | 0.88714   | 0.716859 | 1.23754  |  0.548267  | 0.914921   |   0.756021  | 1         |       0.288635  |       9 |      5.47973 |
| LCE2B  | lymphocyte                                              |      1.75138 |          384 |       0.00284461  |          0.000470368 |               5198 | 0.529281    | 0.88714   | 0.716859 | 1.23754  |  0.548267  | 0.914921   |   0.756021  | 1         |       0.288635  |       9 |      5.47973 |
| LCE2C  | lymphocyte                                              |      1.60669 |            1 |       0.000144739 |          1.22492e-06 |               5198 | 0.0688321   | 0.88714   | 0.716859 | 1.23754  |  0.548267  | 0.914921   |   0.756021  | 1         |       0.288635  |       9 |      5.47973 |
| WFDC12 | lymphocyte                                              |      1.90443 |            9 |       0.000224495 |          1.10243e-05 |               5198 | 0.197511    | 0.88714   | 0.716859 | 1.23754  |  0.548267  | 0.914921   |   0.756021  | 1         |       0.288635  |       9 |      5.47973 |
| ARG1   | squamous epithelial cell                                |      2.34323 |          217 |       0.00203938  |          0.000265807 |               1115 | 0.593579    | 0.157633  | 0.507655 | 0.310513 |  2.13924   | 0.122953   |   0.490251  | 0.105956  |       0.498517  |       9 |      5.1339  |
| FLG2   | squamous epithelial cell                                |      2.55309 |          383 |       0.00292702  |          0.000469143 |               1115 | 0.852152    | 0.157633  | 0.507655 | 0.310513 |  2.13924   | 0.122953   |   0.490251  | 0.105956  |       0.498517  |       9 |      5.1339  |
| KPRP   | squamous epithelial cell                                |      2.15194 |          110 |       0.000852555 |          0.000134741 |               1115 | 0.403362    | 0.157633  | 0.507655 | 0.310513 |  2.13924   | 0.122953   |   0.490251  | 0.105956  |       0.498517  |       9 |      5.1339  |
| LCE1A  | squamous epithelial cell                                |      2.6115  |           33 |       0.000255766 |          4.04223e-05 |               1115 | 0.50806     | 0.157633  | 0.507655 | 0.310513 |  2.13924   | 0.122953   |   0.490251  | 0.105956  |       0.498517  |       9 |      5.1339  |
| LCE1B  | squamous epithelial cell                                |      2.16157 |           80 |       0.000595921 |          9.79933e-05 |               1115 | 0.375693    | 0.157633  | 0.507655 | 0.310513 |  2.13924   | 0.122953   |   0.490251  | 0.105956  |       0.498517  |       9 |      5.1339  |
| LCE1E  | squamous epithelial cell                                |      1.91861 |          139 |       0.00106401  |          0.000170263 |               1115 | 0.339579    | 0.157633  | 0.507655 | 0.310513 |  2.13924   | 0.122953   |   0.490251  | 0.105956  |       0.498517  |       9 |      5.1339  |
| LCE2B  | squamous epithelial cell                                |      2.22676 |          108 |       0.000825373 |          0.000132291 |               1115 | 0.431459    | 0.157633  | 0.507655 | 0.310513 |  2.13924   | 0.122953   |   0.490251  | 0.105956  |       0.498517  |       9 |      5.1339  |
| LCE2C  | squamous epithelial cell                                |      2.84726 |           30 |       0.000232515 |          3.67475e-05 |               1115 | 0.600121    | 0.157633  | 0.507655 | 0.310513 |  2.13924   | 0.122953   |   0.490251  | 0.105956  |       0.498517  |       9 |      5.1339  |
| WFDC12 | squamous epithelial cell                                |      2.55144 |           15 |       0.000116449 |          1.83738e-05 |               1115 | 0.46489     | 0.157633  | 0.507655 | 0.310513 |  2.13924   | 0.122953   |   0.490251  | 0.105956  |       0.498517  |       9 |      5.1339  |
| ARG1   | stratified squamous epithelial cell                     |      2.3558  |          212 |       0.00200626  |          0.000259682 |               1110 | 0.59341     | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| FLG2   | stratified squamous epithelial cell                     |      2.55309 |          383 |       0.00292702  |          0.000469143 |               1110 | 0.852152    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| KPRP   | stratified squamous epithelial cell                     |      2.15194 |          110 |       0.000852555 |          0.000134741 |               1110 | 0.403362    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE1A  | stratified squamous epithelial cell                     |      2.6115  |           33 |       0.000255766 |          4.04223e-05 |               1110 | 0.50806     | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE1B  | stratified squamous epithelial cell                     |      2.16157 |           80 |       0.000595921 |          9.79933e-05 |               1110 | 0.375693    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE1E  | stratified squamous epithelial cell                     |      1.91861 |          139 |       0.00106401  |          0.000170263 |               1110 | 0.339579    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE2B  | stratified squamous epithelial cell                     |      2.22676 |          108 |       0.000825373 |          0.000132291 |               1110 | 0.431459    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE2C  | stratified squamous epithelial cell                     |      2.84726 |           30 |       0.000232515 |          3.67475e-05 |               1110 | 0.600121    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| WFDC12 | stratified squamous epithelial cell                     |      2.55144 |           15 |       0.000116449 |          1.83738e-05 |               1110 | 0.46489     | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| ARG1   | stratified epithelial cell                              |      2.3558  |          212 |       0.00200626  |          0.000259682 |               1110 | 0.59341     | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| FLG2   | stratified epithelial cell                              |      2.55309 |          383 |       0.00292702  |          0.000469143 |               1110 | 0.852152    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| KPRP   | stratified epithelial cell                              |      2.15194 |          110 |       0.000852555 |          0.000134741 |               1110 | 0.403362    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE1A  | stratified epithelial cell                              |      2.6115  |           33 |       0.000255766 |          4.04223e-05 |               1110 | 0.50806     | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE1B  | stratified epithelial cell                              |      2.16157 |           80 |       0.000595921 |          9.79933e-05 |               1110 | 0.375693    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE1E  | stratified epithelial cell                              |      1.91861 |          139 |       0.00106401  |          0.000170263 |               1110 | 0.339579    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE2B  | stratified epithelial cell                              |      2.22676 |          108 |       0.000825373 |          0.000132291 |               1110 | 0.431459    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE2C  | stratified epithelial cell                              |      2.84726 |           30 |       0.000232515 |          3.67475e-05 |               1110 | 0.600121    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| WFDC12 | stratified epithelial cell                              |      2.55144 |           15 |       0.000116449 |          1.83738e-05 |               1110 | 0.46489     | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| ARG1   | keratinocyte                                            |      2.3558  |          212 |       0.00200626  |          0.000259682 |               1110 | 0.59341     | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| FLG2   | keratinocyte                                            |      2.55309 |          383 |       0.00292702  |          0.000469143 |               1110 | 0.852152    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| KPRP   | keratinocyte                                            |      2.15194 |          110 |       0.000852555 |          0.000134741 |               1110 | 0.403362    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE1A  | keratinocyte                                            |      2.6115  |           33 |       0.000255766 |          4.04223e-05 |               1110 | 0.50806     | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE1B  | keratinocyte                                            |      2.16157 |           80 |       0.000595921 |          9.79933e-05 |               1110 | 0.375693    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE1E  | keratinocyte                                            |      1.91861 |          139 |       0.00106401  |          0.000170263 |               1110 | 0.339579    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE2B  | keratinocyte                                            |      2.22676 |          108 |       0.000825373 |          0.000132291 |               1110 | 0.431459    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| LCE2C  | keratinocyte                                            |      2.84726 |           30 |       0.000232515 |          3.67475e-05 |               1110 | 0.600121    | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| WFDC12 | keratinocyte                                            |      2.55144 |           15 |       0.000116449 |          1.83738e-05 |               1110 | 0.46489     | 0.157622  | 0.507636 | 0.310502 |  2.14211   | 0.122941   |   0.490228  | 0.105945  |       0.498895  |       9 |      5.13343 |
| ARG1   | epidermal cell                                          |      2.34822 |          223 |       0.00210445  |          0.000273156 |               1164 | 0.601927    | 0.163378  | 0.510697 | 0.319911 |  2.81907   | 0.12919    |   0.494115  | 0.11502   |       0.588199  |       9 |      5.06376 |
| FLG2   | epidermal cell                                          |      2.55281 |          408 |       0.00311101  |          0.000499766 |               1164 | 0.878292    | 0.163378  | 0.510697 | 0.319911 |  2.81907   | 0.12919    |   0.494115  | 0.11502   |       0.588199  |       9 |      5.06376 |
| KPRP   | epidermal cell                                          |      2.14807 |          115 |       0.00088926  |          0.000140865 |               1164 | 0.407049    | 0.163378  | 0.510697 | 0.319911 |  2.81907   | 0.12919    |   0.494115  | 0.11502   |       0.588199  |       9 |      5.06376 |
| LCE1A  | epidermal cell                                          |      2.59836 |           34 |       0.000262912 |          4.16472e-05 |               1164 | 0.5038      | 0.163378  | 0.510697 | 0.319911 |  2.81907   | 0.12919    |   0.494115  | 0.11502   |       0.588199  |       9 |      5.06376 |
| LCE1B  | epidermal cell                                          |      2.16246 |           84 |       0.000624336 |          0.000102893 |               1164 | 0.380252    | 0.163378  | 0.510697 | 0.319911 |  2.81907   | 0.12919    |   0.494115  | 0.11502   |       0.588199  |       9 |      5.06376 |
| LCE1E  | epidermal cell                                          |      1.91548 |          140 |       0.00106923  |          0.000171488 |               1164 | 0.33936     | 0.163378  | 0.510697 | 0.319911 |  2.81907   | 0.12919    |   0.494115  | 0.11502   |       0.588199  |       9 |      5.06376 |
| LCE2B  | epidermal cell                                          |      2.22691 |          113 |       0.000861629 |          0.000138416 |               1164 | 0.436769    | 0.163378  | 0.510697 | 0.319911 |  2.81907   | 0.12919    |   0.494115  | 0.11502   |       0.588199  |       9 |      5.06376 |
| LCE2C  | epidermal cell                                          |      2.80197 |           32 |       0.000247446 |          3.91973e-05 |               1164 | 0.583931    | 0.163378  | 0.510697 | 0.319911 |  2.81907   | 0.12919    |   0.494115  | 0.11502   |       0.588199  |       9 |      5.06376 |
| WFDC12 | epidermal cell                                          |      2.55144 |           15 |       0.000116449 |          1.83738e-05 |               1164 | 0.46489     | 0.163378  | 0.510697 | 0.319911 |  2.81907   | 0.12919    |   0.494115  | 0.11502   |       0.588199  |       9 |      5.06376 |
| ARG1   | epithelial cell                                         |      2.30239 |          243 |       0.00162244  |          0.000297655 |               1438 | 0.603473    | 0.177184  | 0.499699 | 0.354582 |  3.65917   | 0.144178   |   0.480144  | 0.148458  |       0.699026  |       9 |      4.83241 |
| FLG2   | epithelial cell                                         |      2.42344 |          492 |       0.00291307  |          0.000602659 |               1438 | 0.913027    | 0.177184  | 0.499699 | 0.354582 |  3.65917   | 0.144178   |   0.480144  | 0.148458  |       0.699026  |       9 |      4.83241 |
| KPRP   | epithelial cell                                         |      2.13447 |          118 |       0.000706299 |          0.00014454  |               1438 | 0.404401    | 0.177184  | 0.499699 | 0.354582 |  3.65917   | 0.144178   |   0.480144  | 0.148458  |       0.699026  |       9 |      4.83241 |
| LCE1A  | epithelial cell                                         |      2.22974 |           59 |       0.00035315  |          7.22701e-05 |               1438 | 0.381043    | 0.177184  | 0.499699 | 0.354582 |  3.65917   | 0.144178   |   0.480144  | 0.148458  |       0.699026  |       9 |      4.83241 |
| LCE1B  | epithelial cell                                         |      1.96004 |          135 |       0.000783563 |          0.000165364 |               1438 | 0.351731    | 0.177184  | 0.499699 | 0.354582 |  3.65917   | 0.144178   |   0.480144  | 0.148458  |       0.699026  |       9 |      4.83241 |
| LCE1E  | epithelial cell                                         |      1.85774 |          207 |       0.00122716  |          0.000253558 |               1438 | 0.385897    | 0.177184  | 0.499699 | 0.354582 |  3.65917   | 0.144178   |   0.480144  | 0.148458  |       0.699026  |       9 |      4.83241 |
| LCE2B  | epithelial cell                                         |      2.13789 |          136 |       0.000805239 |          0.000166589 |               1438 | 0.424631    | 0.177184  | 0.499699 | 0.354582 |  3.65917   | 0.144178   |   0.480144  | 0.148458  |       0.699026  |       9 |      4.83241 |
| LCE2C  | epithelial cell                                         |      2.7606  |           33 |       0.000205249 |          4.04223e-05 |               1438 | 0.568199    | 0.177184  | 0.499699 | 0.354582 |  3.65917   | 0.144178   |   0.480144  | 0.148458  |       0.699026  |       9 |      4.83241 |
| WFDC12 | epithelial cell                                         |      2.55144 |           15 |       0.000116449 |          1.83738e-05 |               1438 | 0.46489     | 0.177184  | 0.499699 | 0.354582 |  3.65917   | 0.144178   |   0.480144  | 0.148458  |       0.699026  |       9 |      4.83241 |
| ARG1   | myeloid cell                                            |      2.20032 |          274 |       0.00420374  |          0.000335627 |                520 | 0.598336    | 0.265216  | 0.471122 | 0.562944 |  0.565518  | 0.239747   |   0.443841  | 0.349407  |       0.290911  |       9 |      4.70414 |
| FLG2   | myeloid cell                                            |      1.82785 |          105 |       0.00165872  |          0.000128616 |                520 | 0.268487    | 0.265216  | 0.471122 | 0.562944 |  0.565518  | 0.239747   |   0.443841  | 0.349407  |       0.290911  |       9 |      4.70414 |
| KPRP   | myeloid cell                                            |      2.64143 |            6 |       0.00011989  |          7.3495e-06  |                520 | 0.491888    | 0.265216  | 0.471122 | 0.562944 |  0.565518  | 0.239747   |   0.443841  | 0.349407  |       0.290911  |       9 |      4.70414 |
| LCE1A  | myeloid cell                                            |      2.4764  |           12 |       0.000158527 |          1.4699e-05  |                520 | 0.43153     | 0.265216  | 0.471122 | 0.562944 |  0.565518  | 0.239747   |   0.443841  | 0.349407  |       0.290911  |       9 |      4.70414 |
| LCE1B  | myeloid cell                                            |      1.90822 |           38 |       0.000502001 |          4.65468e-05 |                520 | 0.229591    | 0.265216  | 0.471122 | 0.562944 |  0.565518  | 0.239747   |   0.443841  | 0.349407  |       0.290911  |       9 |      4.70414 |
| LCE1E  | myeloid cell                                            |      1.99938 |           39 |       0.000616464 |          4.77718e-05 |                520 | 0.26762     | 0.265216  | 0.471122 | 0.562944 |  0.565518  | 0.239747   |   0.443841  | 0.349407  |       0.290911  |       9 |      4.70414 |
| LCE2B  | myeloid cell                                            |      1.89752 |           42 |       0.000648358 |          5.14465e-05 |                520 | 0.229648    | 0.265216  | 0.471122 | 0.562944 |  0.565518  | 0.239747   |   0.443841  | 0.349407  |       0.290911  |       9 |      4.70414 |
| LCE2C  | myeloid cell                                            |      3.91286 |            2 |       0.00198413  |          2.44983e-06 |                520 | 1.00402     | 0.265216  | 0.471122 | 0.562944 |  0.565518  | 0.239747   |   0.443841  | 0.349407  |       0.290911  |       9 |      4.70414 |
| WFDC12 | myeloid cell                                            |      3.21392 |            2 |       0.000148798 |          2.44983e-06 |                520 | 0.718985    | 0.265216  | 0.471122 | 0.562944 |  0.565518  | 0.239747   |   0.443841  | 0.349407  |       0.290911  |       9 |      4.70414 |
| ARG1   | stem cell of epidermis                                  |      2.20396 |           10 |       0.000320154 |          1.22492e-05 |                373 | 0.319666    | 0.0521022 | 0.241975 | 0.215321 | -1.40143   | 0.00838663 |   0.152736  | 0.0141505 |       0.0314313 |       9 |      3.61366 |
| FLG2   | stem cell of epidermis                                  |      1.83718 |          119 |       0.00380983  |          0.000145765 |                373 | 0.290027    | 0.0521022 | 0.241975 | 0.215321 | -1.40143   | 0.00838663 |   0.152736  | 0.0141505 |       0.0314313 |       9 |      3.61366 |
| KPRP   | stem cell of epidermis                                  |      2.17081 |            8 |       0.000256123 |          9.79933e-06 |                373 | 0.304104    | 0.0521022 | 0.241975 | 0.215321 | -1.40143   | 0.00838663 |   0.152736  | 0.0141505 |       0.0314313 |       9 |      3.61366 |
| LCE1A  | stem cell of epidermis                                  |      1.89022 |           16 |       0.000493629 |          1.95987e-05 |                373 | 0.199452    | 0.0521022 | 0.241975 | 0.215321 | -1.40143   | 0.00838663 |   0.152736  | 0.0141505 |       0.0314313 |       9 |      3.61366 |
| LCE1B  | stem cell of epidermis                                  |      1.75944 |           40 |       0.00123407  |          4.89967e-05 |                373 | 0.172682    | 0.0521022 | 0.241975 | 0.215321 | -1.40143   | 0.00838663 |   0.152736  | 0.0141505 |       0.0314313 |       9 |      3.61366 |
| LCE1E  | stem cell of epidermis                                  |      1.75982 |          101 |       0.00323355  |          0.000123717 |                373 | 0.239217    | 0.0521022 | 0.241975 | 0.215321 | -1.40143   | 0.00838663 |   0.152736  | 0.0141505 |       0.0314313 |       9 |      3.61366 |
| LCE2B  | stem cell of epidermis                                  |      1.76426 |           69 |       0.00220906  |          8.45193e-05 |                373 | 0.206222    | 0.0521022 | 0.241975 | 0.215321 | -1.40143   | 0.00838663 |   0.152736  | 0.0141505 |       0.0314313 |       9 |      3.61366 |
| LCE2C  | stem cell of epidermis                                  |      1.92572 |            5 |       0.000154259 |          6.12458e-06 |                373 | 0.20185     | 0.0521022 | 0.241975 | 0.215321 | -1.40143   | 0.00838663 |   0.152736  | 0.0141505 |       0.0314313 |       9 |      3.61366 |
| WFDC12 | stem cell of epidermis                                  |      2.03144 |            5 |       0.000160077 |          6.12458e-06 |                373 | 0.244554    | 0.0521022 | 0.241975 | 0.215321 | -1.40143   | 0.00838663 |   0.152736  | 0.0141505 |       0.0314313 |       9 |      3.61366 |
| ARG1   | basal cell of epidermis                                 |      2.20396 |           10 |       0.000320154 |          1.22492e-05 |                370 | 0.319666    | 0.049392  | 0.246162 | 0.200648 | -0.965313  | 0.00544442 |   0.158055  | 0         |       0.0889638 |       9 |      3.61078 |
| FLG2   | basal cell of epidermis                                 |      1.83718 |          119 |       0.00380983  |          0.000145765 |                370 | 0.290027    | 0.049392  | 0.246162 | 0.200648 | -0.965313  | 0.00544442 |   0.158055  | 0         |       0.0889638 |       9 |      3.61078 |
| KPRP   | basal cell of epidermis                                 |      2.17081 |            8 |       0.000256123 |          9.79933e-06 |                370 | 0.304104    | 0.049392  | 0.246162 | 0.200648 | -0.965313  | 0.00544442 |   0.158055  | 0         |       0.0889638 |       9 |      3.61078 |
| LCE1A  | basal cell of epidermis                                 |      1.91824 |           15 |       0.000480231 |          1.83738e-05 |                370 | 0.209709    | 0.049392  | 0.246162 | 0.200648 | -0.965313  | 0.00544442 |   0.158055  | 0         |       0.0889638 |       9 |      3.61078 |
| LCE1B  | basal cell of epidermis                                 |      1.75827 |           39 |       0.0012486   |          4.77718e-05 |                370 | 0.171191    | 0.049392  | 0.246162 | 0.200648 | -0.965313  | 0.00544442 |   0.158055  | 0         |       0.0889638 |       9 |      3.61078 |
| LCE1E  | basal cell of epidermis                                 |      1.75982 |          101 |       0.00323355  |          0.000123717 |                370 | 0.239217    | 0.049392  | 0.246162 | 0.200648 | -0.965313  | 0.00544442 |   0.158055  | 0         |       0.0889638 |       9 |      3.61078 |
| LCE2B  | basal cell of epidermis                                 |      1.76426 |           69 |       0.00220906  |          8.45193e-05 |                370 | 0.206222    | 0.049392  | 0.246162 | 0.200648 | -0.965313  | 0.00544442 |   0.158055  | 0         |       0.0889638 |       9 |      3.61078 |
| LCE2C  | basal cell of epidermis                                 |      2.00001 |            4 |       0.000128061 |          4.89967e-06 |                370 | 0.230772    | 0.049392  | 0.246162 | 0.200648 | -0.965313  | 0.00544442 |   0.158055  | 0         |       0.0889638 |       9 |      3.61078 |
| WFDC12 | basal cell of epidermis                                 |      2.03144 |            5 |       0.000160077 |          6.12458e-06 |                370 | 0.244554    | 0.049392  | 0.246162 | 0.200648 | -0.965313  | 0.00544442 |   0.158055  | 0         |       0.0889638 |       9 |      3.61078 |
| ARG1   | connective tissue cell                                  |      1.83212 |           72 |       0.000284654 |          8.8194e-05  |               1873 | 0.23386     | 0.253594  | 0.331547 | 0.764881 | -0.141738  | 0.22713    |   0.266526  | 0.544159  |       0.19761   |       9 |      3.50711 |
| FLG2   | connective tissue cell                                  |      1.74152 |          553 |       0.00217518  |          0.000677379 |               1873 | 0.699897    | 0.253594  | 0.331547 | 0.764881 | -0.141738  | 0.22713    |   0.266526  | 0.544159  |       0.19761   |       9 |      3.50711 |
| KPRP   | connective tissue cell                                  |      1.66398 |           16 |       6.29346e-05 |          1.95987e-05 |               1873 | 0.107433    | 0.253594  | 0.331547 | 0.764881 | -0.141738  | 0.22713    |   0.266526  | 0.544159  |       0.19761   |       9 |      3.50711 |
| LCE1A  | connective tissue cell                                  |      1.66252 |           75 |       0.000295006 |          9.18688e-05 |               1873 | 0.168499    | 0.253594  | 0.331547 | 0.764881 | -0.141738  | 0.22713    |   0.266526  | 0.544159  |       0.19761   |       9 |      3.50711 |
| LCE1B  | connective tissue cell                                  |      1.67311 |          225 |       0.000885018 |          0.000275606 |               1873 | 0.329521    | 0.253594  | 0.331547 | 0.764881 | -0.141738  | 0.22713    |   0.266526  | 0.544159  |       0.19761   |       9 |      3.50711 |
| LCE1E  | connective tissue cell                                  |      1.70798 |          650 |       0.00255672  |          0.000796196 |               1873 | 0.787714    | 0.253594  | 0.331547 | 0.764881 | -0.141738  | 0.22713    |   0.266526  | 0.544159  |       0.19761   |       9 |      3.50711 |
| LCE2B  | connective tissue cell                                  |      1.72284 |          267 |       0.00105022  |          0.000327053 |               1873 | 0.393491    | 0.253594  | 0.331547 | 0.764881 | -0.141738  | 0.22713    |   0.266526  | 0.544159  |       0.19761   |       9 |      3.50711 |
| LCE2C  | connective tissue cell                                  |      1.69651 |            9 |       7.95918e-05 |          1.10243e-05 |               1873 | 0.113323    | 0.253594  | 0.331547 | 0.764881 | -0.141738  | 0.22713    |   0.266526  | 0.544159  |       0.19761   |       9 |      3.50711 |
| WFDC12 | connective tissue cell                                  |      1.7957  |            6 |       2.56257e-05 |          7.3495e-06  |               1873 | 0.150186    | 0.253594  | 0.331547 | 0.764881 | -0.141738  | 0.22713    |   0.266526  | 0.544159  |       0.19761   |       9 |      3.50711 |
| ARG1   | epithelial fate stem cell                               |      2.10091 |           15 |       0.000426937 |          1.83738e-05 |                548 | 0.283404    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| FLG2   | epithelial fate stem cell                               |      1.78299 |          203 |       0.00577788  |          0.000248658 |                548 | 0.358375    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| KPRP   | epithelial fate stem cell                               |      2.17081 |            8 |       0.000256123 |          9.79933e-06 |                548 | 0.304104    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| LCE1A  | epithelial fate stem cell                               |      1.7461  |           34 |       0.000967724 |          4.16472e-05 |                548 | 0.160659    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| LCE1B  | epithelial fate stem cell                               |      1.68141 |           79 |       0.00224853  |          9.67684e-05 |                548 | 0.183214    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| LCE1E  | epithelial fate stem cell                               |      1.75269 |          111 |       0.00315933  |          0.000135966 |                548 | 0.246615    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| LCE2B  | epithelial fate stem cell                               |      1.74494 |           89 |       0.00253316  |          0.000109018 |                548 | 0.219688    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| LCE2C  | epithelial fate stem cell                               |      2.00001 |            4 |       0.000128061 |          4.89967e-06 |                548 | 0.230772    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| WFDC12 | epithelial fate stem cell                               |      2.03144 |            5 |       0.000160077 |          6.12458e-06 |                548 | 0.244554    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| ARG1   | basal cell                                              |      2.10091 |           15 |       0.000426937 |          1.83738e-05 |                548 | 0.283404    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| FLG2   | basal cell                                              |      1.78299 |          203 |       0.00577788  |          0.000248658 |                548 | 0.358375    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| KPRP   | basal cell                                              |      2.17081 |            8 |       0.000256123 |          9.79933e-06 |                548 | 0.304104    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| LCE1A  | basal cell                                              |      1.7461  |           34 |       0.000967724 |          4.16472e-05 |                548 | 0.160659    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| LCE1B  | basal cell                                              |      1.68141 |           79 |       0.00224853  |          9.67684e-05 |                548 | 0.183214    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| LCE1E  | basal cell                                              |      1.75269 |          111 |       0.00315933  |          0.000135966 |                548 | 0.246615    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| LCE2B  | basal cell                                              |      1.74494 |           89 |       0.00253316  |          0.000109018 |                548 | 0.219688    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| LCE2C  | basal cell                                              |      2.00001 |            4 |       0.000128061 |          4.89967e-06 |                548 | 0.230772    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| WFDC12 | basal cell                                              |      2.03144 |            5 |       0.000160077 |          6.12458e-06 |                548 | 0.244554    | 0.0607469 | 0.247932 | 0.245015 |  0.0825274 | 0.0177715  |   0.160303  | 0.0427882 |       0.227195  |       9 |      3.45093 |
| ARG1   | stem cell                                               |      2.10091 |           15 |       0.000426937 |          1.83738e-05 |                551 | 0.283404    | 0.0626802 | 0.244656 | 0.256197 | -0.169984  | 0.0198704  |   0.156142  | 0.0535728 |       0.193884  |       9 |      3.4431  |
| FLG2   | stem cell                                               |      1.78299 |          203 |       0.00577788  |          0.000248658 |                551 | 0.358375    | 0.0626802 | 0.244656 | 0.256197 | -0.169984  | 0.0198704  |   0.156142  | 0.0535728 |       0.193884  |       9 |      3.4431  |
| KPRP   | stem cell                                               |      2.17081 |            8 |       0.000256123 |          9.79933e-06 |                551 | 0.304104    | 0.0626802 | 0.244656 | 0.256197 | -0.169984  | 0.0198704  |   0.156142  | 0.0535728 |       0.193884  |       9 |      3.4431  |
| LCE1A  | stem cell                                               |      1.73821 |           35 |       0.000963869 |          4.28721e-05 |                551 | 0.158506    | 0.0626802 | 0.244656 | 0.256197 | -0.169984  | 0.0198704  |   0.156142  | 0.0535728 |       0.193884  |       9 |      3.4431  |
| LCE1B  | stem cell                                               |      1.68296 |           80 |       0.00220313  |          9.79933e-05 |                551 | 0.18481     | 0.0626802 | 0.244656 | 0.256197 | -0.169984  | 0.0198704  |   0.156142  | 0.0535728 |       0.193884  |       9 |      3.4431  |
| LCE1E  | stem cell                                               |      1.75269 |          111 |       0.00315933  |          0.000135966 |                551 | 0.246615    | 0.0626802 | 0.244656 | 0.256197 | -0.169984  | 0.0198704  |   0.156142  | 0.0535728 |       0.193884  |       9 |      3.4431  |
| LCE2B  | stem cell                                               |      1.74494 |           89 |       0.00253316  |          0.000109018 |                551 | 0.219688    | 0.0626802 | 0.244656 | 0.256197 | -0.169984  | 0.0198704  |   0.156142  | 0.0535728 |       0.193884  |       9 |      3.4431  |
| LCE2C  | stem cell                                               |      1.92572 |            5 |       0.000154259 |          6.12458e-06 |                551 | 0.20185     | 0.0626802 | 0.244656 | 0.256197 | -0.169984  | 0.0198704  |   0.156142  | 0.0535728 |       0.193884  |       9 |      3.4431  |
| WFDC12 | stem cell                                               |      2.03144 |            5 |       0.000160077 |          6.12458e-06 |                551 | 0.244554    | 0.0626802 | 0.244656 | 0.256197 | -0.169984  | 0.0198704  |   0.156142  | 0.0535728 |       0.193884  |       9 |      3.4431  |
| ARG1   | fibroblast                                              |      1.86545 |           57 |       0.000243444 |          6.98203e-05 |               1242 | 0.231674    | 0.170065  | 0.262255 | 0.648471 |  0.231146  | 0.13645    |   0.1785    | 0.431891  |       0.246801  |       9 |      3.09092 |
| FLG2   | fibroblast                                              |      1.74877 |          359 |       0.00153327  |          0.000439745 |               1242 | 0.500285    | 0.170065  | 0.262255 | 0.648471 |  0.231146  | 0.13645    |   0.1785    | 0.431891  |       0.246801  |       9 |      3.09092 |
| KPRP   | fibroblast                                              |      1.70178 |            9 |       3.84385e-05 |          1.10243e-05 |               1242 | 0.115391    | 0.170065  | 0.262255 | 0.648471 |  0.231146  | 0.13645    |   0.1785    | 0.431891  |       0.246801  |       9 |      3.09092 |
| LCE1A  | fibroblast                                              |      1.64295 |           56 |       0.000239173 |          6.85953e-05 |               1242 | 0.140766    | 0.170065  | 0.262255 | 0.648471 |  0.231146  | 0.13645    |   0.1785    | 0.431891  |       0.246801  |       9 |      3.09092 |
| LCE1B  | fibroblast                                              |      1.68077 |          144 |       0.000615017 |          0.000176388 |               1242 | 0.248043    | 0.170065  | 0.262255 | 0.648471 |  0.231146  | 0.13645    |   0.1785    | 0.431891  |       0.246801  |       9 |      3.09092 |
| LCE1E  | fibroblast                                              |      1.70263 |          455 |       0.00194328  |          0.000557337 |               1242 | 0.582013    | 0.170065  | 0.262255 | 0.648471 |  0.231146  | 0.13645    |   0.1785    | 0.431891  |       0.246801  |       9 |      3.09092 |
| LCE2B  | fibroblast                                              |      1.74871 |          147 |       0.00062783  |          0.000180063 |               1242 | 0.278619    | 0.170065  | 0.262255 | 0.648471 |  0.231146  | 0.13645    |   0.1785    | 0.431891  |       0.246801  |       9 |      3.09092 |
| LCE2C  | fibroblast                                              |      1.69651 |            9 |       7.95918e-05 |          1.10243e-05 |               1242 | 0.113323    | 0.170065  | 0.262255 | 0.648471 |  0.231146  | 0.13645    |   0.1785    | 0.431891  |       0.246801  |       9 |      3.09092 |
| WFDC12 | fibroblast                                              |      1.7957  |            6 |       2.56257e-05 |          7.3495e-06  |               1242 | 0.150186    | 0.170065  | 0.262255 | 0.648471 |  0.231146  | 0.13645    |   0.1785    | 0.431891  |       0.246801  |       9 |      3.09092 |
| ARG1   | precursor cell                                          |      2.28423 |          212 |       0.00481249  |          0.000259682 |                777 | 0.568716    | 0.125889  | 0.279427 | 0.450526 |  3.33495   | 0.0884911  |   0.200314  | 0.240988  |       0.656254  |       9 |      3.07482 |
| FLG2   | precursor cell                                          |      1.77619 |          220 |       0.00499909  |          0.000269482 |                777 | 0.372125    | 0.125889  | 0.279427 | 0.450526 |  3.33495   | 0.0884911  |   0.200314  | 0.240988  |       0.656254  |       9 |      3.07482 |
| KPRP   | precursor cell                                          |      2.17081 |            8 |       0.000256123 |          9.79933e-06 |                777 | 0.304104    | 0.125889  | 0.279427 | 0.450526 |  3.33495   | 0.0884911  |   0.200314  | 0.240988  |       0.656254  |       9 |      3.07482 |
| LCE1A  | precursor cell                                          |      1.72994 |           36 |       0.000796707 |          4.4097e-05  |                777 | 0.155954    | 0.125889  | 0.279427 | 0.450526 |  3.33495   | 0.0884911  |   0.200314  | 0.240988  |       0.656254  |       9 |      3.07482 |
| LCE1B  | precursor cell                                          |      1.69341 |           85 |       0.00188111  |          0.000104118 |                777 | 0.193744    | 0.125889  | 0.279427 | 0.450526 |  3.33495   | 0.0884911  |   0.200314  | 0.240988  |       0.656254  |       9 |      3.07482 |
| LCE1E  | precursor cell                                          |      1.76037 |          114 |       0.00259044  |          0.000139641 |                777 | 0.251981    | 0.125889  | 0.279427 | 0.450526 |  3.33495   | 0.0884911  |   0.200314  | 0.240988  |       0.656254  |       9 |      3.07482 |
| LCE2B  | precursor cell                                          |      1.74412 |           92 |       0.00209053  |          0.000112692 |                777 | 0.22181     | 0.125889  | 0.279427 | 0.450526 |  3.33495   | 0.0884911  |   0.200314  | 0.240988  |       0.656254  |       9 |      3.07482 |
| LCE2C  | precursor cell                                          |      1.92572 |            5 |       0.000154259 |          6.12458e-06 |                777 | 0.20185     | 0.125889  | 0.279427 | 0.450526 |  3.33495   | 0.0884911  |   0.200314  | 0.240988  |       0.656254  |       9 |      3.07482 |
| WFDC12 | precursor cell                                          |      2.03144 |            5 |       0.000160077 |          6.12458e-06 |                777 | 0.244554    | 0.125889  | 0.279427 | 0.450526 |  3.33495   | 0.0884911  |   0.200314  | 0.240988  |       0.656254  |       9 |      3.07482 |
| ARG1   | progenitor cell                                         |      2.29364 |          207 |       0.00515528  |          0.000253558 |                596 | 0.567833    | 0.117375  | 0.275471 | 0.426089 |  5.94066   | 0.0792482  |   0.195288  | 0.217421  |       1         |       9 |      2.76331 |
| FLG2   | progenitor cell                                         |      1.81941 |          136 |       0.00339076  |          0.000166589 |                596 | 0.299886    | 0.117375  | 0.275471 | 0.426089 |  5.94066   | 0.0792482  |   0.195288  | 0.217421  |       1         |       9 |      2.76331 |
| KPRP   | progenitor cell                                         |      2.17081 |            8 |       0.000256123 |          9.79933e-06 |                596 | 0.304104    | 0.117375  | 0.275471 | 0.426089 |  5.94066   | 0.0792482  |   0.195288  | 0.217421  |       1         |       9 |      2.76331 |
| LCE1A  | progenitor cell                                         |      1.88838 |           16 |       0.000398913 |          1.95987e-05 |                596 | 0.198566    | 0.117375  | 0.275471 | 0.426089 |  5.94066   | 0.0792482  |   0.195288  | 0.217421  |       1         |       9 |      2.76331 |
| LCE1B  | progenitor cell                                         |      1.7699  |           44 |       0.00109701  |          5.38963e-05 |                596 | 0.180858    | 0.117375  | 0.275471 | 0.426089 |  5.94066   | 0.0792482  |   0.195288  | 0.217421  |       1         |       9 |      2.76331 |
| LCE1E  | progenitor cell                                         |      1.76803 |          104 |       0.00259293  |          0.000127391 |                596 | 0.244691    | 0.117375  | 0.275471 | 0.426089 |  5.94066   | 0.0792482  |   0.195288  | 0.217421  |       1         |       9 |      2.76331 |
| LCE2B  | progenitor cell                                         |      1.7624  |           72 |       0.00179511  |          8.8194e-05  |                596 | 0.20797     | 0.117375  | 0.275471 | 0.426089 |  5.94066   | 0.0792482  |   0.195288  | 0.217421  |       1         |       9 |      2.76331 |
| LCE2C  | progenitor cell                                         |      2.00001 |            4 |       0.000128061 |          4.89967e-06 |                596 | 0.230772    | 0.117375  | 0.275471 | 0.426089 |  5.94066   | 0.0792482  |   0.195288  | 0.217421  |       1         |       9 |      2.76331 |
| WFDC12 | progenitor cell                                         |      2.03144 |            5 |       0.000160077 |          6.12458e-06 |                596 | 0.244554    | 0.117375  | 0.275471 | 0.426089 |  5.94066   | 0.0792482  |   0.195288  | 0.217421  |       1         |       9 |      2.76331 |
| ARG1   | skin fibroblast                                         |      1.9666  |           17 |       0.00015034  |          2.08236e-05 |                145 | 0.230822    | 0.044377  | 0.150554 | 0.294759 | -0.0764263 | 0          |   0.0365955 | 0.0907626 |       0.206226  |       9 |      2.66326 |
| FLG2   | skin fibroblast                                         |      1.72827 |           24 |       0.000212245 |          2.9398e-05  |                145 | 0.141935    | 0.044377  | 0.150554 | 0.294759 | -0.0764263 | 0          |   0.0365955 | 0.0907626 |       0.206226  |       9 |      2.66326 |
| KPRP   | skin fibroblast                                         |      1.68973 |            7 |       6.19047e-05 |          8.57442e-06 |                145 | 0.108478    | 0.044377  | 0.150554 | 0.294759 | -0.0764263 | 0          |   0.0365955 | 0.0907626 |       0.206226  |       9 |      2.66326 |
| LCE1A  | skin fibroblast                                         |      1.70943 |           18 |       0.000159184 |          2.20485e-05 |                145 | 0.128012    | 0.044377  | 0.150554 | 0.294759 | -0.0764263 | 0          |   0.0365955 | 0.0907626 |       0.206226  |       9 |      2.66326 |
| LCE1B  | skin fibroblast                                         |      1.77415 |           26 |       0.000229932 |          3.18478e-05 |                145 | 0.162569    | 0.044377  | 0.150554 | 0.294759 | -0.0764263 | 0          |   0.0365955 | 0.0907626 |       0.206226  |       9 |      2.66326 |
| LCE1E  | skin fibroblast                                         |      1.67346 |           20 |       0.000176871 |          2.44983e-05 |                145 | 0.11559     | 0.044377  | 0.150554 | 0.294759 | -0.0764263 | 0          |   0.0365955 | 0.0907626 |       0.206226  |       9 |      2.66326 |
| LCE2B  | skin fibroblast                                         |      1.73329 |           20 |       0.000176871 |          2.44983e-05 |                145 | 0.139752    | 0.044377  | 0.150554 | 0.294759 | -0.0764263 | 0          |   0.0365955 | 0.0907626 |       0.206226  |       9 |      2.66326 |
| LCE2C  | skin fibroblast                                         |      1.69651 |            9 |       7.95918e-05 |          1.10243e-05 |                145 | 0.113323    | 0.044377  | 0.150554 | 0.294759 | -0.0764263 | 0          |   0.0365955 | 0.0907626 |       0.206226  |       9 |      2.66326 |
| WFDC12 | skin fibroblast                                         |      1.96006 |            4 |       3.53741e-05 |          4.89967e-06 |                145 | 0.214501    | 0.044377  | 0.150554 | 0.294759 | -0.0764263 | 0          |   0.0365955 | 0.0907626 |       0.206226  |       9 |      2.66326 |
| ARG1   | T cell                                                  |      1.76918 |         1900 |       0.0194191   |          0.00232734  |               3854 | 2.13656     | 0.734759  | 0.628786 | 1.16853  |  1.79193   | 0.749492   |   0.644135  | 0.933452  |       0.452699  |       8 |      4.29864 |
| FLG2   | T cell                                                  |      1.73732 |         1193 |       0.0123805   |          0.00146133  |               3854 | 1.37851     | 0.734759  | 0.628786 | 1.16853  |  1.79193   | 0.749492   |   0.644135  | 0.933452  |       0.452699  |       8 |      4.29864 |
| LCE1A  | T cell                                                  |      1.67576 |           41 |       0.000425483 |          5.02216e-05 |               3854 | 0.138712    | 0.734759  | 0.628786 | 1.16853  |  1.79193   | 0.749492   |   0.644135  | 0.933452  |       0.452699  |       8 |      4.29864 |
| LCE1B  | T cell                                                  |      1.64557 |          297 |       0.00308216  |          0.0003638   |               3854 | 0.396507    | 0.734759  | 0.628786 | 1.16853  |  1.79193   | 0.749492   |   0.644135  | 0.933452  |       0.452699  |       8 |      4.29864 |
| LCE1E  | T cell                                                  |      1.76773 |          161 |       0.0016708   |          0.000197212 |               3854 | 0.302413    | 0.734759  | 0.628786 | 1.16853  |  1.79193   | 0.749492   |   0.644135  | 0.933452  |       0.452699  |       8 |      4.29864 |
| LCE2B  | T cell                                                  |      1.77071 |          254 |       0.00263592  |          0.000311129 |               3854 | 0.401695    | 0.734759  | 0.628786 | 1.16853  |  1.79193   | 0.749492   |   0.644135  | 0.933452  |       0.452699  |       8 |      4.29864 |
| LCE2C  | T cell                                                  |      1.60669 |            1 |       0.000144739 |          1.22492e-06 |               3854 | 0.0688321   | 0.734759  | 0.628786 | 1.16853  |  1.79193   | 0.749492   |   0.644135  | 0.933452  |       0.452699  |       8 |      4.29864 |
| WFDC12 | T cell                                                  |      1.93284 |            7 |       0.000324856 |          8.57442e-06 |               3854 | 0.207059    | 0.734759  | 0.628786 | 1.16853  |  1.79193   | 0.749492   |   0.644135  | 0.933452  |       0.452699  |       8 |      4.29864 |
| ARG1   | phagocyte                                               |      2.24668 |          228 |       0.00331405  |          0.000279281 |                480 | 0.567925    | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| FLG2   | phagocyte                                               |      1.75228 |          116 |       0.0015903   |          0.00014209  |                480 | 0.249293    | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| KPRP   | phagocyte                                               |      1.7265  |            5 |       8.81399e-05 |          6.12458e-06 |                480 | 0.121291    | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| LCE1A  | phagocyte                                               |      1.79362 |           11 |       0.00013799  |          1.34741e-05 |                480 | 0.154706    | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| LCE1B  | phagocyte                                               |      1.74065 |           37 |       0.000464148 |          4.53219e-05 |                480 | 0.16082     | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| LCE1E  | phagocyte                                               |      1.80569 |           41 |       0.000562383 |          5.02216e-05 |                480 | 0.191392    | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| LCE2B  | phagocyte                                               |      1.71833 |           41 |       0.000536032 |          5.02216e-05 |                480 | 0.15607     | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| WFDC12 | phagocyte                                               |      2.37459 |            1 |       8.04311e-05 |          1.22492e-06 |                480 | 0.378865    | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| ARG1   | mononuclear phagocyte                                   |      2.24668 |          228 |       0.00331405  |          0.000279281 |                480 | 0.567925    | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| FLG2   | mononuclear phagocyte                                   |      1.75228 |          116 |       0.0015903   |          0.00014209  |                480 | 0.249293    | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| KPRP   | mononuclear phagocyte                                   |      1.7265  |            5 |       8.81399e-05 |          6.12458e-06 |                480 | 0.121291    | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| LCE1A  | mononuclear phagocyte                                   |      1.79362 |           11 |       0.00013799  |          1.34741e-05 |                480 | 0.154706    | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| LCE1B  | mononuclear phagocyte                                   |      1.74065 |           37 |       0.000464148 |          4.53219e-05 |                480 | 0.16082     | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| LCE1E  | mononuclear phagocyte                                   |      1.80569 |           41 |       0.000562383 |          5.02216e-05 |                480 | 0.191392    | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| LCE2B  | mononuclear phagocyte                                   |      1.71833 |           41 |       0.000536032 |          5.02216e-05 |                480 | 0.15607     | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| WFDC12 | mononuclear phagocyte                                   |      2.37459 |            1 |       8.04311e-05 |          1.22492e-06 |                480 | 0.378865    | 0.152827  | 0.247545 | 0.617369 |  2.20086   | 0.117735   |   0.159812  | 0.401896  |       0.506646  |       8 |      2.47393 |
| ARG1   | myeloid leukocyte                                       |      2.25604 |          246 |       0.00383823  |          0.00030133  |                477 | 0.591196    | 0.160368  | 0.253426 | 0.632802 |  2.4883    | 0.125922   |   0.167282  | 0.416779  |       0.544565  |       8 |      2.46697 |
| FLG2   | myeloid leukocyte                                       |      1.7724  |          103 |       0.00165345  |          0.000126166 |                477 | 0.244004    | 0.160368  | 0.253426 | 0.632802 |  2.4883    | 0.125922   |   0.167282  | 0.416779  |       0.544565  |       8 |      2.46697 |
| KPRP   | myeloid leukocyte                                       |      1.63116 |            4 |       8.15694e-05 |          4.89967e-06 |                477 | 0.0817363   | 0.160368  | 0.253426 | 0.632802 |  2.4883    | 0.125922   |   0.167282  | 0.416779  |       0.544565  |       8 |      2.46697 |
| LCE1A  | myeloid leukocyte                                       |      1.87828 |           10 |       0.000133889 |          1.22492e-05 |                477 | 0.187853    | 0.160368  | 0.253426 | 0.632802 |  2.4883    | 0.125922   |   0.167282  | 0.416779  |       0.544565  |       8 |      2.46697 |
| LCE1B  | myeloid leukocyte                                       |      1.77249 |           36 |       0.000481999 |          4.4097e-05  |                477 | 0.172666    | 0.160368  | 0.253426 | 0.632802 |  2.4883    | 0.125922   |   0.167282  | 0.416779  |       0.544565  |       8 |      2.46697 |
| LCE1E  | myeloid leukocyte                                       |      1.84419 |           37 |       0.00059432  |          4.53219e-05 |                477 | 0.202831    | 0.160368  | 0.253426 | 0.632802 |  2.4883    | 0.125922   |   0.167282  | 0.416779  |       0.544565  |       8 |      2.46697 |
| LCE2B  | myeloid leukocyte                                       |      1.75073 |           40 |       0.000627244 |          4.89967e-05 |                477 | 0.168254    | 0.160368  | 0.253426 | 0.632802 |  2.4883    | 0.125922   |   0.167282  | 0.416779  |       0.544565  |       8 |      2.46697 |
| WFDC12 | myeloid leukocyte                                       |      2.37459 |            1 |       8.04311e-05 |          1.22492e-06 |                477 | 0.378865    | 0.160368  | 0.253426 | 0.632802 |  2.4883    | 0.125922   |   0.167282  | 0.416779  |       0.544565  |       8 |      2.46697 |
| ARG1   | professional antigen presenting cell                    |      1.93645 |           36 |       0.000600761 |          4.4097e-05  |                259 | 0.239064    | 0.0821301 | 0.203046 | 0.40449  |  2.78807   | 0.0409856  |   0.103281  | 0.19659   |       0.58411   |       8 |      2.32103 |
| FLG2   | professional antigen presenting cell                    |      1.76212 |           99 |       0.00154523  |          0.000121267 |                259 | 0.235534    | 0.0821301 | 0.203046 | 0.40449  |  2.78807   | 0.0409856  |   0.103281  | 0.19659   |       0.58411   |       8 |      2.32103 |
| KPRP   | professional antigen presenting cell                    |      1.7265  |            5 |       8.81399e-05 |          6.12458e-06 |                259 | 0.121291    | 0.0821301 | 0.203046 | 0.40449  |  2.78807   | 0.0409856  |   0.103281  | 0.19659   |       0.58411   |       8 |      2.32103 |
| LCE1A  | professional antigen presenting cell                    |      1.82893 |           10 |       0.000141159 |          1.22492e-05 |                259 | 0.167934    | 0.0821301 | 0.203046 | 0.40449  |  2.78807   | 0.0409856  |   0.103281  | 0.19659   |       0.58411   |       8 |      2.32103 |
| LCE1B  | professional antigen presenting cell                    |      1.7219  |           32 |       0.000451709 |          3.91973e-05 |                259 | 0.148033    | 0.0821301 | 0.203046 | 0.40449  |  2.78807   | 0.0409856  |   0.103281  | 0.19659   |       0.58411   |       8 |      2.32103 |
| LCE1E  | professional antigen presenting cell                    |      1.78683 |           38 |       0.000593472 |          4.65468e-05 |                259 | 0.180704    | 0.0821301 | 0.203046 | 0.40449  |  2.78807   | 0.0409856  |   0.103281  | 0.19659   |       0.58411   |       8 |      2.32103 |
| LCE2B  | professional antigen presenting cell                    |      1.71822 |           38 |       0.000562014 |          4.65468e-05 |                259 | 0.152946    | 0.0821301 | 0.203046 | 0.40449  |  2.78807   | 0.0409856  |   0.103281  | 0.19659   |       0.58411   |       8 |      2.32103 |
| WFDC12 | professional antigen presenting cell                    |      2.37459 |            1 |       8.04311e-05 |          1.22492e-06 |                259 | 0.378865    | 0.0821301 | 0.203046 | 0.40449  |  2.78807   | 0.0409856  |   0.103281  | 0.19659   |       0.58411   |       8 |      2.32103 |
| ARG1   | innate lymphoid cell                                    |      1.71475 |          222 |       0.00731418  |          0.000271932 |               1318 | 0.352861    | 0.26003   | 0.285152 | 0.911901 |  5.07797   | 0.234118   |   0.207587  | 0.685949  |       0.886194  |       8 |      2.11748 |
| FLG2   | innate lymphoid cell                                    |      1.7179  |          707 |       0.0246814   |          0.000866016 |               1318 | 0.884133    | 0.26003   | 0.285152 | 0.911901 |  5.07797   | 0.234118   |   0.207587  | 0.685949  |       0.886194  |       8 |      2.11748 |
| KPRP   | innate lymphoid cell                                    |      1.56872 |            2 |       0.000107863 |          2.44983e-06 |               1318 | 0.0544826   | 0.26003   | 0.285152 | 0.911901 |  5.07797   | 0.234118   |   0.207587  | 0.685949  |       0.886194  |       8 |      2.11748 |
| LCE1A  | innate lymphoid cell                                    |      1.62602 |           24 |       0.000837843 |          2.9398e-05  |               1318 | 0.101577    | 0.26003   | 0.285152 | 0.911901 |  5.07797   | 0.234118   |   0.207587  | 0.685949  |       0.886194  |       8 |      2.11748 |
| LCE1B  | innate lymphoid cell                                    |      1.66818 |          146 |       0.00509688  |          0.000178838 |               1318 | 0.25176     | 0.26003   | 0.285152 | 0.911901 |  5.07797   | 0.234118   |   0.207587  | 0.685949  |       0.886194  |       8 |      2.11748 |
| LCE1E  | innate lymphoid cell                                    |      1.78201 |           89 |       0.003107    |          0.000109018 |               1318 | 0.235521    | 0.26003   | 0.285152 | 0.911901 |  5.07797   | 0.234118   |   0.207587  | 0.685949  |       0.886194  |       8 |      2.11748 |
| LCE2B  | innate lymphoid cell                                    |      1.7203  |          126 |       0.00439867  |          0.00015434  |               1318 | 0.250982    | 0.26003   | 0.285152 | 0.911901 |  5.07797   | 0.234118   |   0.207587  | 0.685949  |       0.886194  |       8 |      2.11748 |
| WFDC12 | innate lymphoid cell                                    |      1.80498 |            2 |       0.000107863 |          2.44983e-06 |               1318 | 0.149898    | 0.26003   | 0.285152 | 0.911901 |  5.07797   | 0.234118   |   0.207587  | 0.685949  |       0.886194  |       8 |      2.11748 |
| ARG1   | melanocyte of skin                                      |      1.76214 |            9 |       0.00353218  |          1.10243e-05 |                199 | 0.145008    | 0.0713325 | 0.129624 | 0.550302 | -0.822833  | 0.0292635  |   0.0100073 | 0.337215  |       0.10776   |       8 |      2.07665 |
| FLG2   | melanocyte of skin                                      |      1.71048 |           65 |       0.0255102   |          7.96196e-05 |                199 | 0.2153      | 0.0713325 | 0.129624 | 0.550302 | -0.822833  | 0.0292635  |   0.0100073 | 0.337215  |       0.10776   |       8 |      2.07665 |
| LCE1A  | melanocyte of skin                                      |      1.63828 |           11 |       0.00431711  |          1.34741e-05 |                199 | 0.0982382   | 0.0713325 | 0.129624 | 0.550302 | -0.822833  | 0.0292635  |   0.0100073 | 0.337215  |       0.10776   |       8 |      2.07665 |
| LCE1B  | melanocyte of skin                                      |      1.57861 |           25 |       0.00981162  |          3.06229e-05 |                199 | 0.0969299   | 0.0713325 | 0.129624 | 0.550302 | -0.822833  | 0.0292635  |   0.0100073 | 0.337215  |       0.10776   |       8 |      2.07665 |
| LCE1E  | melanocyte of skin                                      |      1.71292 |           64 |       0.0251177   |          7.83947e-05 |                199 | 0.214658    | 0.0713325 | 0.129624 | 0.550302 | -0.822833  | 0.0292635  |   0.0100073 | 0.337215  |       0.10776   |       8 |      2.07665 |
| LCE2B  | melanocyte of skin                                      |      1.78615 |           23 |       0.00902669  |          2.81731e-05 |                199 | 0.177491    | 0.0713325 | 0.129624 | 0.550302 | -0.822833  | 0.0292635  |   0.0100073 | 0.337215  |       0.10776   |       8 |      2.07665 |
| LCE2C  | melanocyte of skin                                      |      1.46698 |            1 |       0.000392465 |          1.22492e-06 |                199 | 0.0127794   | 0.0713325 | 0.129624 | 0.550302 | -0.822833  | 0.0292635  |   0.0100073 | 0.337215  |       0.10776   |       8 |      2.07665 |
| WFDC12 | melanocyte of skin                                      |      1.62498 |            1 |       0.000392465 |          1.22492e-06 |                199 | 0.0765899   | 0.0713325 | 0.129624 | 0.550302 | -0.822833  | 0.0292635  |   0.0100073 | 0.337215  |       0.10776   |       8 |      2.07665 |
| ARG1   | melanocyte                                              |      1.76214 |            9 |       0.00353218  |          1.10243e-05 |                210 | 0.145008    | 0.062089  | 0.121747 | 0.509984 | -0.411171  | 0.0192285  |   0         | 0.298331  |       0.162066  |       8 |      2.0163  |
| FLG2   | melanocyte                                              |      1.71266 |           72 |       0.00595632  |          8.8194e-05  |                210 | 0.194123    | 0.062089  | 0.121747 | 0.509984 | -0.411171  | 0.0192285  |   0         | 0.298331  |       0.162066  |       8 |      2.0163  |
| LCE1A  | melanocyte                                              |      1.63828 |           11 |       0.00431711  |          1.34741e-05 |                210 | 0.0982382   | 0.062089  | 0.121747 | 0.509984 | -0.411171  | 0.0192285  |   0         | 0.298331  |       0.162066  |       8 |      2.0163  |
| LCE1B  | melanocyte                                              |      1.60135 |           26 |       0.00215089  |          3.18478e-05 |                210 | 0.0956636   | 0.062089  | 0.121747 | 0.509984 | -0.411171  | 0.0192285  |   0         | 0.298331  |       0.162066  |       8 |      2.0163  |
| LCE1E  | melanocyte                                              |      1.71335 |           65 |       0.00537723  |          7.96196e-05 |                210 | 0.186258    | 0.062089  | 0.121747 | 0.509984 | -0.411171  | 0.0192285  |   0         | 0.298331  |       0.162066  |       8 |      2.0163  |
| LCE2B  | melanocyte                                              |      1.7767  |           25 |       0.00206817  |          3.06229e-05 |                210 | 0.165316    | 0.062089  | 0.121747 | 0.509984 | -0.411171  | 0.0192285  |   0         | 0.298331  |       0.162066  |       8 |      2.0163  |
| LCE2C  | melanocyte                                              |      1.46698 |            1 |       0.000392465 |          1.22492e-06 |                210 | 0.0127794   | 0.062089  | 0.121747 | 0.509984 | -0.411171  | 0.0192285  |   0         | 0.298331  |       0.162066  |       8 |      2.0163  |
| WFDC12 | melanocyte                                              |      1.62498 |            1 |       0.000392465 |          1.22492e-06 |                210 | 0.0765899   | 0.062089  | 0.121747 | 0.509984 | -0.411171  | 0.0192285  |   0         | 0.298331  |       0.162066  |       8 |      2.0163  |
| ARG1   | natural killer cell                                     |      1.72748 |          168 |       0.00906051  |          0.000205786 |               1009 | 0.304514    | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| FLG2   | natural killer cell                                     |      1.72372 |          529 |       0.0285298   |          0.000647981 |               1009 | 0.707303    | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| KPRP   | natural killer cell                                     |      1.56872 |            2 |       0.000107863 |          2.44983e-06 |               1009 | 0.0544826   | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| LCE1A  | natural killer cell                                     |      1.62616 |           18 |       0.000970769 |          2.20485e-05 |               1009 | 0.0956011   | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| LCE1B  | natural killer cell                                     |      1.66517 |          122 |       0.00657966  |          0.00014944  |               1009 | 0.227832    | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| LCE1E  | natural killer cell                                     |      1.75872 |           66 |       0.00355949  |          8.08445e-05 |               1009 | 0.202896    | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| LCE2B  | natural killer cell                                     |      1.73013 |          102 |       0.00550102  |          0.000124942 |               1009 | 0.231665    | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| WFDC12 | natural killer cell                                     |      1.80498 |            2 |       0.000107863 |          2.44983e-06 |               1009 | 0.149898    | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| ARG1   | group 1 innate lymphoid cell                            |      1.72748 |          168 |       0.00906051  |          0.000205786 |               1009 | 0.304514    | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| FLG2   | group 1 innate lymphoid cell                            |      1.72372 |          529 |       0.0285298   |          0.000647981 |               1009 | 0.707303    | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| KPRP   | group 1 innate lymphoid cell                            |      1.56872 |            2 |       0.000107863 |          2.44983e-06 |               1009 | 0.0544826   | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| LCE1A  | group 1 innate lymphoid cell                            |      1.62616 |           18 |       0.000970769 |          2.20485e-05 |               1009 | 0.0956011   | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| LCE1B  | group 1 innate lymphoid cell                            |      1.66517 |          122 |       0.00657966  |          0.00014944  |               1009 | 0.227832    | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| LCE1E  | group 1 innate lymphoid cell                            |      1.75872 |           66 |       0.00355949  |          8.08445e-05 |               1009 | 0.202896    | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| LCE2B  | group 1 innate lymphoid cell                            |      1.73013 |          102 |       0.00550102  |          0.000124942 |               1009 | 0.231665    | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| WFDC12 | group 1 innate lymphoid cell                            |      1.80498 |            2 |       0.000107863 |          2.44983e-06 |               1009 | 0.149898    | 0.202536  | 0.246774 | 0.820737 |  4.67089   | 0.171701   |   0.158832  | 0.598028  |       0.832491  |       8 |      2.00768 |
| ARG1   | endothelial cell                                        |      1.79131 |           20 |       0.000456538 |          2.44983e-05 |                274 | 0.163606    | 0.071793  | 0.133573 | 0.537483 |  0.864107  | 0.0297634  |   0.0150231 | 0.324852  |       0.330301  |       8 |      1.9362  |
| FLG2   | endothelial cell                                        |      1.79505 |           84 |       0.00222534  |          0.000102893 |                274 | 0.234269    | 0.071793  | 0.133573 | 0.537483 |  0.864107  | 0.0297634  |   0.0150231 | 0.324852  |       0.330301  |       8 |      1.9362  |
| KPRP   | endothelial cell                                        |      1.61329 |            3 |       7.94765e-05 |          3.67475e-06 |                274 | 0.0734773   | 0.071793  | 0.133573 | 0.537483 |  0.864107  | 0.0297634  |   0.0150231 | 0.324852  |       0.330301  |       8 |      1.9362  |
| LCE1A  | endothelial cell                                        |      1.72843 |           25 |       0.000662304 |          3.06229e-05 |                274 | 0.143715    | 0.071793  | 0.133573 | 0.537483 |  0.864107  | 0.0297634  |   0.0150231 | 0.324852  |       0.330301  |       8 |      1.9362  |
| LCE1B  | endothelial cell                                        |      1.62663 |           51 |       0.0013511   |          6.24708e-05 |                274 | 0.130649    | 0.071793  | 0.133573 | 0.537483 |  0.864107  | 0.0297634  |   0.0150231 | 0.324852  |       0.330301  |       8 |      1.9362  |
| LCE1E  | endothelial cell                                        |      1.7371  |           67 |       0.00177498  |          8.20694e-05 |                274 | 0.192525    | 0.071793  | 0.133573 | 0.537483 |  0.864107  | 0.0297634  |   0.0150231 | 0.324852  |       0.330301  |       8 |      1.9362  |
| LCE2B  | endothelial cell                                        |      1.70054 |           23 |       0.00060932  |          2.81731e-05 |                274 | 0.130292    | 0.071793  | 0.133573 | 0.537483 |  0.864107  | 0.0297634  |   0.0150231 | 0.324852  |       0.330301  |       8 |      1.9362  |
| LCE2C  | endothelial cell                                        |      1.43679 |            1 |       3.17874e-05 |          1.22492e-06 |                274 | 4.76811e-05 | 0.071793  | 0.133573 | 0.537483 |  0.864107  | 0.0297634  |   0.0150231 | 0.324852  |       0.330301  |       8 |      1.9362  |
| ARG1   | endothelial cell of vascular tree                       |      1.77719 |           19 |       0.000493725 |          2.32734e-05 |                273 | 0.156921    | 0.0714315 | 0.132737 | 0.538143 |  0.917079  | 0.0293709  |   0.0139615 | 0.325488  |       0.337289  |       8 |      1.92447 |
| FLG2   | endothelial cell of vascular tree                       |      1.79505 |           84 |       0.00222534  |          0.000102893 |                273 | 0.234269    | 0.0714315 | 0.132737 | 0.538143 |  0.917079  | 0.0293709  |   0.0139615 | 0.325488  |       0.337289  |       8 |      1.92447 |
| KPRP   | endothelial cell of vascular tree                       |      1.61329 |            3 |       7.94765e-05 |          3.67475e-06 |                273 | 0.0734773   | 0.0714315 | 0.132737 | 0.538143 |  0.917079  | 0.0293709  |   0.0139615 | 0.325488  |       0.337289  |       8 |      1.92447 |
| LCE1A  | endothelial cell of vascular tree                       |      1.72843 |           25 |       0.000662304 |          3.06229e-05 |                273 | 0.143715    | 0.0714315 | 0.132737 | 0.538143 |  0.917079  | 0.0293709  |   0.0139615 | 0.325488  |       0.337289  |       8 |      1.92447 |
| LCE1B  | endothelial cell of vascular tree                       |      1.62663 |           51 |       0.0013511   |          6.24708e-05 |                273 | 0.130649    | 0.0714315 | 0.132737 | 0.538143 |  0.917079  | 0.0293709  |   0.0139615 | 0.325488  |       0.337289  |       8 |      1.92447 |
| LCE1E  | endothelial cell of vascular tree                       |      1.7371  |           67 |       0.00177498  |          8.20694e-05 |                273 | 0.192525    | 0.0714315 | 0.132737 | 0.538143 |  0.917079  | 0.0293709  |   0.0139615 | 0.325488  |       0.337289  |       8 |      1.92447 |
| LCE2B  | endothelial cell of vascular tree                       |      1.70054 |           23 |       0.00060932  |          2.81731e-05 |                273 | 0.130292    | 0.0714315 | 0.132737 | 0.538143 |  0.917079  | 0.0293709  |   0.0139615 | 0.325488  |       0.337289  |       8 |      1.92447 |
| LCE2C  | endothelial cell of vascular tree                       |      1.43679 |            1 |       3.17874e-05 |          1.22492e-06 |                273 | 4.76811e-05 | 0.0714315 | 0.132737 | 0.538143 |  0.917079  | 0.0293709  |   0.0139615 | 0.325488  |       0.337289  |       8 |      1.92447 |
| ARG1   | mature T cell                                           |      1.76916 |         1898 |       0.0208725   |          0.00232489  |               3842 | 2.13666     | 0.755791  | 0.707792 | 1.06782  |  1.15924   | 0.772326   |   0.744502  | 0.836316  |       0.369234  |       7 |      4.36355 |
| FLG2   | mature T cell                                           |      1.73724 |         1192 |       0.0133256   |          0.0014601   |               3842 | 1.37886     | 0.755791  | 0.707792 | 1.06782  |  1.15924   | 0.772326   |   0.744502  | 0.836316  |       0.369234  |       7 |      4.36355 |
| LCE1A  | mature T cell                                           |      1.67763 |           39 |       0.000435988 |          4.77718e-05 |               3842 | 0.137403    | 0.755791  | 0.707792 | 1.06782  |  1.15924   | 0.772326   |   0.744502  | 0.836316  |       0.369234  |       7 |      4.36355 |
| LCE1B  | mature T cell                                           |      1.64482 |          296 |       0.00330904  |          0.000362575 |               3842 | 0.395506    | 0.755791  | 0.707792 | 1.06782  |  1.15924   | 0.772326   |   0.744502  | 0.836316  |       0.369234  |       7 |      4.36355 |
| LCE1E  | mature T cell                                           |      1.76755 |          158 |       0.00176631  |          0.000193537 |               3842 | 0.299365    | 0.755791  | 0.707792 | 1.06782  |  1.15924   | 0.772326   |   0.744502  | 0.836316  |       0.369234  |       7 |      4.36355 |
| LCE2B  | mature T cell                                           |      1.77023 |          252 |       0.00281715  |          0.000308679 |               3842 | 0.399695    | 0.755791  | 0.707792 | 1.06782  |  1.15924   | 0.772326   |   0.744502  | 0.836316  |       0.369234  |       7 |      4.36355 |
| WFDC12 | mature T cell                                           |      1.93284 |            7 |       0.000324856 |          8.57442e-06 |               3842 | 0.207059    | 0.755791  | 0.707792 | 1.06782  |  1.15924   | 0.772326   |   0.744502  | 0.836316  |       0.369234  |       7 |      4.36355 |
| ARG1   | cytotoxic T cell                                        |      2.11815 |          197 |       0.0175987   |          0.000241309 |                376 | 0.505233    | 0.113546  | 0.313147 | 0.362596 | -0.301398  | 0.075091   |   0.243151  | 0.156186  |       0.176548  |       7 |      3.00596 |
| FLG2   | cytotoxic T cell                                        |      2.05539 |          107 |       0.00955869  |          0.000131066 |                376 | 0.374309    | 0.113546  | 0.313147 | 0.362596 | -0.301398  | 0.075091   |   0.243151  | 0.156186  |       0.176548  |       7 |      3.00596 |
| LCE1A  | cytotoxic T cell                                        |      1.92896 |            4 |       0.000357334 |          4.89967e-06 |                376 | 0.202422    | 0.113546  | 0.313147 | 0.362596 | -0.301398  | 0.075091   |   0.243151  | 0.156186  |       0.176548  |       7 |      3.00596 |
| LCE1B  | cytotoxic T cell                                        |      1.86535 |           14 |       0.00125067  |          1.71488e-05 |                376 | 0.188462    | 0.113546  | 0.313147 | 0.362596 | -0.301398  | 0.075091   |   0.243151  | 0.156186  |       0.176548  |       7 |      3.00596 |
| LCE1E  | cytotoxic T cell                                        |      2.24646 |           16 |       0.00142934  |          1.95987e-05 |                376 | 0.344728    | 0.113546  | 0.313147 | 0.362596 | -0.301398  | 0.075091   |   0.243151  | 0.156186  |       0.176548  |       7 |      3.00596 |
| LCE2B  | cytotoxic T cell                                        |      2.19513 |           35 |       0.00312668  |          4.28721e-05 |                376 | 0.346286    | 0.113546  | 0.313147 | 0.362596 | -0.301398  | 0.075091   |   0.243151  | 0.156186  |       0.176548  |       7 |      3.00596 |
| WFDC12 | cytotoxic T cell                                        |      2.0016  |            3 |       0.000268001 |          3.67475e-06 |                376 | 0.230588    | 0.113546  | 0.313147 | 0.362596 | -0.301398  | 0.075091   |   0.243151  | 0.156186  |       0.176548  |       7 |      3.00596 |
| ARG1   | mature alpha-beta T cell                                |      1.74303 |          501 |       0.0423321   |          0.000613683 |               1030 | 0.706712    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| FLG2   | mature alpha-beta T cell                                |      1.75784 |          282 |       0.0272359   |          0.000345427 |               1030 | 0.462492    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| LCE1A  | mature alpha-beta T cell                                |      1.81255 |           14 |       0.00135213  |          1.71488e-05 |               1030 | 0.167292    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| LCE1B  | mature alpha-beta T cell                                |      1.65688 |           80 |       0.00772648  |          9.79933e-05 |               1030 | 0.18256     | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| LCE1E  | mature alpha-beta T cell                                |      1.75065 |           64 |       0.00618119  |          7.83947e-05 |               1030 | 0.201488    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| LCE2B  | mature alpha-beta T cell                                |      1.71256 |           85 |       0.00820939  |          0.000104118 |               1030 | 0.210971    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| WFDC12 | mature alpha-beta T cell                                |      1.88127 |            4 |       0.000386324 |          4.89967e-06 |               1030 | 0.183206    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| ARG1   | alpha-beta T cell                                       |      1.74303 |          501 |       0.0423321   |          0.000613683 |               1030 | 0.706712    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| FLG2   | alpha-beta T cell                                       |      1.75784 |          282 |       0.0272359   |          0.000345427 |               1030 | 0.462492    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| LCE1A  | alpha-beta T cell                                       |      1.81255 |           14 |       0.00135213  |          1.71488e-05 |               1030 | 0.167292    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| LCE1B  | alpha-beta T cell                                       |      1.65688 |           80 |       0.00772648  |          9.79933e-05 |               1030 | 0.18256     | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| LCE1E  | alpha-beta T cell                                       |      1.75065 |           64 |       0.00618119  |          7.83947e-05 |               1030 | 0.201488    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| LCE2B  | alpha-beta T cell                                       |      1.71256 |           85 |       0.00820939  |          0.000104118 |               1030 | 0.210971    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| WFDC12 | alpha-beta T cell                                       |      1.88127 |            4 |       0.000386324 |          4.89967e-06 |               1030 | 0.183206    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| ARG1   | CD8-positive, alpha-beta T cell                         |      1.74303 |          501 |       0.0423321   |          0.000613683 |               1030 | 0.706712    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| FLG2   | CD8-positive, alpha-beta T cell                         |      1.75784 |          282 |       0.0272359   |          0.000345427 |               1030 | 0.462492    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| LCE1A  | CD8-positive, alpha-beta T cell                         |      1.81255 |           14 |       0.00135213  |          1.71488e-05 |               1030 | 0.167292    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| LCE1B  | CD8-positive, alpha-beta T cell                         |      1.65688 |           80 |       0.00772648  |          9.79933e-05 |               1030 | 0.18256     | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| LCE1E  | CD8-positive, alpha-beta T cell                         |      1.75065 |           64 |       0.00618119  |          7.83947e-05 |               1030 | 0.201488    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| LCE2B  | CD8-positive, alpha-beta T cell                         |      1.71256 |           85 |       0.00820939  |          0.000104118 |               1030 | 0.210971    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| WFDC12 | CD8-positive, alpha-beta T cell                         |      1.88127 |            4 |       0.000386324 |          4.89967e-06 |               1030 | 0.183206    | 0.205938  | 0.302103 | 0.681681 |  1.96022   | 0.175394   |   0.229121  | 0.46392   |       0.474899  |       7 |      2.44274 |
| ARG1   | memory T cell                                           |      1.74212 |          500 |       0.0482905   |          0.000612458 |               1029 | 0.714243    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| FLG2   | memory T cell                                           |      1.75784 |          282 |       0.0272359   |          0.000345427 |               1029 | 0.462492    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| LCE1A  | memory T cell                                           |      1.81255 |           14 |       0.00135213  |          1.71488e-05 |               1029 | 0.167292    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| LCE1B  | memory T cell                                           |      1.65688 |           80 |       0.00772648  |          9.79933e-05 |               1029 | 0.18256     | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| LCE1E  | memory T cell                                           |      1.75065 |           64 |       0.00618119  |          7.83947e-05 |               1029 | 0.201488    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| LCE2B  | memory T cell                                           |      1.71256 |           85 |       0.00820939  |          0.000104118 |               1029 | 0.210971    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| WFDC12 | memory T cell                                           |      1.88127 |            4 |       0.000386324 |          4.89967e-06 |               1029 | 0.183206    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| ARG1   | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.74212 |          500 |       0.0482905   |          0.000612458 |               1029 | 0.714243    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| FLG2   | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.75784 |          282 |       0.0272359   |          0.000345427 |               1029 | 0.462492    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| LCE1A  | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.81255 |           14 |       0.00135213  |          1.71488e-05 |               1029 | 0.167292    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| LCE1B  | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.65688 |           80 |       0.00772648  |          9.79933e-05 |               1029 | 0.18256     | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| LCE1E  | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.75065 |           64 |       0.00618119  |          7.83947e-05 |               1029 | 0.201488    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| LCE2B  | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.71256 |           85 |       0.00820939  |          0.000104118 |               1029 | 0.210971    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| WFDC12 | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.88127 |            4 |       0.000386324 |          4.89967e-06 |               1029 | 0.183206    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| ARG1   | CD8-positive, alpha-beta memory T cell                  |      1.74212 |          500 |       0.0482905   |          0.000612458 |               1029 | 0.714243    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| FLG2   | CD8-positive, alpha-beta memory T cell                  |      1.75784 |          282 |       0.0272359   |          0.000345427 |               1029 | 0.462492    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| LCE1A  | CD8-positive, alpha-beta memory T cell                  |      1.81255 |           14 |       0.00135213  |          1.71488e-05 |               1029 | 0.167292    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| LCE1B  | CD8-positive, alpha-beta memory T cell                  |      1.65688 |           80 |       0.00772648  |          9.79933e-05 |               1029 | 0.18256     | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| LCE1E  | CD8-positive, alpha-beta memory T cell                  |      1.75065 |           64 |       0.00618119  |          7.83947e-05 |               1029 | 0.201488    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| LCE2B  | CD8-positive, alpha-beta memory T cell                  |      1.71256 |           85 |       0.00820939  |          0.000104118 |               1029 | 0.210971    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| WFDC12 | CD8-positive, alpha-beta memory T cell                  |      1.88127 |            4 |       0.000386324 |          4.89967e-06 |               1029 | 0.183206    | 0.208409  | 0.303179 | 0.687413 |  2.0545    | 0.178076   |   0.230488  | 0.469447  |       0.487338  |       7 |      2.43499 |
| ARG1   | contractile cell                                        |      1.63344 |           14 |       0.000782123 |          1.71488e-05 |                575 | 0.0940999   | 0.0994588 | 0.190036 | 0.523368 | -1.50766   | 0.0597979  |   0.0867531 | 0.311239  |       0.0174175 |       7 |      2.25317 |
| FLG2   | contractile cell                                        |      1.72417 |          175 |       0.00977654  |          0.00021436  |                575 | 0.311526    | 0.0994588 | 0.190036 | 0.523368 | -1.50766   | 0.0597979  |   0.0867531 | 0.311239  |       0.0174175 |       7 |      2.25317 |
| KPRP   | contractile cell                                        |      1.57405 |            6 |       0.000335196 |          7.3495e-06  |                575 | 0.0611297   | 0.0994588 | 0.190036 | 0.523368 | -1.50766   | 0.0597979  |   0.0867531 | 0.311239  |       0.0174175 |       7 |      2.25317 |
| LCE1A  | contractile cell                                        |      1.76178 |           15 |       0.000837989 |          1.83738e-05 |                575 | 0.147057    | 0.0994588 | 0.190036 | 0.523368 | -1.50766   | 0.0597979  |   0.0867531 | 0.311239  |       0.0174175 |       7 |      2.25317 |
| LCE1B  | contractile cell                                        |      1.6668  |           74 |       0.00398535  |          9.06438e-05 |                575 | 0.174722    | 0.0994588 | 0.190036 | 0.523368 | -1.50766   | 0.0597979  |   0.0867531 | 0.311239  |       0.0174175 |       7 |      2.25317 |
| LCE1E  | contractile cell                                        |      1.72466 |          177 |       0.00953253  |          0.00021681  |                575 | 0.313436    | 0.0994588 | 0.190036 | 0.523368 | -1.50766   | 0.0597979  |   0.0867531 | 0.311239  |       0.0174175 |       7 |      2.25317 |
| LCE2B  | contractile cell                                        |      1.6885  |          114 |       0.0061396   |          0.000139641 |                575 | 0.228281    | 0.0994588 | 0.190036 | 0.523368 | -1.50766   | 0.0597979  |   0.0867531 | 0.311239  |       0.0174175 |       7 |      2.25317 |
| ARG1   | supporting cell                                         |      1.63344 |           14 |       0.000782123 |          1.71488e-05 |                569 | 0.0940999   | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| FLG2   | supporting cell                                         |      1.72417 |          175 |       0.00977654  |          0.00021436  |                569 | 0.311526    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| KPRP   | supporting cell                                         |      1.57405 |            6 |       0.000335196 |          7.3495e-06  |                569 | 0.0611297   | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE1A  | supporting cell                                         |      1.76178 |           15 |       0.000837989 |          1.83738e-05 |                569 | 0.147057    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE1B  | supporting cell                                         |      1.66265 |           72 |       0.00402235  |          8.8194e-05  |                569 | 0.171025    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE1E  | supporting cell                                         |      1.72441 |          176 |       0.0098324   |          0.000215585 |                569 | 0.312745    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE2B  | supporting cell                                         |      1.69139 |          111 |       0.00620112  |          0.000135966 |                569 | 0.226423    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| ARG1   | perivascular cell                                       |      1.63344 |           14 |       0.000782123 |          1.71488e-05 |                569 | 0.0940999   | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| FLG2   | perivascular cell                                       |      1.72417 |          175 |       0.00977654  |          0.00021436  |                569 | 0.311526    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| KPRP   | perivascular cell                                       |      1.57405 |            6 |       0.000335196 |          7.3495e-06  |                569 | 0.0611297   | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE1A  | perivascular cell                                       |      1.76178 |           15 |       0.000837989 |          1.83738e-05 |                569 | 0.147057    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE1B  | perivascular cell                                       |      1.66265 |           72 |       0.00402235  |          8.8194e-05  |                569 | 0.171025    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE1E  | perivascular cell                                       |      1.72441 |          176 |       0.0098324   |          0.000215585 |                569 | 0.312745    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE2B  | perivascular cell                                       |      1.69139 |          111 |       0.00620112  |          0.000135966 |                569 | 0.226423    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| ARG1   | pericyte                                                |      1.63344 |           14 |       0.000782123 |          1.71488e-05 |                569 | 0.0940999   | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| FLG2   | pericyte                                                |      1.72417 |          175 |       0.00977654  |          0.00021436  |                569 | 0.311526    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| KPRP   | pericyte                                                |      1.57405 |            6 |       0.000335196 |          7.3495e-06  |                569 | 0.0611297   | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE1A  | pericyte                                                |      1.76178 |           15 |       0.000837989 |          1.83738e-05 |                569 | 0.147057    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE1B  | pericyte                                                |      1.66265 |           72 |       0.00402235  |          8.8194e-05  |                569 | 0.171025    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE1E  | pericyte                                                |      1.72441 |          176 |       0.0098324   |          0.000215585 |                569 | 0.312745    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE2B  | pericyte                                                |      1.69139 |          111 |       0.00620112  |          0.000135966 |                569 | 0.226423    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| ARG1   | mural cell                                              |      1.63344 |           14 |       0.000782123 |          1.71488e-05 |                569 | 0.0940999   | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| FLG2   | mural cell                                              |      1.72417 |          175 |       0.00977654  |          0.00021436  |                569 | 0.311526    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| KPRP   | mural cell                                              |      1.57405 |            6 |       0.000335196 |          7.3495e-06  |                569 | 0.0611297   | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE1A  | mural cell                                              |      1.76178 |           15 |       0.000837989 |          1.83738e-05 |                569 | 0.147057    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE1B  | mural cell                                              |      1.66265 |           72 |       0.00402235  |          8.8194e-05  |                569 | 0.171025    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE1E  | mural cell                                              |      1.72441 |          176 |       0.0098324   |          0.000215585 |                569 | 0.312745    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| LCE2B  | mural cell                                              |      1.69139 |          111 |       0.00620112  |          0.000135966 |                569 | 0.226423    | 0.0993017 | 0.189144 | 0.525007 | -1.49791   | 0.0596274  |   0.0856197 | 0.312819  |       0.018703  |       7 |      2.24573 |
| ARG1   | macrophage                                              |      1.90637 |           30 |       0.000593437 |          3.67475e-05 |                180 | 0.220671    | 0.0570535 | 0.169488 | 0.336623 | -0.35391   | 0.0137619  |   0.0606496 | 0.131137  |       0.16962   |       7 |      2.17702 |
| FLG2   | macrophage                                              |      1.73277 |           66 |       0.0013459   |          8.08445e-05 |                180 | 0.189093    | 0.0570535 | 0.169488 | 0.336623 | -0.35391   | 0.0137619  |   0.0606496 | 0.131137  |       0.16962   |       7 |      2.17702 |
| KPRP   | macrophage                                              |      1.63116 |            4 |       8.15694e-05 |          4.89967e-06 |                180 | 0.0817363   | 0.0570535 | 0.169488 | 0.336623 | -0.35391   | 0.0137619  |   0.0606496 | 0.131137  |       0.16962   |       7 |      2.17702 |
| LCE1A  | macrophage                                              |      2.05177 |            4 |       8.15694e-05 |          4.89967e-06 |                180 | 0.25161     | 0.0570535 | 0.169488 | 0.336623 | -0.35391   | 0.0137619  |   0.0606496 | 0.131137  |       0.16962   |       7 |      2.17702 |
| LCE1B  | macrophage                                              |      1.70431 |           21 |       0.000428239 |          2.57233e-05 |                180 | 0.129467    | 0.0570535 | 0.169488 | 0.336623 | -0.35391   | 0.0137619  |   0.0606496 | 0.131137  |       0.16962   |       7 |      2.17702 |
| LCE1E  | macrophage                                              |      1.78152 |           26 |       0.000530201 |          3.18478e-05 |                180 | 0.165995    | 0.0570535 | 0.169488 | 0.336623 | -0.35391   | 0.0137619  |   0.0606496 | 0.131137  |       0.16962   |       7 |      2.17702 |
| LCE2B  | macrophage                                              |      1.7287  |           29 |       0.000573655 |          3.55226e-05 |                180 | 0.147846    | 0.0570535 | 0.169488 | 0.336623 | -0.35391   | 0.0137619  |   0.0606496 | 0.131137  |       0.16962   |       7 |      2.17702 |
| ARG1   | secretory cell                                          |      1.55463 |            2 |       0.000460405 |          2.44983e-06 |                110 | 0.0493211   | 0.0496663 | 0.145101 | 0.342287 |  2.3377    | 0.00574211 |   0.0296688 | 0.1366    |       0.524697  |       7 |      1.77845 |
| FLG2   | secretory cell                                          |      1.83537 |           38 |       0.00581395  |          4.65468e-05 |                110 | 0.208137    | 0.0496663 | 0.145101 | 0.342287 |  2.3377    | 0.00574211 |   0.0296688 | 0.1366    |       0.524697  |       7 |      1.77845 |
| KPRP   | secretory cell                                          |      1.86333 |            1 |       0.000456204 |          1.22492e-06 |                110 | 0.172947    | 0.0496663 | 0.145101 | 0.342287 |  2.3377    | 0.00574211 |   0.0296688 | 0.1366    |       0.524697  |       7 |      1.77845 |
| LCE1A  | secretory cell                                          |      1.73471 |            7 |       0.00107099  |          8.57442e-06 |                110 | 0.12816     | 0.0496663 | 0.145101 | 0.342287 |  2.3377    | 0.00574211 |   0.0296688 | 0.1366    |       0.524697  |       7 |      1.77845 |
| LCE1B  | secretory cell                                          |      1.72517 |           18 |       0.00275398  |          2.20485e-05 |                110 | 0.138261    | 0.0496663 | 0.145101 | 0.342287 |  2.3377    | 0.00574211 |   0.0296688 | 0.1366    |       0.524697  |       7 |      1.77845 |
| LCE1E  | secretory cell                                          |      1.76951 |           27 |       0.00413097  |          3.30728e-05 |                110 | 0.167585    | 0.0496663 | 0.145101 | 0.342287 |  2.3377    | 0.00574211 |   0.0296688 | 0.1366    |       0.524697  |       7 |      1.77845 |
| LCE2B  | secretory cell                                          |      1.76059 |           17 |       0.00260098  |          2.08236e-05 |                110 | 0.151297    | 0.0496663 | 0.145101 | 0.342287 |  2.3377    | 0.00574211 |   0.0296688 | 0.1366    |       0.524697  |       7 |      1.77845 |
| ARG1   | regulatory T cell                                       |      1.76247 |          350 |       0.0138603   |          0.000428721 |                890 | 0.514957    | 0.188936  | 0.262646 | 0.719355 | -1.63969   | 0.156936   |   0.178996  | 0.500253  |       0         |       6 |      2.15747 |
| FLG2   | regulatory T cell                                       |      1.72154 |          335 |       0.0132663   |          0.000410347 |                890 | 0.481949    | 0.188936  | 0.262646 | 0.719355 | -1.63969   | 0.156936   |   0.178996  | 0.500253  |       0         |       6 |      2.15747 |
| LCE1A  | regulatory T cell                                       |      1.54658 |           12 |       0.00047521  |          1.4699e-05  |                890 | 0.0564833   | 0.188936  | 0.262646 | 0.719355 | -1.63969   | 0.156936   |   0.178996  | 0.500253  |       0         |       6 |      2.15747 |
| LCE1B  | regulatory T cell                                       |      1.64569 |           86 |       0.00340567  |          0.000105343 |                890 | 0.177795    | 0.188936  | 0.262646 | 0.719355 | -1.63969   | 0.156936   |   0.178996  | 0.500253  |       0         |       6 |      2.15747 |
| LCE1E  | regulatory T cell                                       |      1.71314 |           39 |       0.00154443  |          4.77718e-05 |                890 | 0.153407    | 0.188936  | 0.262646 | 0.719355 | -1.63969   | 0.156936   |   0.178996  | 0.500253  |       0         |       6 |      2.15747 |
| LCE2B  | regulatory T cell                                       |      1.72804 |           68 |       0.00269286  |          8.32943e-05 |                890 | 0.191283    | 0.188936  | 0.262646 | 0.719355 | -1.63969   | 0.156936   |   0.178996  | 0.500253  |       0         |       6 |      2.15747 |
| ARG1   | helper T cell                                           |      1.70644 |          850 |       0.0199287   |          0.00104118  |               1546 | 1.02096     | 0.377336  | 0.356632 | 1.05805  |  1.0755    | 0.361467   |   0.298394  | 0.826903  |       0.358187  |       6 |      2.12532 |
| FLG2   | helper T cell                                           |      1.66333 |          468 |       0.0109725   |          0.000573261 |               1546 | 0.593195    | 0.377336  | 0.356632 | 1.05805  |  1.0755    | 0.361467   |   0.298394  | 0.826903  |       0.358187  |       6 |      2.12532 |
| LCE1A  | helper T cell                                           |      1.53077 |            9 |       0.00021101  |          1.10243e-05 |               1546 | 0.0465823   | 0.377336  | 0.356632 | 1.05805  |  1.0755    | 0.361467   |   0.298394  | 0.826903  |       0.358187  |       6 |      2.12532 |
| LCE1B  | helper T cell                                           |      1.60926 |          116 |       0.00271968  |          0.00014209  |               1546 | 0.193225    | 0.377336  | 0.356632 | 1.05805  |  1.0755    | 0.361467   |   0.298394  | 0.826903  |       0.358187  |       6 |      2.12532 |
| LCE1E  | helper T cell                                           |      1.65323 |           39 |       0.000914377 |          4.77718e-05 |               1546 | 0.128267    | 0.377336  | 0.356632 | 1.05805  |  1.0755    | 0.361467   |   0.298394  | 0.826903  |       0.358187  |       6 |      2.12532 |
| LCE2B  | helper T cell                                           |      1.65926 |           64 |       0.00150052  |          7.83947e-05 |               1546 | 0.157558    | 0.377336  | 0.356632 | 1.05805  |  1.0755    | 0.361467   |   0.298394  | 0.826903  |       0.358187  |       6 |      2.12532 |
| ARG1   | monocyte                                                |      2.30484 |          192 |       0.0216362   |          0.000235184 |                221 | 0.581493    | 0.200045  | 0.208008 | 0.961715 |  3.10147   | 0.168996   |   0.109585  | 0.733991  |       0.625454  |       6 |      1.34319 |
| FLG2   | monocyte                                                |      1.695   |           17 |       0.00191571  |          2.08236e-05 |                221 | 0.123781    | 0.200045  | 0.208008 | 0.961715 |  3.10147   | 0.168996   |   0.109585  | 0.733991  |       0.625454  |       6 |      1.34319 |
| LCE1A  | monocyte                                                |      1.44047 |            1 |       0.000112689 |          1.22492e-06 |                221 | 0.00165463  | 0.200045  | 0.208008 | 0.961715 |  3.10147   | 0.168996   |   0.109585  | 0.733991  |       0.625454  |       6 |      1.34319 |
| LCE1B  | monocyte                                                |      1.86066 |            5 |       0.000563444 |          6.12458e-06 |                221 | 0.176189    | 0.200045  | 0.208008 | 0.961715 |  3.10147   | 0.168996   |   0.109585  | 0.733991  |       0.625454  |       6 |      1.34319 |
| LCE1E  | monocyte                                                |      2.0446  |            3 |       0.000338066 |          3.67475e-06 |                221 | 0.248059    | 0.200045  | 0.208008 | 0.961715 |  3.10147   | 0.168996   |   0.109585  | 0.733991  |       0.625454  |       6 |      1.34319 |
| LCE2B  | monocyte                                                |      1.71978 |            3 |       0.000338066 |          3.67475e-06 |                221 | 0.116872    | 0.200045  | 0.208008 | 0.961715 |  3.10147   | 0.168996   |   0.109585  | 0.733991  |       0.625454  |       6 |      1.34319 |